In [ ]:
def convert_BirthYearToAge (fileName, year):
    # Read the contents of the text file
    with open(fileName, 'r') as file:
        data = file.readlines()
  
    new_data = []
    for line in data:
        json_obj = json.loads(line)
        # Extract the birth_year and calculate the age
        birth_year = float(json_obj['birth_year'])
        age = year - birth_year

        json_obj['age'] = age
        del json_obj['birth_year']

        # Append the modified JSON object to the new data list
        new_data.append(json.dumps(json_obj, ensure_ascii=False))

    new_json = '\n'.join(new_data) ##modified by line

    with open('27.Donem_Milletvekili_Link.txt', 'w') as file:
        file.write(new_json)

    # Download the modified data file
    files.download('27.Donem_Milletvekili_Link.txt')


In [ ]:
def mergeing_ageWcomperhensiveData():
    # Read the contents of the files
    with open("27.Donem_Milletvekili_Link.txt", "r", encoding="utf-8") as link_file:
        link_data = link_file.readlines()

    with open("27Donem_age.csv", "r", encoding="windows-1254") as age_file:
        age_reader = csv.DictReader(age_file, delimiter=";")
        age_entries = [entry for entry in age_reader]

    # Merge the data
    link_entries = []
    for line in link_data:
        entry = json.loads(line)
        link_entries.append(entry)

    for link_entry in link_entries:
        for age_entry in age_entries:
            if (
                link_entry["name"] == age_entry["name"]
                and str(link_entry["term"]) == age_entry["term"]
                and link_entry["party"] == age_entry["party"]
                and link_entry["city"] == age_entry["city"]
            ):
                link_entry["birth_year"] = age_entry["birth_year"]
                break

    # Convert the merged data back to a string
    merged_data = "\n".join(json.dumps(entry, ensure_ascii=False) for entry in link_entries)

    # Write the merged data to a new file
    output_file_path = "27.txt"
    with open(output_file_path, "w", encoding="utf-8") as merged_file:
        merged_file.write(merged_data)


In [ ]:
def obtaningBirthYearWithSelenium(name):

    df = pd.read_csv(name, sep=";", encoding='windows-1254')
    print(df.head())

    # Initialize the Chrome driver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    for i, row in df.iterrows():
        # Only search for people without birth years comparing with csvDile
        if pd.isnull(row["birth_year"]):
            name = row["name"]
            
            driver.get("https://www.google.com")
            sleep(2)

            # find the search box, clear it, enter query and submit
            search_box = driver.find_element(By.NAME, "q")
            search_box.clear()
            search_box.send_keys(name + " yaş")
            search_box.send_keys(Keys.RETURN)
            sleep(2)

            try:
                birth_info = driver.find_element(By.CSS_SELECTOR, '.yxAsKe').text
                # Extract birth year using regex
                birth_year = re.search(r"\d{4}", birth_info).group()

                # Update DataFrame
                df.loc[i, "birth_year"] = birth_year
            except Exception as e:
                print(f"Birth year for {name} could not be retrieved due to error: {e}")


    driver.quit()

    # Write DataFrame back to CSV
    df.to_csv(name, index=False)

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

baseURL='https://tr.wikipedia.org'
URL = 'https://tr.wikipedia.org/wiki/TBMM_24._d%C3%B6nem_milletvekilleri_listesi'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
failed_links = []

# output.txt isimli bir dosya açıp yazılabilir hale getiriyoruz.
output_file = open("output.txt", "w")

def extract_year_from_paragraph(paragrap,links):
    try:
        match = re.search(r"\b(19|20)\d{2}\b", paragraph)
        idx = paragraph.find('(')
        name = paragraph[:idx].strip()
        if match and (len(name)<50):  ##ensure this bio paragraph is extracted

            output_file.write(f"{name}, {int(match.group())}, {links}\n")
        
    except Exception as e:
        output_file.write(f"Error occurred: {e}\n")
    return None

# This will find the table with the MPs
table = soup.find('table', {'class': 'wikitable'}) 

table=table[1] ## table which contains MPs name,city, parties together and directing their biographical wiki

for row in table.find_all('tr'):
    cells = row.find_all('td')
    for cell in cells:
        rowspan_attr = cell.get('rowspan')
        if rowspan_attr is None:
            link = cell.find('a')
            if link is not None:
              links=baseURL+link.get('href')      ##reaching biographical wiki
              pageNew = requests.get(links)
              soupNew = BeautifulSoup(pageNew.content, 'html.parser')
              try:
                mv_par = soupNew.find_all('div', {'class': 'mw-parser-output'})
                # Find the first paragraph tag and get its text
                paragraph = mv_par[0].find('p').text   
                extract_year_from_paragraph(paragraph,links). ##first paragraph contains name, birthdate
                ##extract_name_and_year(paragraph)
              except AttributeError:
                output_file.write(f"Failed to parse: {baseURL + link.get('href')}\n")  # link.get('href') is used here.
                failed_links.append(baseURL + link.get('href'))  # link.get('href') is used here.
# Print out failed links
output_file.write("Failed links:\n")
for link in failed_links:
    output_file.write(f"{link}\n")

output_file.close()


KeyError: ignored

In [ ]:
27.dönem
Jülide Sarıeroğlu, 1979, https://tr.wikipedia.org/wiki/J%C3%BClide_Sar%C4%B1ero%C4%9Flu
Tamer Dağlı, 1970, https://tr.wikipedia.org/wiki/Tamer_Da%C4%9Fl%C4%B1
Mehmet Şükrü Erdinç,, 1976, https://tr.wikipedia.org/wiki/Mehmet_%C5%9E%C3%BCkr%C3%BC_Erdin%C3%A7
Ahmet Zenbilci, 1966, https://tr.wikipedia.org/wiki/Ahmet_Zenbilci
Abdullah Doğru, 1965, https://tr.wikipedia.org/wiki/Abdullah_Do%C4%9Fru
Ayhan Barut, 1964, https://tr.wikipedia.org/wiki/Ayhan_Barut
Orhan Sümer, 1963, https://tr.wikipedia.org/wiki/Orhan_S%C3%BCmer
Müzeyyen Şevkin, 1961, https://tr.wikipedia.org/wiki/M%C3%BCzeyyen_%C5%9Eevkin
Burhanettin Bulut, 1970, https://tr.wikipedia.org/wiki/Burhanettin_Bulut
Tülay Hatimoğulları Oruç, 1977, https://tr.wikipedia.org/wiki/Tulay_Hat%C4%B1mo%C4%9Fullar%C4%B1_Oru%C3%A7
Kemal Peköz, 1951, https://tr.wikipedia.org/wiki/Kemal_Pek%C3%B6z
İsmail Koncuk, 1959, https://tr.wikipedia.org/wiki/%C4%B0smail_Koncuk
Mehmet Metanet Çulhaoğlu, 1954, https://tr.wikipedia.org/wiki/Mehmet_Metanet_%C3%87ulhao%C4%9Flu
Muharrem Varlı, 1969, https://tr.wikipedia.org/wiki/Muharrem_Varl%C4%B1
Ayşe Sibel Ersoy,, 1964, https://tr.wikipedia.org/wiki/Ay%C5%9Fe_Sibel_Ersoy
Ahmet Aydın,, 1971, https://tr.wikipedia.org/wiki/Ahmet_Ayd%C4%B1n_(Ad%C4%B1yamanl%C4%B1)
İbrahim Halil Fırat, 1973, https://tr.wikipedia.org/wiki/%C4%B0brahim_Halil_F%C4%B1rat
Muhammed Fatih Toprak, 1985, https://tr.wikipedia.org/wiki/Muhammed_Fatih_Toprak
Yakup Taş,, 1959, https://tr.wikipedia.org/wiki/Yakup_Ta%C5%9F
Abdurrahman Tutdere, 1976, https://tr.wikipedia.org/wiki/Abdurrahman_Tutdere
Veysel Eroğlu, 1948, https://tr.wikipedia.org/wiki/Veysel_Ero%C4%9Flu
Ali Özkaya, 1970, https://tr.wikipedia.org/wiki/Ali_%C3%96zkaya
İbrahim Yurdunuseven, 1969, https://tr.wikipedia.org/wiki/%C4%B0brahim_Yurdunuseven
Burcu Köksal, 1980, https://tr.wikipedia.org/wiki/Burcu_K%C3%B6ksal
Mehmet Taytak,, 1959, https://tr.wikipedia.org/wiki/Mehmet_Taytak
Gültekin Uysal, 1976, https://tr.wikipedia.org/wiki/G%C3%BCltekin_Uysal
Berdan Öztürk, 1980, https://tr.wikipedia.org/wiki/Berdan_%C3%96zt%C3%BCrk
Dirayet Taşdemir, 1982, https://tr.wikipedia.org/wiki/Dirayet_Ta%C5%9Fdemir
Abdullah Koç, 1972, https://tr.wikipedia.org/wiki/Abdullah_Ko%C3%A7
Ekrem Çelebi, 1965, https://tr.wikipedia.org/wiki/Ekrem_%C3%87elebi
İlknur İnceöz, 1973, https://tr.wikipedia.org/wiki/%C4%B0lknur_%C4%B0nce%C3%B6z
Cengiz Aydoğdu,, 1958, https://tr.wikipedia.org/wiki/Cengiz_Aydo%C4%9Fdu
Ramazan Kaşlı,, 1969, https://tr.wikipedia.org/wiki/Ramazan_Ka%C5%9Fl%C4%B1
Ayhan Erel, 1957, https://tr.wikipedia.org/wiki/Ayhan_Erel
Mustafa Levent Karahocagil,, 1959, https://tr.wikipedia.org/wiki/Mustafa_Levent_Karahocagil
Hasan Çilez, 1974, https://tr.wikipedia.org/wiki/Hasan_%C3%87ilez
Mustafa Tuncer, 1971, https://tr.wikipedia.org/wiki/Mustafa_Tuncer
Tekin Bingöl, 1955, https://tr.wikipedia.org/wiki/Tekin_Bing%C3%B6l
Levent Gök, 1959, https://tr.wikipedia.org/wiki/Levent_G%C3%B6k
Ali Haydar Hakverdi, 1979, https://tr.wikipedia.org/wiki/Ali_Haydar_Hakverdi
Bülent Kuşoğlu,, 1958, https://tr.wikipedia.org/wiki/B%C3%BClent_Ku%C5%9Fo%C4%9Flu
Gamze Taşcıer,, 1982, https://tr.wikipedia.org/wiki/Gamze_Ta%C5%9Fc%C4%B1er
Yıldırım Tuğrul Türkeş[1][2], 1954, https://tr.wikipedia.org/wiki/Y%C4%B1ld%C4%B1r%C4%B1m_Tu%C4%9Frul_T%C3%BCrke%C5%9F
Fatih Şahin,, 1979, https://tr.wikipedia.org/wiki/Fatih_%C5%9Eahin_(milletvekili)
Yalçın Akdoğan, 1969, https://tr.wikipedia.org/wiki/Yal%C3%A7%C4%B1n_Akdo%C4%9Fan
Ali İhsan Arslan,, 1969, https://tr.wikipedia.org/wiki/Ali_%C4%B0hsan_Arslan
Koray Aydın, 1955, https://tr.wikipedia.org/wiki/Koray_Ayd%C4%B1n
İbrahim Halil Oral, 1958, https://tr.wikipedia.org/wiki/%C4%B0brahim_Halil_Oral
Mevlüt Karakaya, 1963, https://tr.wikipedia.org/wiki/Mevl%C3%BCt_Karakaya
Filiz Kerestecioğlu Demir, 1961, https://tr.wikipedia.org/wiki/Filiz_Kerestecio%C4%9Flu_Demir
Mustafa Destici, 1966, https://tr.wikipedia.org/wiki/Mustafa_Destici
Lütfiye Selva Çam, 1970, https://tr.wikipedia.org/wiki/L%C3%BCtfiye_Selva_%C3%87am
Emrullah İşler, 1960, https://tr.wikipedia.org/wiki/Emrullah_%C4%B0%C5%9Fler
Orhan Yegin, 1978, https://tr.wikipedia.org/wiki/Orhan_Yegin
Arife Polat Düzgün, 1969, https://tr.wikipedia.org/wiki/Arife_Polat_D%C3%BCzg%C3%BCn
Zeynep Yıldız,, 1992, https://tr.wikipedia.org/wiki/Zeynep_Y%C4%B1ld%C4%B1z
Murat Emir, 1969, https://tr.wikipedia.org/wiki/Murat_Emir
Yıldırım Kaya, 1958, https://tr.wikipedia.org/wiki/Y%C4%B1ld%C4%B1r%C4%B1m_Kaya
Sadir Durmaz, 1963, https://tr.wikipedia.org/wiki/Sadir_Durmaz
Nevin Taşlıçay, 1974, https://tr.wikipedia.org/wiki/Nevin_Ta%C5%9Fl%C4%B1%C3%A7ay
Şenol Sunat, 1956, https://tr.wikipedia.org/wiki/%C5%9Eenol_Bal
Mehmet Naci Bostancı, 1957, https://tr.wikipedia.org/wiki/Mehmet_Naci_Bostanc%C4%B1
Asuman Erdoğan,, 1980, https://tr.wikipedia.org/wiki/Asuman_Erdo%C4%9Fan
Nevzat Ceylan, 1953, https://tr.wikipedia.org/wiki/Nevzat_Ceylan
Barış Aydın, 1975, https://tr.wikipedia.org/wiki/Bar%C4%B1%C5%9F_Ayd%C4%B1n
Hacı Turan, 1968, https://tr.wikipedia.org/wiki/Hac%C4%B1_Turan
Ahmet Haluk Koç, 1954, https://tr.wikipedia.org/wiki/Ahmet_Haluk_Ko%C3%A7
Nihat Yeşil, 1957, https://tr.wikipedia.org/wiki/Nihat_Ye%C5%9Fil
Servet Ünsal, 1957, https://tr.wikipedia.org/wiki/Servet_%C3%9Cnsal
Yaşar Yıldırım,, 1956, https://tr.wikipedia.org/wiki/Ya%C5%9Far_Y%C4%B1ld%C4%B1r%C4%B1m
Erkan Haberal[1][2], 1972, https://tr.wikipedia.org/wiki/Erkan_Haberal
Durmuş Yılmaz, 1947, https://tr.wikipedia.org/wiki/Durmu%C5%9F_Y%C4%B1lmaz
Ayhan Altıntaş, 1958, https://tr.wikipedia.org/wiki/Ayhan_Alt%C4%B1nta%C5%9F
Mevlüt Çavuşoğlu, 1968, https://tr.wikipedia.org/wiki/Mevl%C3%BCt_%C3%87avu%C5%9Fo%C4%9Flu
Failed to parse: https://tr.wikipedia.org/wiki/T%C3%BCrkiye_Cumhuriyeti_D%C4%B1%C5%9Fi%C5%9Fleri_Bakanl%C4%B1%C4%9F%C4%B1
Mustafa Köse, 1977, https://tr.wikipedia.org/wiki/Mustafa_K%C3%B6se
Sena Nur Çelik, 1986, https://tr.wikipedia.org/wiki/Sena_Nur_%C3%87elik
İbrahim Aydın, 1962, https://tr.wikipedia.org/wiki/%C4%B0brahim_Ayd%C4%B1n_(1962_do%C4%9Fumlu_siyaset%C3%A7i)
Atay Uslu, 1970, https://tr.wikipedia.org/wiki/Atay_Uslu
Kemal Çelik,, 1956, https://tr.wikipedia.org/wiki/Kemal_%C3%87elik_(Serik_do%C4%9Fumlu)
Deniz Baykal, 1938, https://tr.wikipedia.org/wiki/Deniz_Baykal
Çetin Osman Budak, 1960, https://tr.wikipedia.org/wiki/%C3%87etin_Osman_Budak
Rafet Zeybek, 1962, https://tr.wikipedia.org/wiki/Rafet_Zeybek
Aydın Özer, 1962, https://tr.wikipedia.org/wiki/Ayd%C4%B1n_%C3%96zer
Cavit Arı, 1968, https://tr.wikipedia.org/wiki/Cavit_Ar%C4%B1
Feridun Bahşi, 1959, https://tr.wikipedia.org/wiki/Feridun_Bah%C5%9Fi
Hasan Subaşı,, 1950, https://tr.wikipedia.org/wiki/Hasan_Suba%C5%9F%C4%B1
Tuba Vural Çokal, 1973, https://tr.wikipedia.org/wiki/Tuba_Vural_%C3%87okal
Abdurrahman Başkan, 1964, https://tr.wikipedia.org/wiki/Abdurrahman_Ba%C5%9Fkan
Kemal Bülbül, 1963, https://tr.wikipedia.org/wiki/Kemal_B%C3%BClb%C3%BCl
Orhan Atalay, 1965, https://tr.wikipedia.org/wiki/Orhan_Atalay
Bağımsız, 2018, https://tr.wikipedia.org/wiki/%C3%96zt%C3%BCrk_Y%C4%B1lmaz
Ertunç Erkan Balta, 1972, https://tr.wikipedia.org/wiki/Ertun%C3%A7_Erkan_Balta
Uğur Bayraktutan, 1965, https://tr.wikipedia.org/wiki/U%C4%9Fur_Bayraktutan
Mustafa Savaş, 1965, https://tr.wikipedia.org/wiki/Mustafa_Sava%C5%9F
Bekir Kuvvet Erim, 1957, https://tr.wikipedia.org/wiki/Bekir_Kuvvet_Erim
Metin Yavuz, 1969, https://tr.wikipedia.org/wiki/Metin_Yavuz
Rıza Posacı, 1963, https://tr.wikipedia.org/wiki/R%C4%B1za_Posac%C4%B1
Bülent Tezcan, 1965, https://tr.wikipedia.org/wiki/B%C3%BClent_Tezcan
Hüseyin Yıldız, 1967, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Y%C4%B1ld%C4%B1z_(siyaset%C3%A7i,_1967)
Süleyman Bülbül, 1964, https://tr.wikipedia.org/wiki/S%C3%BCleyman_B%C3%BClb%C3%BCl
Aydın Adnan Sezgin, 1956, https://tr.wikipedia.org/wiki/Ayd%C4%B1n_Adnan_Sezgin
Adil Çelik, 1968, https://tr.wikipedia.org/wiki/Adil_%C3%87elik
Belgin Uygur, 1977, https://tr.wikipedia.org/wiki/Belgin_Uygur
Yavuz Subaşı, 1961, https://tr.wikipedia.org/wiki/Yavuz_Suba%C5%9F%C4%B1
Mustafa Canbey, 1975, https://tr.wikipedia.org/wiki/Mustafa_Canbey
Pakize Mutlu Aydemir,, 1967, https://tr.wikipedia.org/wiki/Pakize_Mutlu_Aydemir
Ahmet Akın, 1973, https://tr.wikipedia.org/wiki/Ahmet_Ak%C4%B1n
Fikret Şahin, 1968, https://tr.wikipedia.org/wiki/Fikret_%C5%9Eahin
Ensar Aytekin, 1972, https://tr.wikipedia.org/wiki/Ensar_Aytekin
İsmail Ok,, 1964, https://tr.wikipedia.org/wiki/%C4%B0smail_Ok
Aysu Bankoğlu, 1989, https://tr.wikipedia.org/wiki/Aysu_Banko%C4%9Flu
Feleknas Uca, 1976, https://tr.wikipedia.org/wiki/Feleknas_Uca
Necdet İpekyüz, 1964, https://tr.wikipedia.org/wiki/Necdet_%C4%B0peky%C3%BCz
Ayşe Acar Başaran, 1985, https://tr.wikipedia.org/wiki/Ay%C5%9Fe_Acar_Ba%C5%9Faran
Mehmet Ruştu Tiryaki, 1971, https://tr.wikipedia.org/wiki/Mehmet_Ru%C5%9Ftu_Tiryaki
Ziver Özdemir, 1965, https://tr.wikipedia.org/wiki/Ziver_%C3%96zdemir
Fetani Battal, 1963, https://tr.wikipedia.org/wiki/Fetani_Battal
Selim Yağcı, 1966, https://tr.wikipedia.org/wiki/Selim_Ya%C4%9Fc%C4%B1
Yaşar Tüzün,, 1966, https://tr.wikipedia.org/wiki/Ya%C5%9Far_T%C3%BCz%C3%BCn
Cevdet Yılmaz, 1967, https://tr.wikipedia.org/wiki/Cevdet_Y%C4%B1lmaz
Feyzi Berdibek, 1957, https://tr.wikipedia.org/wiki/Feyzi_Berdibek
Erdal Aydemir, 1968, https://tr.wikipedia.org/wiki/Erdal_Aydemir
Cemal Taşar, 1963, https://tr.wikipedia.org/wiki/Cemal_Ta%C5%9Far
Vahit Kiler, 1966, https://tr.wikipedia.org/wiki/Vahit_Kiler
Mahmut Celadet Gaydalı, 1951, https://tr.wikipedia.org/wiki/Mahmut_Celadet_Gaydal%C4%B1
Arzu Aydın, 1969, https://tr.wikipedia.org/wiki/Arzu_Ayd%C4%B1n
Fehmi Küpçü, 1974, https://tr.wikipedia.org/wiki/Fehmi_K%C3%BCp%C3%A7%C3%BC
Tanju Özcan, 1973, https://tr.wikipedia.org/wiki/Tanju_%C3%96zcan
Bayram Özçelik,, 1966, https://tr.wikipedia.org/wiki/Bayram_%C3%96z%C3%A7elik
Yasin Uğur, 1978, https://tr.wikipedia.org/wiki/Yasin_U%C4%9Fur
Hakan Çavuşoğlu, 1972, https://tr.wikipedia.org/wiki/Hakan_%C3%87avu%C5%9Fo%C4%9Flu
Mustafa Esgin, 1967, https://tr.wikipedia.org/wiki/Mustafa_Esgin
Refik Özen, 1975, https://tr.wikipedia.org/wiki/Refik_%C3%96zen
Emine Yavuz Gözgeç, 1969, https://tr.wikipedia.org/wiki/Emine_Yavuz_G%C3%B6zge%C3%A7
Ahmet Kılıç, 1986, https://tr.wikipedia.org/wiki/Ahmet_K%C4%B1l%C4%B1%C3%A7_(1986_do%C4%9Fumlu_siyaset%C3%A7i)
Lale Karabıyık, 1965, https://tr.wikipedia.org/wiki/Lale_Karab%C4%B1y%C4%B1k
Erkan Aydın, 1974, https://tr.wikipedia.org/wiki/Erkan_Ayd%C4%B1n
İsmail Tatlıoğlu, 1960, https://tr.wikipedia.org/wiki/%C4%B0smail_Tatl%C4%B1o%C4%9Flu
İsmet Büyükataman, 1953, https://tr.wikipedia.org/wiki/%C4%B0smet_B%C3%BCy%C3%BCkataman
Efkan Âlâ, 1965, https://tr.wikipedia.org/wiki/Efkan_Ala
Osman Mesten, 1971, https://tr.wikipedia.org/wiki/Osman_Mesten
Vildan Yılmaz Gürel, 1976, https://tr.wikipedia.org/wiki/Vildan_Y%C4%B1lmaz_G%C3%BCrel
Zafer Işık, 1972, https://tr.wikipedia.org/wiki/Zafer_I%C5%9F%C4%B1k
Muhammet Müfit Aydın, 1954, https://tr.wikipedia.org/wiki/Muhammet_M%C3%BCfit_Ayd%C4%B1n
Atilla Ödünç, 1977, https://tr.wikipedia.org/wiki/Atilla_%C3%96d%C3%BCn%C3%A7
Orhan Sarıbal, 1966, https://tr.wikipedia.org/wiki/Orhan_Sar%C4%B1bal
Yüksel Özkan, 1960, https://tr.wikipedia.org/wiki/Y%C3%BCksel_%C3%96zkan
Mustafa Hidayet Vahapoğlu[1][2], 1956, https://tr.wikipedia.org/wiki/Mustafa_Hidayet_Vahapo%C4%9Flu
Ahmet Erozan, 1950, https://tr.wikipedia.org/wiki/Ahmet_Kamil_Erozan
Bülent Turan, 1975, https://tr.wikipedia.org/wiki/B%C3%BClent_Turan
Jülide İskenderoğlu, 1974, https://tr.wikipedia.org/wiki/J%C3%BClide_%C4%B0skendero%C4%9Flu
Muharrem Erkek,, 1969, https://tr.wikipedia.org/wiki/Muharrem_Erkek
Özgür Ceylan, 1973, https://tr.wikipedia.org/wiki/%C3%96zg%C3%BCr_Ceylan
Muhammet Emin Akbaşoğlu,, 1968, https://tr.wikipedia.org/wiki/Muhammet_Emin_Akba%C5%9Fo%C4%9Flu
Ahmet Sami Ceylan, 1971, https://tr.wikipedia.org/wiki/Ahmet_Sami_Ceylan
Oğuzhan Kaya, 1976, https://tr.wikipedia.org/wiki/O%C4%9Fuzhan_Kaya
Erol Kavuncu, 1963, https://tr.wikipedia.org/wiki/Erol_Kavuncu
Tufan Köse, 1967, https://tr.wikipedia.org/wiki/Tufan_K%C3%B6se
Cahit Özkan, 1976, https://tr.wikipedia.org/wiki/Cahit_%C3%96zkan
Şahin Tin, 1966, https://tr.wikipedia.org/wiki/%C5%9Eahin_Tin
Nilgün Ök, 1976, https://tr.wikipedia.org/wiki/Nilg%C3%BCn_%C3%96k
Gülizar Biçer Karaca, 1967, https://tr.wikipedia.org/wiki/G%C3%BClizar_Bi%C3%A7er_Karaca
Kazım Arslan, 1954, https://tr.wikipedia.org/wiki/Kaz%C4%B1m_Arslan_(1954_do%C4%9Fumlu_siyaset%C3%A7i)
Teoman Sancar, 1970, https://tr.wikipedia.org/wiki/Ha%C5%9Fim_Teoman_Sancar
Yasin Öztürk, 1972, https://tr.wikipedia.org/wiki/Yasin_%C3%96zt%C3%BCrk
Adnan Selçuk Mızraklı, 1963, https://tr.wikipedia.org/wiki/Adnan_Sel%C3%A7uk_M%C4%B1zrakl%C4%B1
Salihe Aydeniz, 1973, https://tr.wikipedia.org/wiki/Salihe_Aydeniz
İmam Taşçıer, 1959, https://tr.wikipedia.org/wiki/%C4%B0mam_Ta%C5%9F%C3%A7%C4%B1er
Remziye Tosun, 1981, https://tr.wikipedia.org/wiki/Remziye_Tosun
Hişyar Özsoy,, 1977, https://tr.wikipedia.org/wiki/Hi%C5%9Fyar_%C3%96zsoy
Semra Güzel, 1984, https://tr.wikipedia.org/wiki/Semra_G%C3%BCzel
Garo Paylan, 1972, https://tr.wikipedia.org/wiki/Garo_Paylan
Dersim Dağ, 1996, https://tr.wikipedia.org/wiki/Dersim_Da%C4%9F
Musa Farisoğulları, 1956, https://tr.wikipedia.org/wiki/Musa_Fariso%C4%9Fullar%C4%B1
Mehmet Mehdi Eker, 1956, https://tr.wikipedia.org/wiki/Mehmet_Mehdi_Eker
Ebubekir Bal, 1964, https://tr.wikipedia.org/wiki/Ebubekir_Bal
Oya Eronat, 1962, https://tr.wikipedia.org/wiki/Oya_Eronat
Fahri Çakır, 1958, https://tr.wikipedia.org/wiki/Fahri_%C3%87ak%C4%B1r
Ayşe Keşir, 1967, https://tr.wikipedia.org/wiki/Ay%C5%9Fe_Ke%C5%9Fir
Ümit Yılmaz, 1971, https://tr.wikipedia.org/wiki/%C3%9Cmit_Y%C4%B1lmaz
Okan Gaytancıoğlu, 1967, https://tr.wikipedia.org/wiki/Okan_Gaytanc%C4%B1o%C4%9Flu
Erdin Bircan,, 1959, https://tr.wikipedia.org/wiki/Erdin_Bircan
Fatma Aksal, 1965, https://tr.wikipedia.org/wiki/Fatma_Aksal
Orhan Çakırlar, 1954, https://tr.wikipedia.org/wiki/Orhan_%C3%87ak%C4%B1rlar
Metin Bulut, 1971, https://tr.wikipedia.org/wiki/Metin_Bulut
Sermin Balık, 1972, https://tr.wikipedia.org/wiki/Sermin_Bal%C4%B1k
Zülfü Tolga Ağar[5][6], 1975, https://tr.wikipedia.org/wiki/Z%C3%BClf%C3%BC_Tolga_A%C4%9Far
Zülfü Demirbağ,, 1956, https://tr.wikipedia.org/wiki/Z%C3%BClf%C3%BC_Demirba%C4%9F
Gürsel Erol, 1963, https://tr.wikipedia.org/wiki/G%C3%BCrsel_Erol
Süleyman Karaman, 1956, https://tr.wikipedia.org/wiki/S%C3%BCleyman_Karaman
Burhan Çakır,, 1969, https://tr.wikipedia.org/wiki/Burhan_%C3%87ak%C4%B1r
Recep Akdağ, 1960, https://tr.wikipedia.org/wiki/Recep_Akda%C4%9F
Zehra Taşkesenlioğlu Ban, 1972, https://tr.wikipedia.org/wiki/Zehra_Ta%C5%9Fkesenlio%C4%9Flu
Selami Altınok, 1966, https://tr.wikipedia.org/wiki/Selami_Alt%C4%B1nok
Kamil Aydın, 1963, https://tr.wikipedia.org/wiki/Kamil_Ayd%C4%B1n
Muhammet Naci Cinisli, 1969, https://tr.wikipedia.org/wiki/Muhammet_Naci_Cinisli
Nabi Avcı,, 1953, https://tr.wikipedia.org/wiki/Nabi_Avc%C4%B1
Emine Nur Günay, 1962, https://tr.wikipedia.org/wiki/Emine_Nur_G%C3%BCnay
Utku Çakırözer, 1970, https://tr.wikipedia.org/wiki/Utku_%C3%87ak%C4%B1r%C3%B6zer
Dr. Jale Nur Süllü,, 1962, https://tr.wikipedia.org/wiki/Jale_Nur_S%C3%BCll%C3%BC
Arslan Kabukcuoğlu, 1957, https://tr.wikipedia.org/wiki/Arslan_Kabukcuo%C4%9Flu
Metin Nurullah Sazak, 1959, https://tr.wikipedia.org/wiki/Metin_Nurullah_Sazak
Abdulhamit Gül, 1977, https://tr.wikipedia.org/wiki/Abdulhamit_G%C3%BCl
Ahmet Uzer, 1956, https://tr.wikipedia.org/wiki/Ahmet_Uzer
Mehmet Erdoğan, 1956, https://tr.wikipedia.org/wiki/Mehmet_Erdo%C4%9Fan_(Gaziantepli)
Abdullah Nejat Koçer, 1963, https://tr.wikipedia.org/wiki/Abdullah_Nejat_Ko%C3%A7er
Derya Bakbak, 1972, https://tr.wikipedia.org/wiki/Derya_Bakbak
Ali Şahin, 1970, https://tr.wikipedia.org/wiki/Ali_%C5%9Eahin_(1970_do%C4%9Fumlu_siyaset%C3%A7i)
Müslüm Yüksel, 1994, https://tr.wikipedia.org/wiki/M%C3%BCsl%C3%BCm_Y%C3%BCksel
Mehmet Sait Kirazoğlu, 1973, https://tr.wikipedia.org/wiki/Mehmet_Sait_Kirazo%C4%9Flu
Bayram Yılmazkaya, 1968, https://tr.wikipedia.org/wiki/Bayram_Y%C4%B1lmazkaya
İrfan Kaplan, 1954, https://tr.wikipedia.org/wiki/%C4%B0rfan_Kaplan
Ali Muhittin Taşdoğan, 1973, https://tr.wikipedia.org/wiki/Ali_Muhittin_Ta%C5%9Fdo%C4%9Fan
Sermet Atay, 1964, https://tr.wikipedia.org/wiki/Sermet_Atay
Mahmut Toğrul, 1967, https://tr.wikipedia.org/wiki/Mahmut_To%C4%9Frul
İmam Hüseyin Filiz, 1950, https://tr.wikipedia.org/wiki/%C4%B0mam_H%C3%BCseyin_Filiz
Cemal Öztürk, 1958, https://tr.wikipedia.org/wiki/Cemal_%C3%96zt%C3%BCrk
Kadir Aydın, 1971, https://tr.wikipedia.org/wiki/Kadir_Ayd%C4%B1n
Sabri Öztürk, 1973, https://tr.wikipedia.org/wiki/Sabri_%C3%96zt%C3%BCrk_(1973_do%C4%9Fumlu_siyaset%C3%A7i)
Necati Tığlı, 1983, https://tr.wikipedia.org/wiki/Necati_T%C4%B1%C4%9Fl%C4%B1
Hacı Osman Akgül, 1960, https://tr.wikipedia.org/wiki/Hac%C4%B1_Osman_Akg%C3%BCl
Cihan Pektaş, 1970, https://tr.wikipedia.org/wiki/Cihan_Pekta%C5%9F
Leyla Güven, 1964, https://tr.wikipedia.org/wiki/Leyla_G%C3%BCven_(siyaset%C3%A7i)
Sait Dede, 1985, https://tr.wikipedia.org/wiki/Sait_Dede
Husret Dinç, 1969, https://tr.wikipedia.org/wiki/Husret_Din%C3%A7
Adalet ve Kalkınma Partisi, 2011, https://tr.wikipedia.org/wiki/Hac%C4%B1_Bayram_T%C3%BCrko%C4%9Flu
Hüseyin Yayman, 1969, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Yayman
Abdulkadir Özel, 1979, https://tr.wikipedia.org/wiki/Abdulkadir_%C3%96zel
Hüseyin Şanverdi, 1963, https://tr.wikipedia.org/wiki/H%C3%BCseyin_%C5%9Eanverdi
Sabahat Özgürsoy Çelik, 1982, https://tr.wikipedia.org/wiki/Sabahat_%C3%96zg%C3%BCrsoy_%C3%87elik
Mehmet Güzelmansur, 1967, https://tr.wikipedia.org/wiki/Mehmet_G%C3%BCzelmansur
Suzan Şahin, 1968, https://tr.wikipedia.org/wiki/Suzan_%C5%9Eahin
Serkan Topal, 1976, https://tr.wikipedia.org/wiki/Serkan_Topal
İsmet Tokdemir, 1959, https://tr.wikipedia.org/wiki/%C4%B0smet_Tokdemir
Lütfi Kaşıkçı, 1982, https://tr.wikipedia.org/wiki/L%C3%BCtfi_Ka%C5%9F%C4%B1k%C3%A7%C4%B1
Habip Eksik, 1985, https://tr.wikipedia.org/wiki/Habip_Eksik
Yaşar Karadağ, 1976, https://tr.wikipedia.org/wiki/Ya%C5%9Far_Karada%C4%9F
Süreyya Sadi Bilgiç, 1961, https://tr.wikipedia.org/wiki/S%C3%BCreyya_Sadi_Bilgi%C3%A7
Recep Özel, 1969, https://tr.wikipedia.org/wiki/Recep_%C3%96zel_(siyaset%C3%A7i,_1969)
Mehmet Uğur Gökgöz, 1968, https://tr.wikipedia.org/wiki/Mehmet_U%C4%9Fur_G%C3%B6kg%C3%B6z
Aylin Cesur,, 1970, https://tr.wikipedia.org/wiki/Aylin_Cesur
Berat Albayrak,, 1978, https://tr.wikipedia.org/wiki/Berat_Albayrak
Nurettin Canikli, 1960, https://tr.wikipedia.org/wiki/Nurettin_Canikli
Erol Kaya,, 1959, https://tr.wikipedia.org/wiki/Erol_Kaya
Vedat Demiröz, 1956, https://tr.wikipedia.org/wiki/Vedat_Demir%C3%B6z
Ravza Kavakcı Kan, 1972, https://tr.wikipedia.org/wiki/Ravza_Kavakc%C4%B1_Kan
Osman Boyraz, 1974, https://tr.wikipedia.org/wiki/Osman_Boyraz
İsmet Uçma, 1955, https://tr.wikipedia.org/wiki/%C4%B0smet_U%C3%A7ma
Mihrimah Belma Satır, 1961, https://tr.wikipedia.org/wiki/Mihrimah_Belma_Sat%C4%B1r
Ahmet Berat Çonkar,, 1976, https://tr.wikipedia.org/wiki/Ahmet_Berat_%C3%87onkar
Hulusi Şentürk, 1963, https://tr.wikipedia.org/wiki/Hulusi_%C5%9Eent%C3%BCrk
Müşerref Pervin Tuba Durgut, 1973, https://tr.wikipedia.org/wiki/M%C3%BC%C5%9Ferref_Pervin_Tuba_Durgut
Eyüp Özsoy, 1983, https://tr.wikipedia.org/wiki/Ey%C3%BCp_%C3%96zsoy
Serkan Bayram, 1974, https://tr.wikipedia.org/wiki/Serkan_Bayram
Fatih Süleyman Denizolgun, 1987, https://tr.wikipedia.org/wiki/Fatih_S%C3%BCleyman_Denizolgun
İbrahim Özden Kaboğlu[1], 1950, https://tr.wikipedia.org/wiki/%C4%B0brahim_%C3%96zden_Kabo%C4%9Flu
Mehmet Akif Hamzaçebi, 1953, https://tr.wikipedia.org/wiki/Mehmet_Akif_Hamza%C3%A7ebi
Gamze Akkuş İlgezdi, 1969, https://tr.wikipedia.org/wiki/Gamze_Akku%C5%9F_%C4%B0lgezdi
Onursal Adıgüzel, 1985, https://tr.wikipedia.org/wiki/Onursal_Ad%C4%B1g%C3%BCzel
Fethi Açıkel, 1970, https://tr.wikipedia.org/wiki/Fethi_A%C3%A7%C4%B1kel
Gürsel Tekin, 1964, https://tr.wikipedia.org/wiki/G%C3%BCrsel_Tekin
İlhan Kesici, 1948, https://tr.wikipedia.org/wiki/%C4%B0lhan_Kesici
Oğuz Kaan Salıcı,, 1972, https://tr.wikipedia.org/wiki/O%C4%9Fuz_Kaan_Sal%C4%B1c%C4%B1
Mahmut Tanal, 1961, https://tr.wikipedia.org/wiki/Mahmut_Tanal
Saliha Sera Kadıgil[1], 1984, https://tr.wikipedia.org/wiki/Saliha_Sera_Kad%C4%B1gil_S%C3%BCtl%C3%BC
Pervin Buldan, 1967 https://tr.wikipedia.org/wiki/Pervin_Buldan
Musa Piroğlu, 1968, https://tr.wikipedia.org/wiki/Musa_Piro%C4%9Flu
Erol Katırcıoğlu, 1951, https://tr.wikipedia.org/wiki/Erol_Kat%C4%B1rc%C4%B1o%C4%9Flu
Erkan Baş, 1979, https://tr.wikipedia.org/wiki/Erkan_Ba%C5%9F
Hayrettin Nuhoğlu, 1953, https://tr.wikipedia.org/wiki/Hayrettin_Nuho%C4%9Flu
Ahmet Çelik, 1957, https://tr.wikipedia.org/wiki/Ahmet_%C3%87elik_(1957_do%C4%9Fumlu_siyaset%C3%A7i)
Yavuz Ağıralioğlu, 1972, https://tr.wikipedia.org/wiki/Yavuz_A%C4%9F%C4%B1ralio%C4%9Flu
Edip Semih Yalçın, 1958, https://tr.wikipedia.org/wiki/Edip_Semih_Yal%C3%A7%C4%B1n
İzzet Ulvi Yönter, 1976, https://tr.wikipedia.org/wiki/%C4%B0zzet_Ulvi_Y%C3%B6nter
Memet Bülent Karataş,, 1972, https://tr.wikipedia.org/wiki/Mehmet_B%C3%BClent_Karata%C5%9F
Süleyman Soylu, 1969, https://tr.wikipedia.org/wiki/S%C3%BCleyman_Soylu
Fatma Betül Sayan Kaya, 1981, https://tr.wikipedia.org/wiki/Fatma_Bet%C3%BCl_Sayan_Kaya
Volkan Bozkır, 1950, https://tr.wikipedia.org/wiki/Volkan_Bozk%C4%B1r
Abdullah Güler, 1969, https://tr.wikipedia.org/wiki/Abdullah_G%C3%BCler_(1969_do%C4%9Fumlu_siyaset%C3%A7i)
Hasan Turan, 1967, https://tr.wikipedia.org/wiki/Hasan_Turan_(siyaset%C3%A7i,_1967)
Serap Yaşar, 1965, https://tr.wikipedia.org/wiki/Serap_Ya%C5%9Far
Aziz Babuşcu, 1964, https://tr.wikipedia.org/wiki/Aziz_Babu%C5%9Fcu
Şirin Ünal, 1954, https://tr.wikipedia.org/wiki/%C5%9Eirin_%C3%9Cnal
Canan Kalsın,, 1967, https://tr.wikipedia.org/wiki/Canan_Kals%C4%B1n
Ahmet Hamdi Çamlı, 1965, https://tr.wikipedia.org/wiki/Ahmet_Hamdi_%C3%87aml%C4%B1
Markar Esayan, 1969, https://tr.wikipedia.org/wiki/Markar_Esayan
Mustafa Demir,, 1959, https://tr.wikipedia.org/wiki/Mustafa_Demir_(1959_do%C4%9Fumlu_siyaset%C3%A7i)
İffet Polat, 1968, https://tr.wikipedia.org/wiki/%C4%B0ffet_Polat
Kadri Enis Berberoğlu,, 1956, https://tr.wikipedia.org/wiki/Kadri_Enis_Berbero%C4%9Flu
Aykut Erdoğdu, 1972, https://tr.wikipedia.org/wiki/Aykut_Erdo%C4%9Fdu
Ahmet Ünal Çeviköz, 1952, https://tr.wikipedia.org/wiki/Ahmet_%C3%9Cnal_%C3%87evik%C3%B6z
Mustafa Sezgin Tanrıkulu, 1963, https://tr.wikipedia.org/wiki/Mustafa_Sezgin_Tanr%C4%B1kulu
Yüksel Mansur Kılınç, 1961, https://tr.wikipedia.org/wiki/Y%C3%BCksel_Mansur_K%C4%B1l%C4%B1n%C3%A7
Yunus Emre, 1982, https://tr.wikipedia.org/wiki/Yunus_Emre_(siyaset%C3%A7i)
Gökan Zeybek, 1964, https://tr.wikipedia.org/wiki/G%C3%B6kan_Zeybek
Ali Şeker, 1968, https://tr.wikipedia.org/wiki/Ali_%C5%9Eeker
Ahmet Şık, 1970, https://tr.wikipedia.org/wiki/Ahmet_%C5%9E%C4%B1k
Oya Ersoy, 1970, https://tr.wikipedia.org/wiki/Oya_Ersoy
Hüda Kaya, 1960, https://tr.wikipedia.org/wiki/H%C3%BCda_Kaya
Celal Adan,, 1951, https://tr.wikipedia.org/wiki/Celal_Adan
Cemal Çetin, 1963, https://tr.wikipedia.org/wiki/Cemal_%C3%87etin
Ümit Özdağ, 1961, https://tr.wikipedia.org/wiki/%C3%9Cmit_%C3%96zda%C4%9F
Failed to parse: https://tr.wikipedia.org/wiki/Zafer_Partisi
Numan Kurtulmuş, 1959, https://tr.wikipedia.org/wiki/Numan_Kurtulmu%C5%9F
Mehmet Muş, 1982, https://tr.wikipedia.org/wiki/Mehmet_Mu%C5%9F
Akif Çağatay Kılıç, 1976, https://tr.wikipedia.org/wiki/Akif_%C3%87a%C4%9Fatay_K%C4%B1l%C4%B1%C3%A7
Mehmet Doğan Kubat, 1969, https://tr.wikipedia.org/wiki/Mehmet_Do%C4%9Fan_Kubat
Tülay Kaynarca, 1969, https://tr.wikipedia.org/wiki/T%C3%BClay_Kaynarca
Halis Dalkılıç, 1968, https://tr.wikipedia.org/wiki/Halis_Dalk%C4%B1l%C4%B1%C3%A7
Emine Sare Aydın, 1975, https://tr.wikipedia.org/wiki/Emine_Sare_Ayd%C4%B1n
Nevzat Şatıroğlu, 1967, https://tr.wikipedia.org/wiki/Nevzat_%C5%9Eat%C4%B1ro%C4%9Flu
Zafer Sırakaya, 1974, https://tr.wikipedia.org/wiki/Zafer_S%C4%B1rakaya
Rümeysa Kadak, 1996, https://tr.wikipedia.org/wiki/R%C3%BCmeysa_Kadak
Ahmet Mücahit Arınç, 1986, https://tr.wikipedia.org/wiki/Ahmet_M%C3%BCcahit_Ar%C4%B1n%C3%A7
Mustafa Yeneroğlu, 1975, https://tr.wikipedia.org/wiki/Mustafa_Yenero%C4%9Flu
Erkan Kandemir, 1981, https://tr.wikipedia.org/wiki/Erkan_Kandemir
Şamil Ayrım,, 1950, https://tr.wikipedia.org/wiki/%C5%9Eamil_Ayr%C4%B1m
Alev Dedegil,, 1958, https://tr.wikipedia.org/wiki/Alev_Dedegil
Engin Altay, 1963, https://tr.wikipedia.org/wiki/Engin_Altay
Erdoğan Toprak, 1959, https://tr.wikipedia.org/wiki/Erdo%C4%9Fan_Toprak
Saadet Partisi, 2001, https://tr.wikipedia.org/wiki/Mehmet_Bekaro%C4%9Flu
Zeynel Emre, 1981, https://tr.wikipedia.org/wiki/Zeynel_Emre
Sibel Özdemir, 1979, https://tr.wikipedia.org/wiki/Sibel_%C3%96zdemir
Özgür Karabat, 1974, https://tr.wikipedia.org/wiki/%C3%96zg%C3%BCr_Karabat
Turan Aydoğan, 1964, https://tr.wikipedia.org/wiki/Turan_Aydo%C4%9Fan
Emine Gülizar Emecan, 1968, https://tr.wikipedia.org/wiki/Emine_G%C3%BClizar_Emecan
Nazır Cihangir İslam, 1959, https://tr.wikipedia.org/wiki/Naz%C4%B1r_Cihangir_%C4%B0slam
Zeynel Özen, 1957, https://tr.wikipedia.org/wiki/Zeynel_%C3%96zen
Dilşat Canbaz Kaya, 1973, https://tr.wikipedia.org/wiki/Dil%C5%9Fat_Canbaz_Kaya
Eşitlik ve Demokrasi Partisi, 2010, https://tr.wikipedia.org/wiki/Hakk%C4%B1_Saruhan_Olu%C3%A7
Ali Kenanoğlu,, 1971, https://tr.wikipedia.org/wiki/Ali_Kenano%C4%9Flu
Züleyha Gülüm, 1971, https://tr.wikipedia.org/wiki/Z%C3%BCleyha_G%C3%BCl%C3%BCm
Feti Yıldız,, 1953, https://tr.wikipedia.org/wiki/Feti_Y%C4%B1ld%C4%B1z
İsmail Faruk Aksu, 1963, https://tr.wikipedia.org/wiki/%C4%B0smail_Faruk_Aksu
Arzu Erdem, 1970, https://tr.wikipedia.org/wiki/Arzu_Erdem
Abdul Ahat Andican, 1951, https://tr.wikipedia.org/wiki/Abdul_Ahat_Andican
Ümit Beyaz, 1971, https://tr.wikipedia.org/wiki/%C3%9Cmit_Beyaz
Hayati Arkaz, 1957, https://tr.wikipedia.org/wiki/Hayati_Arkaz
Ahmet Tuncay Özkan, 1966, https://tr.wikipedia.org/wiki/Ahmet_Tuncay_%C3%96zkan
2018 - İzmir, 2018, https://tr.wikipedia.org/wiki/Murat_Bakan
Özcan Purçu, 1977, https://tr.wikipedia.org/wiki/%C3%96zcan_Pur%C3%A7u
Kani Beko, 1953, https://tr.wikipedia.org/wiki/Kani_Beko
Mehmet Ali Çelebi, 1984, https://tr.wikipedia.org/wiki/Mehmet_Ali_%C3%87elebi
Tacettin Bayır, 1960, https://tr.wikipedia.org/wiki/Tacettin_Bay%C4%B1r
Ednan Arslan, 1979, https://tr.wikipedia.org/wiki/Ednan_Arslan
Binali Yıldırım, 1955, https://tr.wikipedia.org/wiki/Binali_Y%C4%B1ld%C4%B1r%C4%B1m
Mahmut Atilla Kaya,, 1973, https://tr.wikipedia.org/wiki/Mahmut_Atilla_Kaya
Ceyda Bölünmez Çankırı, 1978, https://tr.wikipedia.org/wiki/Ceyda_B%C3%B6l%C3%BCnmez_%C3%87ank%C4%B1r%C4%B1
Cemal Bekle, 1978, https://tr.wikipedia.org/wiki/Cemal_Bekle
Serpil Kemalbay Pekgözegü, 1964, https://tr.wikipedia.org/wiki/Serpil_Kemalbay_Pekg%C3%B6zeg%C3%BC
Dursun Müsavat Dervişoğlu, 1960, https://tr.wikipedia.org/wiki/Dursun_M%C3%BCsavat_Dervi%C5%9Fo%C4%9Flu
Hasan Kalyoncu, 1969, https://tr.wikipedia.org/wiki/Hasan_Kalyoncu
Failed to parse: https://tr.wikipedia.org/wiki/Kemal_K%C4%B1l%C4%B1%C3%A7daro%C4%9Flu
Selin Sayek Böke[1][2], 1972, https://tr.wikipedia.org/wiki/Selin_Sayek_B%C3%B6ke
Kamil Okyay Sındır,, 1962, https://tr.wikipedia.org/wiki/Kamil_Okyay_S%C4%B1nd%C4%B1r
Atila Sertel, 1956, https://tr.wikipedia.org/wiki/Atilla_Sertel
Sevda Erdan Kılıç, 1980, https://tr.wikipedia.org/wiki/Sevda_Erdan_K%C4%B1l%C4%B1%C3%A7
Mahir Polat,, 1975, https://tr.wikipedia.org/wiki/Mahir_Polat
Bedri Serter, 1960, https://tr.wikipedia.org/wiki/Bedri_Serter
Hamza Dağ, 1980, https://tr.wikipedia.org/wiki/Hamza_Da%C4%9F
Necip Nasır, 1959, https://tr.wikipedia.org/wiki/Necip_Nas%C4%B1r
Yaşar Kırkpınar, 1973, https://tr.wikipedia.org/wiki/Ya%C5%9Far_K%C4%B1rkp%C4%B1nar
Aytun Çıray, 1957, https://tr.wikipedia.org/wiki/Aytun_%C3%87%C4%B1ray
Murat Çepni, 1974, https://tr.wikipedia.org/wiki/Murat_%C3%87epni
Tamer Osmanağaoğlu, 1963, https://tr.wikipedia.org/wiki/Tamer_Osmana%C4%9Fao%C4%9Flu
Mahir Ünal,, 1966, https://tr.wikipedia.org/wiki/Mahir_%C3%9Cnal
Ahmet Özdemir, 1977, https://tr.wikipedia.org/wiki/Ahmet_%C3%96zdemir_(Kahramanmara%C5%9F_do%C4%9Fumlu)
Celalettin Güvenç, 1959, https://tr.wikipedia.org/wiki/Celalettin_G%C3%BCven%C3%A7
İmran Kılıç, 1957, https://tr.wikipedia.org/wiki/%C4%B0mran_K%C4%B1l%C4%B1%C3%A7
Mehmet Cihat Sezal, 1980, https://tr.wikipedia.org/wiki/Mehmet_Cihat_Sezal
Habibe Öçal, 1968, https://tr.wikipedia.org/wiki/Habibe_%C3%96%C3%A7al
Prof. Dr. Sefer Aycan,, 1963, https://tr.wikipedia.org/wiki/Sefer_Aycan
Ali Öztunç, 1976, https://tr.wikipedia.org/wiki/Ali_%C3%96ztun%C3%A7
Cumhur Ünal, 1966, https://tr.wikipedia.org/wiki/Cumhur_%C3%9Cnal
Niyazi Güneş, 1953, https://tr.wikipedia.org/wiki/Niyazi_G%C3%BCne%C5%9F
Memleket Partisi, 2021, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Avni_Aksoy_(1956_do%C4%9Fumlu_siyaset%C3%A7i)
Dr. Recep Şeker, 1972, https://tr.wikipedia.org/wiki/Recep_%C5%9Eeker
Av. Selman Oğuzhan Eser, 1983, https://tr.wikipedia.org/wiki/Selman_O%C4%9Fuzhan_Eser
Av. İsmail Atakan Ünver, 1974, https://tr.wikipedia.org/wiki/%C4%B0smail_Atakan_%C3%9Cnver
Ahmet Arslan, 1962, https://tr.wikipedia.org/wiki/Ahmet_Arslan_(siyaset%C3%A7i)
Yunus Kılıç, 1965, https://tr.wikipedia.org/wiki/Yunus_K%C4%B1l%C4%B1%C3%A7
Ayhan Bilgen, 1971, https://tr.wikipedia.org/wiki/Ayhan_Bilgen
Hakkı Köylü, 1948, https://tr.wikipedia.org/wiki/Hakk%C4%B1_K%C3%B6yl%C3%BC
Metin Çelik, 1977, https://tr.wikipedia.org/wiki/Metin_%C3%87elik
Hasan Baltacı, 1978, https://tr.wikipedia.org/wiki/Hasan_Baltac%C4%B1
Mehmet Özhaseki, 1957, https://tr.wikipedia.org/wiki/Mehmet_%C3%96zhaseki
Mustafa Elitaş, 1957, https://tr.wikipedia.org/wiki/Mustafa_Elita%C5%9F
Taner Yıldız, 1962, https://tr.wikipedia.org/wiki/Taner_Y%C4%B1ld%C4%B1z
İsmail Tamer, 1956, https://tr.wikipedia.org/wiki/%C4%B0smail_Tamer
Hülya Nergis, 1967, https://tr.wikipedia.org/wiki/H%C3%BClya_Nergis
İsmail Özdemir, 1985, https://tr.wikipedia.org/wiki/%C4%B0smail_%C3%96zdemir
Mustafa Baki Ersoy, 1981, https://tr.wikipedia.org/wiki/Mustafa_Baki_Ersoy
Çetin Arık, 1971, https://tr.wikipedia.org/wiki/%C3%87etin_Ar%C4%B1k
Dursun Ataş, 1966, https://tr.wikipedia.org/wiki/Dursun_Ata%C5%9F
Ramazan Can, 1970, https://tr.wikipedia.org/wiki/Ramazan_Can
Halil Öztürk, 1975, https://tr.wikipedia.org/wiki/Halil_%C3%96zt%C3%BCrk_(K%C4%B1r%C4%B1kkale_do%C4%9Fumlu)
Ahmet Önal, 1983, https://tr.wikipedia.org/wiki/Ahmet_%C3%96nal
Türabi Kayan, 1955, https://tr.wikipedia.org/wiki/T%C3%BCrabi_Kayan
Vecdi Gündoğdu, 1969, https://tr.wikipedia.org/wiki/Vecdi_G%C3%BCndo%C4%9Fdu
Selahattin Minsolmaz, 1970, https://tr.wikipedia.org/wiki/Selahattin_Minsolmaz
Mustafa Kendirli, 1975, https://tr.wikipedia.org/wiki/Mustafa_Kendirli
Metin İlhan, 1972, https://tr.wikipedia.org/wiki/Metin_%C4%B0lhan_(1972_do%C4%9Fumlu_siyaset%C3%A7i)
Hilmi Dülger, 1962, https://tr.wikipedia.org/wiki/Mustafa_Hilmi_D%C3%BClger
Ahmet Salih Dal,, 1971, https://tr.wikipedia.org/wiki/Ahmet_Salih_Dal
Fikri Işık, 1965, https://tr.wikipedia.org/wiki/Fikri_I%C5%9F%C4%B1k
İlyas Şeker, 1960, https://tr.wikipedia.org/wiki/%C4%B0lyas_%C5%9Eeker
Radiye Sezer Katırcıoğlu, 1965, https://tr.wikipedia.org/wiki/Radiye_Sezer_Kat%C4%B1rc%C4%B1o%C4%9Flu
Mehmet Akif Yılmaz, 1975, https://tr.wikipedia.org/wiki/Mehmet_Akif_Y%C4%B1lmaz
Sami Çakır, 1958, https://tr.wikipedia.org/wiki/Sami_%C3%87ak%C4%B1r
Cemil Yaman, 1970, https://tr.wikipedia.org/wiki/Cemil_Yaman
Emine Zeybek, 1967, https://tr.wikipedia.org/wiki/Emine_Zeybek
Fatma Kaplan Hürriyet, 1982, https://tr.wikipedia.org/wiki/Fatma_Kaplan_H%C3%BCrriyet
Haydar Akar,, 1963, https://tr.wikipedia.org/wiki/Haydar_Akar
Tahsin Tarhan, 1964, https://tr.wikipedia.org/wiki/Tahsin_Tarhan
Lütfü Türkkan, 1959, https://tr.wikipedia.org/wiki/L%C3%BCtf%C3%BC_T%C3%BCrkkan
Ömer Faruk Gergerlioğlu, 1965, https://tr.wikipedia.org/wiki/%C3%96mer_Faruk_Gergerlio%C4%9Flu
Ahmet Sorgun, 1958, https://tr.wikipedia.org/wiki/Ahmet_Sorgun
Tahir Akyürek,, 1959, https://tr.wikipedia.org/wiki/Tahir_Aky%C3%BCrek
Orhan Erdem, 1963, https://tr.wikipedia.org/wiki/Orhan_Erdem
Leyla Şahin Usta, 1973, https://tr.wikipedia.org/wiki/Leyla_%C5%9Eahin_Usta
Ziya Altunyaldız, 1963, https://tr.wikipedia.org/wiki/Ziya_Altunyald%C4%B1z
Hacı Ahmet Özdemir, 1961, https://tr.wikipedia.org/wiki/Hac%C4%B1_Ahmet_%C3%96zdemir
Av. Gülay Samancı, 1977, https://tr.wikipedia.org/wiki/G%C3%BClay_Samanc%C4%B1
Abdullah Ağralı, 1969, https://tr.wikipedia.org/wiki/Abdullah_A%C4%9Fral%C4%B1
Selman Özboyacı, 1989, https://tr.wikipedia.org/wiki/Selman_%C3%96zboyac%C4%B1
Halil Etyemez, 1965, https://tr.wikipedia.org/wiki/Halil_Etyemez
Mustafa Kalaycı, 1960, https://tr.wikipedia.org/wiki/Mustafa_Kalayc%C4%B1
Esin Kara, 1976, https://tr.wikipedia.org/wiki/Esin_Kara
Abdüllatif Şener, 1954, https://tr.wikipedia.org/wiki/Abdullatif_%C5%9Eener
Abdulkadir Karaduman, 1992, https://tr.wikipedia.org/wiki/Abdulkadir_Karaduman
Fahrettin Yokuş, 1958, https://tr.wikipedia.org/wiki/Fahrettin_Yoku%C5%9F
Ahmet Tan, 1965, https://tr.wikipedia.org/wiki/Ahmet_Tan_(siyaset%C3%A7i,_1965)
Ceyda Çetin Erenler, 1975, https://tr.wikipedia.org/wiki/Ceyda_%C3%87etin_Erenler
Ahmet Erbaş, 1966, https://tr.wikipedia.org/wiki/Ahmet_Erba%C5%9F
Ali Fazıl Kasap, 1965, https://tr.wikipedia.org/wiki/Ali_Faz%C4%B1l_Kasap
Bülent Tüfenkci, 1966, https://tr.wikipedia.org/wiki/B%C3%BClent_T%C3%BCfenkci
Öznur Çalık, 1965, https://tr.wikipedia.org/wiki/%C3%96znur_%C3%87al%C4%B1k
Hakan Kahtalı, 1966, https://tr.wikipedia.org/wiki/Hakan_Kahtal%C4%B1
Cumhuriyet Halk Partisi, 1995, https://tr.wikipedia.org/wiki/Veli_A%C4%9Fbaba
Mehmet Celal Fendoğlu, 1961, https://tr.wikipedia.org/wiki/Mehmet_Celal_Fendo%C4%9Flu
Murat Baybatur, 1978, https://tr.wikipedia.org/wiki/Murat_Baybatur
Uğur Aydemir, 1967, https://tr.wikipedia.org/wiki/U%C4%9Fur_Aydemir
İsmail Bilen, 1961, https://tr.wikipedia.org/wiki/%C4%B0smail_Bilen_(1961_do%C4%9Fumlu_siyaset%C3%A7i)
Semra Kaplan Kıvırcık, 1976, https://tr.wikipedia.org/wiki/Semra_Kaplan_K%C4%B1v%C4%B1rc%C4%B1k
Mehmet Ali Özkan, 1973, https://tr.wikipedia.org/wiki/Mehmet_Ali_%C3%96zkan
Özgür Özel, 1974, https://tr.wikipedia.org/wiki/%C3%96zg%C3%BCr_%C3%96zel
Ahmet Vehbi Bakırlıoğlu, 1974, https://tr.wikipedia.org/wiki/Ahmet_Vehbi_Bak%C4%B1rl%C4%B1o%C4%9Flu
Bekir Başevirgen, 1975, https://tr.wikipedia.org/wiki/Bekir_Ba%C5%9Fevirgen
Tamer Akkal, 1974, https://tr.wikipedia.org/wiki/Tamer_Akkal
Erkan Akçay, 1961, https://tr.wikipedia.org/wiki/Erkan_Ak%C3%A7ay
Mithat Sancar, 1963, https://tr.wikipedia.org/wiki/Mithat_Sancar
Pero Dundar, 1971, https://tr.wikipedia.org/wiki/Pero_Dundar
Ebrü Günay, 1982, https://tr.wikipedia.org/wiki/Ebru_G%C3%BCnay
Tuma Çelik, 1966, https://tr.wikipedia.org/wiki/Tuma_%C3%87elik
Şeyhmus Dinçel, 1966, https://tr.wikipedia.org/wiki/%C5%9Eeyhmus_Din%C3%A7el
Cengiz Demirkaya, 1968, https://tr.wikipedia.org/wiki/Cengiz_Demirkaya
Lütfi Elvan, 1962, https://tr.wikipedia.org/wiki/L%C3%BCtfi_Elvan
Ali Cumhur Taşkın, 1967, https://tr.wikipedia.org/wiki/Ali_Cumhur_Ta%C5%9Fk%C4%B1n
Zeynep Gül Yılmaz,, 1976, https://tr.wikipedia.org/wiki/Zeynep_G%C3%BCl_Y%C4%B1lmaz
Hacı Özkan, 1963, https://tr.wikipedia.org/wiki/Hac%C4%B1_%C3%96zkan
Cengiz Gökçel, 1965, https://tr.wikipedia.org/wiki/Cengiz_G%C3%B6k%C3%A7el
Alpay Antmen, 1969, https://tr.wikipedia.org/wiki/Alpay_Antmen
Ali Mahir Başarır, 1975, https://tr.wikipedia.org/wiki/Ali_Mahir_Ba%C5%9Far%C4%B1r
Fatma Kurtulan, 1964, https://tr.wikipedia.org/wiki/Fatma_Kurtulan
Rıdvan Turan, 1971, https://tr.wikipedia.org/wiki/R%C4%B1dvan_Turan
Behiç Çelik, 1955, https://tr.wikipedia.org/wiki/Behi%C3%A7_%C3%87elik
Zeki Hakan Sıdalı, 1969, https://tr.wikipedia.org/wiki/Zeki_Hakan_S%C4%B1dal%C4%B1
Olcay Kılavuz, 1985, https://tr.wikipedia.org/wiki/Olcay_K%C4%B1lavuz
Baki Şimşek, 1969, https://tr.wikipedia.org/wiki/Baki_%C5%9Eim%C5%9Fek
Mürsel Alban, 1970, https://tr.wikipedia.org/wiki/M%C3%BCrsel_Alban
Suat Özcan, 1960, https://tr.wikipedia.org/wiki/Suat_%C3%96zcan
Burak Erbay, 1978, https://tr.wikipedia.org/wiki/Burak_Erbay
Süleyman Girgin, 1962, https://tr.wikipedia.org/wiki/S%C3%BCleyman_Girgin
Mehmet Yavuz Demir, 1953, https://tr.wikipedia.org/wiki/Mehmet_Yavuz_Demir
Yelda Erol Gökcan, 1976, https://tr.wikipedia.org/wiki/Yelda_Erol_G%C3%B6kcan
Metin Ergun, 1963, https://tr.wikipedia.org/wiki/Metin_Ergun
Gülüstan Kılıç Koçyiğit, 1979, https://tr.wikipedia.org/wiki/G%C3%BCl%C3%BCstan_K%C4%B1l%C4%B1%C3%A7_Ko%C3%A7yi%C4%9Fit
Mensur Işık, 1975, https://tr.wikipedia.org/wiki/Mensur_I%C5%9F%C4%B1k
Sevin Coşkun, 1980, https://tr.wikipedia.org/wiki/%C5%9Eevin_Co%C5%9Fkun
Mehmet Emin Şimşek, 1964, https://tr.wikipedia.org/wiki/Mehmet_Emin_%C5%9Eim%C5%9Fek
Mustafa Açıkgöz, 1971, https://tr.wikipedia.org/wiki/Mustafa_A%C3%A7%C4%B1kg%C3%B6z
Yücel Menekşe, 1963, https://tr.wikipedia.org/wiki/Y%C3%BCcel_Menek%C5%9Fe
Faruk Sarıaslan, 1953, https://tr.wikipedia.org/wiki/Faruk_Sar%C4%B1aslan
Yavuz Ergun, 1974, https://tr.wikipedia.org/wiki/Yavuz_Ergun
Selim Gültekin, 1979, https://tr.wikipedia.org/wiki/Selim_G%C3%BCltekin
Ömer Fethi Gürer, 1957, https://tr.wikipedia.org/wiki/%C3%96mer_Fethi_G%C3%BCrer
Şenel Yediyıldız, 1953, https://tr.wikipedia.org/wiki/%C5%9Eenel_Yediy%C4%B1ld%C4%B1z
Ergün Taşçı, 1974, https://tr.wikipedia.org/wiki/Erg%C3%BCn_Ta%C5%9Fc%C4%B1
Metin Gündoğdu, 1973, https://tr.wikipedia.org/wiki/Metin_G%C3%BCndo%C4%9Fdu
Seyit Torun, 1968, https://tr.wikipedia.org/wiki/Seyit_Torun
Mustafa Adıgüzel,, 1967, https://tr.wikipedia.org/wiki/Mustafa_Ad%C4%B1g%C3%BCzel
Cemal Enginyurt, 1965, https://tr.wikipedia.org/wiki/Cemal_Enginyurt
Mücahit Durmuşoğlu, 1965, https://tr.wikipedia.org/wiki/M%C3%BCcahit_Durmu%C5%9Fo%C4%9Flu
İsmail Kaya, 1983, https://tr.wikipedia.org/wiki/%C4%B0smail_Kaya
Devlet Bahçeli, 1948, https://tr.wikipedia.org/wiki/Devlet_Bah%C3%A7eli
Baha Ünlü, 1955, https://tr.wikipedia.org/wiki/Baha_%C3%9Cnl%C3%BC
Hayati Yazıcı, 1952, https://tr.wikipedia.org/wiki/Hayati_Yaz%C4%B1c%C4%B1
Osman Aşkın Bak, 1966, https://tr.wikipedia.org/wiki/Osman_A%C5%9Fk%C4%B1n_Bak
Muhammed Avcı, 1980, https://tr.wikipedia.org/wiki/Muhammed_Avc%C4%B1
Ali İhsan Yavuz, 1969, https://tr.wikipedia.org/wiki/Ali_%C4%B0hsan_Yavuz
Recep Uncuoğlu, 1970, https://tr.wikipedia.org/wiki/Recep_Uncuo%C4%9Flu
Kenan Sofuoğlu[1][2], 1983, https://tr.wikipedia.org/wiki/Kenan_Sofuo%C4%9Flu
Çiğdem Erdoğan Atabek, 1981, https://tr.wikipedia.org/wiki/%C3%87i%C4%9Fdem_Erdo%C4%9Fan_Atabek
Engin Özkoç, 1962, https://tr.wikipedia.org/wiki/Engin_%C3%96zko%C3%A7
Muhammed Levent Bülbül, 1980, https://tr.wikipedia.org/wiki/Muhammed_Levent_B%C3%BClb%C3%BCl
Ümit Dikbayır, 1968, https://tr.wikipedia.org/wiki/%C3%9Cmit_Dikbay%C4%B1r
Ahmet Demircan, 1954, https://tr.wikipedia.org/wiki/Ahmet_Demircan
Yusuf Ziya Yılmaz, 1951, https://tr.wikipedia.org/wiki/Yusuf_Ziya_Y%C4%B1lmaz_(Samsunlu)
Çiğdem Karaaslan, 1979, https://tr.wikipedia.org/wiki/%C3%87i%C4%9Fdem_Karaaslan
Fuat Köktaş, 1964, https://tr.wikipedia.org/wiki/Fuat_K%C3%B6kta%C5%9F
Orhan Kırcalı, 1968, https://tr.wikipedia.org/wiki/Orhan_K%C4%B1rcal%C4%B1
Kemal Zeybek, 1962, https://tr.wikipedia.org/wiki/Kemal_Zeybek
Neslihan Hancıoğlu, 1963, https://tr.wikipedia.org/wiki/Neslihan_Hanc%C4%B1o%C4%9Flu
Erhan Usta, 1966, https://tr.wikipedia.org/wiki/Erhan_Usta
Bedri Yaşar, 1963, https://tr.wikipedia.org/wiki/Bedri_Ya%C5%9Far
Meral Danış Beştaş, 1967, https://tr.wikipedia.org/wiki/Meral_Dan%C4%B1%C5%9F_Be%C5%9Fta%C5%9F
Sıdık Taş, 1970, https://tr.wikipedia.org/wiki/S%C4%B1d%C4%B1k_Ta%C5%9F
Osman Ören, 1966, https://tr.wikipedia.org/wiki/Osman_%C3%96ren
Nazım Maviş, 1971, https://tr.wikipedia.org/wiki/Naz%C4%B1m_Mavi%C5%9F
Barış Karadeniz, 1975, https://tr.wikipedia.org/wiki/Bar%C4%B1%C5%9F_Karadeniz
İsmet Yılmaz, 1961, https://tr.wikipedia.org/wiki/%C4%B0smet_Y%C4%B1lmaz
Habib Soluk, 1950, https://tr.wikipedia.org/wiki/Habip_Soluk
Semiha Ekinci,, 1972, https://tr.wikipedia.org/wiki/Semiha_Ekinci
Ahmet Özyürek, 1964, https://tr.wikipedia.org/wiki/Ahmet_%C3%96zy%C3%BCrek
Ulaş Karasu, 1980, https://tr.wikipedia.org/wiki/Ula%C5%9F_Karasu
Ahmet Eşref Fakıbaba, 1951, https://tr.wikipedia.org/wiki/Ahmet_E%C5%9Fref_Fak%C4%B1baba
Mehmet Kasım Gülpınar, 1969, https://tr.wikipedia.org/wiki/Mehmet_Kas%C4%B1m_G%C3%BClp%C4%B1nar
Halil Özcan, 1964, https://tr.wikipedia.org/wiki/Halil_%C3%96zcan
Zemzem Gülender Açanal, 1957, https://tr.wikipedia.org/wiki/Zemzem_G%C3%BClender_A%C3%A7anal
Mehmet Ali Cevheri, 1961, https://tr.wikipedia.org/wiki/Mehmet_Ali_Cevheri
İbrahim Halil Yıldız, 1976, https://tr.wikipedia.org/wiki/%C4%B0brahim_Halil_Y%C4%B1ld%C4%B1z
Halil Özşavlı, 1981, https://tr.wikipedia.org/wiki/Halil_%C3%96z%C5%9Favl%C4%B1
Ahmet Akay, 1959, https://tr.wikipedia.org/wiki/Ahmet_Akay_(%C5%9Eanl%C4%B1urfa_do%C4%9Fumlu)
Nimetullah Erdoğmuş, 1960, https://tr.wikipedia.org/wiki/Nimetullah_Erdo%C4%9Fmu%C5%9F
Ayşe Sürücü, 1983, https://tr.wikipedia.org/wiki/Ay%C5%9Fe_S%C3%BCr%C3%BCc%C3%BC
Nusrettin Maçin, 1963, https://tr.wikipedia.org/wiki/Nusrettin_Ma%C3%A7in
Ömer Öcalan, 1987, https://tr.wikipedia.org/wiki/%C3%96mer_%C3%96calan
İbrahim Özyavuz, 1963, https://tr.wikipedia.org/wiki/%C4%B0brahim_%C3%96zyavuz
Aziz Aydınlık,, 1960, https://tr.wikipedia.org/wiki/Aziz_Ayd%C4%B1nl%C4%B1k
Hasan Özgüneş, 1957, https://tr.wikipedia.org/wiki/Hasan_%C3%96zg%C3%BCne%C5%9F
Nuran İmir, 1977, https://tr.wikipedia.org/wiki/Nuran_%C4%B0mir
Hüseyin Kaçmaz, 1986, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Ka%C3%A7maz
Rizgin Birlik, 1966, https://tr.wikipedia.org/wiki/Rizgin_Birlik
İlhami Özcan Aygun, 1969, https://tr.wikipedia.org/wiki/%C4%B0lhami_%C3%96zcan_Aygun
Candan Yüceer, 1973, https://tr.wikipedia.org/wiki/Candan_Y%C3%BCceer
Mustafa Şentop, 1968, https://tr.wikipedia.org/wiki/Mustafa_%C5%9Eentop
Mustafa Yel, 1968, https://tr.wikipedia.org/wiki/Mustafa_Yel
Çiğdem Koncagül,, 1969, https://tr.wikipedia.org/wiki/%C3%87i%C4%9Fdem_Koncag%C3%BCl
Enez Kaplan, 1962, https://tr.wikipedia.org/wiki/Enez_Kaplan
Yusuf Beyazıt, 1962, https://tr.wikipedia.org/wiki/Yusuf_Beyaz%C4%B1t
Mustafa Arslan, 1967, https://tr.wikipedia.org/wiki/Mustafa_Arslan_(siyaset%C3%A7i)
Özlem Zengin, 1969, https://tr.wikipedia.org/wiki/%C3%96zlem_Zengin
Yücel Bulut, 1980, https://tr.wikipedia.org/wiki/Y%C3%BCcel_Bulut
Muhammet Balta, 1966, https://tr.wikipedia.org/wiki/Muhammet_Balta
Kadim Durmaz, 1953, https://tr.wikipedia.org/wiki/Kadim_Durmaz
Adnan Günnar,, 1966, https://tr.wikipedia.org/wiki/Adnan_G%C3%BCnnar
Bahar Ayvazoğlu, 1976, https://tr.wikipedia.org/wiki/Bahar_Ayvazo%C4%9Flu
Salih Cora, 1981, https://tr.wikipedia.org/wiki/Salih_Cora
Ahmet Kaya, 1971, https://tr.wikipedia.org/wiki/Ahmet_Kaya_(siyaset%C3%A7i)
Hüseyin Örs,, 1964, https://tr.wikipedia.org/wiki/H%C3%BCseyin_%C3%96rs
Alican Önlü,, 1967, https://tr.wikipedia.org/wiki/Alican_%C3%96nl%C3%BC
Polat Şaroğlu, 1969, https://tr.wikipedia.org/wiki/Polat_%C5%9Earo%C4%9Flu
Mehmet Altay, 1972, https://tr.wikipedia.org/wiki/Mehmet_Altay
İsmail Güneş, 1967, https://tr.wikipedia.org/wiki/%C4%B0smail_G%C3%BCne%C5%9F_(siyaset%C3%A7i)
Özkan Yalım, 1969, https://tr.wikipedia.org/wiki/%C3%96zkan_Yal%C4%B1m
Sezai Temelli, 1963, https://tr.wikipedia.org/wiki/Sezai_Temelli
Bedia Özgökçe Ertan, 1975, https://tr.wikipedia.org/wiki/Bedia_%C3%96zg%C3%B6k%C3%A7e_Ertan
Murat Sarısaç, 1983, https://tr.wikipedia.org/wiki/Murat_Sar%C4%B1sa%C3%A7
Muazzez Orhan, 1972, https://tr.wikipedia.org/wiki/Muazzez_Orhan
Tayip Temel, 1982, https://tr.wikipedia.org/wiki/Tayip_Temel
Osman Nuri Gülaçar, 1965, https://tr.wikipedia.org/wiki/Osman_Nuri_G%C3%BCla%C3%A7ar
İrfan Kartal, 1949, https://tr.wikipedia.org/wiki/%C4%B0rfan_Kartal
Abdulahat Arvas, 1969, https://tr.wikipedia.org/wiki/Abdulahat_Arvas
Ahmet Büyükgümüş, 1990, https://tr.wikipedia.org/wiki/Ahmet_B%C3%BCy%C3%BCkg%C3%BCm%C3%BC%C5%9F
Meliha Akyol, 1960, https://tr.wikipedia.org/wiki/Meliha_Akyol
Özcan Özel, 1959, https://tr.wikipedia.org/wiki/%C3%96zcan_%C3%96zel
İbrahim Ethem Sedef,1980, https://tr.wikipedia.org/wiki/%C4%B0brahim_Ethem_Sedef
Ali Keven, 1951, https://tr.wikipedia.org/wiki/Ali_Keven
Polat Türkmen, 1957, https://tr.wikipedia.org/wiki/Polat_T%C3%BCrkmen
Ahmet Çolakoğlu, 1978, https://tr.wikipedia.org/wiki/Ahmet_%C3%87olako%C4%9Flu
Hamdi Uçar, 1963, https://tr.wikipedia.org/wiki/Hamdi_U%C3%A7ar
Ünal Demirtaş, 1971, https://tr.wikipedia.org/wiki/%C3%9Cnal_Demirta%C5%9F
Deniz Yavuzyılmaz, 1979, https://tr.wikipedia.org/wiki/Deniz_Yavuzy%C4%B1lmaz
Failed links:
https://tr.wikipedia.org/wiki/T%C3%BCrkiye_Cumhuriyeti_D%C4%B1%C5%9Fi%C5%9Fleri_Bakanl%C4%B1%C4%9F%C4%B1
https://tr.wikipedia.org/wiki/Zafer_Partisi
https://tr.wikipedia.org/wiki/Kemal_K%C4%B1l%C4%B1%C3%A7daro%C4%9Flu


SyntaxError: ignored

In [ ]:
26.dönem
Ömer Çelik, 1968, https://tr.wikipedia.org/wiki/%C3%96mer_%C3%87elik
Necdet Ünüvar,, 1960, https://tr.wikipedia.org/wiki/Necdet_%C3%9Cn%C3%BCvar
Fatma Güldemet Sarı, 1970, https://tr.wikipedia.org/wiki/Fatma_G%C3%BCldemet_Sar%C4%B1
Talip Küçükcan, 1963, https://tr.wikipedia.org/wiki/Talip_K%C3%BC%C3%A7%C3%BCkcan
Mehmet Şükrü Erdinç,, 1976, https://tr.wikipedia.org/wiki/Mehmet_%C5%9E%C3%BCkr%C3%BC_Erdin%C3%A7
Tamer Dağlı, 1970, https://tr.wikipedia.org/wiki/Tamer_Da%C4%9Fl%C4%B1
Elif Doğan Türkmen, 1962, https://tr.wikipedia.org/wiki/Elif_Do%C4%9Fan_T%C3%BCrkmen
Zülfikar İnönü Tümer, 1966, https://tr.wikipedia.org/wiki/Z%C3%BClfikar_%C4%B0n%C3%B6n%C3%BC_T%C3%BCmer
İbrahim Özdiş, 1956, https://tr.wikipedia.org/wiki/%C4%B0brahim_%C3%96zdi%C5%9F
Aydın Uslupehlivan, 1954, https://tr.wikipedia.org/wiki/Ayd%C4%B1n_Uslupehlivan
Mevlüt Karakaya, 1963, https://tr.wikipedia.org/wiki/Mevl%C3%BCt_Karakaya
Muharrem Varlı, 1969, https://tr.wikipedia.org/wiki/Muharrem_Varl%C4%B1
Seyfettin Yılmaz, 1968, https://tr.wikipedia.org/wiki/Seyfettin_Y%C4%B1lmaz
Meral Danış Beştaş, 1967, https://tr.wikipedia.org/wiki/Meral_Dan%C4%B1%C5%9F_Be%C5%9Fta%C5%9F
Ahmet Aydın,, 1971, https://tr.wikipedia.org/wiki/Ahmet_Ayd%C4%B1n_(Ad%C4%B1yamanl%C4%B1)
Adnan Boynukara, 1964, https://tr.wikipedia.org/wiki/Adnan_Boynukara
İbrahim Halil Fırat, 1973, https://tr.wikipedia.org/wiki/%C4%B0brahim_Halil_F%C4%B1rat
Salih Fırat, 1960, https://tr.wikipedia.org/wiki/Salih_F%C4%B1rat
Behçet Yıldırım, 1959, https://tr.wikipedia.org/wiki/Beh%C3%A7et_Y%C4%B1ld%C4%B1r%C4%B1m
Veysel Eroğlu, 1948, https://tr.wikipedia.org/wiki/Veysel_Ero%C4%9Flu
Ali Özkaya, 1970, https://tr.wikipedia.org/wiki/Ali_%C3%96zkaya
Hatice Dudu Özkal, 1966, https://tr.wikipedia.org/wiki/Hatice_Dudu_%C3%96zkal
Mehmet Parsak, 1979, https://tr.wikipedia.org/wiki/Mehmet_Parsak
Burcu Köksal, 1980, https://tr.wikipedia.org/wiki/Burcu_K%C3%B6ksal
Leyla Zana, 1961, https://tr.wikipedia.org/wiki/Leyla_Zana
Berdan Öztürk, 1980, https://tr.wikipedia.org/wiki/Berdan_%C3%96zt%C3%BCrk
Dirayet Taşdemir, 1982, https://tr.wikipedia.org/wiki/Dirayet_Ta%C5%9Fdemir
Cesim Gökçe, 1968, https://tr.wikipedia.org/wiki/Cesim_G%C3%B6k%C3%A7e
İlknur İnceöz, 1973, https://tr.wikipedia.org/wiki/%C4%B0lknur_%C4%B0nce%C3%B6z
Cengiz Aydoğdu,, 1958, https://tr.wikipedia.org/wiki/Cengiz_Aydo%C4%9Fdu
Mustafa Serdengeçti, 1968, https://tr.wikipedia.org/wiki/Mustafa_Serdenge%C3%A7ti
Mehmet Naci Bostancı, 1957, https://tr.wikipedia.org/wiki/Naci_Bostanc%C4%B1
Haluk İpek, 1963, https://tr.wikipedia.org/wiki/Haluk_%C4%B0pek
Mustafa Tuncer, 1971, https://tr.wikipedia.org/wiki/Mustafa_Tuncer
Ali Babacan, 1967, https://tr.wikipedia.org/wiki/Ali_Babacan
Yalçın Akdoğan, 1969, https://tr.wikipedia.org/wiki/Yal%C3%A7%C4%B1n_Akdo%C4%9Fan
Ahmet Gündoğdu, 1961, https://tr.wikipedia.org/wiki/Ahmet_G%C3%BCndo%C4%9Fdu
Ali İhsan Arslan,, 1969, https://tr.wikipedia.org/wiki/Ali_%C4%B0hsan_Arslan
Jülide Sarıeroğlu, 1979, https://tr.wikipedia.org/wiki/J%C3%BClide_Sar%C4%B1ero%C4%9Flu
Murat Alparslan, 1974, https://tr.wikipedia.org/wiki/Murat_Alparslan
Ertan Aydın, 1971, https://tr.wikipedia.org/wiki/Ertan_Ayd%C4%B1n
Ayşe Gülsün Bilgehan, 1957, https://tr.wikipedia.org/wiki/Ay%C5%9Fe_G%C3%BCls%C3%BCn_Bilgehan
Tekin Bingöl, 1955, https://tr.wikipedia.org/wiki/Tekin_Bing%C3%B6l
Aylin Nazlıaka, 1968, https://tr.wikipedia.org/wiki/Aylin_Nazl%C4%B1aka
Levent Gök, 1959, https://tr.wikipedia.org/wiki/Levent_G%C3%B6k
Ali Haydar Hakverdi, 1979, https://tr.wikipedia.org/wiki/Ali_Haydar_Hakverdi
Bülent Kuşoğlu,, 1958, https://tr.wikipedia.org/wiki/B%C3%BClent_Ku%C5%9Fo%C4%9Flu
Zühal Topcu, 1960, https://tr.wikipedia.org/wiki/Z%C3%BChal_Topcu
Erkan Haberal[1][2], 1972, https://tr.wikipedia.org/wiki/Erkan_Haberal
Sırrı Süreyya Önder, 1962, https://tr.wikipedia.org/wiki/S%C4%B1rr%C4%B1_S%C3%BCreyya_%C3%96nder
Cemil Çiçek, 1946, https://tr.wikipedia.org/wiki/Cemil_%C3%87i%C3%A7ek
Yıldırım Tuğrul Türkeş[1][2], 1954, https://tr.wikipedia.org/wiki/Y%C4%B1ld%C4%B1r%C4%B1m_Tu%C4%9Frul_T%C3%BCrke%C5%9F
Emrullah İşler, 1960, https://tr.wikipedia.org/wiki/Emrullah_%C4%B0%C5%9Fler
Ahmet İyimaya, 1950, https://tr.wikipedia.org/wiki/Ahmet_%C4%B0yimaya
Lütfiye Selva Çam, 1970, https://tr.wikipedia.org/wiki/L%C3%BCtfiye_Selva_%C3%87am
Vedat Bilgin, 1954, https://tr.wikipedia.org/wiki/Vedat_Bilgin
Aydın Ünal, 1970, https://tr.wikipedia.org/wiki/Ayd%C4%B1n_%C3%9Cnal
Nevzat Ceylan, 1953, https://tr.wikipedia.org/wiki/Nevzat_Ceylan
Şenal Sarıhan, 1948, https://tr.wikipedia.org/wiki/%C5%9Eenal_Sar%C4%B1han
Murat Emir, 1969, https://tr.wikipedia.org/wiki/Murat_Emir
Ahmet Haluk Koç, 1954, https://tr.wikipedia.org/wiki/Ahmet_Haluk_Ko%C3%A7
Nihat Yeşil, 1957, https://tr.wikipedia.org/wiki/Nihat_Ye%C5%9Fil
Şefkat Çetin, 1954, https://tr.wikipedia.org/wiki/%C5%9Eefkat_%C3%87etin
Mustafa Mit, 1950, https://tr.wikipedia.org/wiki/Mustafa_Mit
Mevlüt Çavuşoğlu, 1968, https://tr.wikipedia.org/wiki/Mevl%C3%BCt_%C3%87avu%C5%9Fo%C4%9Flu
Sena Nur Çelik, 1986, https://tr.wikipedia.org/wiki/Sena_Nur_%C3%87elik
Mustafa Köse, 1977, https://tr.wikipedia.org/wiki/Mustafa_K%C3%B6se
İbrahim Aydın, 1962, https://tr.wikipedia.org/wiki/%C4%B0brahim_Ayd%C4%B1n_(1962_do%C4%9Fumlu_siyaset%C3%A7i)
Hüseyin Samani, 1964, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Samani
Gökcen Özdoğan Enç, 1976, https://tr.wikipedia.org/wiki/G%C3%B6kcen_%C3%96zdo%C4%9Fan_En%C3%A7
Atay Uslu, 1970, https://tr.wikipedia.org/wiki/Atay_Uslu
Çetin Osman Budak, 1960, https://tr.wikipedia.org/wiki/%C3%87etin_Osman_Budak
Devrim Kök, 1977, https://tr.wikipedia.org/wiki/Devrim_K%C3%B6k
Deniz Baykal, 1938, https://tr.wikipedia.org/wiki/Deniz_Baykal
Mustafa Akaydın, 1952, https://tr.wikipedia.org/wiki/Mustafa_Akayd%C4%B1n
Niyazi Nefi Kara, 1969, https://tr.wikipedia.org/wiki/Niyazi_Nefi_Kara
Ahmet Selim Yurdakul, 1966, https://tr.wikipedia.org/wiki/Ahmet_Selim_Yurdakul
Mehmet Günal, 1964, https://tr.wikipedia.org/wiki/Mehmet_G%C3%BCnal
Orhan Atalay, 1965, https://tr.wikipedia.org/wiki/Orhan_Atalay
Bağımsız, 2018, https://tr.wikipedia.org/wiki/%C3%96zt%C3%BCrk_Y%C4%B1lmaz
İsrafil Kışla, 1959, https://tr.wikipedia.org/wiki/%C4%B0srafil_K%C4%B1%C5%9Fla
Uğur Bayraktutan, 1965, https://tr.wikipedia.org/wiki/U%C4%9Fur_Bayraktutan
Bülent Tezcan, 1965, https://tr.wikipedia.org/wiki/B%C3%BClent_Tezcan
Hüseyin Yıldız, 1967, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Y%C4%B1ld%C4%B1z_(siyaset%C3%A7i,_1967)
Metin Lütfi Baydar,, 1960, https://tr.wikipedia.org/wiki/Metin_L%C3%BCtfi_Baydar
Abdurrahman Öz, 1971, https://tr.wikipedia.org/wiki/Abdurrahman_%C3%96z
Mehmet Erdem, 1966, https://tr.wikipedia.org/wiki/Mehmet_Erdem_(1966_do%C4%9Fumlu_siyaset%C3%A7i)
Mustafa Savaş, 1965, https://tr.wikipedia.org/wiki/Mustafa_Sava%C5%9F
Deniz Depboylu, 1968, https://tr.wikipedia.org/wiki/Deniz_Depboylu
Ali Aydınlıoğlu, 1958, https://tr.wikipedia.org/wiki/Ali_Ayd%C4%B1nl%C4%B1o%C4%9Flu
Kasım Bostan, 1962, https://tr.wikipedia.org/wiki/Kas%C4%B1m_Bostan
Mahmut Poyrazlı,, 1965, https://tr.wikipedia.org/wiki/Mahmut_Poyrazl%C4%B1
Sema Kırcı, 1971, https://tr.wikipedia.org/wiki/Sema_K%C4%B1rc%C4%B1
Ahmet Akın, 1973, https://tr.wikipedia.org/wiki/Ahmet_Ak%C4%B1n
Mehmet Tüm, 1957, https://tr.wikipedia.org/wiki/Mehmet_T%C3%BCm
Namık Havutça, 1957, https://tr.wikipedia.org/wiki/Nam%C4%B1k_Havut%C3%A7a
İsmail Ok,, 1964, https://tr.wikipedia.org/wiki/%C4%B0smail_Ok
Muhammet Rıza Yalçınkaya, 1955, https://tr.wikipedia.org/wiki/Muhammet_R%C4%B1za_Yal%C3%A7%C4%B1nkaya
Ayşe Acar Başaran, 1985, https://tr.wikipedia.org/wiki/Ay%C5%9Fe_Acar_Ba%C5%9Faran
Mehmet Ali Aslan, 1975, https://tr.wikipedia.org/wiki/Mehmet_Ali_Aslan
Saadet Becerekli, 1955, https://tr.wikipedia.org/wiki/Saadet_Becerekli
Ataullah Hamidi, 1954, https://tr.wikipedia.org/wiki/Ataullah_Hamidi
Naci Ağbal, 1968, https://tr.wikipedia.org/wiki/Naci_A%C4%9Fbal
Şahap Kavcıoğlu, 1967, https://tr.wikipedia.org/wiki/%C5%9Eahap_Kavc%C4%B1o%C4%9Flu
Halil Eldemir,, 1973, https://tr.wikipedia.org/wiki/Halil_Eldemir
Yaşar Tüzün,, 1966, https://tr.wikipedia.org/wiki/Ya%C5%9Far_T%C3%BCz%C3%BCn
Cevdet Yılmaz, 1967, https://tr.wikipedia.org/wiki/Cevdet_Y%C4%B1lmaz
Enver Fehmioğlu, 1956, https://tr.wikipedia.org/wiki/Enver_Fehmio%C4%9Flu
Hişyar Özsoy,, 1977, https://tr.wikipedia.org/wiki/Hi%C5%9Fyar_%C3%96zsoy
Mahmut Celadet Gaydalı, 1951, https://tr.wikipedia.org/wiki/Mahmut_Celadet_Gaydal%C4%B1
Mizgin Irgat, 1978, https://tr.wikipedia.org/wiki/Mizgin_Irgat
Vedat Demiröz, 1956, https://tr.wikipedia.org/wiki/Vedat_Demir%C3%B6z
Ali Ercoşkun, 1971, https://tr.wikipedia.org/wiki/Ali_Erco%C5%9Fkun
Fehmi Küpçü, 1974, https://tr.wikipedia.org/wiki/Fehmi_K%C3%BCp%C3%A7%C3%BC
Tanju Özcan, 1973, https://tr.wikipedia.org/wiki/Tanju_%C3%96zcan
Reşat Petek, 1955, https://tr.wikipedia.org/wiki/Re%C5%9Fat_Petek
Bayram Özçelik,, 1966, https://tr.wikipedia.org/wiki/Bayram_%C3%96z%C3%A7elik
Efkan Âlâ, 1965, https://tr.wikipedia.org/wiki/Efkan_Ala
Mehmet Müezzinoğlu,, 1955, https://tr.wikipedia.org/wiki/Mehmet_M%C3%BCezzino%C4%9Flu
Muhammet Müfit Aydın, 1954, https://tr.wikipedia.org/wiki/Muhammet_M%C3%BCfit_Ayd%C4%B1n
Bennur Karaburun Ateş, 1973, https://tr.wikipedia.org/wiki/Bennur_Karaburun
Cemalettin Kani Torun, 1959, https://tr.wikipedia.org/wiki/Cemalettin_Kani_Torun
Osman Mesten, 1971, https://tr.wikipedia.org/wiki/Osman_Mesten
Zekeriya Birkan, 1967, https://tr.wikipedia.org/wiki/Zekeriya_Birkan
Emine Yavuz Gözgeç, 1969, https://tr.wikipedia.org/wiki/Emine_Yavuz_G%C3%B6zge%C3%A7
Hakan Çavuşoğlu, 1972, https://tr.wikipedia.org/wiki/Hakan_%C3%87avu%C5%9Fo%C4%9Flu
Failed to parse: https://tr.wikipedia.org/wiki/%C4%B0smail_Ayd%C4%B1n
Lale Karabıyık, 1965, https://tr.wikipedia.org/wiki/Lale_Karab%C4%B1y%C4%B1k
Erkan Aydın, 1974, https://tr.wikipedia.org/wiki/Erkan_Ayd%C4%B1n
Ceyhun İrgil, 1965, https://tr.wikipedia.org/wiki/Ceyhun_%C4%B0rgil
Orhan Sarıbal, 1966, https://tr.wikipedia.org/wiki/Orhan_Sar%C4%B1bal
İsmet Büyükataman, 1953, https://tr.wikipedia.org/wiki/%C4%B0smet_B%C3%BCy%C3%BCkataman
Kadir Koçdemir, 1964, https://tr.wikipedia.org/wiki/Kadir_Ko%C3%A7demir
Ayhan Gider, 1969, https://tr.wikipedia.org/wiki/Ayhan_Gider
Bülent Turan, 1975, https://tr.wikipedia.org/wiki/B%C3%BClent_Turan
Muharrem Erkek,, 1969, https://tr.wikipedia.org/wiki/Muharrem_Erkek
Bülent Öz, 1972, https://tr.wikipedia.org/wiki/B%C3%BClent_%C3%96z
Hüseyin Filiz, 1958, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Filiz
Muhammet Emin Akbaşoğlu,, 1968, https://tr.wikipedia.org/wiki/Muhammet_Emin_Akba%C5%9Fo%C4%9Flu
Salim Uslu, 1955, https://tr.wikipedia.org/wiki/Salim_Uslu
Lütfiye İlksen Ceritoğlu Kurt, 1970, https://tr.wikipedia.org/wiki/L%C3%BCtfiye_%C4%B0lksen_Cerito%C4%9Flu_Kurt
Ahmet Sami Ceylan, 1971, https://tr.wikipedia.org/wiki/Ahmet_Sami_Ceylan
Tufan Köse, 1967, https://tr.wikipedia.org/wiki/Tufan_K%C3%B6se
Nihat Zeybekci, 1961, https://tr.wikipedia.org/wiki/Nihat_Zeybekci
Cahit Özkan, 1976, https://tr.wikipedia.org/wiki/Cahit_%C3%96zkan
Sema Ramazanoğlu, 1959, https://tr.wikipedia.org/wiki/Sema_Ramazano%C4%9Flu
Şahin Tin, 1966, https://tr.wikipedia.org/wiki/%C5%9Eahin_Tin
Kazım Arslan, 1954, https://tr.wikipedia.org/wiki/Kaz%C4%B1m_Arslan_(siyaset%C3%A7i,_1954)
Melike Basmacı, 1978, https://tr.wikipedia.org/wiki/Melike_Basmac%C4%B1
Emin Haluk Ayhan, 1957, https://tr.wikipedia.org/wiki/Emin_Haluk_Ayhan
Altan Tan, 1958, https://tr.wikipedia.org/wiki/Altan_Tan
İdris Baluken, 1976, https://tr.wikipedia.org/wiki/%C4%B0dris_Baluken
İmam Taşçıer, 1959, https://tr.wikipedia.org/wiki/%C4%B0mam_Ta%C5%9F%C3%A7%C4%B1er
Nursel Aydoğan, 1958, https://tr.wikipedia.org/wiki/Nursel_Aydo%C4%9Fan
Feleknas Uca, 1976, https://tr.wikipedia.org/wiki/Feleknas_Uca
Nimetullah Erdoğmuş, 1960, https://tr.wikipedia.org/wiki/Nimetullah_Erdo%C4%9Fmu%C5%9F
Çağlar Demirel, 1969, https://tr.wikipedia.org/wiki/%C3%87a%C4%9Flar_Demirel
Sibel Yiğitalp,, 1971, https://tr.wikipedia.org/wiki/Sibel_Yi%C4%9Fitalp
Ziya Pir, 1970, https://tr.wikipedia.org/wiki/Ziya_Pir
Mehmet Galip Ensarioğlu, 1966, https://tr.wikipedia.org/wiki/Mehmet_Galip_Ensario%C4%9Flu
Ebubekir Bal, 1964, https://tr.wikipedia.org/wiki/Ebubekir_Bal
Fevai Arslan, 1966, https://tr.wikipedia.org/wiki/Fevai_Arslan
Faruk Özlü[1], 1962, https://tr.wikipedia.org/wiki/Faruk_%C3%96zl%C3%BC
Ayşe Keşir, 1967, https://tr.wikipedia.org/wiki/Ay%C5%9Fe_Ke%C5%9Fir
Okan Gaytancıoğlu, 1967, https://tr.wikipedia.org/wiki/Okan_Gaytanc%C4%B1o%C4%9Flu
Erdin Bircan,, 1959, https://tr.wikipedia.org/wiki/Erdin_Bircan
Rafet Sezen, 1956, https://tr.wikipedia.org/wiki/Rafet_Sezen
Ejder Açıkkapı, 1968, https://tr.wikipedia.org/wiki/Ejder_A%C3%A7%C4%B1kkap%C4%B1
Metin Bulut, 1971, https://tr.wikipedia.org/wiki/Metin_Bulut
Ömer Serdar, 1963, https://tr.wikipedia.org/wiki/%C3%96mer_Serdar
Tahir Öztürk, 1950, https://tr.wikipedia.org/wiki/Tahir_%C3%96zt%C3%BCrk
Sebahattin Karakelle,, 1950, https://tr.wikipedia.org/wiki/Sebahattin_Karakelle
Serkan Bayram, 1974, https://tr.wikipedia.org/wiki/Serkan_Bayram
Recep Akdağ, 1960, https://tr.wikipedia.org/wiki/Recep_Akda%C4%9F
Orhan Deligöz, 1965, https://tr.wikipedia.org/wiki/Orhan_Delig%C3%B6z
Zehra Taşkesenlioğlu Ban, 1972, https://tr.wikipedia.org/wiki/Zehra_Ta%C5%9Fkesenlio%C4%9Flu
Mustafa Ilıcalı, 1954, https://tr.wikipedia.org/wiki/Mustafa_Il%C4%B1cal%C4%B1
Kamil Aydın, 1963, https://tr.wikipedia.org/wiki/Kamil_Ayd%C4%B1n_(siyaset%C3%A7i)
Nabi Avcı,, 1953, https://tr.wikipedia.org/wiki/Nabi_Avc%C4%B1
Emine Nur Günay, 1962, https://tr.wikipedia.org/wiki/Emine_Nur_G%C3%BCnay
Utku Çakırözer, 1970, https://tr.wikipedia.org/wiki/Utku_%C3%87ak%C4%B1r%C3%B6zer
Kasım 2015 – Eskişehir, 2015, https://tr.wikipedia.org/wiki/Gaye_Usluer
Cemal Okan Yüksel, 1973, https://tr.wikipedia.org/wiki/Cemal_Okan_Y%C3%BCksel
Mehmet Şimşek,, 1967, https://tr.wikipedia.org/wiki/Mehmet_%C5%9Eim%C5%9Fek
Abdulhamit Gül, 1977, https://tr.wikipedia.org/wiki/Abdulhamit_G%C3%BCl
Abdullah Nejat Koçer, 1963, https://tr.wikipedia.org/wiki/Abdullah_Nejat_Ko%C3%A7er
Ahmet Uzer, 1956, https://tr.wikipedia.org/wiki/Ahmet_Uzer
Canan Candemir Çelik, 1974, https://tr.wikipedia.org/wiki/Canan_Candemir_%C3%87elik
Mehmet Erdoğan, 1956, https://tr.wikipedia.org/wiki/Mehmet_Erdo%C4%9Fan_(Gaziantepli)
Abdulkadir Yüksel, 1962, https://tr.wikipedia.org/wiki/Abdulkadir_Y%C3%BCksel
Şamil Tayyar, 1965, https://tr.wikipedia.org/wiki/%C5%9Eamil_Tayyar
Akif Ekici, 1955, https://tr.wikipedia.org/wiki/Akif_Ekici
Mehmet Gökdağ, 1961, https://tr.wikipedia.org/wiki/Mehmet_G%C3%B6kda%C4%9F
Ümit Özdağ, 1961, https://tr.wikipedia.org/wiki/%C3%9Cmit_%C3%96zda%C4%9F
Mahmut Toğrul, 1967, https://tr.wikipedia.org/wiki/Mahmut_To%C4%9Frul
Nurettin Canikli, 1960, https://tr.wikipedia.org/wiki/Nurettin_Canikli
Sabri Öztürk, 1973, https://tr.wikipedia.org/wiki/Sabri_%C3%96zt%C3%BCrk_(1973_do%C4%9Fumlu_siyaset%C3%A7i)
Cemal Öztürk, 1958, https://tr.wikipedia.org/wiki/Cemal_%C3%96zt%C3%BCrk
Bülent Yener Bektaşoğlu, 1953, https://tr.wikipedia.org/wiki/B%C3%BClent_Yener_Bekta%C5%9Fo%C4%9Flu
Cihan Pektaş, 1970, https://tr.wikipedia.org/wiki/Cihan_Pekta%C5%9F
Hacı Osman Akgül, 1960, https://tr.wikipedia.org/wiki/Hac%C4%B1_Osman_Akg%C3%BCl
Abdullah Zeydan, 1972, https://tr.wikipedia.org/wiki/Abdullah_Zeydan
Nihat Akdoğan, 1980, https://tr.wikipedia.org/wiki/Nihat_Akdo%C4%9Fan
Adem Yeşildal, 1976, https://tr.wikipedia.org/wiki/Adem_Ye%C5%9Fildal
Fevzi Şanverdi, 1963, https://tr.wikipedia.org/wiki/Fevzi_%C5%9Eanverdi
Adalet ve Kalkınma Partisi, 2011, https://tr.wikipedia.org/wiki/Hac%C4%B1_Bayram_T%C3%BCrko%C4%9Flu
Mehmet Öntürk, 1970, https://tr.wikipedia.org/wiki/Mehmet_%C3%96nt%C3%BCrk
Orhan Karasayar,, 1965, https://tr.wikipedia.org/wiki/Orhan_Karasayar
Hilmi Yarayıcı, 1968, https://tr.wikipedia.org/wiki/Hilmi_Yaray%C4%B1c%C4%B1
Mevlüt Dudu, 1966, https://tr.wikipedia.org/wiki/Mevl%C3%BCt_Dudu
Serkan Topal, 1976, https://tr.wikipedia.org/wiki/Serkan_Topal
Birol Ertem, 1968, https://tr.wikipedia.org/wiki/Birol_Ertem
Mehmet Necmettin Ahrazoğlu,, 1955, https://tr.wikipedia.org/wiki/Mehmet_Necmettin_Ahrazo%C4%9Flu
Mehmet Emin Adıyaman, 1961, https://tr.wikipedia.org/wiki/Mehmet_Emin_Ad%C4%B1yaman
Nurettin Aras, 1954, https://tr.wikipedia.org/wiki/Nurettin_Aras
Süreyya Sadi Bilgiç, 1961, https://tr.wikipedia.org/wiki/S%C3%BCreyya_Sadi_Bilgi%C3%A7
Said Yüce, 1963, https://tr.wikipedia.org/wiki/Said_Y%C3%BCce
İrfan Bakır, 1954, https://tr.wikipedia.org/wiki/%C4%B0rfan_Bak%C4%B1r
Nuri Okutan, 1962, https://tr.wikipedia.org/wiki/Nuri_Okutan
İsmail Kahraman, 1940, https://tr.wikipedia.org/wiki/%C4%B0smail_Kahraman_(siyaset%C3%A7i)
Mihrimah Belma Satır, 1961, https://tr.wikipedia.org/wiki/Mihrimah_Belma_Sat%C4%B1r
Erol Kaya,, 1959, https://tr.wikipedia.org/wiki/Erol_Kaya
Ahmet Berat Çonkar,, 1976, https://tr.wikipedia.org/wiki/Ahmet_Berat_%C3%87onkar
Berat Albayrak,, 1978, https://tr.wikipedia.org/wiki/Berat_Albayrak
İsmet Uçma, 1955, https://tr.wikipedia.org/wiki/%C4%B0smet_U%C3%A7ma
Metin Külünk, 1960, https://tr.wikipedia.org/wiki/Metin_K%C3%BCl%C3%BCnk
Ravza Kavakcı Kan, 1972, https://tr.wikipedia.org/wiki/Ravza_Kavak%C3%A7%C4%B1_Kan
Azmi Ekinci, 1968, https://tr.wikipedia.org/wiki/Azmi_Ekinci
Osman Boyraz, 1974, https://tr.wikipedia.org/wiki/Osman_Boyraz
Hasan Turan, 1967, https://tr.wikipedia.org/wiki/Hasan_Turan_(siyaset%C3%A7i,_1967)
Hasan Sert, 1961, https://tr.wikipedia.org/wiki/Hasan_Sert
Mehmet Ali Pulcu, 1960, https://tr.wikipedia.org/wiki/Mehmet_Ali_Pulcu
Hurşit Yıldırım, 1974, https://tr.wikipedia.org/wiki/Hur%C5%9Fit_Y%C4%B1ld%C4%B1r%C4%B1m
Hulusi Şentürk, 1963, https://tr.wikipedia.org/wiki/Hulusi_%C5%9Eent%C3%BCrk
Şafak Pavey, 1976, https://tr.wikipedia.org/wiki/%C5%9Eafak_Pavey
Gamze Akkuş İlgezdi, 1969, https://tr.wikipedia.org/wiki/Gamze_Akku%C5%9F_%C4%B0lgezdi
Gürsel Tekin, 1964, https://tr.wikipedia.org/wiki/G%C3%BCrsel_Tekin
Barış Yarkadaş, 1974, https://tr.wikipedia.org/wiki/Bar%C4%B1%C5%9F_Yarkada%C5%9F
Mehmet Akif Hamzaçebi, 1953, https://tr.wikipedia.org/wiki/Mehmet_Akif_Hamza%C3%A7ebi
Mahmut Tanal, 1961, https://tr.wikipedia.org/wiki/Mahmut_Tanal
İlhan Kesici, 1948, https://tr.wikipedia.org/wiki/%C4%B0lhan_Kesici
Oğuz Kaan Salıcı,, 1972, https://tr.wikipedia.org/wiki/O%C4%9Fuz_Kaan_Sal%C4%B1c%C4%B1
Onursal Adıgüzel, 1985, https://tr.wikipedia.org/wiki/Onursal_Ad%C4%B1g%C3%BCzel
Ali Özcan, 1945, https://tr.wikipedia.org/wiki/Ali_%C3%96zcan
Yakup Akkaya, 1957, https://tr.wikipedia.org/wiki/Yakup_Akkaya
Selahattin Demirtaş, 1973, https://tr.wikipedia.org/wiki/Selahattin_Demirta%C5%9F
Hüda Kaya, 1960, https://tr.wikipedia.org/wiki/H%C3%BCda_Kaya
Edip Semih Yalçın, 1958, https://tr.wikipedia.org/wiki/Edip_Semih_Yal%C3%A7%C4%B1n
İzzet Ulvi Yönter, 1976, https://tr.wikipedia.org/wiki/%C4%B0zzet_Ulvi_Y%C3%B6nter
Hayati Yazıcı, 1952, https://tr.wikipedia.org/wiki/Hayati_Yaz%C4%B1c%C4%B1
Volkan Bozkır, 1950, https://tr.wikipedia.org/wiki/Volkan_Bozk%C4%B1r
Burhan Kuzu, 1955, https://tr.wikipedia.org/wiki/Burhan_Kuzu
Ayşe Nur Bahçekapılı, 1954, https://tr.wikipedia.org/wiki/Ay%C5%9Fe_Nur_Bah%C3%A7ekap%C4%B1l%C4%B1
Fatma Betül Sayan Kaya, 1981, https://tr.wikipedia.org/wiki/Fatma_Bet%C3%BCl_Sayan_Kaya
Ekrem Erdem, 1948, https://tr.wikipedia.org/wiki/Ekrem_Erdem
Aziz Babuşcu, 1964, https://tr.wikipedia.org/wiki/Aziz_Babu%C5%9Fcu
Durmuş Ali Sarıkaya, 1974, https://tr.wikipedia.org/wiki/Durmu%C5%9F_Ali_Sar%C4%B1kaya
HV.PLT.1975-3, 1975, https://tr.wikipedia.org/wiki/%C5%9Eirin_%C3%9Cnal
Fatma Benli Yalçın, 1982, https://tr.wikipedia.org/wiki/Fatma_Benli
Hüseyin Bürge, 1955, https://tr.wikipedia.org/wiki/H%C3%BCseyin_B%C3%BCrge
Haydar Ali Yıldız, 1971, https://tr.wikipedia.org/wiki/Haydar_Ali_Y%C4%B1ld%C4%B1z
Ahmet Hamdi Çamlı, 1965, https://tr.wikipedia.org/wiki/Ahmet_Hamdi_%C3%87aml%C4%B1
Markar Esayan, 1969, https://tr.wikipedia.org/wiki/Markar_Esayan
Selina Özuzun Doğan, 1977, https://tr.wikipedia.org/wiki/Selina_Do%C4%9Fan
Aykut Erdoğdu, 1972, https://tr.wikipedia.org/wiki/Aykut_Erdo%C4%9Fdu
Mustafa Sezgin Tanrıkulu, 1963, https://tr.wikipedia.org/wiki/Mustafa_Sezgin_Tanr%C4%B1kulu
DZ.P.1980-3, 1980, https://tr.wikipedia.org/wiki/Dursun_%C3%87i%C3%A7ek
Gülay Yedekci, 1976, https://tr.wikipedia.org/wiki/G%C3%BClay_Yedekci
Kadri Enis Berberoğlu,, 1956, https://tr.wikipedia.org/wiki/Enis_Berbero%C4%9Flu
Didem Engin, 1977, https://tr.wikipedia.org/wiki/Didem_Engin
Celal Doğan, 1943, https://tr.wikipedia.org/wiki/Celal_Do%C4%9Fan
Filiz Kerestecioğlu Demir, 1961, https://tr.wikipedia.org/wiki/Filiz_Kerestecio%C4%9Flu
Ekmeleddin Mehmet İhsanoğlu, 1943, https://tr.wikipedia.org/wiki/Ekmeleddin_%C4%B0hsano%C4%9Flu
Celal Adan,, 1951, https://tr.wikipedia.org/wiki/Celal_Adan
Mehmet Mehdi Eker, 1956, https://tr.wikipedia.org/wiki/Mehmet_Mehdi_Eker
Mustafa Şentop, 1968, https://tr.wikipedia.org/wiki/Mustafa_%C5%9Eentop
Mehmet Doğan Kubat, 1969, https://tr.wikipedia.org/wiki/Mehmet_Do%C4%9Fan_Kubat
Serap Yaşar, 1965, https://tr.wikipedia.org/wiki/Serap_Ya%C5%9Far
Feyzullah Kıyıklık, 1949, https://tr.wikipedia.org/wiki/Feyzullah_K%C4%B1y%C4%B1kl%C4%B1k
Mehmet Muş, 1982, https://tr.wikipedia.org/wiki/Mehmet_Mu%C5%9F
Nureddin Nebati,, 1964, https://tr.wikipedia.org/wiki/Nureddin_Nebati
Mehmet Metiner, 1960, https://tr.wikipedia.org/wiki/Mehmet_Metiner
Harun Karaca, 1962, https://tr.wikipedia.org/wiki/Harun_Karaca
Tülay Kaynarca, 1969, https://tr.wikipedia.org/wiki/T%C3%BClay_Kaynarca
Mustafa Yeneroğlu, 1975, https://tr.wikipedia.org/wiki/Mustafa_Yenero%C4%9Flu
Erkan Kandemir, 1981, https://tr.wikipedia.org/wiki/Erkan_Kandemir
Halis Dalkılıç, 1968, https://tr.wikipedia.org/wiki/Halis_Dalk%C4%B1l%C4%B1%C3%A7
Mürteza Zengin, 1964, https://tr.wikipedia.org/wiki/M%C3%BCrteza_Zengin
Yıldız Seferinoğlu, 1969, https://tr.wikipedia.org/wiki/Y%C4%B1ld%C4%B1z_Seferino%C4%9Flu
Fazilet Partisi, 1998, https://tr.wikipedia.org/wiki/Abdullah_Ba%C5%9Fc%C4%B1
Bihlun Tamaylıgil, 1966, https://tr.wikipedia.org/wiki/Bihlun_Tamayl%C4%B1gil
İlhan Cihaner, 1968, https://tr.wikipedia.org/wiki/%C4%B0lhan_Cihaner
Engin Altay, 1963, https://tr.wikipedia.org/wiki/Engin_Altay
Ali Şeker, 1968, https://tr.wikipedia.org/wiki/Ali_%C5%9Eeker
Erdoğan Toprak, 1959, https://tr.wikipedia.org/wiki/Erdo%C4%9Fan_Toprak
Eren Erdem, 1986, https://tr.wikipedia.org/wiki/Eren_Erdem
Saadet Partisi, 2001, https://tr.wikipedia.org/wiki/Mehmet_Bekaro%C4%9Flu
Zeynel Emre, 1981, https://tr.wikipedia.org/wiki/Zeynel_Emre
Sibel Özdemir, 1979, https://tr.wikipedia.org/wiki/Sibel_%C3%96zdemir
2023 - Van, 2023, https://tr.wikipedia.org/wiki/Pervin_Buldan
Garo Paylan, 1972, https://tr.wikipedia.org/wiki/Garo_Paylan
Erdal Ataş, 1974, https://tr.wikipedia.org/wiki/Erdal_Ata%C5%9F
Atila Kaya,, 1957, https://tr.wikipedia.org/wiki/Atila_Kaya
İsmail Faruk Aksu, 1963, https://tr.wikipedia.org/wiki/%C4%B0smail_Faruk_Aksu
Arzu Erdem, 1970, https://tr.wikipedia.org/wiki/Arzu_Erdem
Selin Sayek Böke[1][2], 1972, https://tr.wikipedia.org/wiki/Selin_Sayek_B%C3%B6ke
Musa Çam, 1953, https://tr.wikipedia.org/wiki/Musa_%C3%87am
Ahmet Tuncay Özkan, 1966, https://tr.wikipedia.org/wiki/Tuncay_%C3%96zkan
Tacettin Bayır, 1960, https://tr.wikipedia.org/wiki/Tacettin_Bay%C4%B1r
Özcan Purçu, 1977, https://tr.wikipedia.org/wiki/%C3%96zcan_Pur%C3%A7u
Ali Yiğit, 1955, https://tr.wikipedia.org/wiki/Ali_Yi%C4%9Fit
2018 - İzmir, 2018, https://tr.wikipedia.org/wiki/Murat_Bakan
Binali Yıldırım, 1955, https://tr.wikipedia.org/wiki/Binali_Y%C4%B1ld%C4%B1r%C4%B1m
Mahmut Atilla Kaya,, 1973, https://tr.wikipedia.org/wiki/Mahmut_Atilla_Kaya
Necip Kalkan, 1950, https://tr.wikipedia.org/wiki/Necip_Kalkan
Oktay Vural, 1956, https://tr.wikipedia.org/wiki/Oktay_Vural
Ertuğrul Kürkcü[1], 1948, https://tr.wikipedia.org/wiki/Ertu%C4%9Frul_K%C3%BCrk%C3%A7%C3%BC
Failed to parse: https://tr.wikipedia.org/wiki/Kemal_K%C4%B1l%C4%B1%C3%A7daro%C4%9Flu
Zeynep Altıok, 1969, https://tr.wikipedia.org/wiki/Zeynep_Alt%C4%B1ok
Zekeriya Temizel, 1948, https://tr.wikipedia.org/wiki/Zekeriya_Temizel
Mustafa Ali Balbay, 1960, https://tr.wikipedia.org/wiki/Mustafa_Balbay
Aytun Çıray, 1957, https://tr.wikipedia.org/wiki/Aytun_%C3%87%C4%B1ray
Atila Sertel, 1956, https://tr.wikipedia.org/wiki/Atilla_Sertel
Kamil Okyay Sındır,, 1962, https://tr.wikipedia.org/wiki/Kamil_Okyay_S%C4%B1nd%C4%B1r
Fatma Seniha Nükhet Hotar, 1961, https://tr.wikipedia.org/wiki/N%C3%BCkhet_Hotar
İbrahim Mustafa Turhan, 1968, https://tr.wikipedia.org/wiki/%C4%B0brahim_Turhan_(ekonomist)
Hamza Dağ, 1980, https://tr.wikipedia.org/wiki/Hamza_Da%C4%9F
Kerem Ali Sürekli,, 1965, https://tr.wikipedia.org/wiki/Kerem_Ali_S%C3%BCrekli
Ahmet Kenan Tanrıkulu, 1958, https://tr.wikipedia.org/wiki/Ahmet_Kenan_Tanr%C4%B1kulu
Müslüm Doğan, 1959, https://tr.wikipedia.org/wiki/M%C3%BCsl%C3%BCm_Do%C4%9Fan
Mahir Ünal,, 1966, https://tr.wikipedia.org/wiki/Mahir_%C3%9Cnal
Celalettin Güvenç, 1959, https://tr.wikipedia.org/wiki/Celalettin_G%C3%BCven%C3%A7
İmran Kılıç, 1957, https://tr.wikipedia.org/wiki/%C4%B0mran_K%C4%B1l%C4%B1%C3%A7
Mehmet İlker Çitil, 1969, https://tr.wikipedia.org/wiki/Mehmet_%C4%B0lker_%C3%87itil
Mehmet Uğur Dilipak, 1972, https://tr.wikipedia.org/wiki/Mehmet_U%C4%9Fur_Dilipak
Nursel Reyhanlıoğlu, 1968, https://tr.wikipedia.org/wiki/Nursel_Reyhanl%C4%B1o%C4%9Flu
Veysi Kaynak, 1962, https://tr.wikipedia.org/wiki/Veysi_Kaynak
Fahrettin Oğuz Tor, 1957, https://tr.wikipedia.org/wiki/Fahrettin_O%C4%9Fuz_Tor
Mehmet Ali Şahin, 1950, https://tr.wikipedia.org/wiki/Mehmet_Ali_%C5%9Eahin
Burhanettin Uysal, 1967, https://tr.wikipedia.org/wiki/Burhanettin_Uysal
Recep Konuk, 1956, https://tr.wikipedia.org/wiki/Recep_Konuk
Dr. Recep Şeker, 1972, https://tr.wikipedia.org/wiki/Recep_%C5%9Eeker
Yusuf Selahattin Beyribey, 1955, https://tr.wikipedia.org/wiki/Yusuf_Selahattin_Beyribey
Ayhan Bilgen, 1971, https://tr.wikipedia.org/wiki/Ayhan_Bilgen
Hakkı Köylü, 1948, https://tr.wikipedia.org/wiki/Hakk%C4%B1_K%C3%B6yl%C3%BC
Metin Çelik, 1977, https://tr.wikipedia.org/wiki/Metin_%C3%87elik
Taner Yıldız, 1962, https://tr.wikipedia.org/wiki/Taner_Y%C4%B1ld%C4%B1z
Mehmet Özhaseki, 1957, https://tr.wikipedia.org/wiki/Mehmet_%C3%96zhaseki
Mustafa Elitaş, 1957, https://tr.wikipedia.org/wiki/Mustafa_Elita%C5%9F
Hülya Nergis, 1967, https://tr.wikipedia.org/wiki/H%C3%BClya_Nergis
İsmail Tamer, 1956, https://tr.wikipedia.org/wiki/%C4%B0smail_Tamer
Sami Dedeoğlu, 1953, https://tr.wikipedia.org/wiki/Sami_Dedeo%C4%9Flu
Yusuf Halaçoğlu, 1949, https://tr.wikipedia.org/wiki/Yusuf_Hala%C3%A7o%C4%9Flu
Çetin Arık, 1971, https://tr.wikipedia.org/wiki/%C3%87etin_Ar%C4%B1k
Mehmet Demir, 1963, https://tr.wikipedia.org/wiki/Mehmet_Demir
Ramazan Can, 1970, https://tr.wikipedia.org/wiki/Ramazan_Can
Mikail Arslan, 1961, https://tr.wikipedia.org/wiki/Mikail_Arslan
Salih Çetinkaya, 1972, https://tr.wikipedia.org/wiki/Salih_%C3%87etinkaya
Türabi Kayan, 1955, https://tr.wikipedia.org/wiki/T%C3%BCrabi_Kayan
Vecdi Gündoğdu, 1969, https://tr.wikipedia.org/wiki/Vecdi_G%C3%BCndo%C4%9Fdu
Selahattin Minsolmaz, 1970, https://tr.wikipedia.org/wiki/Selahattin_Minsolmaz
Hilmi Dülger, 1962, https://tr.wikipedia.org/wiki/Mustafa_Hilmi_D%C3%BClger
Reşit Polat, 1975, https://tr.wikipedia.org/wiki/Re%C5%9Fit_Polat
Fikri Işık, 1965, https://tr.wikipedia.org/wiki/Fikri_I%C5%9F%C4%B1k
Cemil Yaman, 1970, https://tr.wikipedia.org/wiki/Cemil_Yaman
İlyas Şeker, 1960, https://tr.wikipedia.org/wiki/%C4%B0lyas_%C5%9Eeker
Mehmet Akif Yılmaz, 1975, https://tr.wikipedia.org/wiki/Mehmet_Akif_Y%C4%B1lmaz
Radiye Sezer Katırcıoğlu, 1965, https://tr.wikipedia.org/wiki/Radiye_Sezer_Kat%C4%B1rc%C4%B1o%C4%9Flu
Sami Çakır, 1958, https://tr.wikipedia.org/wiki/Sami_%C3%87ak%C4%B1r
Zeki Aygün, 1953, https://tr.wikipedia.org/wiki/Zeki_Ayg%C3%BCn
Haydar Akar,, 1963, https://tr.wikipedia.org/wiki/Haydar_Akar
Fatma Kaplan Hürriyet, 1982, https://tr.wikipedia.org/wiki/Fatma_Kaplan_H%C3%BCrriyet
Tahsin Tarhan, 1964, https://tr.wikipedia.org/wiki/Tahsin_Tarhan
Millet İttifakı, 2022, https://tr.wikipedia.org/wiki/Ahmet_Davuto%C4%9Flu
Mustafa Baloğlu, 1967, https://tr.wikipedia.org/wiki/Mustafa_Balo%C4%9Flu
Halil Etyemez, 1965, https://tr.wikipedia.org/wiki/Halil_Etyemez
Abdullah Ağralı, 1969, https://tr.wikipedia.org/wiki/Abdullah_A%C4%9Fral%C4%B1
Ahmet Sorgun, 1958, https://tr.wikipedia.org/wiki/Ahmet_Sorgun
Ömer Ünal, 1970, https://tr.wikipedia.org/wiki/%C3%96mer_%C3%9Cnal
Ziya Altunyaldız, 1963, https://tr.wikipedia.org/wiki/Ziya_Altunyald%C4%B1z
Hacı Ahmet Özdemir, 1961, https://tr.wikipedia.org/wiki/Hac%C4%B1_Ahmet_%C3%96zdemir
Hüsnüye Erdoğan, 1965, https://tr.wikipedia.org/wiki/H%C3%BCsn%C3%BCye_Erdo%C4%9Fan
Leyla Şahin Usta, 1973, https://tr.wikipedia.org/wiki/Leyla_%C5%9Eahin_Usta
Mehmet Babaoğlu, 1965, https://tr.wikipedia.org/wiki/Mehmet_Babao%C4%9Flu
Muhammet Uğur Kaleli, 1958, https://tr.wikipedia.org/wiki/Muhammet_U%C4%9Fur_Kaleli
Mustafa Kalaycı, 1960, https://tr.wikipedia.org/wiki/Mustafa_Kalayc%C4%B1
Mustafa Hüsnü Bozkurt, 1950, https://tr.wikipedia.org/wiki/Mustafa_H%C3%BCsn%C3%BC_Bozkurt
Ahmet Tan, 1965, https://tr.wikipedia.org/wiki/Ahmet_Tan_(siyaset%C3%A7i,_1965)
Mustafa Şükrü Nazlı, 1966, https://tr.wikipedia.org/wiki/Mustafa_%C5%9E%C3%BCkr%C3%BC_Nazl%C4%B1
Vural Kavuncu, 1966, https://tr.wikipedia.org/wiki/Vural_Kavuncu
Taha Özhan, 1977, https://tr.wikipedia.org/wiki/Taha_%C3%96zhan
Öznur Çalık, 1965, https://tr.wikipedia.org/wiki/%C3%96znur_%C3%87al%C4%B1k
Nurettin Yaşar, 1961, https://tr.wikipedia.org/wiki/Nurettin_Ya%C5%9Far
Bülent Tüfenkci, 1966, https://tr.wikipedia.org/wiki/B%C3%BClent_T%C3%BCfenkci
Cumhuriyet Halk Partisi, 1995, https://tr.wikipedia.org/wiki/Veli_A%C4%9Fbaba
Recai Berber, 1959, https://tr.wikipedia.org/wiki/Recai_Berber
Büyük Birlik Partisi, 1993, https://tr.wikipedia.org/wiki/Sel%C3%A7uk_%C3%96zda%C4%9F
Uğur Aydemir, 1967, https://tr.wikipedia.org/wiki/U%C4%9Fur_Aydemir
İsmail Bilen, 1961, https://tr.wikipedia.org/wiki/%C4%B0smail_Bilen_(1961_do%C4%9Fumlu_siyaset%C3%A7i)
Murat Baybatur, 1978, https://tr.wikipedia.org/wiki/Murat_Baybatur
Özgür Özel, 1974, https://tr.wikipedia.org/wiki/%C3%96zg%C3%BCr_%C3%96zel
Tur Yıldız Biçer, 1968, https://tr.wikipedia.org/wiki/Tur_Y%C4%B1ld%C4%B1z_Bi%C3%A7er
Erkan Akçay, 1961, https://tr.wikipedia.org/wiki/Erkan_Ak%C3%A7ay
Mithat Sancar, 1963, https://tr.wikipedia.org/wiki/Mithat_Sancar
Erol Dora, 1964, https://tr.wikipedia.org/wiki/Erol_Dora
Gülser Yıldırım, 1963, https://tr.wikipedia.org/wiki/G%C3%BClser_Y%C4%B1ld%C4%B1r%C4%B1m
Ali Atalan, 1968, https://tr.wikipedia.org/wiki/Ali_Atalan
Orhan Miroğlu,, 1953, https://tr.wikipedia.org/wiki/Orhan_Miro%C4%9Flu
Ceyda Bölünmez Çankırı, 1978, https://tr.wikipedia.org/wiki/Ceyda_B%C3%B6l%C3%BCnmez_%C3%87ank%C4%B1r%C4%B1
Lütfi Elvan, 1962, https://tr.wikipedia.org/wiki/L%C3%BCtfi_Elvan
Prof. Dr. Yılmaz Tezcan, 1968, https://tr.wikipedia.org/wiki/Y%C4%B1lmaz_Tezcan
Hacı Özkan, 1963, https://tr.wikipedia.org/wiki/Hac%C4%B1_%C3%96zkan
Ali Cumhur Taşkın, 1967, https://tr.wikipedia.org/wiki/Ali_Cumhur_Ta%C5%9Fk%C4%B1n
Durmuş Fikri Sağlar, 1953, https://tr.wikipedia.org/wiki/Fikri_Sa%C4%9Flar
Aytuğ Atıcı, 1964, https://tr.wikipedia.org/wiki/Aytu%C4%9F_At%C4%B1c%C4%B1
Serdal Kuyucuoğlu, 1950, https://tr.wikipedia.org/wiki/Serdal_Kuyucuo%C4%9Flu
Hüseyin Çamak, 1948, https://tr.wikipedia.org/wiki/H%C3%BCseyin_%C3%87amak
Oktay Öztürk,, 1951, https://tr.wikipedia.org/wiki/Oktay_%C3%96zt%C3%BCrk
Baki Şimşek, 1969, https://tr.wikipedia.org/wiki/Baki_%C5%9Eim%C5%9Fek
Dengir Mir Mehmet Fırat, 1943, https://tr.wikipedia.org/wiki/Dengir_Mir_Mehmet_F%C4%B1rat
Nurettin Demir, 1953, https://tr.wikipedia.org/wiki/Nurettin_Demir
Ömer Süha Aldan, 1959, https://tr.wikipedia.org/wiki/%C3%96mer_S%C3%BCha_Aldan
Akın Üstündağ, 1968, https://tr.wikipedia.org/wiki/Ak%C4%B1n_%C3%9Cst%C3%BCnda%C4%9F
Nihat Öztürk,, 1973, https://tr.wikipedia.org/wiki/Nihat_%C3%96zt%C3%BCrk
Hasan Özyer, 1960, https://tr.wikipedia.org/wiki/Hasan_%C3%96zyer
Failed to parse: https://tr.wikipedia.org/wiki/Mehmet_Erdo%C4%9Fan
Kasım 2015 – Muş, 2015, https://tr.wikipedia.org/wiki/Burcu_%C3%87elik
Mehmet Emin Şimşek, 1964, https://tr.wikipedia.org/wiki/Mehmet_Emin_%C5%9Eim%C5%9Fek
Ebubekir Gizligider, 1979, https://tr.wikipedia.org/wiki/Ebubekir_Gizligider
Murat Göktürk, 1970, https://tr.wikipedia.org/wiki/Murat_G%C3%B6kt%C3%BCrk
Mustafa Açıkgöz, 1971, https://tr.wikipedia.org/wiki/Mustafa_A%C3%A7%C4%B1kg%C3%B6z
Alpaslan Kavaklıoğlu, 1962, https://tr.wikipedia.org/wiki/Alpaslan_Kavakl%C4%B1o%C4%9Flu
Erdoğan Özegen, 1965, https://tr.wikipedia.org/wiki/Erdo%C4%9Fan_%C3%96zegen
Ömer Fethi Gürer, 1957, https://tr.wikipedia.org/wiki/%C3%96mer_Fethi_G%C3%BCrer
Numan Kurtulmuş, 1959, https://tr.wikipedia.org/wiki/Numan_Kurtulmu%C5%9F
Metin Gündoğdu, 1973, https://tr.wikipedia.org/wiki/Metin_G%C3%BCndo%C4%9Fdu
Ergün Taşçı, 1974, https://tr.wikipedia.org/wiki/Erg%C3%BCn_Ta%C5%9Fc%C4%B1
Oktay Çanak, 1975, https://tr.wikipedia.org/wiki/Oktay_%C3%87anak
Seyit Torun, 1968, https://tr.wikipedia.org/wiki/Seyit_Torun
Mücahit Durmuşoğlu, 1965, https://tr.wikipedia.org/wiki/M%C3%BCcahit_Durmu%C5%9Fo%C4%9Flu
Suat Önal, 1961, https://tr.wikipedia.org/wiki/Suat_%C3%96nal
Devlet Bahçeli, 1948, https://tr.wikipedia.org/wiki/Devlet_Bah%C3%A7eli
Ruhi Ersoy, 1972, https://tr.wikipedia.org/wiki/Ruhi_Ersoy
Hasan Karal, 1973, https://tr.wikipedia.org/wiki/Hasan_Karal
Hikmet Ayar, 1957, https://tr.wikipedia.org/wiki/Hikmet_Ayar
Osman Aşkın Bak, 1966, https://tr.wikipedia.org/wiki/Osman_A%C5%9Fk%C4%B1n_Bak
Mustafa İsen, 1953, https://tr.wikipedia.org/wiki/Mustafa_%C4%B0sen
Ayhan Sefer Üstün, 1970, https://tr.wikipedia.org/wiki/Ayhan_Sefer_%C3%9Cst%C3%BCn
Ali İhsan Yavuz, 1969, https://tr.wikipedia.org/wiki/Ali_%C4%B0hsan_Yavuz
Recep Uncuoğlu, 1970, https://tr.wikipedia.org/wiki/Recep_Uncuo%C4%9Flu
Şaban Dişli, 1958, https://tr.wikipedia.org/wiki/%C5%9Eaban_Di%C5%9Fli
Engin Özkoç, 1962, https://tr.wikipedia.org/wiki/Engin_%C3%96zko%C3%A7
Zihni Açba, 1956, https://tr.wikipedia.org/wiki/Zihni_A%C3%A7ba
Akif Çağatay Kılıç, 1976, https://tr.wikipedia.org/wiki/Akif_%C3%87a%C4%9Fatay_K%C4%B1l%C4%B1%C3%A7
Ahmet Demircan, 1954, https://tr.wikipedia.org/wiki/Ahmet_Demircan
Hasan Basri Kurt, 1978, https://tr.wikipedia.org/wiki/Hasan_Basri_Kurt
Çiğdem Karaaslan, 1979, https://tr.wikipedia.org/wiki/%C3%87i%C4%9Fdem_Karaaslan
Fuat Köktaş, 1964, https://tr.wikipedia.org/wiki/Fuat_K%C3%B6kta%C5%9F
Orhan Kırcalı, 1968, https://tr.wikipedia.org/wiki/Orhan_K%C4%B1rcal%C4%B1
Hayati Tekin, 1959, https://tr.wikipedia.org/wiki/Hayati_Tekin
Kemal Zeybek, 1962, https://tr.wikipedia.org/wiki/Kemal_Zeybek
Erhan Usta, 1966, https://tr.wikipedia.org/wiki/Erhan_Usta
Besime Konca, 1970, https://tr.wikipedia.org/wiki/Besime_Konca
Siirt, 2015, https://tr.wikipedia.org/wiki/Kadri_Y%C4%B1ld%C4%B1r%C4%B1m
Yasin Aktay, 1966, https://tr.wikipedia.org/wiki/Yasin_Aktay
Nazım Maviş, 1971, https://tr.wikipedia.org/wiki/Naz%C4%B1m_Mavi%C5%9F
Barış Karadeniz, 1975, https://tr.wikipedia.org/wiki/Bar%C4%B1%C5%9F_Karadeniz
İsmet Yılmaz, 1961, https://tr.wikipedia.org/wiki/%C4%B0smet_Y%C4%B1lmaz
Habib Soluk, 1950, https://tr.wikipedia.org/wiki/Habip_Soluk
Selim Dursun,, 1957, https://tr.wikipedia.org/wiki/Selim_Dursun
Hilmi Bilgin[1], 1972, https://tr.wikipedia.org/wiki/Hilmi_Bilgin
Ali Akyıldız, 1964, https://tr.wikipedia.org/wiki/Ali_Aky%C4%B1ld%C4%B1z
Faruk Çelik, 1956, https://tr.wikipedia.org/wiki/Faruk_%C3%87elik
Halil Özcan, 1964, https://tr.wikipedia.org/wiki/Halil_%C3%96zcan
Ahmet Eşref Fakıbaba, 1951, https://tr.wikipedia.org/wiki/Ahmet_E%C5%9Fref_Fak%C4%B1baba
İbrahim Halil Yıldız, 1976, https://tr.wikipedia.org/wiki/%C4%B0brahim_Halil_Y%C4%B1ld%C4%B1z
Kemalettin Yılmaztekin, 1980, https://tr.wikipedia.org/wiki/Kemalettin_Y%C4%B1lmaztekin
Mahmut Kaçar,, 1975, https://tr.wikipedia.org/wiki/Mahmut_Ka%C3%A7ar
Mehmet Akyürek, 1950, https://tr.wikipedia.org/wiki/Mehmet_Aky%C3%BCrek
Mehmet Ali Cevheri, 1961, https://tr.wikipedia.org/wiki/Mehmet_Ali_Cevheri
Mehmet Kasım Gülpınar, 1969, https://tr.wikipedia.org/wiki/Mehmet_Kas%C4%B1m_G%C3%BClp%C4%B1nar
Osman Baydemir, 1971, https://tr.wikipedia.org/wiki/Osman_Baydemir
İbrahim Ayhan, 1968, https://tr.wikipedia.org/wiki/%C4%B0brahim_Ayhan
Dilek Öcalan, 1987, https://tr.wikipedia.org/wiki/Dilek_%C3%96calan
Faysal Sarıyıldız, 1975, https://tr.wikipedia.org/wiki/Faysal_Sar%C4%B1y%C4%B1ld%C4%B1z
Ferhat Encü,, 1985, https://tr.wikipedia.org/wiki/Ferhat_Enc%C3%BC
Leyla Birlik,, 1974, https://tr.wikipedia.org/wiki/Leyla_Birlik
Aycan İrmez, 1985, https://tr.wikipedia.org/wiki/Aycan_%C4%B0rmez
Candan Yüceer, 1973, https://tr.wikipedia.org/wiki/Candan_Y%C3%BCceer
Emre Köprülü, 1981, https://tr.wikipedia.org/wiki/Emre_K%C3%B6pr%C3%BCl%C3%BC
Mustafa Yel, 1968, https://tr.wikipedia.org/wiki/Mustafa_Yel
Kasım 2015 – Tekirdağ, 2015, https://tr.wikipedia.org/wiki/Ay%C5%9Fe_Do%C4%9Fan_(siyaset%C3%A7i,_1971)
Metin Akgün, 1958, https://tr.wikipedia.org/wiki/Metin_Akg%C3%BCn
Zeyid Aslan,, 1965, https://tr.wikipedia.org/wiki/Zeyid_Aslan
Celil Göçer, 1970, https://tr.wikipedia.org/wiki/Celil_G%C3%B6%C3%A7er
Coşkun Çakır, 1966, https://tr.wikipedia.org/wiki/Co%C5%9Fkun_%C3%87ak%C4%B1r
Yusuf Beyazıt, 1962, https://tr.wikipedia.org/wiki/Yusuf_Beyaz%C4%B1t
Kadim Durmaz, 1953, https://tr.wikipedia.org/wiki/Kadim_Durmaz
Süleyman Soylu, 1969, https://tr.wikipedia.org/wiki/S%C3%BCleyman_Soylu
Salih Cora, 1981, https://tr.wikipedia.org/wiki/Salih_Cora
Muhammet Balta, 1966, https://tr.wikipedia.org/wiki/Muhammet_Balta
Adnan Günnar,, 1966, https://tr.wikipedia.org/wiki/Adnan_G%C3%BCnnar
Ayşe Sula Köseoğlu, 1969, https://tr.wikipedia.org/wiki/Ay%C5%9Fe_Sula_K%C3%B6seo%C4%9Flu
Haluk Pekşen, 1961, https://tr.wikipedia.org/wiki/Haluk_Pek%C5%9Fen
Alican Önlü,, 1967, https://tr.wikipedia.org/wiki/Alican_%C3%96nl%C3%BC
Gürsel Erol, 1963, https://tr.wikipedia.org/wiki/G%C3%BCrsel_Erol
Alim Tunç, 1964, https://tr.wikipedia.org/wiki/Alim_Tun%C3%A7
Mehmet Altay, 1972, https://tr.wikipedia.org/wiki/Mehmet_Altay
Özkan Yalım, 1969, https://tr.wikipedia.org/wiki/%C3%96zkan_Yal%C4%B1m
Figen Yüksekdağ Şenoğlu, 1971, https://tr.wikipedia.org/wiki/Figen_Y%C3%BCksekda%C4%9F
Nadir Yıldırım, 1981, https://tr.wikipedia.org/wiki/Nadir_Y%C4%B1ld%C4%B1r%C4%B1m
Adem Geveri, 1978, https://tr.wikipedia.org/wiki/Adem_Geveri
Bedia Özgökçe Ertan, 1975, https://tr.wikipedia.org/wiki/Bedia_%C3%96zg%C3%B6k%C3%A7e_Ertan
Tuğba Hezer Öztürk,, 1989, https://tr.wikipedia.org/wiki/Tu%C4%9Fba_Hezer_%C3%96zt%C3%BCrk
Beşir Atalay, 1947, https://tr.wikipedia.org/wiki/Be%C5%9Fir_Atalay
Burhan Kayatürk,, 1970, https://tr.wikipedia.org/wiki/Burhan_Kayat%C3%BCrk
Fikri Demirel, 1956, https://tr.wikipedia.org/wiki/Fikri_Demirel
Muharrem İnce, 1964, https://tr.wikipedia.org/wiki/Muharrem_%C4%B0nce
Haziran 2015 – Yozgat 
Kasım 2015 – Yozgat, 2015, https://tr.wikipedia.org/wiki/Abdulkadir_Akg%C3%BCl
Ertuğrul Soysal, 1963, https://tr.wikipedia.org/wiki/Ertu%C4%9Frul_Soysal
Faruk Çaturoğlu, 1961, https://tr.wikipedia.org/wiki/Faruk_%C3%87aturo%C4%9Flu
Hüseyin Özbakır, 1950, https://tr.wikipedia.org/wiki/H%C3%BCseyin_%C3%96zbak%C4%B1r
Özcan Ulupınar, 1970, https://tr.wikipedia.org/wiki/%C3%96zcan_Ulup%C4%B1nar
Şerafettin Turpcu, 1955, https://tr.wikipedia.org/wiki/%C5%9Eerafettin_Turpcu
Ünal Demirtaş, 1971, https://tr.wikipedia.org/wiki/%C3%9Cnal_Demirta%C5%9F
Failed links:
https://tr.wikipedia.org/wiki/%C4%B0smail_Ayd%C4%B1n
https://tr.wikipedia.org/wiki/Kemal_K%C4%B1l%C4%B1%C3%A7daro%C4%9Flu
https://tr.wikipedia.org/wiki/Mehmet_Erdo%C4%9Fan


In [ ]:
25.Dönem Milletvekili 

Necdet Ünüvar,, 1960, https://tr.wikipedia.org/wiki/Necdet_%C3%9Cn%C3%BCvar
Fatma Güldemet Sarı, 1970, https://tr.wikipedia.org/wiki/Fatma_G%C3%BCldemet_Sar%C4%B1
Sadullah Kısacık, 1977, https://tr.wikipedia.org/wiki/Sadullah_K%C4%B1sac%C4%B1k
Talip Küçükcan, 1963, https://tr.wikipedia.org/wiki/Talip_K%C3%BC%C3%A7%C3%BCkcan
Mehmet Şükrü Erdinç,, 1976, https://tr.wikipedia.org/wiki/Mehmet_%C5%9E%C3%BCkr%C3%BC_Erdin%C3%A7
Elif Doğan Türkmen, 1962, https://tr.wikipedia.org/wiki/Elif_Do%C4%9Fan_T%C3%BCrkmen
Zülfikar İnönü Tümer, 1966, https://tr.wikipedia.org/wiki/Z%C3%BClfikar_%C4%B0n%C3%B6n%C3%BC_T%C3%BCmer
İbrahim Özdiş, 1956, https://tr.wikipedia.org/wiki/%C4%B0brahim_%C3%96zdi%C5%9F
Aydın Uslupehlivan, 1954, https://tr.wikipedia.org/wiki/Ayd%C4%B1n_Uslupehlivan
Mevlüt Karakaya, 1963, https://tr.wikipedia.org/wiki/Mevl%C3%BCt_Karakaya
Muharrem Varlı, 1969, https://tr.wikipedia.org/wiki/Muharrem_Varl%C4%B1
Seyfettin Yılmaz, 1968, https://tr.wikipedia.org/wiki/Seyfettin_Y%C4%B1lmaz
Rıdvan Turan, 1971, https://tr.wikipedia.org/wiki/R%C4%B1dvan_Turan
Meral Danış Beştaş, 1967, https://tr.wikipedia.org/wiki/Meral_Dan%C4%B1%C5%9F_Be%C5%9Fta%C5%9F
Ahmet Aydın,, 1971, https://tr.wikipedia.org/wiki/Ahmet_Ayd%C4%B1n_(Ad%C4%B1yamanl%C4%B1)
Adnan Boynukara, 1964, https://tr.wikipedia.org/wiki/Adnan_Boynukara
İbrahim Halil Fırat, 1973, https://tr.wikipedia.org/wiki/%C4%B0brahim_Halil_F%C4%B1rat
Salih Fırat, 1960, https://tr.wikipedia.org/wiki/Salih_F%C4%B1rat
Behçet Yıldırım, 1959, https://tr.wikipedia.org/wiki/Beh%C3%A7et_Y%C4%B1ld%C4%B1r%C4%B1m
Halil Ürün, 1968, https://tr.wikipedia.org/wiki/Halil_%C3%9Cr%C3%BCn_(Afyonlu)
Ali Özkaya, 1970, https://tr.wikipedia.org/wiki/Ali_%C3%96zkaya
Remziye Sıvacı, 1965, https://tr.wikipedia.org/wiki/Remziye_S%C4%B1vac%C4%B1
Mehmet Parsak, 1979, https://tr.wikipedia.org/wiki/Mehmet_Parsak
Burcu Köksal, 1980, https://tr.wikipedia.org/wiki/Burcu_K%C3%B6ksal
Leyla Zana, 1961, https://tr.wikipedia.org/wiki/Leyla_Zana
Berdan Öztürk, 1980, https://tr.wikipedia.org/wiki/Berdan_%C3%96zt%C3%BCrk
Dirayet Taşdemir, 1982, https://tr.wikipedia.org/wiki/Dirayet_Ta%C5%9Fdemir
Mehmet Emin İlhan, 1979, https://tr.wikipedia.org/wiki/Mehmet_Emin_%C4%B0lhan
İlknur İnceöz, 1973, https://tr.wikipedia.org/wiki/%C4%B0lknur_%C4%B0nce%C3%B6z
Nevzat Palta, 1962, https://tr.wikipedia.org/wiki/Nevzat_Palta
İyi Parti, 2023, https://tr.wikipedia.org/wiki/Turan_Yald%C4%B1r
Mehmet Naci Bostancı, 1957, https://tr.wikipedia.org/wiki/Naci_Bostanc%C4%B1
Said Yüce, 1963, https://tr.wikipedia.org/wiki/Said_Y%C3%BCce
Mustafa Tuncer, 1971, https://tr.wikipedia.org/wiki/Mustafa_Tuncer
Yalçın Akdoğan, 1969, https://tr.wikipedia.org/wiki/Yal%C3%A7%C4%B1n_Akdo%C4%9Fan
Ahmet Gündoğdu, 1961, https://tr.wikipedia.org/wiki/Ahmet_G%C3%BCndo%C4%9Fdu
Ali İhsan Arslan,, 1969, https://tr.wikipedia.org/wiki/Ali_%C4%B0hsan_Arslan
Jülide Sarıeroğlu, 1979, https://tr.wikipedia.org/wiki/J%C3%BClide_Sar%C4%B1ero%C4%9Flu
Murat Alparslan, 1974, https://tr.wikipedia.org/wiki/Murat_Alparslan
Ertan Aydın, 1971, https://tr.wikipedia.org/wiki/Ertan_Ayd%C4%B1n
Tülay Selamoğlu, 1966, https://tr.wikipedia.org/wiki/T%C3%BClay_Selamo%C4%9Flu
Ayşe Gülsün Bilgehan, 1957, https://tr.wikipedia.org/wiki/Ay%C5%9Fe_G%C3%BCls%C3%BCn_Bilgehan
Tekin Bingöl, 1955, https://tr.wikipedia.org/wiki/Tekin_Bing%C3%B6l
Aylin Nazlıaka, 1968, https://tr.wikipedia.org/wiki/Aylin_Nazl%C4%B1aka
Levent Gök, 1959, https://tr.wikipedia.org/wiki/Levent_G%C3%B6k
Ali Haydar Hakverdi, 1979, https://tr.wikipedia.org/wiki/Ali_Haydar_Hakverdi
Bülent Kuşoğlu,, 1958, https://tr.wikipedia.org/wiki/B%C3%BClent_Ku%C5%9Fo%C4%9Flu
Yıldırım Tuğrul Türkeş[1][2], 1954, https://tr.wikipedia.org/wiki/Y%C4%B1ld%C4%B1r%C4%B1m_Tu%C4%9Frul_T%C3%BCrke%C5%9F
Zühal Topcu, 1960, https://tr.wikipedia.org/wiki/Z%C3%BChal_Topcu
Mustafa Erdem, 1957, https://tr.wikipedia.org/wiki/Mustafa_Erdem
Sırrı Süreyya Önder, 1962, https://tr.wikipedia.org/wiki/S%C4%B1rr%C4%B1_S%C3%BCreyya_%C3%96nder
Emrullah İşler, 1960, https://tr.wikipedia.org/wiki/Emrullah_%C4%B0%C5%9Fler
Ahmet İyimaya, 1950, https://tr.wikipedia.org/wiki/Ahmet_%C4%B0yimaya
Lütfiye Selva Çam, 1970, https://tr.wikipedia.org/wiki/L%C3%BCtfiye_Selva_%C3%87am
Vedat Bilgin, 1954, https://tr.wikipedia.org/wiki/Vedat_Bilgin
Aydın Ünal, 1970, https://tr.wikipedia.org/wiki/Ayd%C4%B1n_%C3%9Cnal
Ayhan Yılmaz, 1961, https://tr.wikipedia.org/wiki/Ayhan_Y%C4%B1lmaz
Mahmut Sami Mallı, 1985, https://tr.wikipedia.org/wiki/Mahmut_Sami_Mall%C4%B1
Nevzat Ceylan, 1953, https://tr.wikipedia.org/wiki/Nevzat_Ceylan
Şenal Sarıhan, 1948, https://tr.wikipedia.org/wiki/%C5%9Eenal_Sar%C4%B1han
Murat Emir, 1969, https://tr.wikipedia.org/wiki/Murat_Emir
Ahmet Haluk Koç, 1954, https://tr.wikipedia.org/wiki/Ahmet_Haluk_Ko%C3%A7
Nihat Yeşil, 1957, https://tr.wikipedia.org/wiki/Nihat_Ye%C5%9Fil
Şefkat Çetin, 1954, https://tr.wikipedia.org/wiki/%C5%9Eefkat_%C3%87etin
Mustafa Mit, 1950, https://tr.wikipedia.org/wiki/Mustafa_Mit
Lütfi Elvan, 1962, https://tr.wikipedia.org/wiki/L%C3%BCtfi_Elvan
Mustafa Köse, 1977, https://tr.wikipedia.org/wiki/Mustafa_K%C3%B6se
Hüseyin Samani, 1964, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Samani
Gökcen Özdoğan Enç, 1976, https://tr.wikipedia.org/wiki/G%C3%B6kcen_%C3%96zdo%C4%9Fan_En%C3%A7
Sena Nur Çelik, 1986, https://tr.wikipedia.org/wiki/Sena_Nur_%C3%87elik
Niyazi Nefi Kara, 1969, https://tr.wikipedia.org/wiki/Niyazi_Nefi_Kara
Deniz Baykal, 1938, https://tr.wikipedia.org/wiki/Deniz_Baykal
Çetin Osman Budak, 1960, https://tr.wikipedia.org/wiki/%C3%87etin_Osman_Budak
Mustafa Akaydın, 1952, https://tr.wikipedia.org/wiki/Mustafa_Akayd%C4%B1n
Devrim Kök, 1977, https://tr.wikipedia.org/wiki/Devrim_K%C3%B6k
Mehmet Günal, 1964, https://tr.wikipedia.org/wiki/Mehmet_G%C3%BCnal
Ahmet Selim Yurdakul, 1966, https://tr.wikipedia.org/wiki/Ahmet_Selim_Yurdakul
Tarkan Akıllı, 1969, https://tr.wikipedia.org/wiki/Tarkan_Ak%C4%B1ll%C4%B1
Eşitlik ve Demokrasi Partisi, 2010, https://tr.wikipedia.org/wiki/Hakk%C4%B1_Saruhan_Olu%C3%A7
Taşkın Aktaş, 1965, https://tr.wikipedia.org/wiki/Ta%C5%9Fk%C4%B1n_Akta%C5%9F
Orhan Atalay, 1965, https://tr.wikipedia.org/wiki/Orhan_Atalay
İsrafil Kışla, 1959, https://tr.wikipedia.org/wiki/%C4%B0srafil_K%C4%B1%C5%9Fla
Uğur Bayraktutan, 1965, https://tr.wikipedia.org/wiki/U%C4%9Fur_Bayraktutan
Bülent Tezcan, 1965, https://tr.wikipedia.org/wiki/B%C3%BClent_Tezcan
Metin Lütfi Baydar,, 1960, https://tr.wikipedia.org/wiki/Metin_L%C3%BCtfi_Baydar
Hüseyin Yıldız, 1967, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Y%C4%B1ld%C4%B1z_(siyaset%C3%A7i,_1967)
Mehmet Fatih Atay, 1957, https://tr.wikipedia.org/wiki/Mehmet_Fatih_Atay
Mehmet Sadık Atay, 1973, https://tr.wikipedia.org/wiki/Mehmet_Sad%C4%B1k_Atay
Abdurrahman Öz, 1971, https://tr.wikipedia.org/wiki/Abdurrahman_%C3%96z
Ali Uzunırmak, 1958, https://tr.wikipedia.org/wiki/Ali_Uzun%C4%B1rmak
Mahmut Poyrazlı,, 1965, https://tr.wikipedia.org/wiki/Mahmut_Poyrazl%C4%B1
Sema Kırcı, 1971, https://tr.wikipedia.org/wiki/Sema_K%C4%B1rc%C4%B1
Ali Aydınlıoğlu, 1958, https://tr.wikipedia.org/wiki/Ali_Ayd%C4%B1nl%C4%B1o%C4%9Flu
Mehmet Tüm, 1957, https://tr.wikipedia.org/wiki/Mehmet_T%C3%BCm
Ahmet Akın, 1973, https://tr.wikipedia.org/wiki/Ahmet_Ak%C4%B1n
Namık Havutça, 1957, https://tr.wikipedia.org/wiki/Nam%C4%B1k_Havut%C3%A7a
İsmail Ok,, 1964, https://tr.wikipedia.org/wiki/%C4%B0smail_Ok
Recep Çetin, 1960, https://tr.wikipedia.org/wiki/Recep_%C3%87etin_(siyaset%C3%A7i)
Muhammet Rıza Yalçınkaya, 1955, https://tr.wikipedia.org/wiki/Muhammed_R%C4%B1za_Yal%C3%A7%C4%B1nkaya
Ali Atalan, 1968, https://tr.wikipedia.org/wiki/Ali_Atalan
Saadet Becerekli, 1955, https://tr.wikipedia.org/wiki/Saadet_Becerekli
Ayşe Acar Başaran, 1985, https://tr.wikipedia.org/wiki/Ay%C5%9Fe_Acar_Ba%C5%9Faran
Ziver Özdemir, 1965, https://tr.wikipedia.org/wiki/Ziver_%C3%96zdemir
Naci Ağbal, 1968, https://tr.wikipedia.org/wiki/Naci_A%C4%9Fbal
Karabey Kadri Karaoğlu, 1966, https://tr.wikipedia.org/wiki/Karabey_Kadri_Karao%C4%9Flu
Halil Eldemir,, 1973, https://tr.wikipedia.org/wiki/Halil_Eldemir
Yaşar Tüzün,, 1966, https://tr.wikipedia.org/wiki/Ya%C5%9Far_T%C3%BCz%C3%BCn
Şebnem Elgörmüş, 1977, https://tr.wikipedia.org/wiki/%C5%9Eebnem_Ko%C3%A7akel%C3%A7i
Enver Fehmioğlu, 1956, https://tr.wikipedia.org/wiki/Enver_Fehmio%C4%9Flu
Hişyar Özsoy,, 1977, https://tr.wikipedia.org/wiki/Hi%C5%9Fyar_%C3%96zsoy
Mizgin Irgat, 1978, https://tr.wikipedia.org/wiki/Mizgin_Irgat
Mahmut Celadet Gaydalı, 1951, https://tr.wikipedia.org/wiki/Mahmut_Celadet_Gaydal%C4%B1
Vedat Demiröz, 1956, https://tr.wikipedia.org/wiki/Vedat_Demir%C3%B6z
Ali Ercoşkun, 1971, https://tr.wikipedia.org/wiki/Ali_Erco%C5%9Fkun
Fehmi Küpçü, 1974, https://tr.wikipedia.org/wiki/Fehmi_K%C3%BCp%C3%A7%C3%BC
Tanju Özcan, 1973, https://tr.wikipedia.org/wiki/Tanju_%C3%96zcan
Reşat Petek, 1955, https://tr.wikipedia.org/wiki/Re%C5%9Fat_Petek
Alparslan Ahmet Dursun, 1973, https://tr.wikipedia.org/wiki/Alparslan_Ahmet_Dursun
Mehmet Müezzinoğlu,, 1955, https://tr.wikipedia.org/wiki/Mehmet_M%C3%BCezzino%C4%9Flu
Emine Yavuz Gözgeç, 1969, https://tr.wikipedia.org/wiki/Emine_Yavuz_G%C3%B6zge%C3%A7
Cemalettin Kani Torun, 1959, https://tr.wikipedia.org/wiki/Cemalettin_Kani_Torun
Hüseyin Şahin,, 1968, https://tr.wikipedia.org/wiki/H%C3%BCseyin_%C5%9Eahin_(siyaset%C3%A7i,_1968)
İsmet Su, 1959, https://tr.wikipedia.org/wiki/%C4%B0smet_Su
Hakan Çavuşoğlu, 1972, https://tr.wikipedia.org/wiki/Hakan_%C3%87avu%C5%9Fo%C4%9Flu
Önder Matlı, 1972, https://tr.wikipedia.org/wiki/%C3%96nder_Matl%C4%B1
Zekeriya Birkan, 1967, https://tr.wikipedia.org/wiki/Zekeriya_Birkan
Bennur Karaburun Ateş, 1973, https://tr.wikipedia.org/wiki/Bennur_Karaburun
Lale Karabıyık, 1965, https://tr.wikipedia.org/wiki/Lale_Karab%C4%B1y%C4%B1k
Ceyhun İrgil, 1965, https://tr.wikipedia.org/wiki/Ceyhun_%C4%B0rgil
Orhan Sarıbal, 1966, https://tr.wikipedia.org/wiki/Orhan_Sar%C4%B1bal
Erkan Aydın, 1974, https://tr.wikipedia.org/wiki/Erkan_Ayd%C4%B1n
İsmet Büyükataman, 1953, https://tr.wikipedia.org/wiki/%C4%B0smet_B%C3%BCy%C3%BCkataman
Kadir Koçdemir, 1964, https://tr.wikipedia.org/wiki/Kadir_Ko%C3%A7demir
Tevfik Topçu, 1958, https://tr.wikipedia.org/wiki/Tevfik_Top%C3%A7u
Asiye Kolçak, 1964, https://tr.wikipedia.org/wiki/Asiye_Kol%C3%A7ak
Muharrem Erkek,, 1969, https://tr.wikipedia.org/wiki/Muharrem_Erkek
Bülent Öz, 1972, https://tr.wikipedia.org/wiki/B%C3%BClent_%C3%96z
Bülent Turan, 1975, https://tr.wikipedia.org/wiki/B%C3%BClent_Turan
İbrahim Kürşat Tuna, 1976, https://tr.wikipedia.org/wiki/%C4%B0brahim_K%C3%BCr%C5%9Fat_Tuna
Muhammet Emin Akbaşoğlu,, 1968, https://tr.wikipedia.org/wiki/Muhammet_Emin_Akba%C5%9Fo%C4%9Flu
Hüseyin Filiz, 1958, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Filiz
Salim Uslu, 1955, https://tr.wikipedia.org/wiki/Salim_Uslu
Cahit Bağcı, 1964, https://tr.wikipedia.org/wiki/Cahit_Ba%C4%9Fc%C4%B1
Lütfiye İlksen Ceritoğlu Kurt, 1970, https://tr.wikipedia.org/wiki/L%C3%BCtfiye_%C4%B0lksen_Cerito%C4%9Flu_Kurt
Tufan Köse, 1967, https://tr.wikipedia.org/wiki/Tufan_K%C3%B6se
Nihat Zeybekci, 1961, https://tr.wikipedia.org/wiki/Nihat_Zeybekci
Bilal Uçar,, 1969, https://tr.wikipedia.org/wiki/Bilal_U%C3%A7ar
Melike Basmacı, 1978, https://tr.wikipedia.org/wiki/Melike_Basmac%C4%B1
Kazım Arslan, 1954, https://tr.wikipedia.org/wiki/Kaz%C4%B1m_Arslan_(1954_do%C4%9Fumlu_siyaset%C3%A7i)
Gülizar Biçer Karaca, 1967, https://tr.wikipedia.org/wiki/G%C3%BClizar_Bi%C3%A7er_Karaca
Emin Haluk Ayhan, 1957, https://tr.wikipedia.org/wiki/Emin_Haluk_Ayhan
İdris Baluken, 1976, https://tr.wikipedia.org/wiki/%C4%B0dris_Baluken
Nursel Aydoğan, 1958, https://tr.wikipedia.org/wiki/Nursel_Aydo%C4%9Fan
Nimetullah Erdoğmuş, 1960, https://tr.wikipedia.org/wiki/Nimetullah_Erdo%C4%9Fmu%C5%9F
Feleknas Uca, 1976, https://tr.wikipedia.org/wiki/Feleknas_Uca
Altan Tan, 1958, https://tr.wikipedia.org/wiki/Altan_Tan
Çağlar Demirel, 1969, https://tr.wikipedia.org/wiki/%C3%87a%C4%9Flar_Demirel
Ziya Pir, 1970, https://tr.wikipedia.org/wiki/Ziya_Pir
İmam Taşçıer, 1959, https://tr.wikipedia.org/wiki/%C4%B0mam_Ta%C5%9F%C3%A7%C4%B1er
Sibel Yiğitalp,, 1971, https://tr.wikipedia.org/wiki/Sibel_Yi%C4%9Fitalp
Edib Berk, 1965, https://tr.wikipedia.org/wiki/Edib_Berk
Cevdet Yılmaz, 1967, https://tr.wikipedia.org/wiki/Cevdet_Y%C4%B1lmaz
Faruk Özlü[1], 1962, https://tr.wikipedia.org/wiki/Faruk_%C3%96zl%C3%BC
Fevai Arslan, 1966, https://tr.wikipedia.org/wiki/Fevai_Arslan
Ayşe Keşir, 1967, https://tr.wikipedia.org/wiki/Ay%C5%9Fe_Ke%C5%9Fir
Okan Gaytancıoğlu, 1967, https://tr.wikipedia.org/wiki/Okan_Gaytanc%C4%B1o%C4%9Flu
Erdin Bircan,, 1959, https://tr.wikipedia.org/wiki/Erdin_Bircan
Şemsettin Emir, 1954, https://tr.wikipedia.org/wiki/%C5%9Eemsettin_Emir
Şuay Alpay, 1960, https://tr.wikipedia.org/wiki/%C5%9Euay_Alpay
Metin Bulut, 1971, https://tr.wikipedia.org/wiki/Metin_Bulut
Serpil Bulut, 1965, https://tr.wikipedia.org/wiki/Serpil_Bulut
Yavuz Temizer, 1976, https://tr.wikipedia.org/wiki/Yavuz_Temizer
Talha Erol Durmaz, 1973, https://tr.wikipedia.org/wiki/Talha_Erol_Durmaz
Erdoğan Özyalçın, 1965, https://tr.wikipedia.org/wiki/Erdo%C4%9Fan_%C3%96zyal%C3%A7%C4%B1n
Efkan Âlâ, 1965, https://tr.wikipedia.org/wiki/Efkan_Ala
Adnan Yılmaz, 1953, https://tr.wikipedia.org/wiki/Adnan_Y%C4%B1lmaz_(milletvekili)
Zehra Taşkesenlioğlu Ban, 1972, https://tr.wikipedia.org/wiki/Zehra_Ta%C5%9Fkesenlio%C4%9Flu
Kamil Aydın, 1963, https://tr.wikipedia.org/wiki/Kamil_Ayd%C4%B1n_(siyaset%C3%A7i)
Seher Akçınar Bayar, 1982, https://tr.wikipedia.org/wiki/Seher_Ak%C3%A7%C4%B1nar_Bayar
Kasım 2015 – Eskişehir, 2015, https://tr.wikipedia.org/wiki/Gaye_Usluer
Utku Çakırözer, 1970, https://tr.wikipedia.org/wiki/Utku_%C3%87ak%C4%B1r%C3%B6zer
Cemal Okan Yüksel, 1973, https://tr.wikipedia.org/wiki/Cemal_Okan_Y%C3%BCksel
Nabi Avcı,, 1953, https://tr.wikipedia.org/wiki/Nabi_Avc%C4%B1
Salih Koca, 1972, https://tr.wikipedia.org/wiki/Salih_Koca
Ruhsar Demirel, 1963, https://tr.wikipedia.org/wiki/Ruhsar_Demirel
Mehmet Şimşek,, 1967, https://tr.wikipedia.org/wiki/Mehmet_%C5%9Eim%C5%9Fek
Abdulhamit Gül, 1977, https://tr.wikipedia.org/wiki/Abd%C3%BClhamit_G%C3%BCl
Şamil Tayyar, 1965, https://tr.wikipedia.org/wiki/%C5%9Eamil_Tayyar
Canan Candemir Çelik, 1974, https://tr.wikipedia.org/wiki/Canan_Candemir_%C3%87elik
Abdullah Nejat Koçer, 1963, https://tr.wikipedia.org/wiki/Abdullah_Nejat_Ko%C3%A7er
Mehmet Erdoğan, 1956, https://tr.wikipedia.org/wiki/Mehmet_Erdo%C4%9Fan_(Gaziantepli)
Ümit Özdağ, 1961, https://tr.wikipedia.org/wiki/%C3%9Cmit_%C3%96zda%C4%9F
Ertuğrul Tolga Orhan, 1979, https://tr.wikipedia.org/wiki/Ertu%C4%9Frul_Tolga_Orhan
Mehmet Şeker,, 1963, https://tr.wikipedia.org/wiki/Mehmet_%C5%9Eeker_(siyaset%C3%A7i,_1963)
Mehmet Gökdağ, 1961, https://tr.wikipedia.org/wiki/Mehmet_G%C3%B6kda%C4%9F
Celal Doğan, 1943, https://tr.wikipedia.org/wiki/Celal_Do%C4%9Fan
Mahmut Toğrul, 1967, https://tr.wikipedia.org/wiki/Mahmut_To%C4%9Frul
Turhan Alçelik,, 1958, https://tr.wikipedia.org/wiki/Turhan_Al%C3%A7elik
Adem Tatlı, 1968, https://tr.wikipedia.org/wiki/Adem_Tatl%C4%B1
Bülent Yener Bektaşoğlu, 1953, https://tr.wikipedia.org/wiki/B%C3%BClent_Yener_Bekta%C5%9Fo%C4%9Flu
Orhan Erzurum, 1971, https://tr.wikipedia.org/wiki/Orhan_Erzurum
Kemalettin Aydın,, 1962, https://tr.wikipedia.org/wiki/Kemalettin_Ayd%C4%B1n
Mustafa Canlı, 1962, https://tr.wikipedia.org/wiki/Mustafa_Canl%C4%B1
Nihat Akdoğan, 1980, https://tr.wikipedia.org/wiki/Nihat_Akdo%C4%9Fan
Abdullah Zeydan, 1972, https://tr.wikipedia.org/wiki/Abdullah_Zeydan
Adem Yeşildal, 1976, https://tr.wikipedia.org/wiki/Adem_Ye%C5%9Fildal
Orhan Karasayar,, 1965, https://tr.wikipedia.org/wiki/Orhan_Karasayar
Mehmet Öntürk, 1970, https://tr.wikipedia.org/wiki/Mehmet_%C3%96nt%C3%BCrk
Mehmet Alğan,, 1968, https://tr.wikipedia.org/wiki/Mehmet_Al%C4%9Fan
Adalet ve Kalkınma Partisi, 2011, https://tr.wikipedia.org/wiki/Hac%C4%B1_Bayram_T%C3%BCrko%C4%9Flu
Hilmi Yarayıcı, 1968, https://tr.wikipedia.org/wiki/Hilmi_Yaray%C4%B1c%C4%B1
Mevlüt Dudu, 1966, https://tr.wikipedia.org/wiki/Mevl%C3%BCt_Dudu
Serkan Topal, 1976, https://tr.wikipedia.org/wiki/Serkan_Topal
Birol Ertem, 1968, https://tr.wikipedia.org/wiki/Birol_Ertem
Adnan Şefik Çirkin, 1962, https://tr.wikipedia.org/wiki/Adnan_%C5%9Eefik_%C3%87irkin
Mehmet Emin Adıyaman, 1961, https://tr.wikipedia.org/wiki/Mehmet_Emin_Ad%C4%B1yaman
Kıznaz Türkeli, 1962, https://tr.wikipedia.org/wiki/K%C4%B1znaz_T%C3%BCrkeli
Süreyya Sadi Bilgiç, 1961, https://tr.wikipedia.org/wiki/S%C3%BCreyya_Sadi_Bilgi%C3%A7
Recep Özel, 1969, https://tr.wikipedia.org/wiki/Recep_%C3%96zel_(siyaset%C3%A7i,_1969)
Nuri Okutan, 1962, https://tr.wikipedia.org/wiki/Nuri_Okutan
İrfan Bakır, 1954, https://tr.wikipedia.org/wiki/%C4%B0rfan_Bak%C4%B1r
İdris Güllüce, 1950, https://tr.wikipedia.org/wiki/%C4%B0dris_G%C3%BCll%C3%BCce
Mihrimah Belma Satır, 1961, https://tr.wikipedia.org/wiki/Mihrimah_Belma_Sat%C4%B1r
Erol Kaya,, 1959, https://tr.wikipedia.org/wiki/Erol_Kaya
Ahmet Berat Çonkar,, 1976, https://tr.wikipedia.org/wiki/Ahmet_Berat_%C3%87onkar
Osman Can, 1968, https://tr.wikipedia.org/wiki/Osman_Can
Berat Albayrak,, 1978, https://tr.wikipedia.org/wiki/Berat_Albayrak
İsmet Uçma, 1955, https://tr.wikipedia.org/wiki/%C4%B0smet_U%C3%A7ma
Metin Külünk, 1960, https://tr.wikipedia.org/wiki/Metin_K%C3%BCl%C3%BCnk
Ravza Kavakcı Kan, 1972, https://tr.wikipedia.org/wiki/Ravza_Kavakc%C4%B1_Kan
Erkan Kandemir, 1981, https://tr.wikipedia.org/wiki/Erkan_Kandemir
Alev Dedegil,, 1958, https://tr.wikipedia.org/wiki/Alev_Dedegil
Mehmet Ali Pulcu, 1960, https://tr.wikipedia.org/wiki/Mehmet_Ali_Pulcu
Hasan Sert, 1961, https://tr.wikipedia.org/wiki/Hasan_Sert
Hüseyin Yayman, 1969, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Yayman
Şafak Pavey, 1976, https://tr.wikipedia.org/wiki/%C5%9Eafak_Pavey
Gamze Akkuş İlgezdi, 1969, https://tr.wikipedia.org/wiki/Gamze_Akku%C5%9F_%C4%B0lgezdi
Gürsel Tekin, 1964, https://tr.wikipedia.org/wiki/G%C3%BCrsel_Tekin
Barış Yarkadaş, 1974, https://tr.wikipedia.org/wiki/Bar%C4%B1%C5%9F_Yarkada%C5%9F
Mehmet Akif Hamzaçebi, 1953, https://tr.wikipedia.org/wiki/Mehmet_Akif_Hamza%C3%A7ebi
Mahmut Tanal, 1961, https://tr.wikipedia.org/wiki/Mahmut_Tanal
İlhan Kesici, 1948, https://tr.wikipedia.org/wiki/%C4%B0lhan_Kesici
Oğuz Kaan Salıcı,, 1972, https://tr.wikipedia.org/wiki/O%C4%9Fuz_Kaan_Sal%C4%B1c%C4%B1
İhsan Özkes, 1957, https://tr.wikipedia.org/wiki/%C4%B0hsan_%C3%96zkes
Onursal Adıgüzel, 1985, https://tr.wikipedia.org/wiki/Onursal_Ad%C4%B1g%C3%BCzel
Ali Özcan, 1945, https://tr.wikipedia.org/wiki/Ali_%C3%96zcan
Selahattin Demirtaş, 1973, https://tr.wikipedia.org/wiki/Selahattin_Demirta%C5%9F
Hüda Kaya, 1960, https://tr.wikipedia.org/wiki/H%C3%BCda_Kaya
Erdal Ataş, 1974, https://tr.wikipedia.org/wiki/Erdal_Ata%C5%9F
Edip Semih Yalçın, 1958, https://tr.wikipedia.org/wiki/Edip_Semih_Yal%C3%A7%C4%B1n
İzzet Ulvi Yönter, 1976, https://tr.wikipedia.org/wiki/%C4%B0zzet_Ulvi_Y%C3%B6nter
Uygar Suphi Aktan, 1973, https://tr.wikipedia.org/wiki/Uygar_Suphi_Aktan
Volkan Bozkır, 1950, https://tr.wikipedia.org/wiki/Volkan_Bozk%C4%B1r
Ayşe Nur Bahçekapılı, 1954, https://tr.wikipedia.org/wiki/Ay%C5%9Fe_Nur_Bah%C3%A7ekap%C4%B1l%C4%B1
Ekrem Erdem, 1948, https://tr.wikipedia.org/wiki/Ekrem_Erdem
Aziz Babuşcu, 1964, https://tr.wikipedia.org/wiki/Aziz_Babu%C5%9Fcu
Durmuş Ali Sarıkaya, 1974, https://tr.wikipedia.org/wiki/Durmu%C5%9F_Ali_Sar%C4%B1kaya
Ethem Tolga,, 1950, https://tr.wikipedia.org/wiki/Ethem_Tolga
HV.PLT.1975-3, 1975, https://tr.wikipedia.org/wiki/%C5%9Eirin_%C3%9Cnal
Fatma Benli Yalçın, 1982, https://tr.wikipedia.org/wiki/Fatma_Benli
Hüseyin Bürge, 1955, https://tr.wikipedia.org/wiki/H%C3%BCseyin_B%C3%BCrge
Sevim Savaşer, 1953, https://tr.wikipedia.org/wiki/Sevim_Sava%C5%9Fer
Haydar Ali Yıldız, 1971, https://tr.wikipedia.org/wiki/Haydar_Ali_Y%C4%B1ld%C4%B1z
Markar Esayan, 1969, https://tr.wikipedia.org/wiki/Markar_Esayan
Selina Özuzun Doğan, 1977, https://tr.wikipedia.org/wiki/Selina_Do%C4%9Fan
Aykut Erdoğdu, 1972, https://tr.wikipedia.org/wiki/Aykut_Erdo%C4%9Fdu
Mustafa Sezgin Tanrıkulu, 1963, https://tr.wikipedia.org/wiki/Mustafa_Sezgin_Tanr%C4%B1kulu
DZ.P.1980-3, 1980, https://tr.wikipedia.org/wiki/Dursun_%C3%87i%C3%A7ek
Gülay Yedekci, 1976, https://tr.wikipedia.org/wiki/G%C3%BClay_Yedekci
Kadri Enis Berberoğlu,, 1956, https://tr.wikipedia.org/wiki/Kadri_Enis_Berbero%C4%9Flu
Didem Engin, 1977, https://tr.wikipedia.org/wiki/Didem_Engin
Turgut Öker, 1960, https://tr.wikipedia.org/wiki/Turgut_%C3%96ker
Filiz Kerestecioğlu Demir, 1961, https://tr.wikipedia.org/wiki/Filiz_Kerestecio%C4%9Flu
Sezai Temelli, 1963, https://tr.wikipedia.org/wiki/Sezai_Temelli
Ekmeleddin Mehmet İhsanoğlu, 1943, https://tr.wikipedia.org/wiki/Ekmeleddin_%C4%B0hsano%C4%9Flu
Celal Adan,, 1951, https://tr.wikipedia.org/wiki/Celal_Adan
Murat Başesgioğlu, 1955, https://tr.wikipedia.org/wiki/Murat_Ba%C5%9Fesgio%C4%9Flu
Mustafa Şentop, 1968, https://tr.wikipedia.org/wiki/Mustafa_%C5%9Eentop
Özlem Zengin, 1969, https://tr.wikipedia.org/wiki/%C3%96zlem_Zengin
Feyzullah Kıyıklık, 1949, https://tr.wikipedia.org/wiki/Feyzullah_K%C4%B1y%C4%B1kl%C4%B1k
Mehmet Muş, 1982, https://tr.wikipedia.org/wiki/Mehmet_Mu%C5%9F
Mehmet Doğan Kubat, 1969, https://tr.wikipedia.org/wiki/Mehmet_Do%C4%9Fan_Kubat
Tülay Kaynarca, 1969, https://tr.wikipedia.org/wiki/T%C3%BClay_Kaynarca
Abdurrahim Boynukalın, 1987, https://tr.wikipedia.org/wiki/Abdurrahim_Boynukal%C4%B1n
Harun Karaca, 1962, https://tr.wikipedia.org/wiki/Harun_Karaca
Uğur Işılak[1][2], 1971, https://tr.wikipedia.org/wiki/U%C4%9Fur_I%C5%9F%C4%B1lak
Mustafa Yeneroğlu, 1975, https://tr.wikipedia.org/wiki/Mustafa_Yenero%C4%9Flu
Mustafa Afşın Yazıcıoğlu[1][2], 1977, https://tr.wikipedia.org/wiki/Mustafa_Af%C5%9Fin_Yaz%C4%B1c%C4%B1o%C4%9Flu
Mehmet Metiner, 1960, https://tr.wikipedia.org/wiki/Mehmet_Metiner
Halis Dalkılıç, 1968, https://tr.wikipedia.org/wiki/Halis_Dalk%C4%B1l%C4%B1%C3%A7
Bihlun Tamaylıgil, 1966, https://tr.wikipedia.org/wiki/Bihlun_Tamayl%C4%B1gil
İlhan Cihaner, 1968, https://tr.wikipedia.org/wiki/%C4%B0lhan_Cihaner
Engin Altay, 1963, https://tr.wikipedia.org/wiki/Engin_Altay
Ali Şeker, 1968, https://tr.wikipedia.org/wiki/Ali_%C5%9Eeker
Erdoğan Toprak, 1959, https://tr.wikipedia.org/wiki/Erdo%C4%9Fan_Toprak
Eren Erdem, 1986, https://tr.wikipedia.org/wiki/Eren_Erdem
Murat Özçelik,, 1954, https://tr.wikipedia.org/wiki/Murat_%C3%96z%C3%A7elik
Zeynel Emre, 1981, https://tr.wikipedia.org/wiki/Zeynel_Emre
Saadet Partisi, 2001, https://tr.wikipedia.org/wiki/Mehmet_Bekaro%C4%9Flu
2023 - Van, 2023, https://tr.wikipedia.org/wiki/Pervin_Buldan
Garo Paylan, 1972, https://tr.wikipedia.org/wiki/Garo_Paylan
Abdullah Levent Tüzel, 1961, https://tr.wikipedia.org/wiki/Abdullah_Levent_T%C3%BCzel
Ali Kenanoğlu,, 1971, https://tr.wikipedia.org/wiki/Ali_Kenano%C4%9Flu
Emine Beyza Üstün, 1957, https://tr.wikipedia.org/wiki/Emine_Beyza_%C3%9Cst%C3%BCn
Meral Akşener, 1956, https://tr.wikipedia.org/wiki/Meral_Ak%C5%9Fener
Atila Kaya,, 1957, https://tr.wikipedia.org/wiki/Atila_Kaya
İsmail Faruk Aksu, 1963, https://tr.wikipedia.org/wiki/%C4%B0smail_Faruk_Aksu
Arzu Erdem, 1970, https://tr.wikipedia.org/wiki/Arzu_Erdem
Selin Sayek Böke[1][2], 1972, https://tr.wikipedia.org/wiki/Selin_Sayek_B%C3%B6ke
Musa Çam, 1953, https://tr.wikipedia.org/wiki/Musa_%C3%87am
Ahmet Tuncay Özkan, 1966, https://tr.wikipedia.org/wiki/Ahmet_Tuncay_%C3%96zkan
Tacettin Bayır, 1960, https://tr.wikipedia.org/wiki/Tacettin_Bay%C4%B1r
Özcan Purçu, 1977, https://tr.wikipedia.org/wiki/%C3%96zcan_Pur%C3%A7u
Ali Yiğit, 1955, https://tr.wikipedia.org/wiki/Ali_Yi%C4%9Fit
Veysel Eroğlu, 1948, https://tr.wikipedia.org/wiki/Veysel_Ero%C4%9Flu
Fatma Seniha Nükhet Hotar, 1961, https://tr.wikipedia.org/wiki/N%C3%BCkhet_Hotar
Cemil Şeboy, 1953, https://tr.wikipedia.org/wiki/Cemil_%C5%9Eeboy
Oktay Vural, 1956, https://tr.wikipedia.org/wiki/Oktay_Vural
Aslan Savaşan, 1965, https://tr.wikipedia.org/wiki/Aslan_Sava%C5%9Fan
Ertuğrul Kürkcü[1], 1948, https://tr.wikipedia.org/wiki/Ertu%C4%9Frul_K%C3%BCrk%C3%A7%C3%BC
Failed to parse: https://tr.wikipedia.org/wiki/Kemal_K%C4%B1l%C4%B1%C3%A7daro%C4%9Flu
Zeynep Altıok, 1969, https://tr.wikipedia.org/wiki/Zeynep_Alt%C4%B1ok
Zekeriya Temizel, 1948, https://tr.wikipedia.org/wiki/Zekeriya_Temizel
Mustafa Ali Balbay, 1960, https://tr.wikipedia.org/wiki/Mustafa_Ali_Balbay
Aytun Çıray, 1957, https://tr.wikipedia.org/wiki/Aytun_%C3%87%C4%B1ray
Kamil Okyay Sındır,, 1962, https://tr.wikipedia.org/wiki/Kamil_Okyay_S%C4%B1nd%C4%B1r
İbrahim Mustafa Turhan, 1968, https://tr.wikipedia.org/wiki/Mustafa_%C4%B0brahim_Turhan
Nesrin Ulema, 1971, https://tr.wikipedia.org/wiki/Nesrin_Ulema
Hamza Dağ, 1980, https://tr.wikipedia.org/wiki/Hamza_Da%C4%9F
Kerem Ali Sürekli,, 1965, https://tr.wikipedia.org/wiki/Kerem_Ali_S%C3%BCrekli
Ahmet Kenan Tanrıkulu, 1958, https://tr.wikipedia.org/wiki/Ahmet_Kenan_Tanr%C4%B1kulu
Murat Koç, 1956, https://tr.wikipedia.org/wiki/Murat_Ko%C3%A7
Müslüm Doğan, 1959, https://tr.wikipedia.org/wiki/M%C3%BCsl%C3%BCm_Do%C4%9Fan
Mahir Ünal,, 1966, https://tr.wikipedia.org/wiki/Mahir_%C3%9Cnal
Nursel Reyhanlıoğlu, 1968, https://tr.wikipedia.org/wiki/Nursel_Reyhanl%C4%B1o%C4%9Flu
Veysi Kaynak, 1962, https://tr.wikipedia.org/wiki/Veysi_Kaynak
Sevde Bayazıt Kaçar,, 1974, https://tr.wikipedia.org/wiki/Sevde_Bayaz%C4%B1t_Ka%C3%A7ar
Mehmet Uğur Dilipak, 1972, https://tr.wikipedia.org/wiki/Mehmet_U%C4%9Fur_Dilipak
Mehmet İlker Çitil, 1969, https://tr.wikipedia.org/wiki/Mehmet_%C4%B0lker_%C3%87itil
Fahrettin Oğuz Tor, 1957, https://tr.wikipedia.org/wiki/Fahrettin_O%C4%9Fuz_Tor
Prof. Dr. Sefer Aycan,, 1963, https://tr.wikipedia.org/wiki/Sefer_Aycan
Osman Kahveci, 1950, https://tr.wikipedia.org/wiki/Osman_Kahveci
Durmuş Yalçın, 1963, https://tr.wikipedia.org/wiki/Durmu%C5%9F_Yal%C3%A7%C4%B1n_(siyaset%C3%A7i)
Recep Konuk, 1956, https://tr.wikipedia.org/wiki/Recep_Konuk
Dr. Recep Şeker, 1972, https://tr.wikipedia.org/wiki/Recep_%C5%9Eeker
Ayhan Bilgen, 1971, https://tr.wikipedia.org/wiki/Ayhan_Bilgen
Şafak Özanlı, 1973, https://tr.wikipedia.org/wiki/%C5%9Eafak_%C3%96zanli
Mehmet Uçum, 1965, https://tr.wikipedia.org/wiki/Mehmet_U%C3%A7um
Mustafa Gökhan Gülşen, 1970, https://tr.wikipedia.org/wiki/Mustafa_G%C3%B6khan_G%C3%BCl%C5%9Fen
Metin Çelik, 1977, https://tr.wikipedia.org/wiki/Metin_%C3%87elik
Emin Çınar, 1967, https://tr.wikipedia.org/wiki/Emin_%C3%87%C4%B1nar
Mehmet Özhaseki, 1957, https://tr.wikipedia.org/wiki/Mehmet_%C3%96zhaseki
Ahmet Doğan, 1961, https://tr.wikipedia.org/wiki/Ahmet_Do%C4%9Fan_(1961_do%C4%9Fumlu_siyaset%C3%A7i)
Yaşar Karayel, 1950, https://tr.wikipedia.org/wiki/Ya%C5%9Far_Karayel
Kemal Tekden, 1959, https://tr.wikipedia.org/wiki/Kemal_Tekden
Havva Talay Çalış, 1971, https://tr.wikipedia.org/wiki/Havva_Talay_%C3%87al%C4%B1%C5%9F
Yusuf Halaçoğlu, 1949, https://tr.wikipedia.org/wiki/Yusuf_Hala%C3%A7o%C4%9Flu
Hasan Ali Kilci, 1951, https://tr.wikipedia.org/wiki/Hasan_Ali_Kilci
Süleyman Korkmaz, 1968, https://tr.wikipedia.org/wiki/S%C3%BCleyman_Korkmaz
Çetin Arık, 1971, https://tr.wikipedia.org/wiki/%C3%87etin_Ar%C4%B1k
Oğuz Kağan Köksal, 1948, https://tr.wikipedia.org/wiki/O%C4%9Fuz_Ka%C4%9Fan_K%C3%B6ksal
Ramazan Can, 1970, https://tr.wikipedia.org/wiki/Ramazan_Can
Seyit Ahmet Göçer, 1979, https://tr.wikipedia.org/wiki/Seyit_Ahmet_G%C3%B6%C3%A7er
Türabi Kayan, 1955, https://tr.wikipedia.org/wiki/T%C3%BCrabi_Kayan
Vecdi Gündoğdu, 1969, https://tr.wikipedia.org/wiki/Vecdi_G%C3%BCndo%C4%9Fdu
Hamdi Irmak, 1961, https://tr.wikipedia.org/wiki/Hamdi_Irmak
Salih Çetinkaya, 1972, https://tr.wikipedia.org/wiki/Salih_%C3%87etinkaya
Cemil Yıldırım Türk, 1962, https://tr.wikipedia.org/wiki/Cemil_Y%C4%B1ld%C4%B1r%C4%B1m_T%C3%BCrk
Reşit Polat, 1975, https://tr.wikipedia.org/wiki/Re%C5%9Fit_Polat
Mustafa Yün, 1968, https://tr.wikipedia.org/wiki/Mustafa_Y%C3%BCn
Fikri Işık, 1965, https://tr.wikipedia.org/wiki/Fikri_I%C5%9F%C4%B1k
Radiye Sezer Katırcıoğlu, 1965, https://tr.wikipedia.org/wiki/Radiye_Sezer_Kat%C4%B1rc%C4%B1o%C4%9Flu
Zeki Aygün, 1953, https://tr.wikipedia.org/wiki/Zeki_Ayg%C3%BCn
İlyas Şeker, 1960, https://tr.wikipedia.org/wiki/%C4%B0lyas_%C5%9Eeker
Mehmet Akif Yılmaz, 1975, https://tr.wikipedia.org/wiki/Mehmet_Akif_Y%C4%B1lmaz
Cemalettin Kaflı, 1971, https://tr.wikipedia.org/wiki/Cemalettin_Kafl%C4%B1
Fatma Kaplan Hürriyet, 1982, https://tr.wikipedia.org/wiki/Fatma_Kaplan_H%C3%BCrriyet
Haydar Akar,, 1963, https://tr.wikipedia.org/wiki/Haydar_Akar
Tahsin Tarhan, 1964, https://tr.wikipedia.org/wiki/Tahsin_Tarhan
Ali Haydar Konca, 1950, https://tr.wikipedia.org/wiki/Ali_Haydar_Konca
Millet İttifakı, 2022, https://tr.wikipedia.org/wiki/Ahmet_Davuto%C4%9Flu
Mustafa Baloğlu, 1967, https://tr.wikipedia.org/wiki/Mustafa_Balo%C4%9Flu
Hüsnüye Erdoğan, 1965, https://tr.wikipedia.org/wiki/H%C3%BCsn%C3%BCye_Erdo%C4%9Fan
Ziya Altunyaldız, 1963, https://tr.wikipedia.org/wiki/Ziya_Altunyald%C4%B1z
Muhammet Uğur Kaleli, 1958, https://tr.wikipedia.org/wiki/Muhammet_U%C4%9Fur_Kaleli
Leyla Şahin Usta, 1973, https://tr.wikipedia.org/wiki/Leyla_%C5%9Eahin_Usta
Mehmet Babaoğlu, 1965, https://tr.wikipedia.org/wiki/Mehmet_Babao%C4%9Flu
Abdullah Ağralı, 1969, https://tr.wikipedia.org/wiki/Abdullah_A%C4%9Fral%C4%B1
Halil Etyemez, 1965, https://tr.wikipedia.org/wiki/Halil_Etyemez
Rüveyde Gülseren Işık, 1985, https://tr.wikipedia.org/wiki/R%C3%BCveyde_G%C3%BClseren_I%C5%9F%C4%B1k
Hacı Ahmet Özdemir, 1961, https://tr.wikipedia.org/wiki/Hac%C4%B1_Ahmet_%C3%96zdemir
Mustafa Kalaycı, 1960, https://tr.wikipedia.org/wiki/Mustafa_Kalayc%C4%B1
Mustafa Sait Gönen, 1960, https://tr.wikipedia.org/wiki/Mustafa_Sait_G%C3%B6nen
Mustafa Hüsnü Bozkurt, 1950, https://tr.wikipedia.org/wiki/Mustafa_H%C3%BCsn%C3%BC_Bozkurt
Mustafa Şükrü Nazlı, 1966, https://tr.wikipedia.org/wiki/Mustafa_%C5%9E%C3%BCkr%C3%BC_Nazl%C4%B1
Vural Kavuncu, 1966, https://tr.wikipedia.org/wiki/Vural_Kavuncu
Alim Işık[1], 1959, https://tr.wikipedia.org/wiki/Alim_I%C5%9F%C4%B1k
Öznur Çalık, 1965, https://tr.wikipedia.org/wiki/%C3%96znur_%C3%87al%C4%B1k
Taha Özhan, 1977, https://tr.wikipedia.org/wiki/Taha_%C3%96zhan
Mustafa Şahin, 1956, https://tr.wikipedia.org/wiki/Mustafa_%C5%9Eahin_(siyaset%C3%A7i,_1956)
Nurettin Yaşar, 1961, https://tr.wikipedia.org/wiki/Nurettin_Ya%C5%9Far
Bülent Tüfenkci, 1966, https://tr.wikipedia.org/wiki/B%C3%BClent_T%C3%BCfenkci
Cumhuriyet Halk Partisi, 1995, https://tr.wikipedia.org/wiki/Veli_A%C4%9Fbaba
Recai Berber, 1959, https://tr.wikipedia.org/wiki/Recai_Berber
Uğur Aydemir, 1967, https://tr.wikipedia.org/wiki/U%C4%9Fur_Aydemir
Büyük Birlik Partisi, 1993, https://tr.wikipedia.org/wiki/Sel%C3%A7uk_%C3%96zda%C4%9F
Murat Baybatur, 1978, https://tr.wikipedia.org/wiki/Murat_Baybatur
Özgür Özel, 1974, https://tr.wikipedia.org/wiki/%C3%96zg%C3%BCr_%C3%96zel
Tur Yıldız Biçer, 1968, https://tr.wikipedia.org/wiki/Tur_Y%C4%B1ld%C4%B1z_Bi%C3%A7er
Erkan Akçay, 1961, https://tr.wikipedia.org/wiki/Erkan_Ak%C3%A7ay
Zeynel Balkız, 1957, https://tr.wikipedia.org/wiki/Zeynel_Balk%C4%B1z
Mithat Sancar, 1963, https://tr.wikipedia.org/wiki/Mithat_Sancar
Gülser Yıldırım, 1963, https://tr.wikipedia.org/wiki/G%C3%BClser_Y%C4%B1ld%C4%B1r%C4%B1m
Erol Dora, 1964, https://tr.wikipedia.org/wiki/Erol_Dora
Mehmet Ali Aslan, 1975, https://tr.wikipedia.org/wiki/Mehmet_Ali_Aslan
Enise Çoban Güneyli, 1986, https://tr.wikipedia.org/wiki/Enise_G%C3%BCneyli
Orhan Miroğlu,, 1953, https://tr.wikipedia.org/wiki/Orhan_Miro%C4%9Flu
Hüseyin Çamak, 1948, https://tr.wikipedia.org/wiki/H%C3%BCseyin_%C3%87amak
Durmuş Fikri Sağlar, 1953, https://tr.wikipedia.org/wiki/Durmu%C5%9F_Fikri_Sa%C4%9Flar
Aytuğ Atıcı, 1964, https://tr.wikipedia.org/wiki/Aytu%C4%9F_At%C4%B1c%C4%B1
Mustafa Muhammet Gültak, 1969, https://tr.wikipedia.org/wiki/Mustafa_Muhammet_G%C3%BCltak
Prof. Dr. Yılmaz Tezcan, 1968, https://tr.wikipedia.org/wiki/Y%C4%B1lmaz_Tezcan
Muhsin Kızılkaya, 1963, https://tr.wikipedia.org/wiki/Muhsin_K%C4%B1z%C4%B1lkaya
Oktay Öztürk,, 1951, https://tr.wikipedia.org/wiki/Oktay_%C3%96zt%C3%BCrk
Baki Şimşek, 1969, https://tr.wikipedia.org/wiki/Baki_%C5%9Eim%C5%9Fek
Haziran 2015 – Mersin, 2015, https://tr.wikipedia.org/wiki/Ali_%C3%96z_(siyaset%C3%A7i)
Dengir Mir Mehmet Fırat, 1943, https://tr.wikipedia.org/wiki/Dengir_Mir_Mehmet_F%C4%B1rat
Çilem Küçükkeleş Öz, 1978, https://tr.wikipedia.org/wiki/%C3%87ilem_K%C3%BC%C3%A7%C3%BCkkele%C5%9F_%C3%96z
Akın Üstündağ, 1968, https://tr.wikipedia.org/wiki/Ak%C4%B1n_%C3%9Cst%C3%BCnda%C4%9F
Ömer Süha Aldan, 1959, https://tr.wikipedia.org/wiki/%C3%96mer_S%C3%BCha_Aldan
Nurettin Demir, 1953, https://tr.wikipedia.org/wiki/Nurettin_Demir
Hasan Özyer, 1960, https://tr.wikipedia.org/wiki/Hasan_%C3%96zyer
Hasan Kökten, 1978, https://tr.wikipedia.org/wiki/Hasan_K%C3%B6kten
Mehmet Erdoğan, 1964, https://tr.wikipedia.org/wiki/Mehmet_Erdo%C4%9Fan_(Mu%C4%9Flal%C4%B1)
Kasım 2015 – Muş, 2015, https://tr.wikipedia.org/wiki/Burcu_%C3%87elik_%C3%96zkan
Mehmet Emin Şimşek, 1964, https://tr.wikipedia.org/wiki/Mehmet_Emin_%C5%9Eim%C5%9Fek
Mustafa Açıkgöz, 1971, https://tr.wikipedia.org/wiki/Mustafa_A%C3%A7%C4%B1kg%C3%B6z
Murat Göktürk, 1970, https://tr.wikipedia.org/wiki/Murat_G%C3%B6kt%C3%BCrk
Mehmet Varol, 1964, https://tr.wikipedia.org/wiki/Mehmet_Varol
Alpaslan Kavaklıoğlu, 1962, https://tr.wikipedia.org/wiki/Alpaslan_Kavakl%C4%B1o%C4%9Flu
Erdoğan Özegen, 1965, https://tr.wikipedia.org/wiki/Erdo%C4%9Fan_%C3%96zegen
Vedat Bayram, 1956, https://tr.wikipedia.org/wiki/Vedat_Bayram
Numan Kurtulmuş, 1959, https://tr.wikipedia.org/wiki/Numan_Kurtulmu%C5%9F
Oktay Çanak, 1975, https://tr.wikipedia.org/wiki/Oktay_%C3%87anak
Seyit Torun, 1968, https://tr.wikipedia.org/wiki/Seyit_Torun
Mustafa Adıgüzel,, 1967, https://tr.wikipedia.org/wiki/Mustafa_Ad%C4%B1g%C3%BCzel
Devlet Bahçeli, 1948, https://tr.wikipedia.org/wiki/Devlet_Bah%C3%A7eli
Ruhi Ersoy, 1972, https://tr.wikipedia.org/wiki/Ruhi_Ersoy
Suat Önal, 1961, https://tr.wikipedia.org/wiki/Suat_%C3%96nal
Mücahit Durmuşoğlu, 1965, https://tr.wikipedia.org/wiki/M%C3%BCcahit_Durmu%C5%9Fo%C4%9Flu
Hasan Karal, 1973, https://tr.wikipedia.org/wiki/Hasan_Karal
Hikmet Ayar, 1957, https://tr.wikipedia.org/wiki/Hikmet_Ayar
Osman Aşkın Bak, 1966, https://tr.wikipedia.org/wiki/Osman_A%C5%9Fk%C4%B1n_Bak
Ayşenur İslam, 1958, https://tr.wikipedia.org/wiki/Ay%C5%9Fenur_%C4%B0slam
Mustafa İsen, 1953, https://tr.wikipedia.org/wiki/Mustafa_%C4%B0sen
Ali İhsan Yavuz, 1969, https://tr.wikipedia.org/wiki/Ali_%C4%B0hsan_Yavuz
Recep Uncuoğlu, 1970, https://tr.wikipedia.org/wiki/Recep_Uncuo%C4%9Flu
Ali İnci, 1959, https://tr.wikipedia.org/wiki/Ali_%C4%B0nci
Zihni Açba, 1956, https://tr.wikipedia.org/wiki/Zihni_A%C3%A7ba
Engin Özkoç, 1962, https://tr.wikipedia.org/wiki/Engin_%C3%96zko%C3%A7
Akif Çağatay Kılıç, 1976, https://tr.wikipedia.org/wiki/Akif_%C3%87a%C4%9Fatay_K%C4%B1l%C4%B1%C3%A7
Ahmet Demircan, 1954, https://tr.wikipedia.org/wiki/Ahmet_Demircan
Fuat Köktaş, 1964, https://tr.wikipedia.org/wiki/Fuat_K%C3%B6kta%C5%9F
Çiğdem Karaaslan, 1979, https://tr.wikipedia.org/wiki/%C3%87i%C4%9Fdem_Karaaslan
Hasan Basri Kurt, 1978, https://tr.wikipedia.org/wiki/Hasan_Basri_Kurt
Kemal Zeybek, 1962, https://tr.wikipedia.org/wiki/Kemal_Zeybek
Hayati Tekin, 1959, https://tr.wikipedia.org/wiki/Hayati_Tekin
Erhan Usta, 1966, https://tr.wikipedia.org/wiki/Erhan_Usta
Hüseyin Edis, 1960, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Edis
Siirt, 2015, https://tr.wikipedia.org/wiki/Kadri_Y%C4%B1ld%C4%B1r%C4%B1m
Hatice Kocaman, 1977, https://tr.wikipedia.org/wiki/Hatice_Seviptekin
Yasin Aktay, 1966, https://tr.wikipedia.org/wiki/Yasin_Aktay
Cengiz Tokmak, 1976, https://tr.wikipedia.org/wiki/Cengiz_Tokmak
Barış Karadeniz, 1975, https://tr.wikipedia.org/wiki/Bar%C4%B1%C5%9F_Karadeniz
İsmet Yılmaz, 1961, https://tr.wikipedia.org/wiki/%C4%B0smet_Y%C4%B1lmaz
Hilmi Bilgin[1], 1972, https://tr.wikipedia.org/wiki/Hilmi_Bilgin
Selim Dursun,, 1957, https://tr.wikipedia.org/wiki/Selim_Dursun
Celal Dağgez, 1954, https://tr.wikipedia.org/wiki/Celal_Da%C4%9Fgez
Ali Akyıldız, 1964, https://tr.wikipedia.org/wiki/Ali_Aky%C4%B1ld%C4%B1z
Nureddin Nebati,, 1964, https://tr.wikipedia.org/wiki/Nureddin_Nebati
Mehmet Kasım Gülpınar, 1969, https://tr.wikipedia.org/wiki/Mehmet_Kas%C4%B1m_G%C3%BClp%C4%B1nar
Seyit Eyyüpoğlu, 1942, https://tr.wikipedia.org/wiki/Seyit_Eyy%C3%BCpo%C4%9Flu
Mazhar Bağlı, 1965, https://tr.wikipedia.org/wiki/Mazhar_Ba%C4%9Fl%C4%B1
Halil Özcan, 1964, https://tr.wikipedia.org/wiki/Halil_%C3%96zcan
Hamide Sürücü, 1980, https://tr.wikipedia.org/wiki/Hamide_S%C3%BCr%C3%BCc%C3%BC
Ahmet Eşref Fakıbaba, 1951, https://tr.wikipedia.org/wiki/Ahmet_E%C5%9Fref_Fak%C4%B1baba
Osman Baydemir, 1971, https://tr.wikipedia.org/wiki/Osman_Baydemir
Dilek Öcalan, 1987, https://tr.wikipedia.org/wiki/Dilek_%C3%96calan
İbrahim Ayhan, 1968, https://tr.wikipedia.org/wiki/%C4%B0brahim_Ayhan
Leyla Güven, 1964, https://tr.wikipedia.org/wiki/Leyla_G%C3%BCven_(siyaset%C3%A7i)
Ziya Çalışkan, 1965, https://tr.wikipedia.org/wiki/Ziya_%C3%87al%C4%B1%C5%9Fkan
Faysal Sarıyıldız, 1975, https://tr.wikipedia.org/wiki/Faysal_Sar%C4%B1y%C4%B1ld%C4%B1z
Leyla Birlik,, 1974, https://tr.wikipedia.org/wiki/Leyla_Birlik
Ferhat Encü,, 1985, https://tr.wikipedia.org/wiki/Ferhat_Enc%C3%BC
Aycan İrmez, 1985, https://tr.wikipedia.org/wiki/Aycan_%C4%B0rmez
Candan Yüceer, 1973, https://tr.wikipedia.org/wiki/Candan_Y%C3%BCceer
Emre Köprülü, 1981, https://tr.wikipedia.org/wiki/Emre_K%C3%B6pr%C3%BCl%C3%BC
Mustafa Yel, 1968, https://tr.wikipedia.org/wiki/Mustafa_Yel
Kasım 2015 – Tekirdağ, 2015, https://tr.wikipedia.org/wiki/Ay%C5%9Fe_Do%C4%9Fan_(1971_do%C4%9Fumlu_siyaset%C3%A7i)
Bülent Belen, 1956, https://tr.wikipedia.org/wiki/B%C3%BClent_Belen
Coşkun Çakır, 1966, https://tr.wikipedia.org/wiki/Co%C5%9Fkun_%C3%87ak%C4%B1r
Celil Göçer, 1970, https://tr.wikipedia.org/wiki/Celil_G%C3%B6%C3%A7er
Fatma  Gaye Yiğit, 1988, https://tr.wikipedia.org/wiki/Fatma_Gaye_Yi%C4%9Fit
Abdurrahman Başkan, 1964, https://tr.wikipedia.org/wiki/Abdurrahman_Ba%C5%9Fkan
Süleyman Soylu, 1969, https://tr.wikipedia.org/wiki/S%C3%BCleyman_Soylu
Muhammet Balta, 1966, https://tr.wikipedia.org/wiki/Muhammet_Balta
Ayşe Sula Köseoğlu, 1969, https://tr.wikipedia.org/wiki/Ay%C5%9Fe_Sula_K%C3%B6seo%C4%9Flu
Adnan Günnar,, 1966, https://tr.wikipedia.org/wiki/Adnan_G%C3%BCnnar
Koray Aydın, 1955, https://tr.wikipedia.org/wiki/Koray_Ayd%C4%B1n
Haluk Pekşen, 1961, https://tr.wikipedia.org/wiki/Haluk_Pek%C5%9Fen
Alican Önlü,, 1967, https://tr.wikipedia.org/wiki/Alican_%C3%96nl%C3%BC
Edibe Şahin,, 1960, https://tr.wikipedia.org/wiki/Edibe_%C5%9Eahin
Mehmet Altay, 1972, https://tr.wikipedia.org/wiki/Mehmet_Altay
Özkan Yalım, 1969, https://tr.wikipedia.org/wiki/%C3%96zkan_Yal%C4%B1m
Durmuş Yılmaz, 1947, https://tr.wikipedia.org/wiki/Durmu%C5%9F_Y%C4%B1lmaz
Figen Yüksekdağ Şenoğlu, 1971, https://tr.wikipedia.org/wiki/Figen_Y%C3%BCksekda%C4%9F
Yurdusev Özsökmenler, 1952, https://tr.wikipedia.org/wiki/Yurdusev_%C3%96zs%C3%B6kmenler
Adem Geveri, 1978, https://tr.wikipedia.org/wiki/Adem_Geveri
Tuğba Hezer Öztürk,, 1989, https://tr.wikipedia.org/wiki/Tu%C4%9Fba_Hezer_%C3%96zt%C3%BCrk
Selami Özyaşar, 1978, https://tr.wikipedia.org/wiki/Selami_%C3%96zya%C5%9Far
Remzi Özgökçe,, 1965, https://tr.wikipedia.org/wiki/Remzi_%C3%96zg%C3%B6k%C3%A7e
Burhan Kayatürk,, 1970, https://tr.wikipedia.org/wiki/Burhan_Kayat%C3%BCrk
Fikri Demirel, 1956, https://tr.wikipedia.org/wiki/Fikri_Demirel
Muharrem İnce, 1964, https://tr.wikipedia.org/wiki/Muharrem_%C4%B0nce
Haziran 2015 – Yozgat 
Kasım 2015 – Yozgat, 2015, https://tr.wikipedia.org/wiki/Abdulkadir_Akg%C3%BCl
Ertuğrul Soysal, 1963, https://tr.wikipedia.org/wiki/Ertu%C4%9Frul_Soysal
Sadir Durmaz, 1963, https://tr.wikipedia.org/wiki/Sadir_Durmaz
Şerafettin Turpcu, 1955, https://tr.wikipedia.org/wiki/%C5%9Eerafettin_Turpcu
Ünal Demirtaş, 1971, https://tr.wikipedia.org/wiki/%C3%9Cnal_Demirta%C5%9F
Hüseyin Özbakır, 1950, https://tr.wikipedia.org/wiki/H%C3%BCseyin_%C3%96zbak%C4%B1r
Faruk Çaturoğlu, 1961, https://tr.wikipedia.org/wiki/Faruk_%C3%87aturo%C4%9Flu
Zeki Çakan, 1950, https://tr.wikipedia.org/wiki/Zeki_%C3%87akan
Failed links:
https://tr.wikipedia.org/wiki/Kemal_K%C4%B1l%C4%B1%C3%A7daro%C4%9Flu


In [ ]:
24.dönem mv
Ömer Çelik, 1968, https://tr.wikipedia.org/wiki/%C3%96mer_%C3%87elik
Mehmet Necati Çetinkaya, 1943, https://tr.wikipedia.org/wiki/Mehmet_Necati_%C3%87etinkaya
Necdet Ünüvar,, 1960, https://tr.wikipedia.org/wiki/Necdet_%C3%9Cn%C3%BCvar
Fatoş Gürkan,, 1966, https://tr.wikipedia.org/wiki/Fato%C5%9F_G%C3%BCrkan
Ali Küçükaydın,, 1948, https://tr.wikipedia.org/wiki/Ali_K%C3%BC%C3%A7%C3%BCkayd%C4%B1n
Mehmet Şükrü Erdinç,, 1976, https://tr.wikipedia.org/wiki/Mehmet_%C5%9E%C3%BCkr%C3%BC_Erdin%C3%A7
Osman Faruk Loğoğlu, 1941, https://tr.wikipedia.org/wiki/Osman_Faruk_Lo%C4%9Fo%C4%9Flu
Ümit Özgümüş, 1956, https://tr.wikipedia.org/wiki/%C3%9Cmit_%C3%96zg%C3%BCm%C3%BC%C5%9F
Tugay Develi, 1962, https://tr.wikipedia.org/wiki/Turgay_Develi
Ali Demirçalı, 1965, https://tr.wikipedia.org/wiki/Ali_Demir%C3%A7al%C4%B1
Muharrem Varlı, 1969, https://tr.wikipedia.org/wiki/Muharrem_Varl%C4%B1
Ali Halaman, 1954, https://tr.wikipedia.org/wiki/Ali_Halaman
Seyfettin Yılmaz, 1968, https://tr.wikipedia.org/wiki/Seyfettin_Y%C4%B1lmaz
Murat Bozlak, 1952, https://tr.wikipedia.org/wiki/Murat_Bozlak
Ahmet Aydın,, 1971, https://tr.wikipedia.org/wiki/Ahmet_Ayd%C4%B1n_(Ad%C4%B1yamanl%C4%B1)
Mehmet Metiner, 1960, https://tr.wikipedia.org/wiki/Mehmet_Metiner
Muhammed Murtaza Yetiş, 1972, https://tr.wikipedia.org/wiki/Muhammed_Murtaza_Yeti%C5%9F
Mehmet Erdoğan, 1966, https://tr.wikipedia.org/wiki/Mehmet_Erdo%C4%9Fan_(Ad%C4%B1yamanl%C4%B1)
Salih Fırat, 1960, https://tr.wikipedia.org/wiki/Salih_F%C4%B1rat
Veysel Eroğlu, 1948, https://tr.wikipedia.org/wiki/Veysel_Ero%C4%9Flu
Sait Açba, 1950, https://tr.wikipedia.org/wiki/Sait_A%C3%A7ba
Halil Ürün, 1968, https://tr.wikipedia.org/wiki/Halil_%C3%9Cr%C3%BCn_(Afyonlu)
Ahmet Toptaş, 1949, https://tr.wikipedia.org/wiki/Ahmet_Topta%C5%9F
Kemalettin Yılmaz, 1956, https://tr.wikipedia.org/wiki/Kemalettin_Y%C4%B1lmaz
Mehmet Kerim Yıldız,, 1965, https://tr.wikipedia.org/wiki/Mehmet_Kerim_Y%C4%B1ld%C4%B1z
Ekrem Çelebi, 1965, https://tr.wikipedia.org/wiki/Ekrem_%C3%87elebi
Fatma Salman, 1970, https://tr.wikipedia.org/wiki/Fatma_Salman_Kotan
Halil Aksoy, 1948, https://tr.wikipedia.org/wiki/Halil_Aksoy
Ali Rıza Alaboyun, 1957, https://tr.wikipedia.org/wiki/Ali_R%C4%B1za_Alaboyun
Ruhi Açıkgöz, 1963, https://tr.wikipedia.org/wiki/Ruhi_A%C3%A7%C4%B1kg%C3%B6z
İlknur İnceöz, 1973, https://tr.wikipedia.org/wiki/%C4%B0lknur_%C4%B0nce%C3%B6z
Mehmet Naci Bostancı, 1957, https://tr.wikipedia.org/wiki/Naci_Bostanc%C4%B1
Avni Erdemir, 1964, https://tr.wikipedia.org/wiki/Avni_Erdemir
Ramis Topal, 1960, https://tr.wikipedia.org/wiki/Ramis_Topal
Ali Babacan, 1967, https://tr.wikipedia.org/wiki/Ali_Babacan
Yalçın Akdoğan, 1969, https://tr.wikipedia.org/wiki/Yal%C3%A7%C4%B1n_Akdo%C4%9Fan
Dr. Bülent Gedikli,, 1964, https://tr.wikipedia.org/wiki/B%C3%BClent_Gedikli
Tülay Selamoğlu, 1966, https://tr.wikipedia.org/wiki/T%C3%BClay_Selamo%C4%9Flu
Nazmi Haluk Özdalga[1], 1948, https://tr.wikipedia.org/wiki/Haluk_%C3%96zdalga
Fatih Şahin,, 1979, https://tr.wikipedia.org/wiki/Fatih_%C5%9Eahin_(milletvekili)
Ülker Güzel, 1944, https://tr.wikipedia.org/wiki/%C3%9Clker_G%C3%BCzel
Mehmet Emrehan Halıcı, 1956, https://tr.wikipedia.org/wiki/Emrehan_Hal%C4%B1c%C4%B1
Mehmet Emrehan Halıcı, 1956, https://tr.wikipedia.org/wiki/Elektronik_Demokrasi_Partisi
Emine Ülker Tarhan, 1963, https://tr.wikipedia.org/wiki/Emine_%C3%9Clker_Tarhan
Aylin Nazlıaka, 1968, https://tr.wikipedia.org/wiki/Aylin_Nazl%C4%B1aka
Bülent Kuşoğlu,, 1958, https://tr.wikipedia.org/wiki/B%C3%BClent_Ku%C5%9Fo%C4%9Flu
Levent Gök, 1959, https://tr.wikipedia.org/wiki/Levent_G%C3%B6k
Yıldırım Tuğrul Türkeş[1][2], 1954, https://tr.wikipedia.org/wiki/Tu%C4%9Frul_T%C3%BCrke%C5%9F
Zühal Topcu, 1960, https://tr.wikipedia.org/wiki/Zuhal_Top%C3%A7u
Cemil Çiçek, 1946, https://tr.wikipedia.org/wiki/Cemil_%C3%87i%C3%A7ek
Haluk İpek, 1963, https://tr.wikipedia.org/wiki/Haluk_%C4%B0pek
Salih Kapusuz, 1954, https://tr.wikipedia.org/wiki/Salih_Kapusuz
Nurdan Şanlı, 1954, https://tr.wikipedia.org/wiki/Nurdan_%C5%9Eanl%C4%B1
Ahmet İyimaya, 1950, https://tr.wikipedia.org/wiki/Ahmet_%C4%B0yimaya
Cevdet Erdöl,, 1958, https://tr.wikipedia.org/wiki/Cevdet_Erd%C3%B6l
Zelkif Kazdal, 1975, https://tr.wikipedia.org/wiki/Zelkif_Kazdal
Emrullah İşler, 1960, https://tr.wikipedia.org/wiki/Emrullah_%C4%B0%C5%9Fler
Seyit Sertçelik, 1964, https://tr.wikipedia.org/wiki/Seyit_Sert%C3%A7elik
Ayşe Gülsün Bilgehan, 1957, https://tr.wikipedia.org/wiki/Ay%C5%9Fe_G%C3%BCls%C3%BCn_Bilgehan
İzzet Çetin, 1953, https://tr.wikipedia.org/wiki/%C4%B0zzet_%C3%87etin
Sinan Aydın Aygün, 1959, https://tr.wikipedia.org/wiki/Sinan_Ayg%C3%BCn
Gökhan Günaydın,, 1964, https://tr.wikipedia.org/wiki/G%C3%B6khan_G%C3%BCnayd%C4%B1n
Mustafa Erdem, 1957, https://tr.wikipedia.org/wiki/Mustafa_Erdem
Özcan Yeniçeri,, 1954, https://tr.wikipedia.org/wiki/%C3%96zcan_Yeni%C3%A7eri
Mehmet Vecdi Gönül, 1939, https://tr.wikipedia.org/wiki/Vecdi_G%C3%B6n%C3%BCl
Mevlüt Çavuşoğlu, 1968, https://tr.wikipedia.org/wiki/Mevl%C3%BCt_%C3%87avu%C5%9Fo%C4%9Flu
Menderes Mehmet Tevfik Türel, 1964, https://tr.wikipedia.org/wiki/Menderes_T%C3%BCrel
Sadık Badak, 1955, https://tr.wikipedia.org/wiki/Sad%C4%B1k_Badak
Hüseyin Samani, 1964, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Samani
Gökcen Özdoğan Enç, 1976, https://tr.wikipedia.org/wiki/G%C3%B6kcen_%C3%96zdo%C4%9Fan_En%C3%A7
Deniz Baykal, 1938, https://tr.wikipedia.org/wiki/Deniz_Baykal
Osman Kaptan, 1946, https://tr.wikipedia.org/wiki/Osman_Kaptan
Gürkut Acar,, 1948, https://tr.wikipedia.org/wiki/G%C3%BCrkut_Acar
Yıldıray Sapan,, 1967, https://tr.wikipedia.org/wiki/Y%C4%B1ld%C4%B1ray_Sapan
Arif Bulut, 1963, https://tr.wikipedia.org/wiki/Arif_Bulut
Tunca Toskay, 1939, https://tr.wikipedia.org/wiki/Tunca_Toskay
Mehmet Günal, 1964, https://tr.wikipedia.org/wiki/Mehmet_G%C3%BCnal
Yusuf Ziya İrbeç, 1959, https://tr.wikipedia.org/wiki/Yusuf_Ziya_%C4%B0rbe%C3%A7
Orhan Atalay, 1965, https://tr.wikipedia.org/wiki/Orhan_Atalay
Ensar Öğüt, 1946, https://tr.wikipedia.org/wiki/Ensar_%C3%96%C4%9F%C3%BCt
İsrafil Kışla, 1959, https://tr.wikipedia.org/wiki/%C4%B0srafil_K%C4%B1%C5%9Fla
Uğur Bayraktutan, 1965, https://tr.wikipedia.org/wiki/U%C4%9Fur_Bayraktutan
Mehmet Erdem, 1966, https://tr.wikipedia.org/wiki/Mehmet_Erdem_(Ayd%C4%B1n_do%C4%9Fumlu)
Semiha Öyüş, 1960, https://tr.wikipedia.org/wiki/Semiha_%C3%96y%C3%BC%C5%9F
Ali Gültekin Kılınç, 1959, https://tr.wikipedia.org/wiki/Ali_G%C3%BCltekin_K%C4%B1l%C4%B1n%C3%A7
Bülent Tezcan, 1965, https://tr.wikipedia.org/wiki/B%C3%BClent_Tezcan
Metin Lütfi Baydar,, 1960, https://tr.wikipedia.org/wiki/Metin_L%C3%BCtfi_Baydar
Osman Aydın, 1955, https://tr.wikipedia.org/wiki/Osman_Ayd%C4%B1n_(milletvekili)
Ali Uzunırmak, 1958, https://tr.wikipedia.org/wiki/Ali_Uzun%C4%B1rmak
Ahmet Edip Uğur, 1950, https://tr.wikipedia.org/wiki/Ahmet_Edip_U%C4%9Fur
Mehmet Cemal Öztaylan, 1954, https://tr.wikipedia.org/wiki/Mehmet_Cemal_%C3%96ztaylan
Tülay Babuşcu,, 1976, https://tr.wikipedia.org/wiki/T%C3%BClay_Babu%C5%9Fcu
Ali Aydınlıoğlu, 1958, https://tr.wikipedia.org/wiki/Ali_Ayd%C4%B1nl%C4%B1o%C4%9Flu
Namık Havutça, 1957, https://tr.wikipedia.org/wiki/Nam%C4%B1k_Havut%C3%A7a
Ayşe Nedret Akova, 1948, https://tr.wikipedia.org/wiki/Ay%C5%9Fe_Nedret_Akova
Haluk Ahmet Gümüş, 1960, https://tr.wikipedia.org/wiki/Haluk_Ahmet_G%C3%BCm%C3%BC%C5%9F
Ahmet Duran Bulut, 1953, https://tr.wikipedia.org/wiki/Ahmet_Duran_Bulut
Muhammet Rıza Yalçınkaya, 1955, https://tr.wikipedia.org/wiki/Muhammed_R%C4%B1za_Yal%C3%A7%C4%B1nkaya
Mehmet Şimşek,, 1967, https://tr.wikipedia.org/wiki/Mehmet_%C5%9Eim%C5%9Fek
Ziver Özdemir, 1965, https://tr.wikipedia.org/wiki/Ziver_%C3%96zdemir
Ayla Akat Ata, 1976, https://tr.wikipedia.org/wiki/Ayla_Akat_Ata
Bengi Yıldız, 1965, https://tr.wikipedia.org/wiki/Bengi_Y%C4%B1ld%C4%B1z
Bünyamin Özbek, 1973, https://tr.wikipedia.org/wiki/B%C3%BCnyamin_%C3%96zbek
Fahrettin Poyraz, 1968, https://tr.wikipedia.org/wiki/Fahrettin_Poyraz
Bahattin Şeker, 1956, https://tr.wikipedia.org/wiki/Bahattin_%C5%9Eeker
Cevdet Yılmaz, 1967, https://tr.wikipedia.org/wiki/Cevdet_Y%C4%B1lmaz
Eşref Taş, 1965, https://tr.wikipedia.org/wiki/E%C5%9Fref_Ta%C5%9F
İdris Baluken, 1976, https://tr.wikipedia.org/wiki/%C4%B0dris_Baluken
Vedat Demiröz, 1956, https://tr.wikipedia.org/wiki/Vedat_Demir%C3%B6z
Vahit Kiler, 1966, https://tr.wikipedia.org/wiki/Vahit_Kiler
Halkların Demokratik Partisi, 2011, https://tr.wikipedia.org/wiki/H%C3%BCsamettin_Zenderlio%C4%9Flu
Ali Ercoşkun, 1971, https://tr.wikipedia.org/wiki/Ali_Erco%C5%9Fkun
Fehmi Küpçü, 1974, https://tr.wikipedia.org/wiki/Fehmi_K%C3%BCp%C3%A7%C3%BC
Tanju Özcan, 1973, https://tr.wikipedia.org/wiki/Tanju_%C3%96zcan
Bayram Özçelik,, 1966, https://tr.wikipedia.org/wiki/Bayram_%C3%96z%C3%A7elik
Hasan Hami Yıldırım, 1963, https://tr.wikipedia.org/wiki/Hasan_Hami_Y%C4%B1ld%C4%B1r%C4%B1m
Ramazan Kerim Özkan, 1958, https://tr.wikipedia.org/wiki/Ramazan_Kerim_%C3%96zkan
Bülent Arınç, 1948, https://tr.wikipedia.org/wiki/B%C3%BClent_Ar%C4%B1n%C3%A7
Gelecek Partisi, 2019, https://tr.wikipedia.org/wiki/Mustafa_%C3%96zt%C3%BCrk_(Bursal%C4%B1)
Hüseyin Şahin,, 1968, https://tr.wikipedia.org/wiki/H%C3%BCseyin_%C5%9Eahin_(siyaset%C3%A7i,_1968)
Canan Candemir Çelik, 1974, https://tr.wikipedia.org/wiki/Canan_Candemir_%C3%87elik
İsmet Su, 1959, https://tr.wikipedia.org/wiki/%C4%B0smet_Su
Mustafa Kemal Şerbetçioğlu, 1969, https://tr.wikipedia.org/wiki/Mustafa_Kemal_%C5%9Eerbet%C3%A7io%C4%9Flu
Hakan Çavuşoğlu, 1972, https://tr.wikipedia.org/wiki/Hakan_%C3%87avu%C5%9Fo%C4%9Flu
İsmail Aydın, 1971, https://tr.wikipedia.org/wiki/%C4%B0smail_Ayd%C4%B1n_(%C4%B0znik_do%C4%9Fumlu)
Tülin Erkal Kara, 1969, https://tr.wikipedia.org/wiki/T%C3%BClin_Erkal_Kara
Bedrettin Yıldırım, 1951, https://tr.wikipedia.org/wiki/Bedrettin_Y%C4%B1ld%C4%B1r%C4%B1m
Önder Matlı, 1972, https://tr.wikipedia.org/wiki/%C3%96nder_Matl%C4%B1
Sena Kaleli, 1956, https://tr.wikipedia.org/wiki/Sena_Kaleli
Turhan Tayan, 1943, https://tr.wikipedia.org/wiki/Turhan_Tayan
İlhan Demiröz, 1951, https://tr.wikipedia.org/wiki/%C4%B0lhan_Demir%C3%B6z
Aykan Erdemir, 1974, https://tr.wikipedia.org/wiki/Aykan_Erdemir
Kemal Ekinci, 1948, https://tr.wikipedia.org/wiki/Kemal_Ekinci
İsmet Büyükataman, 1953, https://tr.wikipedia.org/wiki/%C4%B0smet_B%C3%BCy%C3%BCkataman
Necati Özensoy, 1961, https://tr.wikipedia.org/wiki/Necati_%C3%96zensoy
Mehmet Daniş, 1971, https://tr.wikipedia.org/wiki/Mehmet_Dani%C5%9F
Ali Sarıbaş, 1955, https://tr.wikipedia.org/wiki/Ali_Sar%C4%B1ba%C5%9F
Mustafa Serdar Soydan, 1967, https://tr.wikipedia.org/wiki/Mustafa_Serdar_Soydan
İdris Şahin, 1974, https://tr.wikipedia.org/wiki/%C4%B0dris_%C5%9Eahin
Hüseyin Filiz, 1958, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Filiz
Salim Uslu, 1955, https://tr.wikipedia.org/wiki/Salim_Uslu
Cahit Bağcı, 1964, https://tr.wikipedia.org/wiki/Cahit_Ba%C4%9Fc%C4%B1
Murat Yıldırım, 1958, https://tr.wikipedia.org/wiki/Murat_Y%C4%B1ld%C4%B1r%C4%B1m_(siyaset%C3%A7i)
Tufan Köse, 1967, https://tr.wikipedia.org/wiki/Tufan_K%C3%B6se
Nihat Zeybekci, 1961, https://tr.wikipedia.org/wiki/Nihat_Zeybekci
Bilal Uçar,, 1969, https://tr.wikipedia.org/wiki/Bilal_U%C3%A7ar
Nurcan Dalbudak, 1979, https://tr.wikipedia.org/wiki/Nurcan_Dalbudak
Adnan Keskin, 1942, https://tr.wikipedia.org/wiki/Adnan_Keskin
İlhan Cihaner, 1968, https://tr.wikipedia.org/wiki/%C4%B0lhan_Cihaner
Emin Haluk Ayhan, 1957, https://tr.wikipedia.org/wiki/Emin_Haluk_Ayhan
Mehmet Mehdi Eker, 1956, https://tr.wikipedia.org/wiki/Mehmet_Mehdi_Eker
Mehmet Galip Ensarioğlu, 1966, https://tr.wikipedia.org/wiki/Mehmet_Galip_Ensario%C4%9Flu
Mine Lök Beyaz,, 1973, https://tr.wikipedia.org/wiki/Mine_L%C3%B6k_Beyaz
Mehmet Süleyman Hamzaoğulları, 1975, https://tr.wikipedia.org/wiki/Mehmet_S%C3%BCleyman_Hamzao%C4%9Fullar%C4%B1
Cuma İçten, 1973, https://tr.wikipedia.org/wiki/Cuma_%C4%B0%C3%A7ten
Oya Eronat, 1962, https://tr.wikipedia.org/wiki/Oya_Eronat
Leyla Zana, 1961, https://tr.wikipedia.org/wiki/Leyla_Zana
Emine Ayna, 1968, https://tr.wikipedia.org/wiki/Emine_Ayna
Altan Tan, 1958, https://tr.wikipedia.org/wiki/Altan_Tan
Nursel Aydoğan, 1958, https://tr.wikipedia.org/wiki/Nursel_Aydo%C4%9Fan
Şerafettin Elçi, 1938, https://tr.wikipedia.org/wiki/%C5%9Eerafettin_El%C3%A7i
İbrahim Korkmaz, 1960, https://tr.wikipedia.org/wiki/%C4%B0brahim_Korkmaz
Fevai Arslan, 1966, https://tr.wikipedia.org/wiki/Fevai_Arslan
Osman Çakır, 1964, https://tr.wikipedia.org/wiki/Osman_%C3%87ak%C4%B1r_(Ak%C3%A7akoca_do%C4%9Fumlu)
Mehmet Müezzinoğlu,, 1955, https://tr.wikipedia.org/wiki/Mehmet_M%C3%BCezzino%C4%9Flu
Recep Gürkan[1], 1964, https://tr.wikipedia.org/wiki/Recep_G%C3%BCrkan
Kemal Değirmendereli, 1958, https://tr.wikipedia.org/wiki/Kemal_De%C4%9Firmendereli
Zülfü Demirbağ,, 1956, https://tr.wikipedia.org/wiki/Z%C3%BClf%C3%BC_Demirba%C4%9F
Şuay Alpay, 1960, https://tr.wikipedia.org/wiki/%C5%9Euay_Alpay
Faruk Septioğlu,, 1961, https://tr.wikipedia.org/wiki/Faruk_Septio%C4%9Flu
Sermin Balık, 1972, https://tr.wikipedia.org/wiki/Sermin_Bal%C4%B1k
Enver Erdem, 1963, https://tr.wikipedia.org/wiki/Enver_Erdem
Sebahattin Karakelle,, 1950, https://tr.wikipedia.org/wiki/Sebahattin_Karakelle
Muharrem Işık, 1966, https://tr.wikipedia.org/wiki/Muharrem_I%C5%9F%C4%B1k
Recep Akdağ, 1960, https://tr.wikipedia.org/wiki/Recep_Akda%C4%9F
Adnan Yılmaz, 1953, https://tr.wikipedia.org/wiki/Adnan_Y%C4%B1lmaz_(milletvekili)
Fazilet Dağcı Çığlık, 1973, https://tr.wikipedia.org/wiki/Fazilet_Da%C4%9Fc%C4%B1_%C3%87%C4%B1%C4%9Fl%C4%B1k
Cengiz Yavilioğlu, 1970, https://tr.wikipedia.org/wiki/Cengiz_Yavilio%C4%9Flu
Muhyettin Aksak, 1957, https://tr.wikipedia.org/wiki/Muhyettin_Aksak
Oktay Öztürk,, 1951, https://tr.wikipedia.org/wiki/Oktay_%C3%96zt%C3%BCrk
Nabi Avcı,, 1953, https://tr.wikipedia.org/wiki/Nabi_Avc%C4%B1
Salih Koca, 1972, https://tr.wikipedia.org/wiki/Salih_Koca
Ülker Can, 1963, https://tr.wikipedia.org/wiki/%C3%9Clker_Can
Bedii Süheyl Batum, 1955, https://tr.wikipedia.org/wiki/S%C3%BCheyl_Batum
Kazım Kurt, 1957, https://tr.wikipedia.org/wiki/Kaz%C4%B1m_Kurt
Ruhsar Demirel, 1963, https://tr.wikipedia.org/wiki/Ruhsar_Demirel
Hüseyin Çelik, 1959, https://tr.wikipedia.org/wiki/H%C3%BCseyin_%C3%87elik
Fatma Şahin[2], 1966, https://tr.wikipedia.org/wiki/Fatma_%C5%9Eahin
Şamil Tayyar, 1965, https://tr.wikipedia.org/wiki/%C5%9Eamil_Tayyar
Abdullah Nejat Koçer, 1963, https://tr.wikipedia.org/wiki/Abdullah_Nejat_Ko%C3%A7er
Mehmet Sarı, 1968, https://tr.wikipedia.org/wiki/Mehmet_Sar%C4%B1_(1968_do%C4%9Fumlu_siyaset%C3%A7i)
Mehmet Erdoğan, 1956, https://tr.wikipedia.org/wiki/Mehmet_Erdo%C4%9Fan_(Gaziantepli)
Derya Bakbak, 1972, https://tr.wikipedia.org/wiki/Derya_Bakbak
İbrahim Halil Mazıcıoğlu, 1960, https://tr.wikipedia.org/wiki/%C4%B0brahim_Halil_Maz%C4%B1c%C4%B1o%C4%9Flu
Ali Şahin, 1970, https://tr.wikipedia.org/wiki/Ali_%C5%9Eahin_(1970_do%C4%9Fumlu_siyaset%C3%A7i)
Ali Serindağ, 1951, https://tr.wikipedia.org/wiki/Ali_Serinda%C4%9F
Mehmet Şeker,, 1963, https://tr.wikipedia.org/wiki/Mehmet_%C5%9Eeker_(siyaset%C3%A7i,_1963)
Edip Semih Yalçın, 1958, https://tr.wikipedia.org/wiki/Edip_Semih_Yal%C3%A7%C4%B1n
Nurettin Canikli, 1960, https://tr.wikipedia.org/wiki/Nurettin_Canikli
Mehmet Geldi, 1960, https://tr.wikipedia.org/wiki/Mehmet_Geldi
Adem Tatlı, 1968, https://tr.wikipedia.org/wiki/Adem_Tatl%C4%B1
Selahattin Karaahmetoğlu, 1960, https://tr.wikipedia.org/wiki/Selahattin_Karaahmeto%C4%9Flu
Kemalettin Aydın,, 1962, https://tr.wikipedia.org/wiki/Kemalettin_Ayd%C4%B1n
Feramuz Üstün,, 1970, https://tr.wikipedia.org/wiki/Feramuz_%C3%9Cst%C3%BCn
Selahattin Demirtaş, 1973, https://tr.wikipedia.org/wiki/Selahattin_Demirta%C5%9F
Esat Canan,, 1950, https://tr.wikipedia.org/wiki/Esat_Canan
Sadullah Ergin, 1964, https://tr.wikipedia.org/wiki/Sadullah_Ergin
Orhan Karasayar,, 1965, https://tr.wikipedia.org/wiki/Orhan_Karasayar
Mehmet Öntürk, 1970, https://tr.wikipedia.org/wiki/Mehmet_%C3%96nt%C3%BCrk
Adem Yeşildal, 1976, https://tr.wikipedia.org/wiki/Adem_Ye%C5%9Fildal
Adalet ve Kalkınma Partisi, 2011, https://tr.wikipedia.org/wiki/Hac%C4%B1_Bayram_T%C3%BCrko%C4%9Flu
Mehmet Ali Ediboğlu, 1968, https://tr.wikipedia.org/wiki/Mehmet_Ali_Edibo%C4%9Flu
Mevlüt Dudu, 1966, https://tr.wikipedia.org/wiki/Mevl%C3%BCt_Dudu
Hasan Akgöl, 1966, https://tr.wikipedia.org/wiki/Hasan_Akg%C3%B6l
Adnan Şefik Çirkin, 1962, https://tr.wikipedia.org/wiki/Adnan_%C5%9Eefik_%C3%87irkin
Sinan Oğan, 1969, https://tr.wikipedia.org/wiki/Sinan_O%C4%9Fan
2023 - Van, 2023, https://tr.wikipedia.org/wiki/Pervin_Buldan
Süreyya Sadi Bilgiç, 1961, https://tr.wikipedia.org/wiki/S%C3%BCreyya_Sadi_Bilgi%C3%A7
Recep Özel, 1969, https://tr.wikipedia.org/wiki/Recep_%C3%96zel_(siyaset%C3%A7i,_1969)
Ali Haydar Öner, 1948, https://tr.wikipedia.org/wiki/Ali_Haydar_%C3%96ner
Süleyman Nevzat Korkmaz, 1965, https://tr.wikipedia.org/wiki/S%C3%BCleyman_Nevzat_Korkmaz
Failed to parse: https://tr.wikipedia.org/wiki/Recep_Tayyip_Erdo%C4%9Fan
Egemen Bağış, 1970, https://tr.wikipedia.org/wiki/Egemen_Ba%C4%9F%C4%B1%C5%9F
İdris Güllüce, 1950, https://tr.wikipedia.org/wiki/%C4%B0dris_G%C3%BCll%C3%BCce
Erol Kaya,, 1959, https://tr.wikipedia.org/wiki/Erol_Kaya
Mihrimah Belma Satır, 1961, https://tr.wikipedia.org/wiki/Mihrimah_Belma_Sat%C4%B1r
Muhammed Çetin, 1963, https://tr.wikipedia.org/wiki/Muhammed_%C3%87etin
İsmet Uçma, 1955, https://tr.wikipedia.org/wiki/%C4%B0smet_U%C3%A7ma
Gürsoy Erol,, 1956, https://tr.wikipedia.org/wiki/G%C3%BCrsoy_Erol
Alev Dedegil,, 1958, https://tr.wikipedia.org/wiki/Alev_Dedegil
Mehmet Domaç,, 1950, https://tr.wikipedia.org/wiki/Mehmet_Doma%C3%A7
Osman Boyraz, 1974, https://tr.wikipedia.org/wiki/Osman_Boyraz
Oktay Saral, 1967, https://tr.wikipedia.org/wiki/Oktay_Saral
Metin Külünk, 1960, https://tr.wikipedia.org/wiki/Metin_K%C3%BCl%C3%BCnk
Ahmet Berat Çonkar,, 1976, https://tr.wikipedia.org/wiki/Ahmet_Berat_%C3%87onkar
Muhammet Bilal Macit,, 1984, https://tr.wikipedia.org/wiki/Muhammet_Bilal_Macit
Gürsel Tekin, 1964, https://tr.wikipedia.org/wiki/G%C3%BCrsel_Tekin
Osman Taney Korutürk, 1944, https://tr.wikipedia.org/wiki/Osman_Taney_Korut%C3%BCrk
Mehmet Akif Hamzaçebi, 1953, https://tr.wikipedia.org/wiki/Mehmet_Akif_Hamza%C3%A7ebi
Şafak Pavey, 1976, https://tr.wikipedia.org/wiki/%C5%9Eafak_Pavey
İhsan Özkes, 1957, https://tr.wikipedia.org/wiki/%C4%B0hsan_%C3%96zkes
Fatma Nur Serter, 1948, https://tr.wikipedia.org/wiki/Fatma_Nur_Serter
Mahmut Tanal, 1961, https://tr.wikipedia.org/wiki/Mahmut_Tanal
Müslim Sarı, 1974, https://tr.wikipedia.org/wiki/M%C3%BCslim_Sar%C4%B1
Celal Dinçer,, 1957, https://tr.wikipedia.org/wiki/Celal_Din%C3%A7er
Kadir Gökmen Öğüt, 1959, https://tr.wikipedia.org/wiki/Kadir_G%C3%B6kmen_%C3%96%C4%9F%C3%BCt
P.1965-13, 1965, https://tr.wikipedia.org/wiki/Engin_Alan
Durmuşali Torlak, 1962, https://tr.wikipedia.org/wiki/Durmu%C5%9Fali_Torlak
Sebahat Tuncel, 1975, https://tr.wikipedia.org/wiki/Sebahat_Tuncel
Nimet Baş, 1965, https://tr.wikipedia.org/wiki/Nimet_Ba%C5%9F
Ayşe Nur Bahçekapılı, 1954, https://tr.wikipedia.org/wiki/Ay%C5%9Fe_Nur_Bah%C3%A7ekap%C4%B1l%C4%B1
Burhan Kuzu, 1955, https://tr.wikipedia.org/wiki/Burhan_Kuzu
Ekrem Erdem, 1948, https://tr.wikipedia.org/wiki/Ekrem_Erdem
Volkan Bozkır, 1950, https://tr.wikipedia.org/wiki/Volkan_Bozk%C4%B1r
2011 – İstanbul, 2011, https://tr.wikipedia.org/wiki/%C4%B0brahim_Yi%C4%9Fit
Ahmet Kutalmış Türkeş, 1978, https://tr.wikipedia.org/wiki/Ahmet_Kutalm%C4%B1%C5%9F_T%C3%BCrke%C5%9F
İsmail Safi, 1968, https://tr.wikipedia.org/wiki/%C4%B0smail_Safi
Prof. Türkan Dağoğlu, 1945, https://tr.wikipedia.org/wiki/T%C3%BCrkan_Da%C4%9Fo%C4%9Flu
HV.PLT.1975-3, 1975, https://tr.wikipedia.org/wiki/%C5%9Eirin_%C3%9Cnal
Sevim Savaşer, 1953, https://tr.wikipedia.org/wiki/Sevim_Sava%C5%9Fer
Hüseyin Bürge, 1955, https://tr.wikipedia.org/wiki/H%C3%BCseyin_B%C3%BCrge
Osman Aşkın Bak, 1966, https://tr.wikipedia.org/wiki/Osman_A%C5%9Fk%C4%B1n_Bak
Ahmet Haldun Ertürk, 1955, https://tr.wikipedia.org/wiki/Ahmet_Haldun_Ert%C3%BCrk
Ahmet Baha Öğütken,, 1961, https://tr.wikipedia.org/wiki/Ahmet_Baha_%C3%96%C4%9F%C3%BCtken
Failed to parse: https://tr.wikipedia.org/wiki/Kemal_K%C4%B1l%C4%B1%C3%A7daro%C4%9Flu
Bihlun Tamaylıgil, 1966, https://tr.wikipedia.org/wiki/Bihlun_Tamayl%C4%B1gil
Mustafa Sezgin Tanrıkulu, 1963, https://tr.wikipedia.org/wiki/Mustafa_Sezgin_Tanr%C4%B1kulu
Aydın Ağan Ayaydın, 1951, https://tr.wikipedia.org/wiki/Ayd%C4%B1n_A%C4%9Fan_Ayayd%C4%B1n
Sedef Küçük, 1958, https://tr.wikipedia.org/wiki/Sedef_K%C3%BC%C3%A7%C3%BCk
Merkez Parti, 2015, https://tr.wikipedia.org/wiki/Ercan_Cengiz
Merkez Parti, 2014, https://tr.wikipedia.org/wiki/Merkez_Parti
Haluk Eyidoğan, 1950, https://tr.wikipedia.org/wiki/Haluk_Eyido%C4%9Fan
Aykut Erdoğdu, 1972, https://tr.wikipedia.org/wiki/Aykut_Erdo%C4%9Fdu
Melda Onur,, 1964, https://tr.wikipedia.org/wiki/Melda_Onur
Murat Başesgioğlu, 1955, https://tr.wikipedia.org/wiki/Murat_Ba%C5%9Fesgio%C4%9Flu
Celal Adan,, 1951, https://tr.wikipedia.org/wiki/Celal_Adan
Sırrı Süreyya Önder, 1962, https://tr.wikipedia.org/wiki/S%C4%B1rr%C4%B1_S%C3%BCreyya_%C3%96nder
Ömer Dinçer, 1956, https://tr.wikipedia.org/wiki/%C3%96mer_Din%C3%A7er
Abdülkadir Aksu, 1944, https://tr.wikipedia.org/wiki/Abd%C3%BClkadir_Aksu
Ünal Kacır,, 1953, https://tr.wikipedia.org/wiki/%C3%9Cnal_Kac%C4%B1r
Halide İncekara, 1959, https://tr.wikipedia.org/wiki/Halide_%C4%B0ncekara
Mustafa Şentop, 1968, https://tr.wikipedia.org/wiki/Mustafa_%C5%9Eentop
Feyzullah Kıyıklık, 1949, https://tr.wikipedia.org/wiki/Feyzullah_K%C4%B1y%C4%B1kl%C4%B1k
Tülay Kaynarca, 1969, https://tr.wikipedia.org/wiki/T%C3%BClay_Kaynarca
Nureddin Nebati,, 1964, https://tr.wikipedia.org/wiki/Nureddin_Nebati
Mehmet Doğan Kubat, 1969, https://tr.wikipedia.org/wiki/Mehmet_Do%C4%9Fan_Kubat
Bülent Turan, 1975, https://tr.wikipedia.org/wiki/B%C3%BClent_Turan
Mehmet Muş, 1982, https://tr.wikipedia.org/wiki/Mehmet_Mu%C5%9F
Gülay Dalyan, 1962, https://tr.wikipedia.org/wiki/G%C3%BClay_Dalyan
Enver Yılmaz,, 1970, https://tr.wikipedia.org/wiki/Enver_Y%C4%B1lmaz_(siyaset%C3%A7i)
Harun Karaca, 1962, https://tr.wikipedia.org/wiki/Harun_Karaca
Erdoğan Toprak, 1959, https://tr.wikipedia.org/wiki/Erdo%C4%9Fan_Toprak
Umut Oran, 1962, https://tr.wikipedia.org/wiki/Umut_Oran
Süleyman Çelebi,, 1953, https://tr.wikipedia.org/wiki/S%C3%BCleyman_%C3%87elebi_(sendikac%C4%B1)
Osman Oktay Ekşi, 1932, https://tr.wikipedia.org/wiki/Oktay_Ek%C5%9Fi
Binnaz Toprak, 1942, https://tr.wikipedia.org/wiki/Binnaz_Toprak
Sabahat Akkiray[1][2], 1957, https://tr.wikipedia.org/wiki/Sabahat_Akkiraz
Faik Tunay, 1981, https://tr.wikipedia.org/wiki/Faik_Tunay
Ali Özgündüz, 1965, https://tr.wikipedia.org/wiki/Ali_%C3%96zg%C3%BCnd%C3%BCz
Ferit Mevlüt Aslanoğlu,, 1952, https://tr.wikipedia.org/wiki/Ferit_Mevl%C3%BCt_Aslano%C4%9Flu
Meral Akşener, 1956, https://tr.wikipedia.org/wiki/Meral_Ak%C5%9Fener
Atila Kaya,, 1957, https://tr.wikipedia.org/wiki/Atila_Kaya
Bağımsız Siyasetçi, 2018, https://tr.wikipedia.org/wiki/%C4%B0hsan_Barut%C3%A7u
Abdullah Levent Tüzel, 1961, https://tr.wikipedia.org/wiki/Abdullah_Levent_T%C3%BCzel
Sosyaldemokrat Halkçı Parti, 1986, https://tr.wikipedia.org/wiki/Ertu%C4%9Frul_G%C3%BCnay
Mehmet Sayım Tekelioğlu,, 1950, https://tr.wikipedia.org/wiki/Mehmet_Say%C4%B1m_Tekelio%C4%9Flu
İlhan İşbilen,, 1946, https://tr.wikipedia.org/wiki/%C4%B0lhan_%C4%B0%C5%9Fbilen
İlknur Denizli, 1964, https://tr.wikipedia.org/wiki/%C4%B0lknur_Denizli
Ali Aşlık, 1973, https://tr.wikipedia.org/wiki/Ali_A%C5%9Fl%C4%B1k
Rıfar Sait, 1966, https://tr.wikipedia.org/wiki/R%C4%B1fat_Sait
Şükran Güldal Mumcu[1][2], 1951, https://tr.wikipedia.org/wiki/G%C3%BCldal_Mumcu
Rıza Mahmut Türmen[1][2], 1941, https://tr.wikipedia.org/wiki/R%C4%B1za_T%C3%BCrmen
Prof.Dr.Oğuz Oyan, 1947, https://tr.wikipedia.org/wiki/O%C4%9Fuz_Oyan
Prof. Dr. Hülya Güven,, 1951, https://tr.wikipedia.org/wiki/H%C3%BClya_G%C3%BCven
Erdal Aksünger, 1968, https://tr.wikipedia.org/wiki/Erdal_Aks%C3%BCnger
Musa Çam, 1953, https://tr.wikipedia.org/wiki/Musa_%C3%87am
Oktay Vural, 1956, https://tr.wikipedia.org/wiki/Oktay_Vural
Binali Yıldırım, 1955, https://tr.wikipedia.org/wiki/Binali_Y%C4%B1ld%C4%B1r%C4%B1m
Erdal Kalkan,, 1946, https://tr.wikipedia.org/wiki/Erdal_Kalkan
Nesrin Ulema, 1971, https://tr.wikipedia.org/wiki/Nesrin_Ulema
Hamza Dağ, 1980, https://tr.wikipedia.org/wiki/Hamza_Da%C4%9F
Alaattin Yüksel,, 1950, https://tr.wikipedia.org/wiki/Alaattin_Y%C3%BCksel
Mustafa Ali Balbay, 1960, https://tr.wikipedia.org/wiki/Mustafa_Balbay
2011 – İzmir, 2011, https://tr.wikipedia.org/wiki/Mehmet_Ali_Susam
Rahmi Aşkın Türeli, 1963, https://tr.wikipedia.org/wiki/Rahmi_A%C5%9Fk%C4%B1n_T%C3%BCreli
Aytun Çıray, 1957, https://tr.wikipedia.org/wiki/Aytun_%C3%87%C4%B1ray
Birgül Ayman Güler, 1961, https://tr.wikipedia.org/wiki/Birg%C3%BCl_Ayman_G%C3%BCler
Mustafa Moroğlu, 1958, https://tr.wikipedia.org/wiki/Mustafa_Moro%C4%9Flu
Ahmet Kenan Tanrıkulu, 1958, https://tr.wikipedia.org/wiki/Ahmet_Kenan_Tanr%C4%B1kulu
Mehmet Sağlam,, 1938, https://tr.wikipedia.org/wiki/Mehmet_Sa%C4%9Flam
Nevzat Pakdil, 1950, https://tr.wikipedia.org/wiki/Nevzat_Pakdil
Mahir Ünal,, 1966, https://tr.wikipedia.org/wiki/Mahir_%C3%9Cnal
Yıldırım Mehmet Ramazanoğlu, 1956, https://tr.wikipedia.org/wiki/Y%C4%B1ld%C4%B1r%C4%B1m_Mehmet_Ramazano%C4%9Flu
Sevde Bayazıt Kaçar,, 1974, https://tr.wikipedia.org/wiki/Sevde_Bayaz%C4%B1t_Ka%C3%A7ar
Sıtkı Güvenç, 1961, https://tr.wikipedia.org/wiki/S%C4%B1tk%C4%B1_G%C3%BCven%C3%A7
Durdu Özbolat, 1962, https://tr.wikipedia.org/wiki/Durdu_%C3%96zbolat
Mesut Dedeoğlu, 1960, https://tr.wikipedia.org/wiki/Mesut_Dedeo%C4%9Flu
Mehmet Ali Şahin, 1950, https://tr.wikipedia.org/wiki/Mehmet_Ali_%C5%9Eahin
Osman Kahveci, 1950, https://tr.wikipedia.org/wiki/Osman_Kahveci
Lütfi Elvan, 1962, https://tr.wikipedia.org/wiki/L%C3%BCtfi_Elvan
Mevlüt Akgün, 1966, https://tr.wikipedia.org/wiki/Mevl%C3%BCt_Akg%C3%BCn
Ahmet Arslan, 1962, https://tr.wikipedia.org/wiki/Ahmet_Arslan_(Milletvekili)
Yunus Kılıç, 1965, https://tr.wikipedia.org/wiki/Yunus_K%C4%B1l%C4%B1%C3%A7
Halkların Demokratik Partisi, 2011, https://tr.wikipedia.org/wiki/M%C3%BClkiye_Birtane
Hakkı Köylü, 1948, https://tr.wikipedia.org/wiki/Hakk%C4%B1_K%C3%B6yl%C3%BC
Mustafa Gökhan Gülşen, 1970, https://tr.wikipedia.org/wiki/Mustafa_G%C3%B6khan_G%C3%BCl%C5%9Fen
Emin Çınar, 1967, https://tr.wikipedia.org/wiki/Emin_%C3%87%C4%B1nar
Taner Yıldız, 1962, https://tr.wikipedia.org/wiki/Taner_Y%C4%B1ld%C4%B1z
Sadık Yakut,, 1956, https://tr.wikipedia.org/wiki/Sad%C4%B1k_Yakut
Mustafa Elitaş, 1957, https://tr.wikipedia.org/wiki/Mustafa_Elita%C5%9F
Yaşar Karayel, 1950, https://tr.wikipedia.org/wiki/Ya%C5%9Far_Karayel
İYİ Parti, 2023, https://tr.wikipedia.org/wiki/Pelin_G%C3%BCnde%C5%9F_Bak%C4%B1r
Ahmet Öksüzkaya, 1960, https://tr.wikipedia.org/wiki/Ahmet_%C3%96ks%C3%BCzkaya
İsmail Tamer, 1956, https://tr.wikipedia.org/wiki/%C4%B0smail_Tamer
2011 – Kayseri, 2011, https://tr.wikipedia.org/wiki/Mehmet_%C5%9Eevki_Kulkulo%C4%9Flu
Yusuf Halaçoğlu, 1949, https://tr.wikipedia.org/wiki/Yusuf_Hala%C3%A7o%C4%9Flu
Fuat Karakuş,, 1973, https://tr.wikipedia.org/wiki/Fuat_Karaku%C5%9F
Ahmet Salih Dal,, 1971, https://tr.wikipedia.org/wiki/Ahmet_Salih_Dal
Şenol Gürşan,, 1964, https://tr.wikipedia.org/wiki/%C5%9Eenol_G%C3%BCr%C5%9Fan
Mehmet Siyam Kesimoğlu[1], 1960, https://tr.wikipedia.org/wiki/Mehmet_Siyam_Kesimo%C4%9Flu
Turgut Dibek, 1966, https://tr.wikipedia.org/wiki/Turgut_Dibek
Beşir Atalay, 1947, https://tr.wikipedia.org/wiki/Be%C5%9Fir_Atalay
Oğuz Kağan Köksal, 1948, https://tr.wikipedia.org/wiki/O%C4%9Fuz_Ka%C4%9Fan_K%C3%B6ksal
Ramazan Can, 1970, https://tr.wikipedia.org/wiki/Ramazan_Can
Abdullah Çalışkan, 1974, https://tr.wikipedia.org/wiki/Abdullah_%C3%87al%C4%B1%C5%9Fkan_(siyaset%C3%A7i,_1974)
Muzaffer Aslan, 1968, https://tr.wikipedia.org/wiki/Muzaffer_Aslan
Nihat Ergün, 1962, https://tr.wikipedia.org/wiki/Nihat_Erg%C3%BCn
Fikri Işık, 1965, https://tr.wikipedia.org/wiki/Fikri_I%C5%9F%C4%B1k
Azize Sibel Gönül, 1966, https://tr.wikipedia.org/wiki/Sibel_G%C3%B6n%C3%BCl
Muzaffer Baştopçu,, 1948, https://tr.wikipedia.org/wiki/Muzaffer_Ba%C5%9Ftop%C3%A7u
Zeki Aygün, 1953, https://tr.wikipedia.org/wiki/Zeki_Ayg%C3%BCn
İlyas Şeker, 1960, https://tr.wikipedia.org/wiki/%C4%B0lyas_%C5%9Eeker
Bünyamin Özbek, 1973, https://tr.wikipedia.org/wiki/Mehmet_Ali_Okur
Hurşit Güneş,, 1957, https://tr.wikipedia.org/wiki/Hur%C5%9Fit_G%C3%BCne%C5%9F
Mehmet Hilal Kaplan, 1959, https://tr.wikipedia.org/wiki/Mehmet_Hilal_Kaplan
Haydar Akar,, 1963, https://tr.wikipedia.org/wiki/Haydar_Akar
Lütfü Türkkan, 1959, https://tr.wikipedia.org/wiki/L%C3%BCtf%C3%BC_T%C3%BCrkkan
Millet İttifakı, 2022, https://tr.wikipedia.org/wiki/Ahmet_Davuto%C4%9Flu
2007 – Konya 
2011 – Konya, 2007, https://tr.wikipedia.org/wiki/Kerim_%C3%96zkul
İlhan Yerlikaya,, 1960, https://tr.wikipedia.org/wiki/%C4%B0lhan_Yerlikaya
Ayşe Türkmenoğlu, 1965, https://tr.wikipedia.org/wiki/Ay%C5%9Fe_T%C3%BCrkmeno%C4%9Flu
Hüseyin Üzülmez, 1957, https://tr.wikipedia.org/wiki/H%C3%BCseyin_%C3%9Cz%C3%BClmez
Mustafa Baloğlu, 1967, https://tr.wikipedia.org/wiki/Mustafa_Balo%C4%9Flu
2011 – Konya, 2011, https://tr.wikipedia.org/wiki/Mustafa_Kabakc%C4%B1
Av. Gülay Samancı, 1977, https://tr.wikipedia.org/wiki/G%C3%BClay_Samanc%C4%B1
Mustafa Akış, 1981, https://tr.wikipedia.org/wiki/Mustafa_Ak%C4%B1%C5%9F
2007 – Konya
2011 – Konya, 2007, https://tr.wikipedia.org/wiki/Harun_T%C3%BCfekci
Atilla Kart, 1954, https://tr.wikipedia.org/wiki/Atilla_Kart
Faruk Bal, 1950, https://tr.wikipedia.org/wiki/Faruk_Bal
Mustafa Kalaycı, 1960, https://tr.wikipedia.org/wiki/Mustafa_Kalayc%C4%B1
Soner Aksoy,, 1941, https://tr.wikipedia.org/wiki/Soner_Aksoy
Hasan Fehmi Kinay,, 1964, https://tr.wikipedia.org/wiki/Hasan_Fehmi_Kinay
İdris Bal, 1968, https://tr.wikipedia.org/wiki/%C4%B0dris_Bal
Demokratik Gelişim Partisi, 2014, https://tr.wikipedia.org/wiki/Demokratik_Geli%C5%9Fim_Partisi
Vural Kavuncu, 1966, https://tr.wikipedia.org/wiki/Vural_Kavuncu
Alim Işık[1], 1959, https://tr.wikipedia.org/wiki/Alim_I%C5%9F%C4%B1k
2011 – Malatya, 2011, https://tr.wikipedia.org/wiki/Mahmut_M%C3%BCcahit_F%C4%B1nd%C4%B1kl%C4%B1
Mustafa Şahin, 1956, https://tr.wikipedia.org/wiki/Mustafa_%C5%9Eahin_(siyaset%C3%A7i,_1956)
Öznur Çalık, 1965, https://tr.wikipedia.org/wiki/%C3%96znur_%C3%87al%C4%B1k
Hüseyin Cemal Akın, 1949, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Cemal_Ak%C4%B1n
Ömer Faruk Öz,, 1966, https://tr.wikipedia.org/wiki/%C3%96mer_Faruk_%C3%96z
Cumhuriyet Halk Partisi, 1995, https://tr.wikipedia.org/wiki/Veli_A%C4%9Fbaba
Hüseyin Tanrıverdi, 1956, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Tanr%C4%B1verdi
Recai Berber, 1959, https://tr.wikipedia.org/wiki/Recai_Berber
Uğur Aydemir, 1967, https://tr.wikipedia.org/wiki/U%C4%9Fur_Aydemir
Muzaffer Yurttaş, 1967, https://tr.wikipedia.org/wiki/Muzaffer_Yurtta%C5%9F
Büyük Birlik Partisi, 1993, https://tr.wikipedia.org/wiki/Sel%C3%A7uk_%C3%96zda%C4%9F
2011 – Manisa, 2011, https://tr.wikipedia.org/wiki/Hasan_%C3%96ren
Özgür Özel, 1974, https://tr.wikipedia.org/wiki/%C3%96zg%C3%BCr_%C3%96zel
Sakine Öz, 1964, https://tr.wikipedia.org/wiki/Sakine_%C3%96z
Sümer Oral, 1938, https://tr.wikipedia.org/wiki/S%C3%BCmer_Oral
Erkan Akçay, 1961, https://tr.wikipedia.org/wiki/Erkan_Ak%C3%A7ay
Muammer Güler, 1949, https://tr.wikipedia.org/wiki/Muammer_G%C3%BCler
Gönül Bekin Şahkulubey, 1970, https://tr.wikipedia.org/wiki/G%C3%B6n%C3%BCl_Bekin_%C5%9Eahkulubey
Abdurrahim Akdağ, 1960, https://tr.wikipedia.org/wiki/Abdurrahim_Akda%C4%9F
Ahmet Türk, 1942, https://tr.wikipedia.org/wiki/Ahmet_T%C3%BCrk
Gülser Yıldırım, 1963, https://tr.wikipedia.org/wiki/G%C3%BClser_Y%C4%B1ld%C4%B1r%C4%B1m
Erol Dora, 1964, https://tr.wikipedia.org/wiki/Erol_Dora
Mehmet Zafer Çağlayan, 1957, https://tr.wikipedia.org/wiki/Mehmet_Zafer_%C3%87a%C4%9Flayan
Ahmet Tevfik Uzun, 1968, https://tr.wikipedia.org/wiki/Ahmet_Tevfik_Uzun
Nebi Bozkurt, 1950, https://tr.wikipedia.org/wiki/Nebi_Bozkurt
Çiğdem Münevver Ökten, 1961, https://tr.wikipedia.org/wiki/%C3%87i%C4%9Fdem_M%C3%BCnevver_%C3%96kten
Aytuğ Atıcı, 1964, https://tr.wikipedia.org/wiki/Aytu%C4%9F_At%C4%B1c%C4%B1
Vahap Seçer[1], 1963, https://tr.wikipedia.org/wiki/Vahap_Se%C3%A7er
İsa Gök,, 1963, https://tr.wikipedia.org/wiki/%C4%B0sa_G%C3%B6k
2011 – Mersin, 2011, https://tr.wikipedia.org/wiki/Ali_R%C4%B1za_%C3%96zt%C3%BCrk
Mehmet Şandır, 1947, https://tr.wikipedia.org/wiki/Mehmet_%C5%9Eand%C4%B1r
Haziran 2015 – Mersin, 2015, https://tr.wikipedia.org/wiki/Ali_%C3%96z_(siyaset%C3%A7i)
Ertuğrul Kürkcü[1], 1948, https://tr.wikipedia.org/wiki/Ertu%C4%9Frul_K%C3%BCrk%C3%A7%C3%BC
Ali Boğa, 1949, https://tr.wikipedia.org/wiki/Ali_Bo%C4%9Fa
2011 – Muğla, 2011, https://tr.wikipedia.org/wiki/Y%C3%BCksel_%C3%96zden
Tolga Çandar[1][2], 1959, https://tr.wikipedia.org/wiki/Tolga_%C3%87andar
Nurettin Demir, 1953, https://tr.wikipedia.org/wiki/Nurettin_Demir
Ömer Süha Aldan, 1959, https://tr.wikipedia.org/wiki/%C3%96mer_S%C3%BCha_Aldan
Mehmet Erdoğan, 1964, https://tr.wikipedia.org/wiki/Mehmet_Erdo%C4%9Fan_(Mu%C4%9Flal%C4%B1)
Faruk Işık, 1963, https://tr.wikipedia.org/wiki/Faruk_I%C5%9F%C4%B1k
Muzaffer Çakar, 1965, https://tr.wikipedia.org/wiki/Muzaffer_%C3%87akar
Sırrı Sakık, 1957, https://tr.wikipedia.org/wiki/S%C4%B1rr%C4%B1_Sak%C4%B1k
Demir Çelik, 1959, https://tr.wikipedia.org/wiki/Demir_%C3%87elik
Ahmet Erdal Feralan, 1962, https://tr.wikipedia.org/wiki/Ahmet_Erdal_Feralan
Murat Göktürk, 1970, https://tr.wikipedia.org/wiki/Murat_G%C3%B6kt%C3%BCrk
Ebubekir Gizligider, 1979, https://tr.wikipedia.org/wiki/Ebubekir_Gizligider
Alpaslan Kavaklıoğlu, 1962, https://tr.wikipedia.org/wiki/Alpaslan_Kavakl%C4%B1o%C4%9Flu
Ömer Selvi, 1976, https://tr.wikipedia.org/wiki/%C3%96mer_Selvi
Doğan Şafak, 1966, https://tr.wikipedia.org/wiki/Do%C4%9Fan_%C5%9Eafak
İdris Naim Şahin, 1956, https://tr.wikipedia.org/wiki/%C4%B0dris_Naim_%C5%9Eahin
2011 – Ordu, 2011, https://tr.wikipedia.org/wiki/Mustafa_Hamarat
Fatih Han Ünal, 1968, https://tr.wikipedia.org/wiki/Fatih_Han_%C3%9Cnal
Harun Çakır, 1954, https://tr.wikipedia.org/wiki/Harun_%C3%87ak%C4%B1r
İdris Yıldız, 1950, https://tr.wikipedia.org/wiki/%C4%B0dris_Y%C4%B1ld%C4%B1z
Suat Önal, 1961, https://tr.wikipedia.org/wiki/Suat_%C3%96nal
Durdu Mehmet Kastal, 1969, https://tr.wikipedia.org/wiki/Durdu_Mehmet_Kastal
Devlet Bahçeli, 1948, https://tr.wikipedia.org/wiki/Devlet_Bah%C3%A7eli
Hasan Hüseyin Türkoğlu, 1968, https://tr.wikipedia.org/wiki/Hasan_H%C3%BCseyin_T%C3%BCrko%C4%9Flu
Hayati Yazıcı, 1952, https://tr.wikipedia.org/wiki/Hayati_Yaz%C4%B1c%C4%B1
Hasan Karal, 1973, https://tr.wikipedia.org/wiki/Hasan_Karal
Nusret Bayraktar, 1951, https://tr.wikipedia.org/wiki/Nusret_Bayraktar
Hasan Ali Çelik, 1959, https://tr.wikipedia.org/wiki/Hasan_Ali_%C3%87elik
Ayhan Sefer Üstün, 1970, https://tr.wikipedia.org/wiki/Ayhan_Sefer_%C3%9Cst%C3%BCn
Şaban Dişli, 1958, https://tr.wikipedia.org/wiki/%C5%9Eaban_Di%C5%9Fli
Ayşenur İslam, 1958, https://tr.wikipedia.org/wiki/Ay%C5%9Fenur_K%C3%BClahl%C4%B1o%C4%9Flu_%C4%B0slam
Ali İhsan Yavuz, 1969, https://tr.wikipedia.org/wiki/Ali_%C4%B0hsan_Yavuz
Engin Özkoç, 1962, https://tr.wikipedia.org/wiki/Engin_%C3%96zko%C3%A7
Münir Kutluata, 1946, https://tr.wikipedia.org/wiki/M%C3%BCnir_Kutluata
Mustafa Demir[1], 1961, https://tr.wikipedia.org/wiki/Mustafa_Demir_(siyaset%C3%A7i,_1961)
Suat Kılıç, 1972, https://tr.wikipedia.org/wiki/Suat_K%C4%B1l%C4%B1%C3%A7
Cemal Yılmaz Demir, 1960, https://tr.wikipedia.org/wiki/Cemal_Y%C4%B1lmaz_Demir
Prof.Dr. Tülay Bakır, 1947, https://tr.wikipedia.org/wiki/T%C3%BClay_Bak%C4%B1r
Akif Çağatay Kılıç, 1976, https://tr.wikipedia.org/wiki/Akif_%C3%87a%C4%9Fatay_K%C4%B1l%C4%B1%C3%A7
Ahmet Yeni, 1955, https://tr.wikipedia.org/wiki/Ahmet_Yeni
Ahmet Haluk Koç, 1954, https://tr.wikipedia.org/wiki/Ahmet_Haluk_Ko%C3%A7
Ahmet İhsan Kalkavan, 1953, https://tr.wikipedia.org/wiki/Ahmet_%C4%B0hsan_Kalkavan
Cemalettin Şimşek, 1954, https://tr.wikipedia.org/wiki/Cemalettin_%C5%9Eim%C5%9Fek
Afif Demirkıran, 1952, https://tr.wikipedia.org/wiki/Afif_Demirk%C4%B1ran
Osman Ören, 1966, https://tr.wikipedia.org/wiki/Osman_%C3%96ren
Gültan Kışanak, 1961, https://tr.wikipedia.org/wiki/G%C3%BCltan_K%C4%B1%C5%9Fanak
Mehmet Ersoy,, 1964, https://tr.wikipedia.org/wiki/Mehmet_Ersoy_(Sinop_do%C4%9Fumlu)
Engin Altay, 1963, https://tr.wikipedia.org/wiki/Engin_Altay
İsmet Yılmaz, 1961, https://tr.wikipedia.org/wiki/%C4%B0smet_Y%C4%B1lmaz
Mesude Nursuna Memecan, 1957, https://tr.wikipedia.org/wiki/Mesude_Nursuna_Memecan
Hilmi Bilgin[1], 1972, https://tr.wikipedia.org/wiki/Hilmi_Bilgin
Ali Turan, 1959, https://tr.wikipedia.org/wiki/Ali_Turan_(siyaset%C3%A7i)
Malik Ecder Özdemir,, 1954, https://tr.wikipedia.org/wiki/Malik_Ecder_%C3%96zdemir
Tevfik Ziyaeddin Akbulut,, 1949, https://tr.wikipedia.org/wiki/Tevfik_Ziyaeddin_Akbulut
Özlem Babadağ, 1971, https://tr.wikipedia.org/wiki/%C3%96zlem_Babada%C4%9F
Candan Yüceer, 1973, https://tr.wikipedia.org/wiki/Candan_Y%C3%BCceer
Emre Köprülü, 1981, https://tr.wikipedia.org/wiki/Emre_K%C3%B6pr%C3%BCl%C3%BC
Bülent Belen, 1956, https://tr.wikipedia.org/wiki/B%C3%BClent_Belen
Zeyid Aslan,, 1965, https://tr.wikipedia.org/wiki/Zeyid_Aslan
Şükrü Ayalan, 1963, https://tr.wikipedia.org/wiki/%C5%9E%C3%BCkr%C3%BC_Ayalan
2011 – Tokat, 2011, https://tr.wikipedia.org/wiki/Dilek_Y%C3%BCksel
Orhan Düzgün, 1967, https://tr.wikipedia.org/wiki/Orhan_D%C3%BCzg%C3%BCn_(siyaset%C3%A7i)
Reşat Doğru, 1955, https://tr.wikipedia.org/wiki/Re%C5%9Fat_Do%C4%9Fru
Faruk Nafız Özak, 1946, https://tr.wikipedia.org/wiki/Faruk_Nafiz_%C3%96zak
Erdoğan Bayraktar, 1948, https://tr.wikipedia.org/wiki/Erdo%C4%9Fan_Bayraktar
Aydın Bıyıklıoğlu, 1954, https://tr.wikipedia.org/wiki/Ayd%C4%B1n_B%C4%B1y%C4%B1kl%C4%B1o%C4%9Flu
Safiye Seymenoğlu, 1964, https://tr.wikipedia.org/wiki/Safiye_Seymeno%C4%9Flu
Mehmet Volkan Canalioğlu, 1950, https://tr.wikipedia.org/wiki/Volkan_Canalio%C4%9Flu
Koray Aydın, 1955, https://tr.wikipedia.org/wiki/Koray_Ayd%C4%B1n
Kamer Genç, 1940, https://tr.wikipedia.org/wiki/Kamer_Gen%C3%A7
Hüseyin Aygün, 1970, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Ayg%C3%BCn
Faruk Çelik, 1956, https://tr.wikipedia.org/wiki/Faruk_%C3%87elik
Seyit Eyyüpoğlu, 1942, https://tr.wikipedia.org/wiki/Seyit_Eyy%C3%BCpo%C4%9Flu
Mehmet Kasım Gülpınar, 1969, https://tr.wikipedia.org/wiki/Mehmet_Kas%C4%B1m_G%C3%BClp%C4%B1nar
Mehmet Akyürek, 1950, https://tr.wikipedia.org/wiki/Mehmet_Aky%C3%BCrek
Zeynep Karahan Uslu, 1969, https://tr.wikipedia.org/wiki/Zeynep_Karahan_Uslu
Halil Özcan, 1964, https://tr.wikipedia.org/wiki/Halil_%C3%96zcan
Mahmut Kaçar,, 1975, https://tr.wikipedia.org/wiki/Mahmut_Ka%C3%A7ar
Abdulkadir Emin Önen, 1976, https://tr.wikipedia.org/wiki/Abdulkadir_Emin_%C3%96nen
Yahya Akman, 1966, https://tr.wikipedia.org/wiki/Yahya_Akman
Abdulkerim Gök, 1970, https://tr.wikipedia.org/wiki/Abdulkerim_G%C3%B6k
İbrahim Ayhan, 1968, https://tr.wikipedia.org/wiki/%C4%B0brahim_Ayhan
İbrahim Binici,, 1962, https://tr.wikipedia.org/wiki/%C4%B0brahim_Binici
Mehmet Emin Dindar,, 1959, https://tr.wikipedia.org/wiki/Mehmet_Emin_Dindar
Hasip Kaplan, 1954, https://tr.wikipedia.org/wiki/Hasip_Kaplan
Faysal Sarıyıldız, 1975, https://tr.wikipedia.org/wiki/Faysal_Sar%C4%B1y%C4%B1ld%C4%B1z
İsmail Güneş, 1967, https://tr.wikipedia.org/wiki/%C4%B0smail_G%C3%BCne%C5%9F_(siyaset%C3%A7i)
Mehmet Altay, 1972, https://tr.wikipedia.org/wiki/Mehmet_Altay
Dilek Akagün Yılmaz, 1963, https://tr.wikipedia.org/wiki/Dilek_Akag%C3%BCn_Y%C4%B1lmaz
Burhan Kayatürk,, 1970, https://tr.wikipedia.org/wiki/Burhan_Kayat%C3%BCrk
Fatih Çiftci, 1971, https://tr.wikipedia.org/wiki/Fatih_%C3%87iftci
Mustafa Bilici,, 1969, https://tr.wikipedia.org/wiki/Mustafa_Bilici
Gülşen Orhan, 1970, https://tr.wikipedia.org/wiki/G%C3%BCl%C5%9Fen_Orhan
Halkların Demokratik Partisi, 2011, https://tr.wikipedia.org/wiki/Kemal_Akta%C5%9F
Aysel Tuğluk, 1965, https://tr.wikipedia.org/wiki/Aysel_Tu%C4%9Fluk
Özdal Üçer, 1974, https://tr.wikipedia.org/wiki/%C3%96zdal_%C3%9C%C3%A7er
Halkların Demokratik Partisi, 2011, https://tr.wikipedia.org/wiki/Nazmi_G%C3%BCr
Temel Coşkun,, 1956, https://tr.wikipedia.org/wiki/Temel_Co%C5%9Fkun
Muharrem İnce, 1964, https://tr.wikipedia.org/wiki/Muharrem_%C4%B0nce
Ertuğrul Soysal, 1963, https://tr.wikipedia.org/wiki/Ertu%C4%9Frul_Soysal
Sadir Durmaz, 1963, https://tr.wikipedia.org/wiki/Sadir_Durmaz
Ercan Candan, 1967, https://tr.wikipedia.org/wiki/Ercan_Candan
Özcan Ulupınar, 1970, https://tr.wikipedia.org/wiki/%C3%96zcan_Ulup%C4%B1nar
Mehmet Haberal[2][1], 1944, https://tr.wikipedia.org/wiki/Mehmet_Haberal
Ali İhsan Köktürk, 1963, https://tr.wikipedia.org/wiki/Ali_%C4%B0hsan_K%C3%B6kt%C3%BCrk
Failed links:
https://tr.wikipedia.org/wiki/Recep_Tayyip_Erdo%C4%9Fan
https://tr.wikipedia.org/wiki/Kemal_K%C4%B1l%C4%B1%C3%A7daro%C4%9Flu


In [ ]:
23.dönem
Dengir Mir Mehmet Fırat, 1943, https://tr.wikipedia.org/wiki/Dengir_Mir_Mehmet_F%C4%B1rat
Ömer Çelik, 1968, https://tr.wikipedia.org/wiki/%C3%96mer_%C3%87elik
Necdet Ünüvar,, 1960, https://tr.wikipedia.org/wiki/Necdet_%C3%9Cn%C3%BCvar
Fatoş Gürkan,, 1966, https://tr.wikipedia.org/wiki/Fato%C5%9F_G%C3%BCrkan
Vahit Kirişci, 1960, https://tr.wikipedia.org/wiki/Vahit_Kiri%C5%9F%C3%A7i
Ali Küçükaydın,, 1948, https://tr.wikipedia.org/wiki/Ali_K%C3%BC%C3%A7%C3%BCkayd%C4%B1n
Nevingaye Erbatur[1], 1950, https://tr.wikipedia.org/wiki/Nevin_Gaye_Erbatur
Hulusi Güvel, 1948, https://tr.wikipedia.org/wiki/Hulusi_G%C3%BCvel
Tacidar Seyhan,, 1959, https://tr.wikipedia.org/wiki/Tacidar_Seyhan
Mustafa Vural,, 1953, https://tr.wikipedia.org/wiki/Mustafa_Vural
Yılmaz Tankut, 1959, https://tr.wikipedia.org/wiki/Y%C4%B1lmaz_Tankut
Kürşat Atılgan, 1956, https://tr.wikipedia.org/wiki/K%C3%BCr%C5%9Fat_At%C4%B1lgan
Muharrem Varlı, 1969, https://tr.wikipedia.org/wiki/Muharrem_Varl%C4%B1
Recai Yıldırım,, 1950, https://tr.wikipedia.org/wiki/Recai_Y%C4%B1ld%C4%B1r%C4%B1m
Ahmet Aydın,, 1971, https://tr.wikipedia.org/wiki/Ahmet_Ayd%C4%B1n_(Ad%C4%B1yamanl%C4%B1)
Mehmet Erdoğan, 1966, https://tr.wikipedia.org/wiki/Mehmet_Erdo%C4%9Fan_(Ad%C4%B1yamanl%C4%B1)
Şevket Gürsoy,, 1953, https://tr.wikipedia.org/wiki/%C5%9Eevket_G%C3%BCrsoy
Fehmi Hüsrev Kutlu,, 1961, https://tr.wikipedia.org/wiki/Fehmi_H%C3%BCsrev_Kutlu
Şevket Köse, 1953, https://tr.wikipedia.org/wiki/%C5%9Eevket_K%C3%B6se
Sait Açba, 1950, https://tr.wikipedia.org/wiki/Sait_A%C3%A7ba
Zekeriya Aslan, 1969, https://tr.wikipedia.org/wiki/Zekeriya_Aslan
Halil Aydoğan, 1950, https://tr.wikipedia.org/wiki/Halil_Aydo%C4%9Fan
Veysel Eroğlu, 1948, https://tr.wikipedia.org/wiki/Veysel_Ero%C4%9Flu
Ahmet Koca,, 1963, https://tr.wikipedia.org/wiki/Ahmet_Koca
Halil Ünlütepe,, 1949, https://tr.wikipedia.org/wiki/Halil_%C3%9Cnl%C3%BCtepe
Abdülkadir Akcan, 1952, https://tr.wikipedia.org/wiki/Abd%C3%BClkadir_Akcan
Mehmet Hanifi Alır, 1959, https://tr.wikipedia.org/wiki/Mehmet_Hanifi_Al%C4%B1r
Abdulkerim Aydemir, 1954, https://tr.wikipedia.org/wiki/Abd%C3%BClkerim_Aydemir
Yaşar Eryılmaz, 1944, https://tr.wikipedia.org/wiki/Ya%C5%9Far_Ery%C4%B1lmaz
Cemal Kaya, 1962, https://tr.wikipedia.org/wiki/Cemal_Kaya
Fatma Salman, 1970, https://tr.wikipedia.org/wiki/Fatma_Kotan
Ruhi Açıkgöz, 1963, https://tr.wikipedia.org/wiki/Ruhi_A%C3%A7%C4%B1kg%C3%B6z
Ali Rıza Alaboyun, 1957, https://tr.wikipedia.org/wiki/Ali_R%C4%B1za_Alaboyun
İlknur İnceöz, 1973, https://tr.wikipedia.org/wiki/%C4%B0lknur_%C4%B0nce%C3%B6z
Osman Ertuğrul,, 1956, https://tr.wikipedia.org/wiki/Osman_Ertu%C4%9Frul
Akif Gülle, 1958, https://tr.wikipedia.org/wiki/Akif_G%C3%BClle
Avni Erdemir, 1964, https://tr.wikipedia.org/wiki/Avni_Erdemir
Hüseyin Ünsal, 1957, https://tr.wikipedia.org/wiki/H%C3%BCseyin_%C3%9Cnsal
Cemil Çiçek, 1946, https://tr.wikipedia.org/wiki/Cemil_%C3%87i%C3%A7ek
Ali Babacan, 1967, https://tr.wikipedia.org/wiki/Ali_Babacan
Zeynep Dağı, 1964, https://tr.wikipedia.org/wiki/Zeynep_Da%C4%9F%C4%B1
Mehmet Zekai Özcan, 1950, https://tr.wikipedia.org/wiki/Mehmet_Zekai_%C3%96zcan
Dr. Bülent Gedikli,, 1964, https://tr.wikipedia.org/wiki/B%C3%BClent_Gedikli
Burhan Kayatürk,, 1970, https://tr.wikipedia.org/wiki/Burhan_Kayat%C3%BCrk
Faruk Koca,, 1964, https://tr.wikipedia.org/wiki/Faruk_Koca
Nazmi Haluk Özdalga[1], 1948, https://tr.wikipedia.org/wiki/Haluk_%C3%96zdalga
Önder Sav, 1937, https://tr.wikipedia.org/wiki/%C3%96nder_Sav
Hakkı Suha Okay, 1953, https://tr.wikipedia.org/wiki/Hakk%C4%B1_Suha_Okay
Mehmet Emrehan Halıcı, 1956, https://tr.wikipedia.org/wiki/Mehmet_Emrehan_Hal%C4%B1c%C4%B1
Nesrin Baytok, 1960, https://tr.wikipedia.org/wiki/Nesrin_Baytok
Eşref Erdem, 1941, https://tr.wikipedia.org/wiki/E%C5%9Fref_Erdem
Yıldırım Tuğrul Türkeş[1][2], 1954, https://tr.wikipedia.org/wiki/Y%C4%B1ld%C4%B1r%C4%B1m_Tu%C4%9Frul_T%C3%BCrke%C5%9F
Bekir Aksoy, 1950, https://tr.wikipedia.org/wiki/Bekir_Aksoy_(siyaset%C3%A7i)
Beşir Atalay, 1947, https://tr.wikipedia.org/wiki/Be%C5%9Fir_Atalay
Salih Kapusuz, 1954, https://tr.wikipedia.org/wiki/Salih_Kapusuz
Haluk İpek, 1963, https://tr.wikipedia.org/wiki/Haluk_%C4%B0pek
Mustafa Sait Yazıcıoğlu, 1949, https://tr.wikipedia.org/wiki/Mustafa_Said_Yaz%C4%B1c%C4%B1o%C4%9Flu
Mehmet Zafer Çağlayan, 1957, https://tr.wikipedia.org/wiki/Mehmet_Zafer_%C3%87a%C4%9Flayan
Aşkın Asan, 1965, https://tr.wikipedia.org/wiki/A%C5%9Fk%C4%B1n_Asan
Ahmet İyimaya, 1950, https://tr.wikipedia.org/wiki/Ahmet_%C4%B0yimaya
Yılmaz Ateş, 1953, https://tr.wikipedia.org/wiki/Y%C4%B1lmaz_Ate%C5%9F
Mücahit Pehlivan,, 1963, https://tr.wikipedia.org/wiki/M%C3%BCcahit_Pehlivan
Tekin Bingöl, 1955, https://tr.wikipedia.org/wiki/Tekin_Bing%C3%B6l
Zekeriya Akıncı, 1957, https://tr.wikipedia.org/wiki/Zekeriya_Ak%C4%B1nc%C4%B1
Mustafa Cihan Paçacı, 1949, https://tr.wikipedia.org/wiki/Mustafa_Cihan_Pa%C3%A7ac%C4%B1
Ahmet Deniz Bölükbaşı, 1949, https://tr.wikipedia.org/wiki/Ahmet_Deniz_B%C3%B6l%C3%BCkba%C5%9F%C4%B1
Mehmet Ali Şahin, 1950, https://tr.wikipedia.org/wiki/Mehmet_Ali_%C5%9Eahin
Mevlüt Çavuşoğlu, 1968, https://tr.wikipedia.org/wiki/Mevl%C3%BCt_%C3%87avu%C5%9Fo%C4%9Flu
Abdurrahman Arıcı, 1951, https://tr.wikipedia.org/wiki/Abdurrahman_Ar%C4%B1c%C4%B1
Sadık Badak, 1955, https://tr.wikipedia.org/wiki/Sad%C4%B1k_Badak
Yusuf Ziya İrbeç, 1959, https://tr.wikipedia.org/wiki/Yusuf_Ziya_%C4%B0rbe%C3%A7
Deniz Baykal, 1938, https://tr.wikipedia.org/wiki/Deniz_Baykal
Osman Kaptan, 1946, https://tr.wikipedia.org/wiki/Osman_Kaptan
Hüsnü Çöllü, 1960, https://tr.wikipedia.org/wiki/H%C3%BCsn%C3%BC_%C3%87%C3%B6ll%C3%BC
Atila Emek,, 1947, https://tr.wikipedia.org/wiki/Atila_Emek
Tayfur Süner, 1943, https://tr.wikipedia.org/wiki/Tayfur_S%C3%BCner
Tunca Toskay, 1939, https://tr.wikipedia.org/wiki/Tunca_Toskay
Mehmet Günal, 1964, https://tr.wikipedia.org/wiki/Mehmet_G%C3%BCnal
Hüseyin Yıldız, 1959, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Y%C4%B1ld%C4%B1z_(siyaset%C3%A7i,_1959)
Saffet Kaya,, 1962, https://tr.wikipedia.org/wiki/Saffet_Kaya
Ensar Öğüt, 1946, https://tr.wikipedia.org/wiki/Ensar_%C3%96%C4%9F%C3%BCt
Ertekin Çolak, 1964, https://tr.wikipedia.org/wiki/Ertekin_%C3%87olak
Metin Arifağaoğlu,, 1946, https://tr.wikipedia.org/wiki/Metin_Arifa%C4%9Fao%C4%9Flu
Atilla Koç, 1946, https://tr.wikipedia.org/wiki/Atilla_Ko%C3%A7
Mehmet Erdem, 1966, https://tr.wikipedia.org/wiki/Mehmet_Erdem_(Ayd%C4%B1n_do%C4%9Fumlu)
Ahmet Ertürk, 1953, https://tr.wikipedia.org/wiki/Ahmet_Ert%C3%BCrk_(siyaset%C3%A7i,_1953)
Özlem Çerçioğlu, 1968, https://tr.wikipedia.org/wiki/%C3%96zlem_%C3%87er%C3%A7io%C4%9Flu
Mehmet Fatih Atay, 1957, https://tr.wikipedia.org/wiki/Fatih_Atay
Ertuğrul Kumcuoğlu, 1938, https://tr.wikipedia.org/wiki/Ertu%C4%9Frul_Kumcuo%C4%9Flu
Recep Taner, 1957, https://tr.wikipedia.org/wiki/Recep_Taner
Ali Uzunırmak, 1958, https://tr.wikipedia.org/wiki/Ali_Uzun%C4%B1rmak
Ahmet Edip Uğur, 1950, https://tr.wikipedia.org/wiki/Ahmet_Edip_U%C4%9Fur
İsmail Özgün, 1956, https://tr.wikipedia.org/wiki/%C4%B0smail_%C3%96zg%C3%BCn
Mehmet Cemal Öztaylan, 1954, https://tr.wikipedia.org/wiki/Mehmet_Cemal_%C3%96ztaylan
2007 – Balıkesir, 2007, https://tr.wikipedia.org/wiki/Ali_Osman_Sali
Ayşe Akbaş, 1971, https://tr.wikipedia.org/wiki/Ay%C5%9Fe_Akba%C5%9F
Hüseyin Pazarcı,, 1944, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Pazarc%C4%B1
Ergün Aydoğan,, 1964, https://tr.wikipedia.org/wiki/Erg%C3%BCn_Aydo%C4%9Fan
Ahmet Duran Bulut, 1953, https://tr.wikipedia.org/wiki/Ahmet_Duran_Bulut
Muhammet Rıza Yalçınkaya, 1955, https://tr.wikipedia.org/wiki/Muhammed_R%C4%B1za_Yal%C3%A7%C4%B1nkaya
Ahmet İnal, 1954, https://tr.wikipedia.org/wiki/Ahmet_%C4%B0nal
Mehmet Emin Ekmen, 1975, https://tr.wikipedia.org/wiki/Mehmet_Emin_Ekmen
Ayla Akat Ata, 1976, https://tr.wikipedia.org/wiki/Ayla_Akat_Ata
Bengi Yıldız, 1965, https://tr.wikipedia.org/wiki/Bengi_Y%C4%B1ld%C4%B1z
Fetani Battal, 1963, https://tr.wikipedia.org/wiki/Fetani_Battal
Ülkü Gökalp Güney, 1935, https://tr.wikipedia.org/wiki/%C3%9Clk%C3%BC_G%C3%B6kalp_G%C3%BCney
Fahrettin Poyraz, 1968, https://tr.wikipedia.org/wiki/Fahrettin_Poyraz
Yaşar Tüzün,, 1966, https://tr.wikipedia.org/wiki/Ya%C5%9Far_T%C3%BCz%C3%BCn
Kazım Ataoğlu, 1952, https://tr.wikipedia.org/wiki/Kaz%C4%B1m_Atao%C4%9Flu
Yusuf Coşkun,, 1965, https://tr.wikipedia.org/wiki/Yusuf_Co%C5%9Fkun
Cevdet Yılmaz, 1967, https://tr.wikipedia.org/wiki/Cevdet_Y%C4%B1lmaz
Zeki Ergezen, 1949, https://tr.wikipedia.org/wiki/Zeki_Ergezen
Vahit Kiler, 1966, https://tr.wikipedia.org/wiki/Vahit_Kiler
Cemal Taşar, 1963, https://tr.wikipedia.org/wiki/Cemal_Ta%C5%9Far
Mehmet Nezir Karabaş, 1960, https://tr.wikipedia.org/wiki/Mehmet_Nezir_Karaba%C5%9F
Yüksel Coşkunyürek, 1965, https://tr.wikipedia.org/wiki/Y%C3%BCksel_Co%C5%9Fkuny%C3%BCrek
Fatih Metin, 1973, https://tr.wikipedia.org/wiki/Fatih_Metin
2007 – Bolu, 2007, https://tr.wikipedia.org/wiki/Metin_Y%C4%B1lmaz_(1968_do%C4%9Fumlu_siyaset%C3%A7i)
Mehmet Alp, 1953, https://tr.wikipedia.org/wiki/Mehmet_Alp_(Bucak_do%C4%9Fumlu)
Bayram Özçelik,, 1966, https://tr.wikipedia.org/wiki/Bayram_%C3%96z%C3%A7elik
Ramazan Kerim Özkan, 1958, https://tr.wikipedia.org/wiki/Ramazan_Kerim_%C3%96zkan
Faruk Çelik, 1956, https://tr.wikipedia.org/wiki/Faruk_%C3%87elik
Hayrettin Çakmak, 1957, https://tr.wikipedia.org/wiki/Hayrettin_%C3%87akmak
Canan Candemir Çelik, 1974, https://tr.wikipedia.org/wiki/Canan_Candemir_%C3%87elik
2007 – Bursa, 2007, https://tr.wikipedia.org/wiki/Sedat_K%C4%B1z%C4%B1lc%C4%B1kl%C4%B1
Mehmet Tunçak, 1969, https://tr.wikipedia.org/wiki/Mehmet_Tun%C3%A7ak
2007 – Bursa, 2007, https://tr.wikipedia.org/wiki/Mehmet_Emin_Tutan
Ali Koyuncu, 1968, https://tr.wikipedia.org/wiki/Ali_Koyuncu
Ali Kul, 1955, https://tr.wikipedia.org/wiki/Ali_Kul
Mehmet Ocaktan, 1955, https://tr.wikipedia.org/wiki/Mehmet_Ocaktan
Onur Başaran Öymen[1], 1940, https://tr.wikipedia.org/wiki/Onur_%C3%96ymen
Kemal Demirel,, 1955, https://tr.wikipedia.org/wiki/Kemal_Demirel
Abdullah Özer, 1950, https://tr.wikipedia.org/wiki/Abdullah_%C3%96zer
İsmet Büyükataman, 1953, https://tr.wikipedia.org/wiki/%C4%B0smet_B%C3%BCy%C3%BCkataman
Hamza Hamit Homriş[1], 1944, https://tr.wikipedia.org/wiki/Hamza_Hamit_Homri%C5%9F
Necati Özensoy, 1961, https://tr.wikipedia.org/wiki/Necati_%C3%96zensoy
Mehmet Daniş, 1971, https://tr.wikipedia.org/wiki/Mehmet_Dani%C5%9F
Müjdat Kuşku, 1958, https://tr.wikipedia.org/wiki/M%C3%BCjdat_Ku%C5%9Fku
Ahmet Küçük,, 1958, https://tr.wikipedia.org/wiki/Ahmet_K%C3%BC%C3%A7%C3%BCk
Mustafa Kemal Cengiz, 1964, https://tr.wikipedia.org/wiki/Mustafa_Kemal_Cengiz
Nurettin Akman, 1951, https://tr.wikipedia.org/wiki/Nurettin_Akman
Suat Kınıklıoğlu, 1965, https://tr.wikipedia.org/wiki/Suat_K%C4%B1n%C4%B1kl%C4%B1o%C4%9Flu
Ahmet Bukan,, 1952, https://tr.wikipedia.org/wiki/Ahmet_Bukan
Ahmet Aydoğmuş, 1970, https://tr.wikipedia.org/wiki/Ahmet_Aydo%C4%9Fmu%C5%9F
Cahit Bağcı, 1964, https://tr.wikipedia.org/wiki/Cahit_Ba%C4%9Fc%C4%B1
Agah Kafkas, 1956, https://tr.wikipedia.org/wiki/Agah_Kafkas
Murat Yıldırım, 1958, https://tr.wikipedia.org/wiki/Murat_Y%C4%B1ld%C4%B1r%C4%B1m_(siyaset%C3%A7i)
Derviş Günday,, 1941, https://tr.wikipedia.org/wiki/Dervi%C5%9F_G%C3%BCnday
Mithat Ekici, 1955, https://tr.wikipedia.org/wiki/Mithat_Ekici
2007 – Denizli, 2007, https://tr.wikipedia.org/wiki/Mehmet_Salih_Erdo%C4%9Fan
Selma Aliye Kavaf, 1962, https://tr.wikipedia.org/wiki/Selma_Aliye_Kavaf
Mehmet Yüksel, 1955, https://tr.wikipedia.org/wiki/Mehmet_Y%C3%BCksel_(siyaset%C3%A7i)
2007 – Denizli, 2007, https://tr.wikipedia.org/wiki/Hasan_Er%C3%A7elebi
Ali Rıza Ertemür, 1962, https://tr.wikipedia.org/wiki/Ali_R%C4%B1za_Ertem%C3%BCr
Emin Haluk Ayhan, 1957, https://tr.wikipedia.org/wiki/Emin_Haluk_Ayhan
Mehmet İhsan Arslan, 1948, https://tr.wikipedia.org/wiki/Mehmet_%C4%B0hsan_Arslan
Kutbettin Arzu, 1955, https://tr.wikipedia.org/wiki/Kutbettin_Arzu
Osman Aslan, 1954, https://tr.wikipedia.org/wiki/Osman_Aslan
Mehmet Mehdi Eker, 1956, https://tr.wikipedia.org/wiki/Mehmet_Mehdi_Eker
Abdurrahman Kurt, 1968, https://tr.wikipedia.org/wiki/Abdurrahman_Kurt
Ali İhsan Merdanoğlu, 1963, https://tr.wikipedia.org/wiki/Ali_%C4%B0hsan_Merdano%C4%9Flu
Gültan Kışanak, 1961, https://tr.wikipedia.org/wiki/G%C3%BCltan_K%C4%B1%C5%9Fanak
Selahattin Demirtaş, 1973, https://tr.wikipedia.org/wiki/Selahattin_Demirta%C5%9F
Aysel Tuğluk, 1965, https://tr.wikipedia.org/wiki/Aysel_Tu%C4%9Fluk
Akın Birdal, 1948, https://tr.wikipedia.org/wiki/Ak%C4%B1n_Birdal
Yaşar Yakış, 1938, https://tr.wikipedia.org/wiki/Ya%C5%9Far_Yak%C4%B1%C5%9F
Metin Kaşıkoğlu, 1966, https://tr.wikipedia.org/wiki/Metin_Ka%C5%9F%C4%B1ko%C4%9Flu
Celal Erbay, 1952, https://tr.wikipedia.org/wiki/Celal_Erbay
Necdet Budak, 1964, https://tr.wikipedia.org/wiki/Necdet_Budak
Rasim Çakır, 1957, https://tr.wikipedia.org/wiki/Rasim_%C3%87ak%C4%B1r
Bilgin Paçarız, 1955, https://tr.wikipedia.org/wiki/Bilgin_Pa%C3%A7ar%C4%B1z
Cemaleddin Uslu, 1956, https://tr.wikipedia.org/wiki/Cemaleddin_Uslu
Mehmet Necati Çetinkaya, 1943, https://tr.wikipedia.org/wiki/Mehmet_Necati_%C3%87etinkaya
Feyzi İşbaşaran, 1956, https://tr.wikipedia.org/wiki/Feyzi_%C4%B0%C5%9Fba%C5%9Faran
Tahir Öztürk, 1950, https://tr.wikipedia.org/wiki/Tahir_%C3%96zt%C3%BCrk
Faruk Septioğlu,, 1961, https://tr.wikipedia.org/wiki/Faruk_Septio%C4%9Flu
Hamza Yanılmaz, 1963, https://tr.wikipedia.org/wiki/Hamza_Yan%C4%B1lmaz
Binali Yıldırım, 1955, https://tr.wikipedia.org/wiki/Binali_Y%C4%B1ld%C4%B1r%C4%B1m
Sebahattin Karakelle,, 1950, https://tr.wikipedia.org/wiki/Sebahattin_Karakelle
Erol Tınastepe,, 1957, https://tr.wikipedia.org/wiki/Erol_T%C4%B1nastepe
Recep Akdağ, 1960, https://tr.wikipedia.org/wiki/Recep_Akda%C4%9F
Muhyettin Aksak, 1957, https://tr.wikipedia.org/wiki/Muhyettin_Aksak
Saadettin Aydın, 1962, https://tr.wikipedia.org/wiki/Saadettin_Ayd%C4%B1n
Fazilet Dağcı Çığlık, 1973, https://tr.wikipedia.org/wiki/Fazilet_Da%C4%9Fc%C4%B1_%C3%87%C4%B1%C4%9Fl%C4%B1k
2007 – Erzurum, 2007, https://tr.wikipedia.org/wiki/Muzaffer_G%C3%BClyurt
İbrahim Kavaz, 1955, https://tr.wikipedia.org/wiki/%C4%B0brahim_Kavaz
Zeki Ertugay,, 1947, https://tr.wikipedia.org/wiki/Zeki_Ertugay
Kemal Unakıtan, 1946, https://tr.wikipedia.org/wiki/Kemal_Unak%C4%B1tan
Hasan Murat Mercan, 1959, https://tr.wikipedia.org/wiki/Hasan_Murat_Mercan
Emin Nedim Öztürk, 1952, https://tr.wikipedia.org/wiki/Emin_Nedim_%C3%96zt%C3%BCrk
Fehmi Murat Sönmez, 1963, https://tr.wikipedia.org/wiki/Fehmi_Murat_S%C3%B6nmez
Hüseyin Tayfun İçli,, 1957, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Tayfun_%C4%B0%C3%A7li
Beytullah Asil, 1959, https://tr.wikipedia.org/wiki/Beytullah_Asil
Mehmet Şimşek, 1967, https://tr.wikipedia.org/wiki/Mehmet_%C5%9Eim%C5%9Fek
Fatma Şahin[2], 1966, https://tr.wikipedia.org/wiki/Fatma_%C5%9Eahin
Mehmet Erdoğan, 1956, https://tr.wikipedia.org/wiki/Mehmet_Erdo%C4%9Fan_(Gaziantepli)
İbrahim Halil Mazıcıoğlu, 1960, https://tr.wikipedia.org/wiki/%C4%B0brahim_Halil_Maz%C4%B1c%C4%B1o%C4%9Flu
Özlem Müftüoğlu, 1966, https://tr.wikipedia.org/wiki/%C3%96zlem_M%C3%BCft%C3%BCo%C4%9Flu
Mehmet Sarı, 1968, https://tr.wikipedia.org/wiki/Mehmet_Sar%C4%B1_(1968_do%C4%9Fumlu_siyaset%C3%A7i)
Akif Ekici, 1955, https://tr.wikipedia.org/wiki/Akif_Ekici
Yaşar Ağyüz, 1949, https://tr.wikipedia.org/wiki/Ya%C5%9Far_A%C4%9Fy%C3%BCz
Hasan Özdemir, 1947, https://tr.wikipedia.org/wiki/Hasan_%C3%96zdemir
Nurettin Canikli, 1960, https://tr.wikipedia.org/wiki/Nurettin_Canikli
Hacı Hasan Sönmez, 1956, https://tr.wikipedia.org/wiki/Hac%C4%B1_Hasan_S%C3%B6nmez
2007 – Giresun, 2007, https://tr.wikipedia.org/wiki/Ali_Tem%C3%BCr
Eşref Karaibrahim, 1956, https://tr.wikipedia.org/wiki/E%C5%9Fref_Karaibrahim
Murat Özkan,, 1963, https://tr.wikipedia.org/wiki/Murat_%C3%96zkan
Kemalettin Aydın,, 1962, https://tr.wikipedia.org/wiki/Kemalettin_Ayd%C4%B1n
Yahya Doğan,, 1949, https://tr.wikipedia.org/wiki/Yahya_Do%C4%9Fan
Abdulmuttalip Özbek, 1963, https://tr.wikipedia.org/wiki/Abdulmuttalip_%C3%96zbek
Cumhuriyet Halk Partisi, 2021, https://tr.wikipedia.org/wiki/R%C3%BCstem_Zeydan
Hamit Geylani,, 1947, https://tr.wikipedia.org/wiki/Hamit_Geylani
Sadullah Ergin, 1964, https://tr.wikipedia.org/wiki/Sadullah_Ergin
Abdülhadi Kahya, 1969, https://tr.wikipedia.org/wiki/Abd%C3%BClhadi_Kahya
Orhan Karasayar,, 1965, https://tr.wikipedia.org/wiki/Orhan_Karasayar
Fevzi Şanverdi, 1963, https://tr.wikipedia.org/wiki/Fevzi_%C5%9Eanverdi
Fuat Çay, 1949, https://tr.wikipedia.org/wiki/Fuat_%C3%87ay
Gökhan Durgun,, 1957, https://tr.wikipedia.org/wiki/G%C3%B6khan_Durgun
Abdulaziz Yazar, 1951, https://tr.wikipedia.org/wiki/Abdulaziz_Yazar
İzzettin Yılmaz, 1967, https://tr.wikipedia.org/wiki/%C4%B0zzettin_Y%C4%B1lmaz
Süleyman Turan Çirkin,, 1949, https://tr.wikipedia.org/wiki/S%C3%BCleyman_Turan_%C3%87irkin
Ali Güner, 1955, https://tr.wikipedia.org/wiki/Ali_G%C3%BCner
2023 - Van, 2023, https://tr.wikipedia.org/wiki/Pervin_Buldan
Süreyya Sadi Bilgiç, 1961, https://tr.wikipedia.org/wiki/S%C3%BCreyya_Sadi_Bilgi%C3%A7
Mehmet Saik Dilek, 1966, https://tr.wikipedia.org/wiki/Mehmet_Sait_Dilek
Haydar Kemal Kurt,, 1963, https://tr.wikipedia.org/wiki/Haydar_Kemal_Kurt
Mevlüt Coşkuner, 1945, https://tr.wikipedia.org/wiki/Mevl%C3%BCt_Co%C5%9Fkuner
Süleyman Nevzat Korkmaz, 1965, https://tr.wikipedia.org/wiki/S%C3%BCleyman_Nevzat_Korkmaz
Failed to parse: https://tr.wikipedia.org/wiki/Recep_Tayyip_Erdo%C4%9Fan
İdris Güllüce, 1950, https://tr.wikipedia.org/wiki/%C4%B0dris_G%C3%BCll%C3%BCce
Sosyaldemokrat Halkçı Parti, 1986, https://tr.wikipedia.org/wiki/Ertu%C4%9Frul_G%C3%BCnay
Mesude Nursuna Memecan, 1957, https://tr.wikipedia.org/wiki/Mesude_Nursuna_Memecan
Nusret Bayraktar, 1951, https://tr.wikipedia.org/wiki/Nusret_Bayraktar
Prof. Dr. Edibe Sözen, 1961, https://tr.wikipedia.org/wiki/Edibe_S%C3%B6zen
Hüseyin Besli, 1953, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Besli
Özlem Öztelcan[1], 1976, https://tr.wikipedia.org/wiki/%C3%96zlem_Piltano%C4%9Flu_T%C3%BCrk%C3%B6ne
İrfan Gündüz, 1950, https://tr.wikipedia.org/wiki/%C4%B0rfan_G%C3%BCnd%C3%BCz
Mehmet Sekmen[1], 1958, https://tr.wikipedia.org/wiki/Mehmet_Sekmen
Hasan Kemal Yardımcı, 1950, https://tr.wikipedia.org/wiki/Hasan_Kemal_Yard%C4%B1mc%C4%B1
2007 – İstanbul, 2007, https://tr.wikipedia.org/wiki/Mehmet_Beyaz%C4%B1t_Denizolgun
Demokratik Sol Parti, 2015, https://tr.wikipedia.org/wiki/Ay%C5%9Fe_Jale_A%C4%9F%C4%B1rba%C5%9F
Algan Hacaloğlu[1], 1940, https://tr.wikipedia.org/wiki/Algan_Hacalo%C4%9Flu
İlhan Kesici, 1948, https://tr.wikipedia.org/wiki/%C4%B0lhan_Kesici
Bayram Ali Meral, 1941, https://tr.wikipedia.org/wiki/Bayramali_Meral
Şinasi Öktem, 1955, https://tr.wikipedia.org/wiki/%C5%9Einasi_%C3%96ktem
Fatma Nur Serter, 1948, https://tr.wikipedia.org/wiki/Fatma_Nur_Serter
Ahmet Tan, 1949, https://tr.wikipedia.org/wiki/Ahmet_Tan
Ali Topuz, 1932, https://tr.wikipedia.org/wiki/Ali_Topuz
Durmuşali Torlak, 1962, https://tr.wikipedia.org/wiki/Durmu%C5%9Fali_Torlak
Gündüz Suphi Aktan, 1941, https://tr.wikipedia.org/wiki/G%C3%BCnd%C3%BCz_Suphi_Aktan
Mehmet Ufuk Uras, 1959, https://tr.wikipedia.org/wiki/Mehmet_Ufuk_Uras
Hayati Yazıcı, 1952, https://tr.wikipedia.org/wiki/Hayati_Yaz%C4%B1c%C4%B1
Murat Başesgioğlu, 1955, https://tr.wikipedia.org/wiki/Murat_Ba%C5%9Fesgio%C4%9Flu
Egemen Bağış, 1970, https://tr.wikipedia.org/wiki/Egemen_Ba%C4%9F%C4%B1%C5%9F
Necat Birinci,, 1943, https://tr.wikipedia.org/wiki/Necat_Birinci
2007 – İstanbul, 2007, https://tr.wikipedia.org/wiki/Alaattin_B%C3%BCy%C3%BCkkaya
Nimet Baş, 1965, https://tr.wikipedia.org/wiki/Nimet_%C3%87ubuk%C3%A7u
Ömer Dinçer, 1956, https://tr.wikipedia.org/wiki/%C3%96mer_Din%C3%A7er
Canan Kalsın,, 1967, https://tr.wikipedia.org/wiki/Canan_Kals%C4%B1n
Recep Koral, 1948, https://tr.wikipedia.org/wiki/Recep_Koral
Burhan Kuzu, 1955, https://tr.wikipedia.org/wiki/Burhan_Kuzu
Ayşe Nur Bahçekapılı, 1954, https://tr.wikipedia.org/wiki/Ay%C5%9Fe_Nur_Bah%C3%A7ekap%C4%B1l%C4%B1
Osman Gazi Yağmurdereli, 1953, https://tr.wikipedia.org/wiki/Osman_Gazi_Ya%C4%9Fmurdereli
Failed to parse: https://tr.wikipedia.org/wiki/Kemal_K%C4%B1l%C4%B1%C3%A7daro%C4%9Flu
Necla Arat,, 1940, https://tr.wikipedia.org/wiki/Necla_Arat
Mustafa Özyürek, 1939, https://tr.wikipedia.org/wiki/Mustafa_%C3%96zy%C3%BCrek
Mehmet Sevigen, 1954, https://tr.wikipedia.org/wiki/Mehmet_Sevigen
Çetin Soysal,, 1963, https://tr.wikipedia.org/wiki/%C3%87etin_Soysal
Bihlun Tamaylıgil, 1966, https://tr.wikipedia.org/wiki/Bihlun_Tamayl%C4%B1gil
2007 – İstanbul, 2007, https://tr.wikipedia.org/wiki/S%C3%BCleyman_Ya%C4%9F%C4%B1z
Mithat Melen, 1947, https://tr.wikipedia.org/wiki/Mithat_Melen
Ümit Şafak, 1955, https://tr.wikipedia.org/wiki/%C3%9Cmit_%C5%9Eafak
Abdülkadir Aksu, 1944, https://tr.wikipedia.org/wiki/Abd%C3%BClkadir_Aksu
İdris Naim Şahin, 1956, https://tr.wikipedia.org/wiki/%C4%B0dris_Naim_%C5%9Eahin
Nazım Ekren, 1956, https://tr.wikipedia.org/wiki/Naz%C4%B1m_Ekren
Güldal Akşit, 1960, https://tr.wikipedia.org/wiki/G%C3%BCldal_Ak%C5%9Fit
Mehmet Müezzinoğlu,, 1955, https://tr.wikipedia.org/wiki/Mehmet_M%C3%BCezzino%C4%9Flu
Lokman Ayva, 1966, https://tr.wikipedia.org/wiki/Lokman_Ayva
Fuat Bol, 1955, https://tr.wikipedia.org/wiki/Fuat_Bol
2011 – İstanbul, 2011, https://tr.wikipedia.org/wiki/%C4%B0brahim_Yi%C4%9Fit
Reha Çamuroğlu,, 1958, https://tr.wikipedia.org/wiki/Reha_%C3%87amuro%C4%9Flu
Alev Dedegil,, 1958, https://tr.wikipedia.org/wiki/Alev_Dedegil
Mehmet Domaç,, 1950, https://tr.wikipedia.org/wiki/Mehmet_Doma%C3%A7
Feyzullah Kıyıklık, 1949, https://tr.wikipedia.org/wiki/Feyzullah_K%C4%B1y%C4%B1kl%C4%B1k
Halide İncekara, 1959, https://tr.wikipedia.org/wiki/Halide_%C4%B0ncekara
Ünal Kacır,, 1953, https://tr.wikipedia.org/wiki/%C3%9Cnal_Kac%C4%B1r
Mustafa Şükrü Elekdağ, 1924, https://tr.wikipedia.org/wiki/Mustafa_%C5%9E%C3%BCkr%C3%BC_Elekda%C4%9F
Birgen Keleş, 1939, https://tr.wikipedia.org/wiki/Birgen_Kele%C5%9F
Esfender Korkmaz, 1940, https://tr.wikipedia.org/wiki/Esfender_Korkmaz
Hasan Macit, 1953, https://tr.wikipedia.org/wiki/Hasan_Macit
Hüseyin Mert,, 1961, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Mert
2002 – İstanbul, 2002, https://tr.wikipedia.org/wiki/Mehmet_Ali_%C3%96zpolat
Sacid Yıldız, 1948, https://tr.wikipedia.org/wiki/Sacid_Y%C4%B1ld%C4%B1z
Meral Akşener, 1956, https://tr.wikipedia.org/wiki/Meral_Ak%C5%9Fener
Atila Kaya,, 1957, https://tr.wikipedia.org/wiki/Atila_Kaya
Mehmet Cihat Özönder, 1947, https://tr.wikipedia.org/wiki/Mehmet_Cihat_%C3%96z%C3%B6nder
Sebahat Tuncel, 1975, https://tr.wikipedia.org/wiki/Sebahat_Tuncel
Mehmet Aydın, 1943, https://tr.wikipedia.org/wiki/Mehmet_Ayd%C4%B1n_(1943_do%C4%9Fumlu_siyaset%C3%A7i)
Fatma Seniha Nükhet Hotar, 1961, https://tr.wikipedia.org/wiki/N%C3%BCkhet_Hotar
Prof. Dr. İbrahim Hasgür,, 1953, https://tr.wikipedia.org/wiki/%C4%B0brahim_Hasg%C3%BCr
Mehmet Sayım Tekelioğlu,, 1950, https://tr.wikipedia.org/wiki/Mehmet_Say%C4%B1m_Tekelio%C4%9Flu
Tuğrul Yemişci, 1944, https://tr.wikipedia.org/wiki/Tu%C4%9Frul_Yemi%C5%9Fci
Şükran Güldal Mumcu[1][2], 1951, https://tr.wikipedia.org/wiki/%C5%9E%C3%BCkran_G%C3%BCldal_Mumcu
Bülent Baratalı, 1947, https://tr.wikipedia.org/wiki/B%C3%BClent_Baratal%C4%B1
Ahmet Ersin,, 1950, https://tr.wikipedia.org/wiki/Ahmet_Ersin
2007 – İzmir, 2007, https://tr.wikipedia.org/wiki/Abdurrezzak_Erten
Harun Öztürk, 1953, https://tr.wikipedia.org/wiki/Harun_%C3%96zt%C3%BCrk
Oktay Vural, 1956, https://tr.wikipedia.org/wiki/Oktay_Vural
Şenol Sunat, 1956, https://tr.wikipedia.org/wiki/%C5%9Eenol_Bal
Mehmet Vecdi Gönül, 1939, https://tr.wikipedia.org/wiki/Vecdi_G%C3%B6n%C3%BCl
Taha Aksoy, 1947, https://tr.wikipedia.org/wiki/Taha_Aksoy
Erdal Kalkan,, 1946, https://tr.wikipedia.org/wiki/Erdal_Kalkan
2007 – İzmir, 2007, https://tr.wikipedia.org/wiki/%C4%B0smail_Katmerci
Kıvılcım Kemal Anadol, 1941, https://tr.wikipedia.org/wiki/K%C4%B1v%C4%B1lc%C4%B1m_Kemal_Anadol
Cânân Aritman[2], 1950, https://tr.wikipedia.org/wiki/Canan_Aritman
Recai Birgün, 1966, https://tr.wikipedia.org/wiki/Recai_Birg%C3%BCn
Selçuk Ayhan, 1953, https://tr.wikipedia.org/wiki/Sel%C3%A7uk_Ayhan
Prof.Dr.Oğuz Oyan, 1947, https://tr.wikipedia.org/wiki/O%C4%9Fuz_Oyan
2011 – İzmir, 2011, https://tr.wikipedia.org/wiki/Mehmet_Ali_Susam
Ahmet Kenan Tanrıkulu, 1958, https://tr.wikipedia.org/wiki/Ahmet_Kenan_Tanr%C4%B1kulu
Kamil Erdal Sipahi, 1948, https://tr.wikipedia.org/wiki/Kamil_Erdal_Sipahi
Nevzat Pakdil, 1950, https://tr.wikipedia.org/wiki/Nevzat_Pakdil
Mehmet Sağlam,, 1938, https://tr.wikipedia.org/wiki/Mehmet_Sa%C4%9Flam
Dr. Cafer Tatlıbal, 1945, https://tr.wikipedia.org/wiki/Cafer_Tatl%C4%B1bal
Avni Doğan, 1951, https://tr.wikipedia.org/wiki/Avni_Do%C4%9Fan_(siyaset%C3%A7i,_1951)
Veysi Kaynak, 1962, https://tr.wikipedia.org/wiki/Veysi_Kaynak
2007 – Kahramanmaraş, 2007, https://tr.wikipedia.org/wiki/Fatih_Ar%C4%B1kan
Durdu Özbolat, 1962, https://tr.wikipedia.org/wiki/Durdu_%C3%96zbolat
Mehmet Akif Paksoy, 1956, https://tr.wikipedia.org/wiki/Mehmet_Akif_Paksoy
Mehmet Ceylan, 1958, https://tr.wikipedia.org/wiki/Mehmet_Ceylan_(Karab%C3%BCkl%C3%BC)
Cumhur Ünal, 1966, https://tr.wikipedia.org/wiki/Cumhur_%C3%9Cnal
Mustafa Ünal, 1948, https://tr.wikipedia.org/wiki/Mustafa_%C3%9Cnal
Lütfi Elvan, 1962, https://tr.wikipedia.org/wiki/L%C3%BCtfi_Elvan
Mevlüt Akgün, 1966, https://tr.wikipedia.org/wiki/Mevl%C3%BCt_Akg%C3%BCn
Hasan Çalış, 1956, https://tr.wikipedia.org/wiki/Hasan_%C3%87al%C4%B1%C5%9F
Zeki Karabayır, 1952, https://tr.wikipedia.org/wiki/Zeki_Karabay%C4%B1r
Mahmut Esat Güven,, 1953, https://tr.wikipedia.org/wiki/Mahmut_Esat_G%C3%BCven
Gürcan Dağdaş, 1960, https://tr.wikipedia.org/wiki/G%C3%BCrcan_Da%C4%9Fda%C5%9F
Hakkı Köylü, 1948, https://tr.wikipedia.org/wiki/Hakk%C4%B1_K%C3%B6yl%C3%BC
Hasan Altan,, 1948, https://tr.wikipedia.org/wiki/Hasan_Altan
Musa Sıvacıoğlu,, 1955, https://tr.wikipedia.org/wiki/Musa_S%C4%B1vac%C4%B1o%C4%9Flu
Mehmet Serdaroğlu, 1947, https://tr.wikipedia.org/wiki/Mehmet_Serdaro%C4%9Flu
Abdullah Gül, 1950, https://tr.wikipedia.org/wiki/Abdullah_G%C3%BCl
Taner Yıldız, 1962, https://tr.wikipedia.org/wiki/Taner_Y%C4%B1ld%C4%B1z
Sadık Yakut,, 1956, https://tr.wikipedia.org/wiki/Sad%C4%B1k_Yakut
Mustafa Elitaş, 1957, https://tr.wikipedia.org/wiki/Mustafa_Elita%C5%9F
Yaşar Karayel, 1950, https://tr.wikipedia.org/wiki/Ya%C5%9Far_Karayel
Ahmet Öksüzkaya, 1960, https://tr.wikipedia.org/wiki/Ahmet_%C3%96ks%C3%BCzkaya
2011 – Kayseri, 2011, https://tr.wikipedia.org/wiki/Mehmet_%C5%9Eevki_Kulkulo%C4%9Flu
Sabahattin Çakmakoğlu, 1930, https://tr.wikipedia.org/wiki/Sabahattin_%C3%87akmako%C4%9Flu
Vahit Erdem, 1944, https://tr.wikipedia.org/wiki/Vahit_Erdem
Mustafa Özbayrak, 1960, https://tr.wikipedia.org/wiki/Mustafa_%C3%96zbayrak
Osman Durmuş, 1947, https://tr.wikipedia.org/wiki/Osman_Durmu%C5%9F
Ahmet Gökhan Sarıçam,, 1969, https://tr.wikipedia.org/wiki/Ahmet_G%C3%B6khan_Sar%C4%B1%C3%A7am
Turgut Dibek, 1966, https://tr.wikipedia.org/wiki/Turgut_Dibek
Tansel Barış,, 1952, https://tr.wikipedia.org/wiki/Tansel_Bar%C4%B1%C5%9F
Mikail Arslan, 1961, https://tr.wikipedia.org/wiki/Mikail_Arslan
Abdullah Çalışkan, 1974, https://tr.wikipedia.org/wiki/Abdullah_%C3%87al%C4%B1%C5%9Fkan_(siyaset%C3%A7i,_1974)
Metin Çobanoğlu, 1958, https://tr.wikipedia.org/wiki/Metin_%C3%87obano%C4%9Flu
Hüseyin Devecioğlu, 1963, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Devecio%C4%9Flu
Hasan Kara, 1969, https://tr.wikipedia.org/wiki/Hasan_Kara
Nihat Ergün, 1962, https://tr.wikipedia.org/wiki/Nihat_Erg%C3%BCn
Osman Pepe, 1954, https://tr.wikipedia.org/wiki/Osman_Pepe
Fikri Işık, 1965, https://tr.wikipedia.org/wiki/Fikri_I%C5%9F%C4%B1k
Azize Sibel Gönül, 1966, https://tr.wikipedia.org/wiki/Azize_Sibel_G%C3%B6n%C3%BCl
2007 – Kocaeli, 2007, https://tr.wikipedia.org/wiki/Ey%C3%BCp_Ayar
Muzaffer Baştopçu,, 1948, https://tr.wikipedia.org/wiki/Muzaffer_Ba%C5%9Ftop%C3%A7u
Mehmet Cevdet Selvi, 1943, https://tr.wikipedia.org/wiki/Cevdet_Selvi
Hikmet Erenkaya, 1953, https://tr.wikipedia.org/wiki/Hikmet_Erenkaya
Cumali Durmuş, 1962, https://tr.wikipedia.org/wiki/Cumali_Durmu%C5%9F
2007 – Konya, 2007, https://tr.wikipedia.org/wiki/Hasan_Ang%C4%B1
2007 – Konya, 2007, https://tr.wikipedia.org/wiki/Ahmet_B%C3%BCy%C3%BCkakka%C5%9Flar
Özkan Öksüz, 1953, https://tr.wikipedia.org/wiki/%C3%96zkan_%C3%96ks%C3%BCz
2007 – Konya 
2011 – Konya, 2007, https://tr.wikipedia.org/wiki/Kerim_%C3%96zkul
Ali Öztürk,, 1954, https://tr.wikipedia.org/wiki/Ali_%C3%96zt%C3%BCrk_(hukuk%C3%A7u)
Hüsnü Tuna, 1954, https://tr.wikipedia.org/wiki/H%C3%BCsn%C3%BC_Tuna
2007 – Konya
2011 – Konya, 2007, https://tr.wikipedia.org/wiki/Harun_T%C3%BCfekci
Ayşe Türkmenoğlu, 1965, https://tr.wikipedia.org/wiki/Ay%C5%9Fe_T%C3%BCrkmeno%C4%9Flu
Muharrem Candan, 1956, https://tr.wikipedia.org/wiki/Muharrem_Candan
2007 – Konya, 2007, https://tr.wikipedia.org/wiki/Abdullah_%C3%87etinkaya
Orhan Erdem, 1963, https://tr.wikipedia.org/wiki/Orhan_Erdem
Sami Güçlü, 1950, https://tr.wikipedia.org/wiki/Sami_G%C3%BC%C3%A7l%C3%BC
2011 – Konya, 2011, https://tr.wikipedia.org/wiki/Mustafa_Kabakc%C4%B1
Atilla Kart, 1954, https://tr.wikipedia.org/wiki/Atilla_Kart
Faruk Bal, 1950, https://tr.wikipedia.org/wiki/Faruk_Bal
Mustafa Kalaycı, 1960, https://tr.wikipedia.org/wiki/Mustafa_Kalayc%C4%B1
Soner Aksoy,, 1941, https://tr.wikipedia.org/wiki/Soner_Aksoy
Hüsnü Tuna, 1952, https://tr.wikipedia.org/wiki/%C4%B0smail_Hakk%C4%B1_Bi%C3%A7er
Hasan Fehmi Kinay,, 1964, https://tr.wikipedia.org/wiki/Hasan_Fehmi_Kinay
Hüsnü Ordu, 1953, https://tr.wikipedia.org/wiki/H%C3%BCsn%C3%BC_Ordu
Hüseyin Tuğcu, 1954, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Tu%C4%9Fcu
Alim Işık[1], 1959, https://tr.wikipedia.org/wiki/Alim_I%C5%9F%C4%B1k
Öznur Çalık, 1965, https://tr.wikipedia.org/wiki/%C3%96znur_%C3%87al%C4%B1k
2011 – Malatya, 2011, https://tr.wikipedia.org/wiki/Mahmut_M%C3%BCcahit_F%C4%B1nd%C4%B1kl%C4%B1
İhsan Koca, 1966, https://tr.wikipedia.org/wiki/%C4%B0hsan_Koca
2007 – Malatya, 2007, https://tr.wikipedia.org/wiki/Fuat_%C3%96lmeztoprak
Ömer Faruk Öz,, 1966, https://tr.wikipedia.org/wiki/%C3%96mer_Faruk_%C3%96z
Mehmet Şahin, 1956, https://tr.wikipedia.org/wiki/Mehmet_%C5%9Eahin_(siyaset%C3%A7i,_1956)
Ferit Mevlüt Aslanoğlu,, 1952, https://tr.wikipedia.org/wiki/Ferit_Mevl%C3%BCt_Aslano%C4%9Flu
Bülent Arınç, 1948, https://tr.wikipedia.org/wiki/B%C3%BClent_Ar%C4%B1n%C3%A7
Hüseyin Tanrıverdi, 1956, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Tanr%C4%B1verdi
Recai Berber, 1959, https://tr.wikipedia.org/wiki/Recai_Berber
İsmail Bilen, 1961, https://tr.wikipedia.org/wiki/%C4%B0smail_Bilen_(1961_do%C4%9Fumlu_siyaset%C3%A7i)
2007 – Manisa, 2007, https://tr.wikipedia.org/wiki/Mehmet_%C3%87er%C3%A7i
Şahin Mengü, 1948, https://tr.wikipedia.org/wiki/%C5%9Eahin_Meng%C3%BC
Mustafa Erdoğan Yetenç,, 1946, https://tr.wikipedia.org/wiki/Mustafa_Erdo%C4%9Fan_Yeten%C3%A7
Ahmet Orhan, 1956, https://tr.wikipedia.org/wiki/Ahmet_Orhan
Erkan Akçay, 1961, https://tr.wikipedia.org/wiki/Erkan_Ak%C3%A7ay
2007 – Manisa, 2007, https://tr.wikipedia.org/wiki/Mustafa_En%C3%B6z
Prof. Dr. Cüneyt Yüksel, 1970, https://tr.wikipedia.org/wiki/C%C3%BCneyt_Y%C3%BCksel
Süleyman Çelebi, 1944, https://tr.wikipedia.org/wiki/S%C3%BCleyman_%C3%87elebi_(milletvekili)
Mehmet Halit Demir, 1970, https://tr.wikipedia.org/wiki/Mehmet_Halit_Demir
Gönül Bekin Şahkulubey, 1970, https://tr.wikipedia.org/wiki/G%C3%B6n%C3%BCl_Bekin_%C5%9Eahkulubey
Ahmet Türk, 1942, https://tr.wikipedia.org/wiki/Ahmet_T%C3%BCrk
Emine Ayna, 1968, https://tr.wikipedia.org/wiki/Emine_Ayna
Recep Tayyip Erdoğan, 2003, https://tr.wikipedia.org/wiki/K%C3%BCr%C5%9Fad_T%C3%BCzmen
2007 – Mersin, 2007, https://tr.wikipedia.org/wiki/%C3%96mer_%C4%B0nan
Mehmet Zafer Üskül, 1944, https://tr.wikipedia.org/wiki/Mehmet_Zafer_%C3%9Csk%C3%BCl
Ali Er, 1949, https://tr.wikipedia.org/wiki/Ali_Er
2011 – Mersin, 2011, https://tr.wikipedia.org/wiki/Ali_R%C4%B1za_%C3%96zt%C3%BCrk
İsa Gök,, 1963, https://tr.wikipedia.org/wiki/%C4%B0sa_G%C3%B6k
2007 – Mersin, 2007, https://tr.wikipedia.org/wiki/Ali_Oksal
Vahap Seçer[1], 1963, https://tr.wikipedia.org/wiki/Vahap_Se%C3%A7er
Akif Akkuş, 1948, https://tr.wikipedia.org/wiki/Akif_Akku%C5%9F
Behiç Çelik, 1955, https://tr.wikipedia.org/wiki/Behi%C3%A7_%C3%87elik
Mehmet Şandır, 1947, https://tr.wikipedia.org/wiki/Mehmet_%C5%9Eand%C4%B1r
Kadir Ural, 1968, https://tr.wikipedia.org/wiki/Kadir_Ural
Mehmet Nil Hıdır, 1956, https://tr.wikipedia.org/wiki/Mehmet_Nil_H%C4%B1d%C4%B1r
2011 – Muğla, 2011, https://tr.wikipedia.org/wiki/Y%C3%BCksel_%C3%96zden
Ali Arslan, 1955, https://tr.wikipedia.org/wiki/Ali_Arslan_(milletvekili)
2007 – Muğla, 2007, https://tr.wikipedia.org/wiki/G%C3%BCrol_Ergin
Fevzi Topuz, 1954, https://tr.wikipedia.org/wiki/Fevzi_Topuz
İYİ Parti, 2017, https://tr.wikipedia.org/wiki/Metin_Ergun
2007 – Muş, 2007, https://tr.wikipedia.org/wiki/Seracettin_Karaya%C4%9F%C4%B1z
Medeni Yılmaz, 1954, https://tr.wikipedia.org/wiki/Medeni_Y%C4%B1lmaz
Sırrı Sakık, 1957, https://tr.wikipedia.org/wiki/S%C4%B1rr%C4%B1_Sak%C4%B1k
Mehmet Nuri Yaman,, 1944, https://tr.wikipedia.org/wiki/Mehmet_Nuri_Yaman
Mahmut Dede, 1961, https://tr.wikipedia.org/wiki/Mahmut_Dede
Ahmet Erdal Feralan, 1962, https://tr.wikipedia.org/wiki/Ahmet_Erdal_Feralan
Rıtvan Köybaşı, 1965, https://tr.wikipedia.org/wiki/R%C4%B1tvan_K%C3%B6yba%C5%9F%C4%B1
İsmail Göksel, 1952, https://tr.wikipedia.org/wiki/%C4%B0smail_G%C3%B6ksel
Muharrem Selamoğlu, 1950, https://tr.wikipedia.org/wiki/Muharrem_Selamo%C4%9Flu
, 1999, https://tr.wikipedia.org/wiki/M%C3%BCmin_%C4%B0nan
Mehmet Hilmi Güler[1], 1949, https://tr.wikipedia.org/wiki/Mehmet_Hilmi_G%C3%BCler
Eyüp Fatsa, 1961, https://tr.wikipedia.org/wiki/Ey%C3%BCp_Fatsa
Enver Yılmaz,, 1970, https://tr.wikipedia.org/wiki/Enver_Y%C4%B1lmaz_(siyaset%C3%A7i)
2011 – Ordu, 2011, https://tr.wikipedia.org/wiki/Mustafa_Hamarat
Ayhan Yılmaz, 1961, https://tr.wikipedia.org/wiki/Ayhan_Y%C4%B1lmaz
Rahmi Güner, 1945, https://tr.wikipedia.org/wiki/Rahmi_G%C3%BCner
Rıdvan Yalçın, 1972, https://tr.wikipedia.org/wiki/R%C4%B1dvan_Yal%C3%A7%C4%B1n
İbrahim Mete Doğruer, 1951, https://tr.wikipedia.org/wiki/%C4%B0brahim_Mete_Do%C4%9Fruer
Durdu Mehmet Kastal, 1969, https://tr.wikipedia.org/wiki/Durdu_Mehmet_Kastal
Devlet Bahçeli, 1948, https://tr.wikipedia.org/wiki/Devlet_Bah%C3%A7eli
Hakan Coşkun, 1967, https://tr.wikipedia.org/wiki/Hakan_Co%C5%9Fkun
Bayram Ali Bayramoğlu, 1958, https://tr.wikipedia.org/wiki/Bayram_Ali_Bayramo%C4%9Flu
Lütfi Çırakoğlu, 1952, https://tr.wikipedia.org/wiki/L%C3%BCtfi_%C3%87%C4%B1rako%C4%9Flu
Ahmet Mesut Yılmaz[1], 1947, https://tr.wikipedia.org/wiki/Ahmet_Mesut_Y%C4%B1lmaz
Erol Aslan Cebeci, 1963, https://tr.wikipedia.org/wiki/Erol_Aslan_Cebeci
Hasan Ali Çelik, 1959, https://tr.wikipedia.org/wiki/Hasan_Ali_%C3%87elik
Şaban Dişli, 1958, https://tr.wikipedia.org/wiki/%C5%9Eaban_Di%C5%9Fli
Ayhan Sefer Üstün, 1970, https://tr.wikipedia.org/wiki/Ayhan_Sefer_%C3%9Cst%C3%BCn
Recep Yıldırım,, 1949, https://tr.wikipedia.org/wiki/Recep_Y%C4%B1ld%C4%B1r%C4%B1m
Münir Kutluata, 1946, https://tr.wikipedia.org/wiki/M%C3%BCnir_Kutluata
Mustafa Demir[1], 1961, https://tr.wikipedia.org/wiki/Mustafa_Demir_(siyaset%C3%A7i,_1961)
Suat Kılıç, 1972, https://tr.wikipedia.org/wiki/Suat_K%C4%B1l%C4%B1%C3%A7
Cemal Yılmaz Demir, 1960, https://tr.wikipedia.org/wiki/Cemal_Y%C4%B1lmaz_Demir
Fatih Öztürk, 1972, https://tr.wikipedia.org/wiki/Fatih_%C3%96zt%C3%BCrk_(1972_do%C4%9Fumlu_siyaset%C3%A7i)
Ahmet Yeni, 1955, https://tr.wikipedia.org/wiki/Ahmet_Yeni
Ahmet Haluk Koç, 1954, https://tr.wikipedia.org/wiki/Haluk_Ko%C3%A7
Suat Binici, 1947, https://tr.wikipedia.org/wiki/Suat_Binici
Osman Çakır, 1952, https://tr.wikipedia.org/wiki/Osman_%C3%87ak%C4%B1r_(S%C3%BCrmene_do%C4%9Fumlu)
Afif Demirkıran, 1952, https://tr.wikipedia.org/wiki/Afif_Demirk%C4%B1ran
Memet Yılmaz Helvacıoğlu, 1949, https://tr.wikipedia.org/wiki/Memet_Y%C4%B1lmaz_Helvac%C4%B1o%C4%9Flu
Halkların Demokratik Partisi, 2014, https://tr.wikipedia.org/wiki/Osman_%C3%96z%C3%A7elik
Abdurrahman Dodurgalı, 1952, https://tr.wikipedia.org/wiki/Abdurrahman_Dodurgal%C4%B1
Kadir Tıngıroğlu, 1966, https://tr.wikipedia.org/wiki/Kadir_T%C4%B1ng%C4%B1ro%C4%9Flu
Engin Altay, 1963, https://tr.wikipedia.org/wiki/Engin_Altay
Hamza Yerlikaya, 1976, https://tr.wikipedia.org/wiki/Hamza_Yerlikaya
Selami Uzun,, 1957, https://tr.wikipedia.org/wiki/Selami_Uzun
Mehmet Mustafa Açıkalın,, 1951, https://tr.wikipedia.org/wiki/Mehmet_Mustafa_A%C3%A7%C4%B1kal%C4%B1n
Osman Kılıç,, 1946, https://tr.wikipedia.org/wiki/Osman_K%C4%B1l%C4%B1%C3%A7_(siyaset%C3%A7i,_1946)
Malik Ecder Özdemir,, 1954, https://tr.wikipedia.org/wiki/Malik_Ecder_%C3%96zdemir
Muhsin Yazıcıoğlu, 1954, https://tr.wikipedia.org/wiki/Muhsin_Yaz%C4%B1c%C4%B1o%C4%9Flu
Yahya Akman, 1966, https://tr.wikipedia.org/wiki/Yahya_Akman
Ramazan Başak, 1968, https://tr.wikipedia.org/wiki/Ramazan_Ba%C5%9Fak
Eyyüp Cenap Gülpınar,, 1943, https://tr.wikipedia.org/wiki/Eyy%C3%BCp_Cenap_G%C3%BClp%C4%B1nar
Zülfükar İzol, 1959, https://tr.wikipedia.org/wiki/Z%C3%BClf%C3%BCkar_%C4%B0zol
Mustafa Kuş,, 1949, https://tr.wikipedia.org/wiki/Mustafa_Ku%C5%9F
Abdulkadir Emin Önen, 1976, https://tr.wikipedia.org/wiki/Abdulkadir_Emin_%C3%96nen
Çağla Aktemur, 1972, https://tr.wikipedia.org/wiki/%C3%87a%C4%9Fla_Aktemur_%C3%96zyavuz
Abdurrahman Müfit Yetkin, 1953, https://tr.wikipedia.org/wiki/Abdurrahman_M%C3%BCfit_Yetkin
Sabahattin Cevheri, 1950, https://tr.wikipedia.org/wiki/Sabahattin_Cevheri
İbrahim Binici,, 1962, https://tr.wikipedia.org/wiki/%C4%B0brahim_Binici
Seyit Eyyüpoğlu, 1942, https://tr.wikipedia.org/wiki/Seyit_Eyy%C3%BCpo%C4%9Flu
Abdullah Veli Seyda, 1958, https://tr.wikipedia.org/wiki/Abdullah_Veli_Seyda
Sevahir Bayındır, 1969, https://tr.wikipedia.org/wiki/Sevahir_Bay%C4%B1nd%C4%B1r
Hasip Kaplan, 1954, https://tr.wikipedia.org/wiki/Hasip_Kaplan
Tevfik Ziyaeddin Akbulut,, 1949, https://tr.wikipedia.org/wiki/Tevfik_Ziyaeddin_Akbulut
Necip Taylan, 1946, https://tr.wikipedia.org/wiki/Necip_Taylan
2002 – Tekirdağ
2007 – Tekirdağ, 2002, https://tr.wikipedia.org/wiki/Enis_T%C3%BCt%C3%BCnc%C3%BC
Kemalettin Nalcı, 1966, https://tr.wikipedia.org/wiki/Kemalettin_Nalc%C4%B1
Zeyid Aslan,, 1965, https://tr.wikipedia.org/wiki/Zeyid_Aslan
Şükrü Ayalan, 1963, https://tr.wikipedia.org/wiki/%C5%9E%C3%BCkr%C3%BC_Ayalan
Osman Demir, 1962, https://tr.wikipedia.org/wiki/Osman_Demir
Hüseyin Gülsün,, 1957, https://tr.wikipedia.org/wiki/H%C3%BCseyin_G%C3%BCls%C3%BCn
2011 – Tokat, 2011, https://tr.wikipedia.org/wiki/Dilek_Y%C3%BCksel
2007 – Tokat, 2007, https://tr.wikipedia.org/wiki/Orhan_Ziya_Diren
Reşat Doğru, 1955, https://tr.wikipedia.org/wiki/Re%C5%9Fat_Do%C4%9Fru
Faruk Nafız Özak, 1946, https://tr.wikipedia.org/wiki/Faruk_Naf%C4%B1z_%C3%96zak
Cevdet Erdöl,, 1958, https://tr.wikipedia.org/wiki/Cevdet_Erd%C3%B6l
Mustafa Cumur, 1964, https://tr.wikipedia.org/wiki/Mustafa_Cumur
Kemalettin Göktaş,, 1950, https://tr.wikipedia.org/wiki/Kemalettin_G%C3%B6kta%C5%9F
Asım Aykan,, 1953, https://tr.wikipedia.org/wiki/As%C4%B1m_Aykan
Safiye Seymenoğlu, 1964, https://tr.wikipedia.org/wiki/Safiye_Seymeno%C4%9Flu
Mehmet Akif Hamzaçebi, 1953, https://tr.wikipedia.org/wiki/Akif_Hamza%C3%A7ebi
Süleyman Latif Yunusoğlu, 1953, https://tr.wikipedia.org/wiki/S%C3%BCleyman_Latif_Yunuso%C4%9Flu
Kamer Genç, 1940, https://tr.wikipedia.org/wiki/Kamer_Gen%C3%A7
Şerafettin Halis, 1956, https://tr.wikipedia.org/wiki/%C5%9Eerafettin_Halis
Nuri Uslu, 1953, https://tr.wikipedia.org/wiki/Nuri_Uslu
Mustafa Çetin,, 1952, https://tr.wikipedia.org/wiki/Mustafa_%C3%87etin
Osman Coşkunoğlu, 1947, https://tr.wikipedia.org/wiki/Osman_Co%C5%9Fkuno%C4%9Flu
Hüseyin Çelik, 1959, https://tr.wikipedia.org/wiki/H%C3%BCseyin_%C3%87elik
Kerem Altun, 1955, https://tr.wikipedia.org/wiki/Kerem_Altun
İkram Dinçer, 1959, https://tr.wikipedia.org/wiki/%C4%B0kram_Din%C3%A7er
Gülşen Orhan, 1970, https://tr.wikipedia.org/wiki/G%C3%BCl%C5%9Fen_Orhan
Kayhan Türkmenoğlu, 1962, https://tr.wikipedia.org/wiki/Kayhan_T%C3%BCrkmeno%C4%9Flu
Fatma Kurtulan, 1964, https://tr.wikipedia.org/wiki/Fatma_Kurtulan
Özdal Üçer, 1974, https://tr.wikipedia.org/wiki/%C3%96zdal_%C3%9C%C3%A7er
İlhan Evcin, 1962, https://tr.wikipedia.org/wiki/%C4%B0lhan_Evcin
Muharrem İnce, 1964, https://tr.wikipedia.org/wiki/Muharrem_%C4%B0nce
Haziran 2015 – Yozgat 
Kasım 2015 – Yozgat, 2015, https://tr.wikipedia.org/wiki/Abdulkadir_Akg%C3%BCl
Osman Coşkun,, 1962, https://tr.wikipedia.org/wiki/Osman_Co%C5%9Fkun_(Yozgatl%C4%B1)
Mehmet Çiçek, 1946, https://tr.wikipedia.org/wiki/Mehmet_%C3%87i%C3%A7ek
Mehmet Yaşar Öztürk, 1967, https://tr.wikipedia.org/wiki/Mehmet_Ya%C5%9Far_%C3%96zt%C3%BCrk
Mehmet Ekici, 1955, https://tr.wikipedia.org/wiki/Mehmet_Ekici_(siyaset%C3%A7i)
Fazlı Erdoğan, 1956, https://tr.wikipedia.org/wiki/Fazl%C4%B1_Erdo%C4%9Fan
Polat Türkmen, 1957, https://tr.wikipedia.org/wiki/Polat_T%C3%BCrkmen
Ali Koçal, 1948, https://tr.wikipedia.org/wiki/Ali_Ko%C3%A7al
Ali İhsan Köktürk, 1963, https://tr.wikipedia.org/wiki/Ali_%C4%B0hsan_K%C3%B6kt%C3%BCrk
Failed links:
https://tr.wikipedia.org/wiki/Recep_Tayyip_Erdo%C4%9Fan
https://tr.wikipedia.org/wiki/Kemal_K%C4%B1l%C4%B1%C3%A7daro%C4%9Flu


In [ ]:
22.dönem
Abdullah Çalışkan, 1958, https://tr.wikipedia.org/wiki/Abdullah_%C3%87al%C4%B1%C5%9Fkan_(siyaset%C3%A7i,_1958)
Ömer Çelik, 1968, https://tr.wikipedia.org/wiki/%C3%96mer_%C3%87elik
Recep Garip, 1956, https://tr.wikipedia.org/wiki/Recep_Garip
Ali Küçükaydın, 1948, https://tr.wikipedia.org/wiki/Ali_K%C3%BC%C3%A7%C3%BCkayd%C4%B1n
Vahit Kirişci, 1960, https://tr.wikipedia.org/wiki/Vahit_Kiri%C5%9Fci
Ayhan Zeynep Tekin Börü, 1950, https://tr.wikipedia.org/wiki/Ayhan_Zeynep_Tekin_B%C3%B6r%C3%BC
Abdullah Torun, 1968, https://tr.wikipedia.org/wiki/Abdullah_Torun
Ziyaettin Yağcı, 1945, https://tr.wikipedia.org/wiki/Ziyaettin_Ya%C4%9Fc%C4%B1
Nevingaye Erbatur, 1950, https://tr.wikipedia.org/wiki/Nevin_Gaye_Erbatur
Uğur Aksöz, 1948, https://tr.wikipedia.org/wiki/U%C4%9Fur_Aks%C3%B6z
Atilla Başoğlu, 1954, https://tr.wikipedia.org/wiki/Atilla_Ba%C5%9Fo%C4%9Flu
Kemal Sağ, 1955, https://tr.wikipedia.org/wiki/Kemal_Sa%C4%9F
Tacidar Seyhan, 1959, https://tr.wikipedia.org/wiki/Tacidar_Seyhan
Mehmet Ziya Yergök, 1951, https://tr.wikipedia.org/wiki/Mehmet_Ziya_Yerg%C3%B6k
Mahmut Göksu,, 1959, https://tr.wikipedia.org/wiki/Mahmut_G%C3%B6ksu
Fehmi Hüsrev Kutlu, 1961, https://tr.wikipedia.org/wiki/Fehmi_H%C3%BCsrev_Kutlu
Mehmet Özyol,, 1937, https://tr.wikipedia.org/wiki/Mehmet_%C3%96zyol
Ahmet Faruk Ünsal, 1963, https://tr.wikipedia.org/wiki/Ahmet_Faruk_%C3%9Cnsal
Şevket Gürsoy, 1953, https://tr.wikipedia.org/wiki/%C5%9Eevket_G%C3%BCrsoy
Sait Açba, 1950, https://tr.wikipedia.org/wiki/Sait_A%C3%A7ba
İbrahim Hakkı Aşkar, 1965, https://tr.wikipedia.org/wiki/%C4%B0brahim_Hakk%C4%B1_A%C5%9Fkar
Halil Aydoğan, 1950, https://tr.wikipedia.org/wiki/Halil_Aydo%C4%9Fan
Ahmet Koca, 1963, https://tr.wikipedia.org/wiki/Ahmet_Koca
Mahmut Koçak, 1964, https://tr.wikipedia.org/wiki/Mahmut_Ko%C3%A7ak
Reyhan Tugay, 1969, https://tr.wikipedia.org/wiki/Reyhan_Tugay
Halil Ünlütepe,, 1949, https://tr.wikipedia.org/wiki/Halil_%C3%9Cnl%C3%BCtepe
Mehmet Kerim Yıldız, 1965, https://tr.wikipedia.org/wiki/Mehmet_Kerim_Y%C4%B1ld%C4%B1z
Mehmet Melik Özmen, 1966, https://tr.wikipedia.org/wiki/Mehmet_Melik_%C3%96zmen
Halil Özyolcu, 1971, https://tr.wikipedia.org/wiki/Halil_%C3%96zyolcu
Adalet ve Kalkınma Partisi, 2005, https://tr.wikipedia.org/wiki/Naci_Aslan
Cemal Kaya, 1962, https://tr.wikipedia.org/wiki/Cemal_Kaya
Ruhi Açıkgöz, 1963, https://tr.wikipedia.org/wiki/Ruhi_A%C3%A7%C4%B1kg%C3%B6z
Ali Rıza Alaboyun, 1957, https://tr.wikipedia.org/wiki/Ali_R%C4%B1za_Alaboyun
Ramazan Toprak, 1960, https://tr.wikipedia.org/wiki/Ramazan_Toprak
Ahmet Yaşar, 1950, https://tr.wikipedia.org/wiki/Ahmet_Ya%C5%9Far
Hamza Albayrak, 1953, https://tr.wikipedia.org/wiki/Hamza_Albayrak
Akif Gülle, 1958, https://tr.wikipedia.org/wiki/Akif_G%C3%BClle
Mustafa Sayar, 1952, https://tr.wikipedia.org/wiki/Mustafa_Sayar_(Amasya_do%C4%9Fumlu)
Cemil Çiçek, 1946, https://tr.wikipedia.org/wiki/Cemil_%C3%87i%C3%A7ek
Beşir Atalay, 1947, https://tr.wikipedia.org/wiki/Be%C5%9Fir_Atalay
Ali Babacan, 1967, https://tr.wikipedia.org/wiki/Ali_Babacan
İsmail Alptekin, 1943, https://tr.wikipedia.org/wiki/%C4%B0smail_Alptekin
Bülent Gedikli, 1964, https://tr.wikipedia.org/wiki/B%C3%BClent_Gedikli
Haluk İpek, 1963, https://tr.wikipedia.org/wiki/Haluk_%C4%B0pek
Salih Kapusuz, 1954, https://tr.wikipedia.org/wiki/Salih_Kapusuz
Telat Karapınar, 1962, https://tr.wikipedia.org/wiki/Telat_Karap%C4%B1nar
Faruk Koca, 1964, https://tr.wikipedia.org/wiki/Faruk_Koca
Ersönmez Yarbay, 1953, https://tr.wikipedia.org/wiki/Ers%C3%B6nmez_Yarbay
Mustafa Sait Yazıcıoğlu, 1949, https://tr.wikipedia.org/wiki/Mustafa_Said_Yaz%C4%B1c%C4%B1o%C4%9Flu
Mehmet Zekai Özcan, 1950, https://tr.wikipedia.org/wiki/Mehmet_Zekai_%C3%96zcan
Remziye Öztoptak, 1949, https://tr.wikipedia.org/wiki/Remziye_%C3%96ztoprak
Eyyüp Sanay, 1942, https://tr.wikipedia.org/wiki/Eyy%C3%BCp_Sanay
Nur Doğan Topaloğlu, 1937, https://tr.wikipedia.org/wiki/Nur_Do%C4%9Fan_Topalo%C4%9Flu
Mustafa Tuna, 1957, https://tr.wikipedia.org/wiki/Mustafa_Tuna
Önder Sav, 1937, https://tr.wikipedia.org/wiki/%C3%96nder_Sav
Oya Araslı, 1943, https://tr.wikipedia.org/wiki/Oya_Arasl%C4%B1
Yılmaz Ateş, 1953, https://tr.wikipedia.org/wiki/Y%C4%B1lmaz_Ate%C5%9F
Eşref Erdem, 1941, https://tr.wikipedia.org/wiki/E%C5%9Fref_Erdem
Zekeriya Akıncı, 1957, https://tr.wikipedia.org/wiki/Zekeriya_Ak%C4%B1nc%C4%B1
Ayşe Gülsün Bilgehan, 1957, https://tr.wikipedia.org/wiki/Ay%C5%9Fe_G%C3%BCls%C3%BCn_Bilgehan
Ahmet İsmet Çanakcı, 1938, https://tr.wikipedia.org/wiki/Ahmet_%C4%B0smet_%C3%87anakc%C4%B1
İsmail Değerli, 1945, https://tr.wikipedia.org/wiki/%C4%B0smail_De%C4%9Ferli
Bayram Ali Meral, 1941, https://tr.wikipedia.org/wiki/Bayramali_Meral
Yakup Kepenek, 1937, https://tr.wikipedia.org/wiki/Yakup_Kepenek
Mehmet Tomanbay, 1953, https://tr.wikipedia.org/wiki/Mehmet_Tomanbay
Anavatan Partisi, 2005, https://tr.wikipedia.org/wiki/Muzaffer_Remzi_Kurtulmu%C5%9Fo%C4%9Flu
Mevlüt Çavuşoğlu, 1968, https://tr.wikipedia.org/wiki/Mevl%C3%BCt_%C3%87avu%C5%9Fo%C4%9Flu
Mehmet Dülger, 1940, https://tr.wikipedia.org/wiki/Mehmet_D%C3%BClger
Osman Akman, 1958, https://tr.wikipedia.org/wiki/Osman_Akman
Fikret Badazlı, 1940, https://tr.wikipedia.org/wiki/Fikret_Badazl%C4%B1
Burhan Kılıç, 1953, https://tr.wikipedia.org/wiki/Burhan_K%C4%B1l%C4%B1%C3%A7
Deniz Baykal, 1938, https://tr.wikipedia.org/wiki/Deniz_Baykal
Osman Kaptan, 1946, https://tr.wikipedia.org/wiki/Osman_Kaptan
Feridun Fikret Baloğlu, 1945, https://tr.wikipedia.org/wiki/Feridun_Fikret_Balo%C4%9Flu
Hüseyin Ekmekcioğlu, 1959, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Ekmekcio%C4%9Flu
Atila Emek, 1947, https://tr.wikipedia.org/wiki/Atila_Emek
Tuncay Ercenk, 1950, https://tr.wikipedia.org/wiki/Tuncay_Ercenk
Nail Kamacı, 1959, https://tr.wikipedia.org/wiki/Nail_Kamac%C4%B1
Osman Özcan, 1971, https://tr.wikipedia.org/wiki/Osman_%C3%96zcan
Kenan Altun, 1956, https://tr.wikipedia.org/wiki/Kenan_Altun
Ensar Öğüt, 1946, https://tr.wikipedia.org/wiki/Ensar_%C3%96%C4%9F%C3%BCt
Orhan Yıldız, 1962, https://tr.wikipedia.org/wiki/Orhan_Y%C4%B1ld%C4%B1z
Yüksel Çorbacıoğlu, 1962, https://tr.wikipedia.org/wiki/Y%C3%BCksel_%C3%87orbac%C4%B1o%C4%9Flu
Atilla Koç, 1946, https://tr.wikipedia.org/wiki/Atilla_Ko%C3%A7
Ahmet Ertürk, 1953, https://tr.wikipedia.org/wiki/Ahmet_Ert%C3%BCrk_(siyaset%C3%A7i,_1953)
Ahmet Rıza Acar, 1947, https://tr.wikipedia.org/wiki/Ahmet_R%C4%B1za_Acar
Semiha Öyüş, 1960, https://tr.wikipedia.org/wiki/Semiha_%C3%96y%C3%BC%C5%9F
Özlem Çerçioğlu, 1968, https://tr.wikipedia.org/wiki/%C3%96zlem_%C3%87er%C3%A7io%C4%9Flu
Mehmet Bozbaş, 1957, https://tr.wikipedia.org/wiki/Mehmet_Bozta%C5%9F
Mehmet Mesut Özakcan, 1952, https://tr.wikipedia.org/wiki/Mehmet_Mesut_%C3%96zakcan
Turhan Çömez, 1965, https://tr.wikipedia.org/wiki/Turhan_%C3%87%C3%B6mez
Ahmet Edip Uğur, 1950, https://tr.wikipedia.org/wiki/Ahmet_Edip_U%C4%9Fur
Ali Aydınlıoğlu, 1958, https://tr.wikipedia.org/wiki/Ali_Ayd%C4%B1nl%C4%B1o%C4%9Flu
İsmail Özgün, 1956, https://tr.wikipedia.org/wiki/%C4%B0smail_%C3%96zg%C3%BCn
2007 – Balıkesir, 2007, https://tr.wikipedia.org/wiki/Ali_Osman_Sali
Ali Kemal Deveciler, 1958, https://tr.wikipedia.org/wiki/Ali_Kemal_Deveciler
Hacı İbrahim Kabarık, 1956, https://tr.wikipedia.org/wiki/Hac%C4%B1_%C4%B0brahim_Kabar%C4%B1k
Mehmet Asım Kulak, 1964, https://tr.wikipedia.org/wiki/Mehmet_As%C4%B1m_Kulak
Afif Demirkıran, 1952, https://tr.wikipedia.org/wiki/Afif_Demirk%C4%B1ran
Ahmet İnal, 1954, https://tr.wikipedia.org/wiki/Ahmet_%C4%B0nal
Memet Ali Suçin, 1954, https://tr.wikipedia.org/wiki/Memet_Ali_Su%C3%A7in
Mehmet Nezir Nasıroğlu, 1960, https://tr.wikipedia.org/wiki/Mehmet_Nezir_Nas%C4%B1ro%C4%9Flu
Fetani Battal, 1963, https://tr.wikipedia.org/wiki/Fetani_Battal
Ülkü Gökalp Güney, 1935, https://tr.wikipedia.org/wiki/%C3%9Clk%C3%BC_G%C3%B6kalp_G%C3%BCney
Fahrettin Poyraz, 1968, https://tr.wikipedia.org/wiki/Fahrettin_Poyraz
Yaşar Tüzün, 1966, https://tr.wikipedia.org/wiki/Ya%C5%9Far_T%C3%BCz%C3%BCn
Adalet ve Kalkınma Partisi, 2001, https://tr.wikipedia.org/wiki/Abdurrahman_An%C4%B1k
Feyzi Berdibek, 1957, https://tr.wikipedia.org/wiki/Feyzi_Berdibek
Mahfuz Güler, 1953, https://tr.wikipedia.org/wiki/Mahfuz_G%C3%BCler
Abdurrahim Aksoy, 1959, https://tr.wikipedia.org/wiki/Abdurrahim_Aksoy
Zeki Ergezen, 1949, https://tr.wikipedia.org/wiki/Zeki_Ergezen
Vahit Kiler, 1966, https://tr.wikipedia.org/wiki/Vahit_Kiler
Edip Safder Gaydalı[1], 1952, https://tr.wikipedia.org/wiki/Edip_Safder_Gaydal%C4%B1
Yüksel Coşkunyürek, 1965, https://tr.wikipedia.org/wiki/Y%C3%BCksel_Co%C5%9Fkuny%C3%BCrek
Adalet ve Kalkınma Partisi, 2001, https://tr.wikipedia.org/wiki/Mehmet_G%C3%BCner
2007 – Bolu, 2007, https://tr.wikipedia.org/wiki/Metin_Y%C4%B1lmaz_(1968_do%C4%9Fumlu_siyaset%C3%A7i)
Mehmet Alp, 1953, https://tr.wikipedia.org/wiki/Mehmet_Alp_(Bucak_do%C4%9Fumlu)
Bayram Özçelik, 1966, https://tr.wikipedia.org/wiki/Bayram_%C3%96z%C3%A7elik
Ramazan Kerim Özkan, 1958, https://tr.wikipedia.org/wiki/Ramazan_Kerim_%C3%96zkan
Ertuğrul Yalçınbayır, 1946, https://tr.wikipedia.org/wiki/Ertu%C4%9Frul_Yal%C3%A7%C4%B1nbay%C4%B1r
Faruk Çelik, 1956, https://tr.wikipedia.org/wiki/Faruk_%C3%87elik
Abdulmecit Alp, 1956, https://tr.wikipedia.org/wiki/Abdulmecit_Alp
Faruk Anbarcıoğlu, 1955, https://tr.wikipedia.org/wiki/Faruk_Anbarc%C4%B1o%C4%9Flu
Mustafa Dündar, 1965, https://tr.wikipedia.org/wiki/Mustafa_D%C3%BCndar_(siyaset%C3%A7i,_1965)
Zafer Hıdıroğlu, 1954, https://tr.wikipedia.org/wiki/Zafer_H%C4%B1d%C4%B1ro%C4%9Flu
2007 – Bursa, 2007, https://tr.wikipedia.org/wiki/Sedat_K%C4%B1z%C4%B1lc%C4%B1kl%C4%B1
Şevket Orhan, 1959, https://tr.wikipedia.org/wiki/%C5%9Eevket_Orhan
Niyazi Pakyürek, 1952, https://tr.wikipedia.org/wiki/Niyazi_Paky%C3%BCrek
2007 – Bursa, 2007, https://tr.wikipedia.org/wiki/Mehmet_Emin_Tutan
Mehmet Küçükaşık, 1960, https://tr.wikipedia.org/wiki/Mehmet_K%C3%BC%C3%A7%C3%BCka%C5%9F%C4%B1k
Mustafa Özyurt, 1941, https://tr.wikipedia.org/wiki/Mustafa_%C3%96zyurt
Kemal Demirel,, 1955, https://tr.wikipedia.org/wiki/Kemal_Demirel
Ali Dinçer, 1945, https://tr.wikipedia.org/wiki/Ali_Din%C3%A7er
Mehmet Daniş, 1971, https://tr.wikipedia.org/wiki/Mehmet_Dani%C5%9F
Ahmet Küçük,, 1958, https://tr.wikipedia.org/wiki/Ahmet_K%C3%BC%C3%A7%C3%BCk
İsmail Özay[1], 1953, https://tr.wikipedia.org/wiki/%C4%B0smail_%C3%96zay
Tevfik Akbak, 1956, https://tr.wikipedia.org/wiki/Tevfik_Akbak
İsmail Ericekli, 1959, https://tr.wikipedia.org/wiki/%C4%B0smail_Ericekli
Hikmet Özdemir, 1946, https://tr.wikipedia.org/wiki/Hikmet_%C3%96zdemir
Agah Kafkas, 1956, https://tr.wikipedia.org/wiki/Agah_Kafkas
Ali Yüksel Kavuştu, 1937, https://tr.wikipedia.org/wiki/Ali_Y%C3%BCksel_Kavu%C5%9Ftu
Muzaffer Külcü, 1971, https://tr.wikipedia.org/wiki/Muzaffer_K%C3%BClc%C3%BC
Murat Yıldırım, 1958, https://tr.wikipedia.org/wiki/Murat_Y%C4%B1ld%C4%B1r%C4%B1m_(siyaset%C3%A7i)
2007 – Denizli, 2007, https://tr.wikipedia.org/wiki/Mehmet_Salih_Erdo%C4%9Fan
Ümmet Kandoğan, 1956, https://tr.wikipedia.org/wiki/%C3%9Cmmet_Kando%C4%9Fan
Doğru Yol Partisi, 1983, https://tr.wikipedia.org/wiki/Do%C4%9Fru_Yol_Partisi
Mehmet Yüksektepe, 1966, https://tr.wikipedia.org/wiki/Mehmet_Y%C3%BCksektepe
Mustafa Gazalcı,, 1945, https://tr.wikipedia.org/wiki/Mustafa_Gazalc%C4%B1
Mehmet Ugur Neşşar, 1951, https://tr.wikipedia.org/wiki/Mehmet_Ugur_Ne%C5%9F%C5%9Far
Veli Haşim Oral, 1955, https://tr.wikipedia.org/wiki/Veli_Ha%C5%9Fim_Oral
Mehmet İhsan Arslan, 1948, https://tr.wikipedia.org/wiki/Mehmet_%C4%B0hsan_Arslan
Osman Aslan, 1954, https://tr.wikipedia.org/wiki/Osman_Aslan
Mehmet Mehdi Eker, 1956, https://tr.wikipedia.org/wiki/Mehmet_Mehdi_Eker
Ali İhsan Merdanoğlu, 1963, https://tr.wikipedia.org/wiki/Ali_%C4%B0hsan_Merdano%C4%9Flu
Mehmet Fehmi Uyanık, 1934, https://tr.wikipedia.org/wiki/Mehmet_Fehmi_Uyan%C4%B1k
İrfan Riza Yazıcıoğlu, 1961, https://tr.wikipedia.org/wiki/%C4%B0rfan_Riza_Yaz%C4%B1c%C4%B1o%C4%9Flu
Adalet ve Kalkınma Partisi, 2015, https://tr.wikipedia.org/wiki/Mesut_De%C4%9Fer
Anavatan Partisi, 2005, https://tr.wikipedia.org/wiki/Muhsin_Ko%C3%A7yi%C4%9Fit
Yaşar Yakış, 1938, https://tr.wikipedia.org/wiki/Ya%C5%9Far_Yak%C4%B1%C5%9F
Fahri Çakır, 1958, https://tr.wikipedia.org/wiki/Fahri_%C3%87ak%C4%B1r
Metin Kaşıkoğlu, 1966, https://tr.wikipedia.org/wiki/Metin_Ka%C5%9F%C4%B1ko%C4%9Flu
Rasim Çakır, 1957, https://tr.wikipedia.org/wiki/Rasim_%C3%87ak%C4%B1r
Necdet Budak, 1964, https://tr.wikipedia.org/wiki/Necdet_Budak
Nejat Gencan, 1952, https://tr.wikipedia.org/wiki/Nejat_Gencan
Ali Ayağ, 1950, https://tr.wikipedia.org/wiki/Ali_Aya%C4%9F
Mehmet Necati Çetinkaya, 1943, https://tr.wikipedia.org/wiki/Mehmet_Necati_%C3%87etinkaya
Zülfü Demirbağ,, 1956, https://tr.wikipedia.org/wiki/Z%C3%BClf%C3%BC_Demirba%C4%9F
Şemsettin Murat, 1952, https://tr.wikipedia.org/wiki/%C5%9Eemsettin_Murat
Abdulbaki Türkoğlu, 1952, https://tr.wikipedia.org/wiki/Abdulbaki_T%C3%BCrko%C4%9Flu
Mehmet Kemal Ağar, 1951, https://tr.wikipedia.org/wiki/Mehmet_A%C4%9Far
Doğru Yol Partisi, 1983, https://tr.wikipedia.org/wiki/Do%C4%9Fru_Yol_Partisi
Talip Kaban, 1955, https://tr.wikipedia.org/wiki/Talip_Kaban
Tevhit Karakaya,, 1955, https://tr.wikipedia.org/wiki/Tevhit_Karakaya
Erol Tınastepe,, 1957, https://tr.wikipedia.org/wiki/Erol_T%C4%B1nastepe
Recep Akdağ, 1960, https://tr.wikipedia.org/wiki/Recep_Akda%C4%9F
Mustafa Nuri Akbulut, 1957, https://tr.wikipedia.org/wiki/Mustafa_Nuri_Akbulut
Mücahit Daloğlu, 1956, https://tr.wikipedia.org/wiki/M%C3%BCcahit_Dalo%C4%9Flu
2007 – Erzurum, 2007, https://tr.wikipedia.org/wiki/Muzaffer_G%C3%BClyurt
Mustafa Ilıcalı, 1954, https://tr.wikipedia.org/wiki/Mustafa_Il%C4%B1cal%C4%B1
İbrahim Özdoğan, 1956, https://tr.wikipedia.org/wiki/%C4%B0brahim_%C3%96zdo%C4%9Fan
Ömer Özyılmaz, 1952, https://tr.wikipedia.org/wiki/%C3%96mer_%C3%96zy%C4%B1lmaz
Fahri Keskin, 1947, https://tr.wikipedia.org/wiki/Fahri_Keskin
Hasan Murat Mercan, 1959, https://tr.wikipedia.org/wiki/Hasan_Murat_Mercan
Muharrem Tozçöken,, 1946, https://tr.wikipedia.org/wiki/Muharrem_Toz%C3%A7%C3%B6ken
Mehmet Cevdet Selvi, 1943, https://tr.wikipedia.org/wiki/Cevdet_Selvi
Mehmet Ali Arıkan, 1943, https://tr.wikipedia.org/wiki/Mehmet_Ali_Ar%C4%B1kan_(Eski%C5%9Fehir_do%C4%9Fumlu)
Mehmet Vedat Yücesan, 1948, https://tr.wikipedia.org/wiki/Mehmet_Vedat_Y%C3%BCcesan
Prof.Dr.Ömer Abuşoğlu,, 1951, https://tr.wikipedia.org/wiki/%C3%96mer_Abu%C5%9Fo%C4%9Flu
Nurettin Aktaş, 1947, https://tr.wikipedia.org/wiki/Nurettin_Akta%C5%9F
Mehmet Sarı, 1968, https://tr.wikipedia.org/wiki/Mehmet_Sar%C4%B1_(1968_do%C4%9Fumlu_siyaset%C3%A7i)
Fatma Şahin[2], 1966, https://tr.wikipedia.org/wiki/Fatma_%C5%9Eahin
Recep Tayyip Erdoğan, 2003, https://tr.wikipedia.org/wiki/K%C3%BCr%C5%9Fad_T%C3%BCzmen
Ahmet Uzer, 1956, https://tr.wikipedia.org/wiki/Ahmet_Uzer
Abdülkadir Ateş, 1944, https://tr.wikipedia.org/wiki/Abd%C3%BClkadir_Ate%C5%9F
Mustafa Yılmaz, 1949, https://tr.wikipedia.org/wiki/Mustafa_Y%C4%B1lmaz_(Gaziantepli)
Ahmet Yılmazkaya, 1946, https://tr.wikipedia.org/wiki/Ahmet_Y%C4%B1lmazkaya
Nurettin Canikli, 1960, https://tr.wikipedia.org/wiki/Nurettin_Canikli
Hasan Aydın, 1971, https://tr.wikipedia.org/wiki/Hasan_Ayd%C4%B1n
Adem Tatlı, 1968, https://tr.wikipedia.org/wiki/Adem_Tatl%C4%B1
2007 – Giresun, 2007, https://tr.wikipedia.org/wiki/Ali_Tem%C3%BCr
Mehmet Işık, 1938, https://tr.wikipedia.org/wiki/Mehmet_I%C5%9F%C4%B1k
Adalet ve Kalkınma Partisi, 2001, https://tr.wikipedia.org/wiki/Sabri_Varan
Temel Yılmaz, 1953, https://tr.wikipedia.org/wiki/Temel_Y%C4%B1lmaz
Fehmi Öztunç, 1960, https://tr.wikipedia.org/wiki/Fehmi_%C3%96ztun%C3%A7
1995 – Hakkari 
2002 – Hakkari, 1995, https://tr.wikipedia.org/wiki/Mustafa_Zeydan
Esat Canan,, 1950, https://tr.wikipedia.org/wiki/Esat_Canan
Sadullah Ergin, 1964, https://tr.wikipedia.org/wiki/Sadullah_Ergin
Doğru Yol Partisi, 2005, https://tr.wikipedia.org/wiki/Mehmet_Eraslan
Bağımsız, 2006, https://tr.wikipedia.org/wiki/Fuat_Ge%C3%A7en
Mehmet Soydan, 1962, https://tr.wikipedia.org/wiki/Mehmet_Soydan
İsmail Soylu, 1955, https://tr.wikipedia.org/wiki/%C4%B0smail_Soylu
Cumhuriyet Halk Partisi, 2005, https://tr.wikipedia.org/wiki/Z%C3%BCheyir_Amber
İnal Batu, 1936, https://tr.wikipedia.org/wiki/%C4%B0nal_Batu
Doğru Yol Partisi, 1983, https://tr.wikipedia.org/wiki/Do%C4%9Fru_Yol_Partisi
Fuat Çay, 1949, https://tr.wikipedia.org/wiki/Fuat_%C3%87ay
Gökhan Durgun,, 1957, https://tr.wikipedia.org/wiki/G%C3%B6khan_Durgun
Abdulaziz Yazar, 1951, https://tr.wikipedia.org/wiki/Abdulaziz_Yazar
Dursun Akdemir,, 1946, https://tr.wikipedia.org/wiki/Dursun_Akdemir
Yücel Artantaş, 1952, https://tr.wikipedia.org/wiki/Y%C3%BCcel_Artanta%C5%9F
Anavatan Partisi, 2005, https://tr.wikipedia.org/wiki/Mehmet_Sait_Arma%C4%9Fan
Erkan Mumcu, 1963, https://tr.wikipedia.org/wiki/Erkan_Mumcu
Recep Özel, 1969, https://tr.wikipedia.org/wiki/Recep_%C3%96zel_(siyaset%C3%A7i,_1969)
Mevlüt Coşkuner, 1945, https://tr.wikipedia.org/wiki/Mevl%C3%BCt_Co%C5%9Fkuner
Abdülkadir Aksu, 1944, https://tr.wikipedia.org/wiki/Abd%C3%BClkadir_Aksu
Murat Başesgioğlu, 1955, https://tr.wikipedia.org/wiki/Murat_Ba%C5%9Fesgio%C4%9Flu
Egemen Bağış, 1970, https://tr.wikipedia.org/wiki/Egemen_Ba%C4%9F%C4%B1%C5%9F
Nazım Ekren, 1956, https://tr.wikipedia.org/wiki/Naz%C4%B1m_Ekren
Ekrem Erdem, 1948, https://tr.wikipedia.org/wiki/Ekrem_Erdem
Nevzat Yalçıntaş, 1933, https://tr.wikipedia.org/wiki/Nevzat_Yal%C3%A7%C4%B1nta%C5%9F
İdris Naim Şahin, 1956, https://tr.wikipedia.org/wiki/%C4%B0dris_Naim_%C5%9Eahin
Mehmet Ali Şahin, 1950, https://tr.wikipedia.org/wiki/Mehmet_Ali_%C5%9Eahin
Tayyar Altıkulaç, 1938, https://tr.wikipedia.org/wiki/Tayyar_Alt%C4%B1kula%C3%A7
Mehmet Mustafa Açıkalın,, 1951, https://tr.wikipedia.org/wiki/Mehmet_Mustafa_A%C3%A7%C4%B1kal%C4%B1n
Güldal Akşit, 1960, https://tr.wikipedia.org/wiki/G%C3%BCldal_Ak%C5%9Fit
İlhan Albayrak, 1954, https://tr.wikipedia.org/wiki/%C4%B0lhan_Albayrak
İsmet Atalay, 1937, https://tr.wikipedia.org/wiki/%C4%B0smet_Atalay
Mehmet Sekmen[1], 1958, https://tr.wikipedia.org/wiki/Mehmet_Sekmen
1999 – İstanbul 
2002 – İstanbul, 1999, https://tr.wikipedia.org/wiki/Azmi_Ate%C5%9F
Lokman Ayva, 1966, https://tr.wikipedia.org/wiki/Lokman_Ayva
Mustafa Baş, 1956, https://tr.wikipedia.org/wiki/Mustafa_Ba%C5%9F
Yahya Baş, 1952, https://tr.wikipedia.org/wiki/Yahya_Ba%C5%9F
Nusret Bayraktar, 1951, https://tr.wikipedia.org/wiki/Nusret_Bayraktar
Hüseyin Besli, 1953, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Besli
2007 – İstanbul, 2007, https://tr.wikipedia.org/wiki/Alaattin_B%C3%BCy%C3%BCkkaya
Ali Coşkun, 1939, https://tr.wikipedia.org/wiki/Ali_Co%C5%9Fkun_(siyaset%C3%A7i)
Nimet Baş, 1965, https://tr.wikipedia.org/wiki/Nimet_%C3%87ubuk%C3%A7u
2007 – İstanbul, 2007, https://tr.wikipedia.org/wiki/Mehmet_Beyaz%C4%B1t_Denizolgun
Gürsoy Erol,, 1956, https://tr.wikipedia.org/wiki/G%C3%BCrsoy_Erol
İrfan Gündüz, 1950, https://tr.wikipedia.org/wiki/%C4%B0rfan_G%C3%BCnd%C3%BCz
Kemal Unakıtan, 1946, https://tr.wikipedia.org/wiki/Kemal_Unak%C4%B1tan
Zeynep Karahan Uslu, 1969, https://tr.wikipedia.org/wiki/Zeynep_Arma%C4%9Fan_Uslu
İnci Özdemir, 1952, https://tr.wikipedia.org/wiki/%C4%B0nci_%C3%96zdemir
Hayati Yazıcı, 1952, https://tr.wikipedia.org/wiki/Hayati_Yaz%C4%B1c%C4%B1
Binali Yıldırım, 1955, https://tr.wikipedia.org/wiki/Binali_Y%C4%B1ld%C4%B1r%C4%B1m
Ali İbiş, 1953, https://tr.wikipedia.org/wiki/Ali_%C4%B0bi%C5%9F
İbrahim Reyhan Özal, 1965, https://tr.wikipedia.org/wiki/%C4%B0brahim_Reyhan_%C3%96zal
Halide İncekara, 1959, https://tr.wikipedia.org/wiki/Halide_%C4%B0ncekara
Ünal Kacır,, 1953, https://tr.wikipedia.org/wiki/%C3%9Cnal_Kac%C4%B1r
1999 – İstanbul 
2002 – İstanbul, 1999, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Kansu
Emin Şirin, 1948, https://tr.wikipedia.org/wiki/Emin_%C5%9Eirin
Cengiz Kaptanoğlu, 1947, https://tr.wikipedia.org/wiki/Cengiz_Kaptano%C4%9Flu
Muharrem Karslı, 1938, https://tr.wikipedia.org/wiki/Muharrem_Karsl%C4%B1
Burhan Kuzu, 1955, https://tr.wikipedia.org/wiki/Burhan_Kuzu
bağımsız 
Adalet ve Kalkınma Partisi, 2002, https://tr.wikipedia.org/wiki/G%C3%B6ksal_K%C3%BC%C3%A7%C3%BCkali
Recep Koral, 1948, https://tr.wikipedia.org/wiki/Recep_Koral
Failed to parse: https://tr.wikipedia.org/wiki/Kemal_K%C4%B1l%C4%B1%C3%A7daro%C4%9Flu
Ömer Zülfü Livaneli[1], 1946, https://tr.wikipedia.org/wiki/%C3%96mer_Z%C3%BClf%C3%BC_Livaneli
Onur Başaran Öymen[1], 1940, https://tr.wikipedia.org/wiki/Onur_%C3%96ymen
Hasan Fehmi Güneş, 1934, https://tr.wikipedia.org/wiki/Hasan_Fehmi_G%C3%BCne%C5%9F
Kemal Derviş, 1949, https://tr.wikipedia.org/wiki/Kemal_Dervi%C5%9F
Yaşar Nuri Öztürk, 1951, https://tr.wikipedia.org/wiki/Ya%C5%9Far_Nuri_%C3%96zt%C3%BCrk
Mustafa Şükrü Elekdağ, 1924, https://tr.wikipedia.org/wiki/Mustafa_%C5%9E%C3%BCkr%C3%BC_Elekda%C4%9F
Hasan Aydın, 1971, https://tr.wikipedia.org/wiki/Hasan_Ayd%C4%B1n
Bağımsız, 2005, https://tr.wikipedia.org/wiki/Memduh_Hac%C4%B1o%C4%9Flu
Halil Akyüz, 1938, https://tr.wikipedia.org/wiki/Halil_Aky%C3%BCz_(1938_do%C4%9Fumlu_siyaset%C3%A7i)
Ersin Arıoğlu, 1940, https://tr.wikipedia.org/wiki/Ersin_Ar%C4%B1o%C4%9Flu
Zeynep Damla Gürel, 1964, https://tr.wikipedia.org/wiki/Zeynep_Damla_G%C3%BCrel
Algan Hacaloğlu[1], 1940, https://tr.wikipedia.org/wiki/Algan_Hacalo%C4%9Flu
Birgen Keleş, 1939, https://tr.wikipedia.org/wiki/Birgen_Kele%C5%9F
Ahmet Güryüz Ketenci, 1938, https://tr.wikipedia.org/wiki/Ahmet_G%C3%BCry%C3%BCz_Ketenci
Sosyaldemokrat Halk Partisi, 2002, https://tr.wikipedia.org/wiki/Sosyaldemokrat_Halk_Partisi
Ali Kemal Kumkumoğlu, 1955, https://tr.wikipedia.org/wiki/Ali_Kemal_Kumkumo%C4%9Flu
Güldal Okuducu, 1958, https://tr.wikipedia.org/wiki/G%C3%BCldal_Alogan
Mehmet Sevigen, 1954, https://tr.wikipedia.org/wiki/Mehmet_Sevigen
Ali Rıza Gülçiçek, 1947, https://tr.wikipedia.org/wiki/Ali_R%C4%B1za_G%C3%BCl%C3%A7i%C3%A7ek
Ahmet Sırrı Özbek,, 1950, https://tr.wikipedia.org/wiki/Ahmet_S%C4%B1rr%C4%B1_%C3%96zbek
2002 – İstanbul, 2002, https://tr.wikipedia.org/wiki/Mehmet_Ali_%C3%96zpolat
Sıdıka Sarıbekir, 1957, https://tr.wikipedia.org/wiki/S%C4%B1d%C4%B1ka_Sar%C4%B1bekir
Berhan Şimşek, 1958, https://tr.wikipedia.org/wiki/Berhan_%C5%9Eim%C5%9Fek
Bihlun Tamaylıgil, 1966, https://tr.wikipedia.org/wiki/Bihlun_Tamayl%C4%B1gil
Bülent Hasan Tanla, 1946, https://tr.wikipedia.org/wiki/B%C3%BClent_Hasan_Tanla
Ali Topuz, 1932, https://tr.wikipedia.org/wiki/Ali_Topuz
Mehmet Aydın, 1943, https://tr.wikipedia.org/wiki/Mehmet_Ayd%C4%B1n_(1943_do%C4%9Fumlu_siyaset%C3%A7i)
Tevfik Ensari, 1948, https://tr.wikipedia.org/wiki/Tevfik_Ensari
Fatma Seniha Nükhet Hotar, 1961, https://tr.wikipedia.org/wiki/N%C3%BCkhet_Hotar
2007 – İzmir, 2007, https://tr.wikipedia.org/wiki/%C4%B0smail_Katmerci
Mehmet Sayım Tekelioğlu,, 1950, https://tr.wikipedia.org/wiki/Mehmet_Say%C4%B1m_Tekelio%C4%9Flu
Anavatan Partisi, 2005, https://tr.wikipedia.org/wiki/Serpil_Y%C4%B1ld%C4%B1z
Zekeriya Akçam, 1968, https://tr.wikipedia.org/wiki/Zekeriya_Ak%C3%A7am
Erdal Karademir, 1954, https://tr.wikipedia.org/wiki/Erdal_Karademir
Kıvılcım Kemal Anadol, 1941, https://tr.wikipedia.org/wiki/K%C4%B1v%C4%B1lc%C4%B1m_Kemal_Anadol
Hakkı Akalın, 1949, https://tr.wikipedia.org/wiki/Hakk%C4%B1_Akal%C4%B1n
Vezir Akdemir, 1954, https://tr.wikipedia.org/wiki/Vezir_Akdemir
Cânân Aritman[2], 1950, https://tr.wikipedia.org/wiki/C%C3%A2n%C3%A2n_Aritman
Ahmet Ersin,, 1950, https://tr.wikipedia.org/wiki/Ahmet_Ersin
2007 – İzmir, 2007, https://tr.wikipedia.org/wiki/Abdurrezzak_Erten
Bülent Baratalı, 1947, https://tr.wikipedia.org/wiki/B%C3%BClent_Baratal%C4%B1
Ali Rıza Bodur, 1946, https://tr.wikipedia.org/wiki/Ali_R%C4%B1za_Bodur
Yılmaz Kaya, 1962, https://tr.wikipedia.org/wiki/Y%C4%B1lmaz_Kaya
Türkan Miçooğulları, 1952, https://tr.wikipedia.org/wiki/T%C3%BCrkan_Mi%C3%A7oo%C4%9Fullar%C4%B1
Prof.Dr.Oğuz Oyan, 1947, https://tr.wikipedia.org/wiki/O%C4%9Fuz_Oyan
Enver Öktem,, 1957, https://tr.wikipedia.org/wiki/Enver_%C3%96ktem
Muharrem Toprak, 1947, https://tr.wikipedia.org/wiki/Muharrem_Toprak
Sedat Uzunbay, 1957, https://tr.wikipedia.org/wiki/Sedat_Uzunbay
Sosyal Demokrasi Partisi, 1983, https://tr.wikipedia.org/wiki/Hakk%C4%B1_%C3%9Clk%C3%BC
Nevzat Pakdil, 1950, https://tr.wikipedia.org/wiki/Nevzat_Pakdil
Avni Doğan, 1951, https://tr.wikipedia.org/wiki/Avni_Do%C4%9Fan_(siyaset%C3%A7i,_1951)
2007 – Kahramanmaraş, 2007, https://tr.wikipedia.org/wiki/Fatih_Ar%C4%B1kan
Mehmet Ali Bulut, 1956, https://tr.wikipedia.org/wiki/Mehmet_Ali_Bulut
Ali Sezal, 1946, https://tr.wikipedia.org/wiki/Ali_Sezal
Mehmet Parlakyiğit, 1941, https://tr.wikipedia.org/wiki/Mehmet_Parlakyi%C4%9Fit
Hasan Bilir, 1961, https://tr.wikipedia.org/wiki/Hasan_Bilir
Mehmet Ceylan, 1958, https://tr.wikipedia.org/wiki/Mehmet_Ceylan_(Karab%C3%BCkl%C3%BC)
Ali Öğüten, 1953, https://tr.wikipedia.org/wiki/Ali_%C3%96%C4%9F%C3%BCten
Mevlüt Akgün, 1966, https://tr.wikipedia.org/wiki/Mevl%C3%BCt_Akg%C3%BCn
Fikret Ünlü, 1943, https://tr.wikipedia.org/wiki/Fikret_%C3%9Cnl%C3%BC
Yusuf Selahattin Beyribey, 1955, https://tr.wikipedia.org/wiki/Yusuf_Selahattin_Beyribey
Zeki Karabayır, 1952, https://tr.wikipedia.org/wiki/Zeki_Karabay%C4%B1r
Bağımsız, 2005, https://tr.wikipedia.org/wiki/Selami_Yi%C4%9Fit
Hakkı Köylü, 1948, https://tr.wikipedia.org/wiki/Hakk%C4%B1_K%C3%B6yl%C3%BC
Sinan Özkan, 1972, https://tr.wikipedia.org/wiki/Sinan_%C3%96zkan_(Kastamonu_do%C4%9Fumlu)
Musa Sıvacıoğlu,, 1955, https://tr.wikipedia.org/wiki/Musa_S%C4%B1vac%C4%B1o%C4%9Flu
Adalet ve Kalkınma Partisi, 2015, https://tr.wikipedia.org/wiki/Mehmet_Y%C4%B1ld%C4%B1r%C4%B1m_(1949_do%C4%9Fumlu_siyaset%C3%A7i)
Abdullah Gül, 1950, https://tr.wikipedia.org/wiki/Abdullah_G%C3%BCl
Taner Yıldız, 1962, https://tr.wikipedia.org/wiki/Taner_Y%C4%B1ld%C4%B1z
Mustafa Elitaş, 1957, https://tr.wikipedia.org/wiki/Mustafa_Elita%C5%9F
Prof. Dr. Adem Baştürk,, 1950, https://tr.wikipedia.org/wiki/Adem_Ba%C5%9Ft%C3%BCrk
Mustafa Duru, 1949, https://tr.wikipedia.org/wiki/Mustafa_Duru
Niyazi Özcan, 1949, https://tr.wikipedia.org/wiki/Niyazi_%C3%96zcan
Sadık Yakut,, 1956, https://tr.wikipedia.org/wiki/Sad%C4%B1k_Yakut
Muharrem Eskiyapan, 1935, https://tr.wikipedia.org/wiki/Muharrem_Eskiyapan
Vahit Erdem, 1944, https://tr.wikipedia.org/wiki/Vahit_Erdem
Ramazan Can, 1970, https://tr.wikipedia.org/wiki/Ramazan_Can
Adalet ve Kalkınma Partisi, 2001, https://tr.wikipedia.org/wiki/Murat_Y%C4%B1lmazer
Halil Tiryaki, 1935, https://tr.wikipedia.org/wiki/Halil_Tiryaki
Ahmet Gökhan Sarıçam,, 1969, https://tr.wikipedia.org/wiki/Ahmet_G%C3%B6khan_Sar%C4%B1%C3%A7am
Yavuz Altınorak, 1962, https://tr.wikipedia.org/wiki/Yavuz_Alt%C4%B1norak
Mehmet Siyam Kesimoğlu[1], 1960, https://tr.wikipedia.org/wiki/Mehmet_Kesimo%C4%9Flu
Mikail Arslan, 1961, https://tr.wikipedia.org/wiki/Mikail_Arslan
Hacı Turan, 1968, https://tr.wikipedia.org/wiki/Hac%C4%B1_Turan
Hüseyin Bayındır, 1965, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Bay%C4%B1nd%C4%B1r
Hasan Kara, 1969, https://tr.wikipedia.org/wiki/Hasan_Kara
Veli Kaya, 1956, https://tr.wikipedia.org/wiki/Veli_Kaya
Nihat Ergün, 1962, https://tr.wikipedia.org/wiki/Nihat_Erg%C3%BCn
Osman Pepe, 1954, https://tr.wikipedia.org/wiki/Osman_Pepe
Mehmet Vecdi Gönül, 1939, https://tr.wikipedia.org/wiki/Vecdi_G%C3%B6n%C3%BCl
2007 – Kocaeli, 2007, https://tr.wikipedia.org/wiki/Ey%C3%BCp_Ayar
Muzaffer Baştopçu,, 1948, https://tr.wikipedia.org/wiki/Muzaffer_Ba%C5%9Ftop%C3%A7u
Nevzat Doğan, 1959, https://tr.wikipedia.org/wiki/Nevzat_Do%C4%9Fan
Mehmet Sefa Sirmen, 1949, https://tr.wikipedia.org/wiki/Mehmet_Sefa_Sirmen
İzzet Çetin, 1953, https://tr.wikipedia.org/wiki/%C4%B0zzet_%C3%87etin
Cumhuriyet Halk Partisi, 1995, https://tr.wikipedia.org/wiki/Salih_G%C3%BCn
2007 – Konya, 2007, https://tr.wikipedia.org/wiki/Hasan_Ang%C4%B1
2007 – Konya, 2007, https://tr.wikipedia.org/wiki/Ahmet_B%C3%BCy%C3%BCkakka%C5%9Flar
Muharrem Candan, 1956, https://tr.wikipedia.org/wiki/Muharrem_Candan
Remzi Çetin,, 1952, https://tr.wikipedia.org/wiki/Remzi_%C3%87etin
2007 – Konya, 2007, https://tr.wikipedia.org/wiki/Abdullah_%C3%87etinkaya
Orhan Erdem, 1963, https://tr.wikipedia.org/wiki/Orhan_Erdem
Sami Güçlü, 1950, https://tr.wikipedia.org/wiki/Sami_G%C3%BC%C3%A7l%C3%BC
Ahmet Işık, 1963, https://tr.wikipedia.org/wiki/Ahmet_I%C5%9F%C4%B1k
Mehmet Kılıç, 1955, https://tr.wikipedia.org/wiki/Mehmet_K%C4%B1l%C4%B1%C3%A7_(Konya_do%C4%9Fumlu)
Özkan Öksüz, 1953, https://tr.wikipedia.org/wiki/%C3%96zkan_%C3%96ks%C3%BCz
2007 – Konya 
2011 – Konya, 2007, https://tr.wikipedia.org/wiki/Kerim_%C3%96zkul
2007 – Konya
2011 – Konya, 2007, https://tr.wikipedia.org/wiki/Harun_T%C3%BCfekci
Mustafa Ünaldı,, 1945, https://tr.wikipedia.org/wiki/Mustafa_%C3%9Cnald%C4%B1
Doç. Dr.Halil Ürün, 1947, https://tr.wikipedia.org/wiki/Halil_%C3%9Cr%C3%BCn_(Konyal%C4%B1)
Atilla Kart, 1954, https://tr.wikipedia.org/wiki/Atilla_Kart
Nezir Büyükcengiz,, 1951, https://tr.wikipedia.org/wiki/Nezir_B%C3%BCy%C3%BCkcengiz
Soner Aksoy,, 1941, https://tr.wikipedia.org/wiki/Soner_Aksoy
Abdullah Erdem Cantimur,, 1961, https://tr.wikipedia.org/wiki/Abdullah_Erdem_Cantimur
Alaettin Güven, 1950, https://tr.wikipedia.org/wiki/Alaettin_G%C3%BCven
Hasan Fehmi Kinay,, 1964, https://tr.wikipedia.org/wiki/Hasan_Fehmi_Kinay
Hüsnü Ordu, 1953, https://tr.wikipedia.org/wiki/H%C3%BCsn%C3%BC_Ordu
Halil İbrahim Yılmaz, 1954, https://tr.wikipedia.org/wiki/Halil_%C4%B0brahim_Y%C4%B1lmaz_(siyaset%C3%A7i)
2007 – Malatya, 2007, https://tr.wikipedia.org/wiki/Fuat_%C3%96lmeztoprak
Ali Osman Başkurt, 1954, https://tr.wikipedia.org/wiki/Ali_Osman_Ba%C5%9Fkurt
Ahmet Münir Erkal, 1954, https://tr.wikipedia.org/wiki/Ahmet_M%C3%BCnir_Erkal
Miraç Akdoğan, 1958, https://tr.wikipedia.org/wiki/Mira%C3%A7_Akdo%C4%9Fan
Anavatan Partisi, 2005, https://tr.wikipedia.org/wiki/S%C3%BCleyman_Sar%C4%B1ba%C5%9F
Ferit Mevlüt Aslanoğlu,, 1952, https://tr.wikipedia.org/wiki/Ferit_Mevl%C3%BCt_Aslano%C4%9Flu
Bülent Arınç, 1948, https://tr.wikipedia.org/wiki/B%C3%BClent_Ar%C4%B1n%C3%A7
Hüseyin Tanrıverdi, 1956, https://tr.wikipedia.org/wiki/H%C3%BCseyin_Tanr%C4%B1verdi
İsmail Bilen, 1961, https://tr.wikipedia.org/wiki/%C4%B0smail_Bilen_(1961_do%C4%9Fumlu_siyaset%C3%A7i)
2007 – Manisa, 2007, https://tr.wikipedia.org/wiki/Mehmet_%C3%87er%C3%A7i
Hakan Taşcı, 1969, https://tr.wikipedia.org/wiki/Hakan_Ta%C5%9Fc%C4%B1
Süleyman Turgut, 1971, https://tr.wikipedia.org/wiki/S%C3%BCleyman_Turgut
Mustafa Erdoğan Yetenç,, 1946, https://tr.wikipedia.org/wiki/Mustafa_Erdo%C4%9Fan_Yeten%C3%A7
Nuri Çilingir, 1952, https://tr.wikipedia.org/wiki/Nuri_%C3%87ilingir
2011 – Manisa, 2011, https://tr.wikipedia.org/wiki/Hasan_%C3%96ren
Ufuk Özkan, 1960, https://tr.wikipedia.org/wiki/Ufuk_%C3%96zkan_(siyaset%C3%A7i)
Selahattin Cebeli, 1957, https://tr.wikipedia.org/wiki/Selahattin_Cebeli
Nihat Eri, 1948, https://tr.wikipedia.org/wiki/Nihat_Eri
Mehmet Beşir Hamidi, 1953, https://tr.wikipedia.org/wiki/Mehmet_Be%C5%9Fir_Hamidi
Mahmut Duyan, 1955, https://tr.wikipedia.org/wiki/Mahmut_Duyan
Anavatan Partisi, 2005, https://tr.wikipedia.org/wiki/Muharrem_Do%C4%9Fan
Süleyman Bölünmez, 1956, https://tr.wikipedia.org/wiki/S%C3%BCleyman_B%C3%B6l%C3%BCnmez
Dengir Mir Mehmet Fırat, 1943, https://tr.wikipedia.org/wiki/Dengir_Mir_Mehmet_F%C4%B1rat
2002 – Mersin, 2002, https://tr.wikipedia.org/wiki/Saffet_Benli
Ali Er, 1949, https://tr.wikipedia.org/wiki/Ali_Er
Adalet ve Kalkınma Partisi, 2001, https://tr.wikipedia.org/wiki/Mustafa_Eyiceo%C4%9Flu
2007 – Mersin, 2007, https://tr.wikipedia.org/wiki/%C3%96mer_%C4%B0nan
Mustafa Özyürek, 1939, https://tr.wikipedia.org/wiki/Mustafa_%C3%96zy%C3%BCrek
Sosyaldemokrat Halk Partisi, 2005, https://tr.wikipedia.org/wiki/Ersoy_Bulut
Vahit Çekmez, 1947, https://tr.wikipedia.org/wiki/Vahit_%C3%87ekmez
Anavatan Partisi, 2005, https://tr.wikipedia.org/wiki/H%C3%BCseyin_G%C3%BCler
2007 – Mersin, 2007, https://tr.wikipedia.org/wiki/Ali_Oksal
Hüseyin Özcan, 1946, https://tr.wikipedia.org/wiki/H%C3%BCseyin_%C3%96zcan_(1946_do%C4%9Fumlu_siyaset%C3%A7i)
Sosyaldemokrat Halkçı Parti, 1985, https://tr.wikipedia.org/wiki/%C5%9Eefik_Zengin
Hasan Özyer, 1960, https://tr.wikipedia.org/wiki/Hasan_%C3%96zyer
Orhan Seyfi Terzibaşıoğlu, 1947, https://tr.wikipedia.org/wiki/Orhan_Seyfi_Terziba%C5%9F%C4%B1o%C4%9Flu
Ali Arslan, 1955, https://tr.wikipedia.org/wiki/Ali_Arslan_(milletvekili)
2007 – Muğla, 2007, https://tr.wikipedia.org/wiki/G%C3%BCrol_Ergin
Fahrettin Üstün, 1966, https://tr.wikipedia.org/wiki/Fahrettin_%C3%9Cst%C3%BCn
Ali Cumhur Yaka, 1950, https://tr.wikipedia.org/wiki/Ali_Cumhur_Yaka
2007 – Muş, 2007, https://tr.wikipedia.org/wiki/Seracettin_Karaya%C4%9F%C4%B1z
Sabahattin Yıldız, 1960, https://tr.wikipedia.org/wiki/Sabahattin_Y%C4%B1ld%C4%B1z
Medeni Yılmaz, 1954, https://tr.wikipedia.org/wiki/Medeni_Y%C4%B1lmaz
Mehmet Şerif Ertuğrul, 1947, https://tr.wikipedia.org/wiki/Mehmet_%C5%9Eerif_Ertu%C4%9Frul
Mehmet Elkatmış, 1947, https://tr.wikipedia.org/wiki/Mehmet_Elkatm%C4%B1%C5%9F
Rıtvan Köybaşı, 1965, https://tr.wikipedia.org/wiki/R%C4%B1tvan_K%C3%B6yba%C5%9F%C4%B1
Osman Seyfi, 1948, https://tr.wikipedia.org/wiki/Osman_Seyfi
Mahmut Uğur Çetin, 1965, https://tr.wikipedia.org/wiki/Mahmut_U%C4%9Fur_%C3%87etin
Erdoğan Özegen, 1965, https://tr.wikipedia.org/wiki/Erdo%C4%9Fan_%C3%96zegen
Orhan Eraslan,, 1954, https://tr.wikipedia.org/wiki/Orhan_Eraslan
Eyüp Fatsa, 1961, https://tr.wikipedia.org/wiki/Ey%C3%BCp_Fatsa
Mehmet Hilmi Güler[1], 1949, https://tr.wikipedia.org/wiki/Mehmet_Hilmi_G%C3%BCler
Hamit Taşcı, 1946, https://tr.wikipedia.org/wiki/Hamit_Ta%C5%9Fc%C4%B1
Cemal Uysal, 1940, https://tr.wikipedia.org/wiki/Cemal_Uysal
Enver Yılmaz,, 1970, https://tr.wikipedia.org/wiki/Enver_Y%C4%B1lmaz_(siyaset%C3%A7i)
İdris Sami Tandoğdu, 1945, https://tr.wikipedia.org/wiki/%C4%B0dris_Sami_Tando%C4%9Fdu
Sosyal Demokrasi Partisi, 1983, https://tr.wikipedia.org/wiki/Kaz%C4%B1m_T%C3%BCrkmen
Durdu Mehmet Kastal, 1969, https://tr.wikipedia.org/wiki/Durdu_Mehmet_Kastal
Mehmet Sarı, 1945, https://tr.wikipedia.org/wiki/Mehmet_Sar%C4%B1_(1945_do%C4%9Fumlu_siyaset%C3%A7i)
2002 – Osmaniye, 2002, https://tr.wikipedia.org/wiki/%C5%9E%C3%BCkr%C3%BC_%C3%9Cnal
Necati Uzdil, 1946, https://tr.wikipedia.org/wiki/Necati_Uzdil
İlyas Çakır, 1966, https://tr.wikipedia.org/wiki/%C4%B0lyas_%C3%87ak%C4%B1r
Abdulkadir Kart, 1964, https://tr.wikipedia.org/wiki/Abdulkadir_Kart
İmdat Sütlüoğlu, 1953, https://tr.wikipedia.org/wiki/%C4%B0mdat_S%C3%BCtl%C3%BCo%C4%9Flu
Erol Aslan Cebeci, 1963, https://tr.wikipedia.org/wiki/Erol_Aslan_Cebeci
Hasan Ali Çelik, 1959, https://tr.wikipedia.org/wiki/Hasan_Ali_%C3%87elik
Şaban Dişli, 1958, https://tr.wikipedia.org/wiki/%C5%9Eaban_Di%C5%9Fli
Ayhan Sefer Üstün, 1970, https://tr.wikipedia.org/wiki/Ayhan_Sefer_%C3%9Cst%C3%BCn
Recep Yıldırım,, 1949, https://tr.wikipedia.org/wiki/Recep_Y%C4%B1ld%C4%B1r%C4%B1m
Mustafa Demir[1], 1961, https://tr.wikipedia.org/wiki/Mustafa_Demir_(siyaset%C3%A7i,_1961)
Suat Kılıç, 1972, https://tr.wikipedia.org/wiki/Suat_K%C4%B1l%C4%B1%C3%A7
Mustafa Çakır, 1949, https://tr.wikipedia.org/wiki/Mustafa_%C3%87ak%C4%B1r
Cemal Yılmaz Demir, 1960, https://tr.wikipedia.org/wiki/Cemal_Y%C4%B1lmaz_Demir
Musa Uzunkaya, 1952, https://tr.wikipedia.org/wiki/Musa_Uzunkaya
Ahmet Yeni, 1955, https://tr.wikipedia.org/wiki/Ahmet_Yeni
Ahmet Haluk Koç, 1954, https://tr.wikipedia.org/wiki/Haluk_Ko%C3%A7
İlyas Sezai Önder, 1942, https://tr.wikipedia.org/wiki/%C4%B0lyas_Sezai_%C3%96nder
Failed to parse: https://tr.wikipedia.org/wiki/Recep_Tayyip_Erdo%C4%9Fan
Öner Ergenç, 1944, https://tr.wikipedia.org/wiki/%C3%96ner_Ergen%C3%A7
Cahit Can, 1956, https://tr.wikipedia.org/wiki/Cahit_Can
Mustafa Öztürk, 1949, https://tr.wikipedia.org/wiki/Mustafa_%C3%96zt%C3%BCrk_(siyaset%C3%A7i,_1949)
Engin Altay, 1963, https://tr.wikipedia.org/wiki/Engin_Altay
Abdüllatif Şener, 1954, https://tr.wikipedia.org/wiki/Abd%C3%BCllatif_%C5%9Eener
Selami Uzun,, 1957, https://tr.wikipedia.org/wiki/Selami_Uzun
Osman Kılıç,, 1946, https://tr.wikipedia.org/wiki/Osman_K%C4%B1l%C4%B1%C3%A7_(siyaset%C3%A7i,_1946)
Ömer Kulaksız, 1968, https://tr.wikipedia.org/wiki/%C3%96mer_Kulaks%C4%B1z
Orhan Taş, 1961, https://tr.wikipedia.org/wiki/Orhan_Ta%C5%9F
Nurettin Sözen, 1937, https://tr.wikipedia.org/wiki/Nurettin_S%C3%B6zen
Yahya Akman, 1966, https://tr.wikipedia.org/wiki/Yahya_Akman
Mehmet Faruk Bayrak, 1964, https://tr.wikipedia.org/wiki/Mehmet_Faruk_Bayrak
Sabahattin Cevheri, 1950, https://tr.wikipedia.org/wiki/Sabahattin_Cevheri
Zülfükar İzol, 1959, https://tr.wikipedia.org/wiki/Z%C3%BClf%C3%BCkar_%C4%B0zol
Mahmut Kaplan, 1947, https://tr.wikipedia.org/wiki/Mahmut_Kaplan
Mehmet Atilla Maraş, 1949, https://tr.wikipedia.org/wiki/Mehmet_Atilla_Mara%C5%9F
Mehmet Özlek, 1939, https://tr.wikipedia.org/wiki/Mehmet_%C3%96zlek
Abdurrahman Müfit Yetkin, 1953, https://tr.wikipedia.org/wiki/Abdurrahman_M%C3%BCfit_Yetkin
Mehmet Vedat Melik, 1954, https://tr.wikipedia.org/wiki/Mehmet_Vedat_Melik
Anavatan Partisi, 2005, https://tr.wikipedia.org/wiki/Turan_T%C3%BCys%C3%BCz
İbrahim Hakkı Birlik, 1952, https://tr.wikipedia.org/wiki/%C4%B0brahim_Hakk%C4%B1_Birlik
Abdullah Veli Seyda, 1958, https://tr.wikipedia.org/wiki/Abdullah_Veli_Seyda
2002 – Şırnak, 2002, https://tr.wikipedia.org/wiki/Mehmet_Tatar_(siyaset%C3%A7i)
Doğru Yol Partisi, 1983, https://tr.wikipedia.org/wiki/Do%C4%9Fru_Yol_Partisi
Tevfik Ziyaeddin Akbulut,, 1949, https://tr.wikipedia.org/wiki/Tevfik_Ziyaeddin_Akbulut
Ahmet Kambur, 1954, https://tr.wikipedia.org/wiki/Ahmet_Kambur
Erdoğan Kaplan, 1954, https://tr.wikipedia.org/wiki/Erdo%C4%9Fan_Kaplan
Mehmet Nuri Saygun, 1952, https://tr.wikipedia.org/wiki/Mehmet_Nuri_Saygun
2002 – Tekirdağ
2007 – Tekirdağ, 2002, https://tr.wikipedia.org/wiki/Enis_T%C3%BCt%C3%BCnc%C3%BC
Zeyid Aslan,, 1965, https://tr.wikipedia.org/wiki/Zeyid_Aslan
Şükrü Ayalan, 1963, https://tr.wikipedia.org/wiki/%C5%9E%C3%BCkr%C3%BC_Ayalan
İbrahim Çakmak, 1961, https://tr.wikipedia.org/wiki/%C4%B0brahim_%C3%87akmak
2002 – Tokat, 2002, https://tr.wikipedia.org/wiki/Mehmet_Erg%C3%BCn_Da%C4%9Fc%C4%B1o%C4%9Flu
Resul Tosun, 1956, https://tr.wikipedia.org/wiki/Resul_Tosun
2007 – Tokat, 2007, https://tr.wikipedia.org/wiki/Orhan_Ziya_Diren
Feramüz Şahin, 1964, https://tr.wikipedia.org/wiki/Feram%C3%BCz_%C5%9Eahin
Faruk Nafız Özak, 1946, https://tr.wikipedia.org/wiki/Faruk_Naf%C4%B1z_%C3%96zak
Asım Aykan,, 1953, https://tr.wikipedia.org/wiki/As%C4%B1m_Aykan
Mustafa Cumur, 1964, https://tr.wikipedia.org/wiki/Mustafa_Cumur
Prof.Dr. Ali Aydın Dumanoğlu, 1944, https://tr.wikipedia.org/wiki/Ali_Ayd%C4%B1n_Dumano%C4%9Flu
Cevdet Erdöl,, 1958, https://tr.wikipedia.org/wiki/Cevdet_Erd%C3%B6l
Kemalettin Göktaş,, 1950, https://tr.wikipedia.org/wiki/Kemalettin_G%C3%B6kta%C5%9F
Mehmet Akif Hamzaçebi, 1953, https://tr.wikipedia.org/wiki/Akif_Hamza%C3%A7ebi
Şevket Arz, 1952, https://tr.wikipedia.org/wiki/%C5%9Eevket_Arz
Hasan Güyüldar, 1946, https://tr.wikipedia.org/wiki/Hasan_G%C3%BCy%C3%BCldar
Vahdet Sinan Yerlikaya, 1951, https://tr.wikipedia.org/wiki/Vahdet_Sinan_Yerlikaya
Alim Tunç, 1964, https://tr.wikipedia.org/wiki/Alim_Tun%C3%A7
Ahmet Çağlayan, 1952, https://tr.wikipedia.org/wiki/Ahmet_%C3%87a%C4%9Flayan
Osman Coşkunoğlu, 1947, https://tr.wikipedia.org/wiki/Osman_Co%C5%9Fkuno%C4%9Flu
Hüseyin Çelik, 1959, https://tr.wikipedia.org/wiki/H%C3%BCseyin_%C3%87elik
Maliki Ejder Arvas, 1950, https://tr.wikipedia.org/wiki/Maliki_Ejder_Arvas
Haci Biner, 1950, https://tr.wikipedia.org/wiki/Haci_Biner
Yekta Haydaroğlu, 1954, https://tr.wikipedia.org/wiki/Yekta_Haydaro%C4%9Flu
Cüneyit Karabıyık, 1949, https://tr.wikipedia.org/wiki/C%C3%BCneyit_Karab%C4%B1y%C4%B1k
Halil Kaya, 1920, https://tr.wikipedia.org/wiki/Halil_Kaya
Mehmet Kartal, 1941, https://tr.wikipedia.org/wiki/Mehmet_Kartal_(Vanl%C4%B1)
Şükrü Önder, 1948, https://tr.wikipedia.org/wiki/%C5%9E%C3%BCkr%C3%BC_%C3%96nder
Muharrem İnce, 1964, https://tr.wikipedia.org/wiki/Muharrem_%C4%B0nce
İlyas Arslan,, 1956, https://tr.wikipedia.org/wiki/%C4%B0lyas_Arslan
Mehmet Çiçek, 1946, https://tr.wikipedia.org/wiki/Mehmet_%C3%87i%C3%A7ek
Mehmet Erdemir, 1953, https://tr.wikipedia.org/wiki/Mehmet_Erdemir
Mehmet Yaşar Öztürk, 1967, https://tr.wikipedia.org/wiki/Mehmet_Ya%C5%9Far_%C3%96zt%C3%BCrk
Emin Koç, 1950, https://tr.wikipedia.org/wiki/Emin_Ko%C3%A7
Polat Türkmen, 1957, https://tr.wikipedia.org/wiki/Polat_T%C3%BCrkmen
Fazlı Erdoğan, 1956, https://tr.wikipedia.org/wiki/Fazl%C4%B1_Erdo%C4%9Fan
Harun Akın, 1963, https://tr.wikipedia.org/wiki/Harun_Ak%C4%B1n
Nadir Saraç, 1956, https://tr.wikipedia.org/wiki/Nadir_Sara%C3%A7
Failed links:
https://tr.wikipedia.org/wiki/Kemal_K%C4%B1l%C4%B1%C3%A7daro%C4%9Flu
https://tr.wikipedia.org/wiki/Recep_Tayyip_Erdo%C4%9Fan

